In [ ]:
-----------------------------------
--
-- Adjust the following variables to customize the install of Stellar BI
--
-- PROJECT: sets the core name of the database, roles and warehouses used for Stellar BI
--
-- ENVIRONMENT: This allows you to deploy the framework as a separate environment (like dev or test), 
--              do not use this if you are only going to deploy a single instance of  Stellar BI
--              in your account.
--
-- ADMIN_USERS: Provide a comma delimited list of user names who will be granted the ADMIN role of 
--              Stellar BI
--              
------------------------------------
-- The core name of the database, roles and warehouses created by the Stellar BI deployment 
SET PROJECT='STELLAR_BI';

-- comma delimited list of users to be granted the STELLAR_BI_ADMIN role 
SET ADMIN_USERS=''; -- example: 'USER_1,USER_2' 

-- DO not use this unless you plan on deploying more than one instance of Stellar BI to you Snowflake Account
SET ENVIRONMENT = 'PROD'; -- example: 'DEV' or 'TEST'
                      

In [ ]:
SET SUFFIX = (SELECT IFF($ENVIRONMENT='PROD','','_'||$ENVIRONMENT));
SET ADMIN = $PROJECT||'_ADMIN'||$SUFFIX;
SET CREATOR = $PROJECT||'_CREATOR'||$SUFFIX;
SET VIEWER = $PROJECT||'_VIEWER'||$SUFFIX;
SET DB = $PROJECT||$SUFFIX;
SET COMMENT_TAG = '{"origin":"sf_sit","name":"stellar_bi","version":{"major":0,"minor":9,"build":1901,"hash":"247f988d9c12","timestamp":1763142031}}';

In [ ]:
USE ROLE SECURITYADMIN;
CREATE ROLE IF NOT EXISTS IDENTIFIER($ADMIN) COMMENT = 'Administrator of the BI framework';
GRANT ROLE IDENTIFIER($ADMIN) TO ROLE SYSADMIN;

CREATE ROLE IF NOT EXISTS IDENTIFIER($CREATOR) COMMENT = 'Creator/editor of projects within the BI framework';
GRANT ROLE IDENTIFIER($CREATOR) TO ROLE IDENTIFIER($ADMIN);

CREATE ROLE IF NOT EXISTS IDENTIFIER($VIEWER) COMMENT = 'Viewer of available projects within the BI framework';
GRANT ROLE IDENTIFIER($VIEWER) TO ROLE IDENTIFIER($ADMIN);

In [ ]:
USE ROLE SYSADMIN;

-- Set warehouse names with _WH suffix
SET ADMIN_WH = $ADMIN||'_WH';
SET CREATOR_WH = $CREATOR||'_WH';
SET VIEWER_WH = $VIEWER||'_WH';

CREATE WAREHOUSE IF NOT EXISTS IDENTIFIER($ADMIN_WH) COMMENT = $COMMENT_TAG;
GRANT ALL ON WAREHOUSE IDENTIFIER($ADMIN_WH) TO ROLE IDENTIFIER($ADMIN);

CREATE WAREHOUSE IF NOT EXISTS IDENTIFIER($CREATOR_WH) COMMENT = $COMMENT_TAG;
GRANT ALL ON WAREHOUSE IDENTIFIER($CREATOR_WH) TO ROLE IDENTIFIER($CREATOR);

CREATE WAREHOUSE IF NOT EXISTS IDENTIFIER($VIEWER_WH) COMMENT = $COMMENT_TAG;
GRANT ALL ON WAREHOUSE IDENTIFIER($VIEWER_WH) TO ROLE IDENTIFIER($VIEWER);


In [ ]:
USE ROLE SYSADMIN;
CREATE DATABASE IF NOT EXISTS IDENTIFIER($DB) COMMENT = $COMMENT_TAG;

GRANT USAGE ON DATABASE IDENTIFIER($DB) TO ROLE IDENTIFIER($ADMIN);
GRANT USAGE ON DATABASE IDENTIFIER($DB) TO ROLE IDENTIFIER($CREATOR);
GRANT USAGE ON DATABASE IDENTIFIER($DB) TO ROLE IDENTIFIER($VIEWER);

USE DATABASE IDENTIFIER($DB);

In [ ]:

-- Create the schema and grant privileges.

-- Use ROLE_TO_USE if it's defined, otherwise use SYSADMIN.
SET ROLE_TO_USE = COALESCE(GETVARIABLE('ROLE_TO_USE'), 'SYSADMIN');
USE ROLE IDENTIFIER($ROLE_TO_USE);

CREATE SCHEMA IF NOT EXISTS DATA;
GRANT ALL ON SCHEMA DATA TO ROLE IDENTIFIER($ADMIN);
GRANT USAGE ON SCHEMA DATA TO ROLE IDENTIFIER($CREATOR);

USE ROLE IDENTIFIER($ADMIN);
CREATE OR REPLACE STAGE DATA.IMPORT DIRECTORY = (ENABLE = TRUE)
                                    FILE_FORMAT = (TYPE = CSV 
                                                   COMPRESSION = NONE
                                                   FIELD_DELIMITER = NONE
                                                   RECORD_DELIMITER = NONE
                                                   FILE_EXTENSION = ''
                                                   ESCAPE_UNENCLOSED_FIELD = NONE
                                                   )
                                    COMMENT = $COMMENT_TAG
;

USE SCHEMA DATA;

-- Create the USAGE table
CREATE TABLE DATA.USAGE IF NOT EXISTS (
    CREATED_ON TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    ORGANIZATION_NAME VARCHAR(255) DEFAULT CURRENT_ORGANIZATION_NAME(),
    ACCOUNT_NAME VARCHAR(255) DEFAULT CURRENT_ACCOUNT_NAME(),
    USER_EMAIL VARCHAR(255),
    CUSTOMER_ACCOUNT VARCHAR(255)
);

GRANT ALL ON TABLE DATA.USAGE TO ROLE IDENTIFIER($ADMIN);


In [ ]:
-- Use ROLE_TO_USE if it's defined, otherwise use SYSADMIN.
SET ROLE_TO_USE = COALESCE(GETVARIABLE('ROLE_TO_USE'), 'SYSADMIN');
USE ROLE IDENTIFIER($ROLE_TO_USE);

USE DATABASE IDENTIFIER($DB);
CREATE SCHEMA IF NOT EXISTS UI;
GRANT ALL ON SCHEMA UI TO ROLE IDENTIFIER($ADMIN);

USE ROLE IDENTIFIER($ADMIN);
USE SCHEMA UI;
CREATE OR REPLACE STAGE UI.STREAMLIT DIRECTORY = (ENABLE = TRUE)
                                    FILE_FORMAT = (TYPE = CSV 
                                                   COMPRESSION = NONE
                                                   FIELD_DELIMITER = NONE
                                                   RECORD_DELIMITER = NONE
                                                   FILE_EXTENSION = ''
                                                   ESCAPE_UNENCLOSED_FIELD = NONE
                                                   )
                                    COMMENT = $COMMENT_TAG;

In [ ]:
COPY INTO @UI.STREAMLIT/ui/tableItemView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIHN0cmVhbWxpdC5kZWx0YV9nZW5lcmF0b3IgaW1wb3J0IERlbHRhR2VuZXJhdG9yCmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gLmV4cHJlc3Npb25WaWV3IGltcG9ydCBFeHByZXNzaW9uVmlldwpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KCmRlZiBUYWJsZUl0ZW1WaWV3KHRhYmxlSXRlbTogVGFibGVJdGVtLCBrZXk6IHN0ciA9ICIiLCBkaXNwbGF5RnVsbE5hbWU6IGJvb2wgPSBUcnVlLCBkaXNwbGF5V2luZG93OiBEZWx0YUdlbmVyYXRvciA9IE5vbmUpOgogICAgZnJvbSAucmVsYXRpb25zaGlwVmlldyBpbXBvcnQgUmVsYXRpb25zaGlwVmlldwogICAgaWYgdGFibGVJdGVtLmlzQ29sdW1uOgogICAgICAgIGtleT1mImNvbHVtbl97dGFibGVJdGVtLl9faGFzaF9fKCl9X3trZXl9Ii5yZXBsYWNlKCItIiwiXyIpCiAgICBlbHNlOgogICAgICAgIGtleT1mInRhYmxlX2l0ZW1fe3RhYmxlSXRlbS5fX2hhc2hfXygpfV97a2V5fSIucmVwbGFjZSgiLSIsIl8iKQogICAgCiAgICBpZiBkaXNwbGF5RnVsbE5hbWUgPT0gVHJ1ZToKICAgICAgICBuYW1lID0gdGFibGVJdGVtLmdldElkZW50aWZpZXIocXVvdGVNaXhlZENhc2U9RmFsc2UsIGluY2x1ZGVUYWJsZUFsaWFzPVRydWUsIGluY2x1ZGVUYWJsZU5hbWU9VHJ1ZSkKICAgIGVsc2U6CiAgICAgICAgbmFtZSA9IHRhYmxlSXRlbS5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPUZhbHNlKQoKICAgIGFsbElzc3VlcyA9IHRhYmxlSXRlbS5HZXRBbGxJc3N1ZXMoKQogICAgCiAgICBpZiBsZW4oYWxsSXNzdWVzKSA+IDA6CiAgICAgICAgaWYgbGVuKGFsbElzc3VlcykgPiAxOgogICAgICAgICAgICBuYW1lID0gZiJ7bmFtZX0gKHtsZW4oYWxsSXNzdWVzKX0gSXNzdWVzKSIKICAgICAgICBlbHNlOgogICAgICAgICAgICBuYW1lID0gZiJ7bmFtZX0gKHtsZW4oYWxsSXNzdWVzKX0gSXNzdWUpIgoKICAgICAgICBjc3MgPSBmIiIiCiAgICAgICAgPHN0eWxlPgogICAgICAgIGRpdltjbGFzcyo9XCJzdC1rZXkte2tleX1cIl0ge3sKICAgICAgICAgICAgYmFja2dyb3VuZC1jb2xvcjogcmVkOwogICAgICAgIH19CiAgICAgICAgPC9zdHlsZT4KIiIiCiAgICAgICAgCiAgICAgICAgc3QuaHRtbChjc3MpCgogICAgd2l0aCBzdC5jb250YWluZXIoa2V5PWtleSk6ICAgIAogICAgICAgIGlmIGRpc3BsYXlXaW5kb3cgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHR5cGVDb2wsIG5hbWVDb2wsIGV4cENvbCwgcmVsQ29sID0gc3QuY29sdW1ucyhbMSwxMCw2LDddLCBnYXA9Tm9uZSkKCiAgICAgICAgICAgIHdpdGggdHlwZUNvbDoKICAgICAgICAgICAgICAgIHN0LnRleHQodGFibGVJdGVtLnR5cGUudG9JY29uKCkpCiAgICAgICAgICAgIHdpdGggbmFtZUNvbDoKICAgICAgICAgICAgICAgIHN0LnRleHQobmFtZSkKICAgICAgICAgICAgCiAgICAgICAgICAgIHdpdGggZXhwQ29sOgogICAgICAgICAgICAgICAgaWYgdGFibGVJdGVtLmV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgaWYgc3QuYnV0dG9uKCJFeHByZXNzaW9uIiwga2V5PWYic2hvd19leHByZXNzaW9uc197a2V5fSIpOgogICAgICAgICAgICAgICAgICAgICAgICBkaXNwbGF5V2luZG93LmVtcHR5KCkKICAgICAgICAgICAgICAgICAgICAgICAgd2l0aCBkaXNwbGF5V2luZG93OgogICAgICAgICAgICAgICAgICAgICAgICAgICAgRXhwcmVzc2lvblZpZXcodGFibGVJdGVtLmV4cHJlc3Npb24sIGtleT1rZXkpCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHN0LnRleHQoIiIpCiAgICAgICAgICAgIHdpdGggcmVsQ29sOgogICAgICAgICAgICAgICAgaWYgbGVuKHRhYmxlSXRlbS5yZWxhdGlvbnNoaXBzKSA+IDA6CiAgICAgICAgICAgICAgICAgICAgaWYgc3QuYnV0dG9uKCJSZWxhdGlvbnNoaXBzIiwga2V5PWYic2hvd19yZWxhdGlvbnNoaXBzX3trZXl9Iik6CiAgICAgICAgICAgICAgICAgICAgICAgIGRpc3BsYXlXaW5kb3cuZW1wdHkoKQogICAgICAgICAgICAgICAgICAgICAgICB3aXRoIGRpc3BsYXlXaW5kb3c6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBmb3IgcmVsYXRpb25zaGlwIGluIHRhYmxlSXRlbS5yZWxhdGlvbnNoaXBzOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFJlbGF0aW9uc2hpcFZpZXcocmVsYXRpb25zaGlwPXJlbGF0aW9uc2hpcCwgaW5jbHVkZUhlYWRlcj1UcnVlLCBrZXk9a2V5KQogICAgICAgICAgICAgICAgCgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHR5cGVDb2wsIG5hbWVDb2wsIGV4cENvbCA9IHN0LmNvbHVtbnMoWzEsNSwxNV0sIGdhcD1Ob25lKQogICAgICAgICAgICB3aXRoIHR5cGVDb2w6CiAgICAgICAgICAgICAgICBpZiB0YWJsZUl0ZW0udHlwZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KHRhYmxlSXRlbS50eXBlLnRvSWNvbigpKQogICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgIHdpdGggbmFtZUNvbDoKICAgICAgICAgICAgICAgIHN0LnRleHQobmFtZSkKICAgICAgICAgICAgd2l0aCBleHBDb2w6CiAgICAgICAgICAgICAgICBpZiB0YWJsZUl0ZW0uZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KHRhYmxlSXRlbS5leHByZXNzaW9uLnZhbHVlKQogICAgICAgICAgICAgICAgCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/expressionEditor.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwppbXBvcnQgc3RyZWFtbGl0IGFzIHN0CmZyb20gYmkuY29yZS5leHByZXNzaW9uQ29kZVR5cGUgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnByb2plY3QgaW1wb3J0IFByb2plY3QKZnJvbSBiaS5yZWFkZXJzLnBiaS5tb2RlbCBpbXBvcnQgTW9kZWwKZnJvbSB1aS5leHByZXNzaW9uVmlldyBpbXBvcnQgRXhwcmVzc2lvblZpZXcKCmNsYXNzIEV4cHJlc3Npb25FZGl0b3IoKToKCiAgICBkZWYgX19pbml0X18oc2VsZiwgZXhwcmVzc2lvblRvRWRpdDogRXhwcmVzc2lvbik6CiAgICAgICAgaWYgImV4cHJlc3Npb25Ub0VkaXQiIGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgICAgIGV4cHJlc3Npb25Ub0VkaXQgPSBzdC5zZXNzaW9uX3N0YXRlWyJleHByZXNzaW9uVG9FZGl0Il0KICAgICAgICAgICAgbW9kZWw6IE1vZGVsID0gZXhwcmVzc2lvblRvRWRpdC5Nb2RlbCAgICAgICAgCiAgICAgICAgICAgIHBhcmVudCA9IGV4cHJlc3Npb25Ub0VkaXQucGFyZW50CiAgICAgICAgICAgIGNvZGVUeXBlID0gZXhwcmVzc2lvblRvRWRpdC5jb2RlVHlwZS5uYW1lCgogICAgICAgIGlmIGV4cHJlc3Npb25Ub0VkaXQgaXMgTm9uZToKICAgICAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9ICIiCiAgICAgICAgICAgIGNvZGVUeXBlID0gRXhwcmVzc2lvbkNvZGVUeXBlLkRBWAogICAgICAgICAgICBpZiAicHJvamVjdCIgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgICAgICAgICAgICAgIHByb2plY3Q6IFByb2plY3QgPSBzdC5zZXNzaW9uX3N0YXRlWyJwcm9qZWN0Il0KICAgICAgICAgICAgICAgIG1vZGVsID0gcHJvamVjdC5tb2RlbAogICAgICAgICAgICAgICAgcGFyZW50ID0gTm9uZQogICAgICAgICAgICAgICAgY29kZVR5cGUgPSAiREFYIgoKICAgICAgICBleHByZXNzaW9uU3RyaW5nID0gZXhwcmVzc2lvblRvRWRpdC5zb3VyY2VTdHJpbmcKICAgICAgIAoKICAgICAgICBleHByZXNzaW9uVHlwZXMgPSBbdCBmb3IgdCBpbiBFeHByZXNzaW9uQ29kZVR5cGVdCgogICAgICAgIHR5cGVJbmRleCA9IGV4cHJlc3Npb25UeXBlcy5pbmRleChjb2RlVHlwZSkKICAgICAgICBzdC5oZWFkZXIoIkV4cHJlc3Npb24gRWRpdG9yIikKICAgICAgICBjb2RlVHlwZSA9IHN0LnNlbGVjdGJveCgiVHlwZSIsIG9wdGlvbnM9ZXhwcmVzc2lvblR5cGVzLCBpbmRleD10eXBlSW5kZXgsICkKICAgICAgICBleHByZXNzaW9uU3RyaW5nID0gc3QudGV4dF9hcmVhKCJFeHByZXNzaW9uIix2YWx1ZT1leHByZXNzaW9uU3RyaW5nLCBwbGFjZWhvbGRlcj0iRW50ZXIgYW4gZXhwcmVzc2lvbiIpCgogICAgICAgIGlmIHN0LmJ1dHRvbigiUHJvY2VzcyIsIGtleT0icHJvY2Vzc19idG4iLCBkaXNhYmxlZD0ocGFyZW50IGlzIE5vbmUgb3IgZXhwcmVzc2lvblN0cmluZyA9PSBleHByZXNzaW9uVG9FZGl0LnNvdXJjZVN0cmluZykpOgogICAgICAgICAgICB0cnk6CiAgICAgICAgICAgICAgICBwYXJzZXIgPSBtb2RlbC5SdW50aW1lLkdldEV4cHJlc3Npb25QYXJzZXIoY29kZVR5cGUpCiAgICAgICAgICAgICAgICBleHByZXNzaW9uOiBFeHByZXNzaW9uID0gcGFyc2VyLkNyZWF0ZUZyb21TdHJpbmcocGFyZW50LCBleHByZXNzaW9uU3RyaW5nKQogICAgICAgICAgICAgICAgaWYgZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBleHByZXNzaW9uVG9FZGl0ID0gZXhwcmVzc2lvbgogICAgICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgICAgICBzdC5lcnJvcihmIkVycm9yIHBhcnNpbmcgRXhwcmVzc2lvbjoge2V9IikKCiAgICAgICAgRXhwcmVzc2lvblZpZXcoZXhwcmVzc2lvblRvRWRpdCkKICAgICAgICBzdC5zZXNzaW9uX3N0YXRlWyJleHByZXNzaW9uVG9FZGl0Il0gPSBleHByZXNzaW9uVG9FZGl0Cg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/semanticView.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/semanticTableView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdAoKZnJvbSB1aS5jb2x1bW5WaWV3IGltcG9ydCBDb2x1bW5WaWV3CmZyb20gdWkucXVlcnlWaWV3IGltcG9ydCBRdWVyeVZpZXcKZnJvbSB1aS50YWJsZVBhcnRpdGlvblZpZXcgaW1wb3J0IFRhYmxlUGFydGl0aW9uVmlldwpmcm9tIGJpLnJlYWRlcnMucGJpLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUsIFRhYmxlCmZyb20gYmkuY29yZS5jb25uZWN0b3IgaW1wb3J0IENvbm5lY3RvciwgQ29ubmVjdG9yVHlwZQoKCmRlZiBTZW1hbnRpY1RhYmxlVmlldyh0YWJsZTogU2VtYW50aWNUYWJsZSwga2V5OiBzdHIgPSAiIik6CiAgICAiIiIKICAgIE1vZGVybiwgZWxlZ2FudCB0YWJsZSB2aWV3IHdpdGggY2FyZC1iYXNlZCBsYXlvdXQuCiAgICBNYXRjaGVzIHRoZSBzZW1hbnRpYyB2aWV3IHJlZGVzaWduIGFlc3RoZXRpYy4KICAgICIiIgogICAgbmFtZSA9IHRhYmxlLm5hbWUKICAgIGtleSA9IGYidGFibGVfe3RhYmxlLl9faGFzaF9fKCl9X3trZXl9IgogICAgCiAgICAjIENsZWFuLCBtaW5pbWFsaXN0IHN0eWxpbmcKICAgIHN0Lmh0bWwoIiIiCiAgICA8c3R5bGU+CiAgICAvKiBDb21wYWN0IHNwYWNpbmcgKi8KICAgIC5tYWluIC5ibG9jay1jb250YWluZXIgewogICAgICAgIHBhZGRpbmctdG9wOiAwLjVyZW0gIWltcG9ydGFudDsKICAgICAgICBwYWRkaW5nLWJvdHRvbTogMC41cmVtICFpbXBvcnRhbnQ7CiAgICAgICAgbWF4LXdpZHRoOiAxMDAlICFpbXBvcnRhbnQ7CiAgICB9CiAgICBkaXZbZGF0YS10ZXN0aWQ9InN0VmVydGljYWxCbG9jayJdIHsKICAgICAgICBnYXA6IDAuM3JlbSAhaW1wb3J0YW50OwogICAgfQogICAgCiAgICAvKiBQYWdlIGhlYWRlciAqLwogICAgLnBhZ2UtaGVhZGVyIHsKICAgICAgICBiYWNrZ3JvdW5kOiBsaW5lYXItZ3JhZGllbnQoMTM1ZGVnLCAjNjY3ZWVhIDAlLCAjNzY0YmEyIDEwMCUpOwogICAgICAgIGNvbG9yOiB3aGl0ZTsKICAgICAgICBwYWRkaW5nOiAyOHB4IDMycHg7CiAgICAgICAgYm9yZGVyLXJhZGl1czogMTJweDsKICAgICAgICBib3gtc2hhZG93OiAwIDJweCAxMnB4IHJnYmEoMTAyLCAxMjYsIDIzNCwgMC4xNSk7CiAgICAgICAgaGVpZ2h0OiAxMDAlOwogICAgICAgIGRpc3BsYXk6IGZsZXg7CiAgICAgICAgZmxleC1kaXJlY3Rpb246IGNvbHVtbjsKICAgICAgICBqdXN0aWZ5LWNvbnRlbnQ6IGNlbnRlcjsKICAgICAgICBtaW4taGVpZ2h0OiAxMTZweDsKICAgIH0KICAgIC5wYWdlLWhlYWRlciBoMSB7CiAgICAgICAgbWFyZ2luOiAwIDAgMTRweCAwOwogICAgICAgIGZvbnQtc2l6ZTogMjZweDsKICAgICAgICBmb250LXdlaWdodDogNzAwOwogICAgICAgIGRpc3BsYXk6IGZsZXg7CiAgICAgICAgYWxpZ24taXRlbXM6IGNlbnRlcjsKICAgICAgICBnYXA6IDEwcHg7CiAgICAgICAgbGluZS1oZWlnaHQ6IDEuMjsKICAgIH0KICAgIC5wYWdlLWhlYWRlciAuc3VidGl0bGUgewogICAgICAgIG1hcmdpbjogMDsKICAgICAgICBvcGFjaXR5OiAwLjk7CiAgICAgICAgZm9udC1zaXplOiAxM3B4OwogICAgICAgIGxpbmUtaGVpZ2h0OiAxLjY7CiAgICB9CiAgICAKICAgIC8qIEJ1dHRvbnMgKi8KICAgIGRpdltkYXRhLXRlc3RpZD0ic3RCdXR0b24iXSBidXR0b25ba2luZD0icHJpbWFyeSJdIHsKICAgICAgICBiYWNrZ3JvdW5kOiBsaW5lYXItZ3JhZGllbnQoMTM1ZGVnLCAjMTBiOTgxIDAlLCAjMDU5NjY5IDEwMCUpICFpbXBvcnRhbnQ7CiAgICAgICAgYm9yZGVyLXJhZGl1czogOHB4ICFpbXBvcnRhbnQ7CiAgICAgICAgcGFkZGluZzogOHB4IDE2cHggIWltcG9ydGFudDsKICAgICAgICBib3JkZXI6IG5vbmUgIWltcG9ydGFudDsKICAgICAgICBib3gtc2hhZG93OiAwIDFweCAzcHggcmdiYSgxNiwgMTg1LCAxMjksIDAuMikgIWltcG9ydGFudDsKICAgICAgICBmb250LXdlaWdodDogNjAwICFpbXBvcnRhbnQ7CiAgICAgICAgZm9udC1zaXplOiAxNHB4ICFpbXBvcnRhbnQ7CiAgICAgICAgdHJhbnNpdGlvbjogYWxsIDAuMnMgZWFzZSAhaW1wb3J0YW50OwogICAgfQogICAgZGl2W2RhdGEtdGVzdGlkPSJzdEJ1dHRvbiJdIGJ1dHRvbltraW5kPSJwcmltYXJ5Il06aG92ZXIgewogICAgICAgIGJveC1zaGFkb3c6IDAgMnB4IDZweCByZ2JhKDE2LCAxODUsIDEyOSwgMC4zKSAhaW1wb3J0YW50OwogICAgICAgIHRyYW5zZm9ybTogdHJhbnNsYXRlWSgtMXB4KTsKICAgIH0KICAgIAogICAgLyogRXhwYW5kZXIgc3R5bGluZyAtIGNsZWFuZXIsIGxlc3MgcHJvbWluZW50ICovCiAgICBkaXZbZGF0YS10ZXN0aWQ9InN0RXhwYW5kZXIiXSB7CiAgICAgICAgYmFja2dyb3VuZDogd2hpdGU7CiAgICAgICAgYm9yZGVyOiAxcHggc29saWQgI2U1ZTdlYjsKICAgICAgICBib3JkZXItcmFkaXVzOiA4cHg7CiAgICAgICAgbWFyZ2luLWJvdHRvbTogNnB4OwogICAgfQogICAgZGl2W2RhdGEtdGVzdGlkPSJzdEV4cGFuZGVyIl0gPiBkaXY6Zmlyc3QtY2hpbGQgewogICAgICAgIGJhY2tncm91bmQ6ICNmYWZhZmE7CiAgICAgICAgYm9yZGVyLXJhZGl1czogOHB4IDhweCAwIDA7CiAgICAgICAgcGFkZGluZzogMTBweCAxNHB4OwogICAgfQogICAgPC9zdHlsZT4KICAgICIiIikKICAgIAogICAgIyBNb2Rlcm4gcGFnZSBoZWFkZXIgKGZ1bGwgd2lkdGgsIG5vIG1ldHJpY3MpCiAgICBzdC5tYXJrZG93bihmIiIiCiAgICA8ZGl2IGNsYXNzPSJwYWdlLWhlYWRlciI+CiAgICAgICAgPGgxPnt0YWJsZS5uYW1lfTwvaDE+CiAgICAgICAgPGRpdiBjbGFzcz0ic3VidGl0bGUiPkNvbmZpZ3VyZSBwYXJ0aXRpb24gbW9kZSBhbmQgU25vd2ZsYWtlIHZpZXdzIGZvciB0aGlzIHRhYmxlPC9kaXY+CiAgICA8L2Rpdj4KICAgICIiIiwgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgIAogICAgIyBBZGQgc2lnbmlmaWNhbnQgc3BhY2luZyBiZWxvdyBoZWFkZXIKICAgIHN0Lm1hcmtkb3duKCI8ZGl2IHN0eWxlPSdtYXJnaW4tYm90dG9tOiA0OHB4Oyc+PC9kaXY+IiwgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgIAogICAgIyBNYWluIGxheW91dDogU2lkZWJhciBmb3IgY29sdW1ucywgbWFpbiBhcmVhIGZvciBjb25maWd1cmF0aW9uCiAgICBjb2wxLCBjb2wyID0gc3QuY29sdW1ucyhbMSwgM10sIGdhcD0ibGFyZ2UiKQogICAgCiAgICAjIExFRlQgU0lERUJBUjogQ29sdW1ucyAmIE1lYXN1cmVzCiAgICB3aXRoIGNvbDE6CiAgICAgICAgIyBBZGQgY29udGFpbmVyIHdpdGggcmlnaHQgcGFkZGluZyB0byBwcmV2ZW50IG92ZXJsYXAKICAgICAgICBzdC5tYXJrZG93bignPGRpdiBzdHlsZT0icGFkZGluZy1yaWdodDogMTJweDsiPicsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgIyBTaW1wbGUgdGV4dCBoZWFkZXIgYWxpZ25lZCB3aXRoIGNvbHVtbnMKICAgICAgICBzdC5tYXJrZG93bigiIiIKICAgICAgICA8ZGl2IHN0eWxlPSJwYWRkaW5nLWxlZnQ6IDBweDsgbWFyZ2luLWJvdHRvbTogMTZweDsiPgogICAgICAgICAgICA8c3BhbiBzdHlsZT0iZm9udC1zaXplOiAyMHB4OyBmb250LXdlaWdodDogNzAwOyBjb2xvcjogIzM3NDE1MTsiPvCfk4sgQ29sdW1uczwvc3Bhbj4KICAgICAgICA8L2Rpdj4KICAgICAgICAiIiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgCiAgICAgICAgZm9yIGNvbCBpbiB0YWJsZS5hbGxJdGVtczoKICAgICAgICAgICAgaWYgY29sLmlzQ29sdW1uOgogICAgICAgICAgICAgICAgQ29sdW1uVmlldyhjb2wsIGRpc3BsYXlGdWxsTmFtZT1GYWxzZSwga2V5PWtleSkKICAgICAgICAKICAgICAgICAjIE1lYXN1cmVzIHNlY3Rpb24KICAgICAgICBpZiBsZW4odGFibGUubWVhc3VyZXMpID4gMDoKICAgICAgICAgICAgc3QubWFya2Rvd24oIiIiCiAgICAgICAgICAgIDxkaXYgc3R5bGU9InBhZGRpbmctbGVmdDogMHB4OyBtYXJnaW4tdG9wOiAyNHB4OyBtYXJnaW4tYm90dG9tOiAxNnB4OyI+CiAgICAgICAgICAgICAgICA8c3BhbiBzdHlsZT0iZm9udC1zaXplOiAyMHB4OyBmb250LXdlaWdodDogNzAwOyBjb2xvcjogIzM3NDE1MTsiPvCfk4ggTWVhc3VyZXM8L3NwYW4+CiAgICAgICAgICAgIDwvZGl2PgogICAgICAgICAgICAiIiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgICAgIAogICAgICAgICAgICBmb3IgbWVhc3VyZSBpbiB0YWJsZS5tZWFzdXJlczoKICAgICAgICAgICAgICAgIENvbHVtblZpZXcobWVhc3VyZSwgZGlzcGxheUZ1bGxOYW1lPUZhbHNlLCBrZXk9a2V5KQogICAgICAgIAogICAgICAgICMgQ2xvc2UgcGFkZGluZyBjb250YWluZXIKICAgICAgICBzdC5tYXJrZG93bignPC9kaXY+JywgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgIAogICAgIyBSSUdIVCBNQUlOIEFSRUE6IFBhcnRpdGlvbnMg4oaSIEFjdGlvbnMg4oaSIFNRTAogICAgd2l0aCBjb2wyOgogICAgICAgICMgU0VDVElPTiAxOiBQQVJUSVRJT05TIChOb3cgRmlyc3QhKQogICAgICAgIHN0Lm1hcmtkb3duKCIiIgogICAgICAgIDxkaXYgc3R5bGU9InBhZGRpbmctbGVmdDogMHB4OyBtYXJnaW4tYm90dG9tOiAxNnB4OyI+CiAgICAgICAgICAgIDxzcGFuIHN0eWxlPSJmb250LXNpemU6IDIwcHg7IGZvbnQtd2VpZ2h0OiA3MDA7IGNvbG9yOiAjMzc0MTUxOyI+8J+Xgu+4jyBQYXJ0aXRpb25zPC9zcGFuPgogICAgICAgIDwvZGl2PgogICAgICAgICIiIiwgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgICAgICAKICAgICAgICBpZiBsZW4odGFibGUucGFydGl0aW9ucykgPT0gMDoKICAgICAgICAgICAgc3QuaW5mbygi4oS577iPIE5vIHBhcnRpdGlvbnMgZGVmaW5lZCBmb3IgdGhpcyB0YWJsZSIpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgIyBEaXNwbGF5IGVhY2ggcGFydGl0aW9uIHdpdGggYWxsIGNvbnRlbnQgaW5zaWRlIHRoZSBib3JkZXIKICAgICAgICAgICAgZm9yIGksIHBhcnRpdGlvbiBpbiBlbnVtZXJhdGUodGFibGUucGFydGl0aW9ucyk6CiAgICAgICAgICAgICAgICBpZiBpID4gMDoKICAgICAgICAgICAgICAgICAgICBzdC5tYXJrZG93bigiPGRpdiBzdHlsZT0nbWFyZ2luLXRvcDogMjRweDsnPjwvZGl2PiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICMgQ29udGFpbmVyIHdpdGggYm9yZGVyIHRoYXQgaW5jbHVkZXMgZXZlcnl0aGluZwogICAgICAgICAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICAgICAgICAgICMgUGFydGl0aW9uIGhlYWRlciB3aXRoIGlubGluZSBzb3VyY2UgaW5mbwogICAgICAgICAgICAgICAgICAgIHNvdXJjZV9pbmZvID0gIiIKICAgICAgICAgICAgICAgICAgICBpZiBwYXJ0aXRpb24uaXNTbm93Zmxha2U6CiAgICAgICAgICAgICAgICAgICAgICAgIGFjY291bnRzID0gc2V0KCkKICAgICAgICAgICAgICAgICAgICAgICAgYWNjb3VudExpc3QgPSAiIgogICAgICAgICAgICAgICAgICAgICAgICBmb3IgZCBpbiBwYXJ0aXRpb24uR2V0QWxsRGVwZW5kZW5jaWVzKCk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGQsIENvbm5lY3Rvcik6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgYWNjb3VudCA9IGQuZ2V0UHJvcGVydHkoImFjY291bnQiKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGFjY291bnQgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGFjY291bnRzLmFkZChhY2NvdW50KQogICAgICAgICAgICAgICAgICAgICAgICBpZiBsZW4oYWNjb3VudHMpID4gMDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGFjY291bnRMaXN0ID0gIiwgIi5qb2luKGxpc3QoYWNjb3VudHMpKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgIHNvdXJjZV9pbmZvID0gZic8c3BhbiBzdHlsZT0iZm9udC13ZWlnaHQ6IDQwMDsgY29sb3I6ICM2YjcyODA7IGZvbnQtc2l6ZTogMTNweDsiPiDigJQgU25vd2ZsYWtlICh7YWNjb3VudExpc3R9KSA8Yj5Jbml0aWFsIE1vZGU6IHtwYXJ0aXRpb24ub3JpZ2luYWxNb2RlfTwvYj48L3NwYW4+JwogICAgICAgICAgICAgICAgICAgIGVsaWYgcGFydGl0aW9uLmlzRmlsZToKICAgICAgICAgICAgICAgICAgICAgICAgcGF0aHMgPSBzZXQoKQogICAgICAgICAgICAgICAgICAgICAgICBwYXRoTGlzdCA9ICIiCiAgICAgICAgICAgICAgICAgICAgICAgIGZvciBkIGluIHBhcnRpdGlvbi5HZXRBbGxEZXBlbmRlbmNpZXMoKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoZCwgQ29ubmVjdG9yKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXRoID0gZC5nZXRQcm9wZXJ0eSgicGF0aCIpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgaWYgcGF0aCBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgcGF0aHMuYWRkKHBhdGgpCiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihwYXRocykgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgcGF0aExpc3QgPSAiLCAiLmpvaW4obGlzdChwYXRocykpCiAgICAgICAgICAgICAgICAgICAgICAgIHNvdXJjZV9pbmZvID0gZic8c3BhbiBzdHlsZT0iZm9udC13ZWlnaHQ6IDQwMDsgY29sb3I6ICM2YjcyODA7IGZvbnQtc2l6ZTogMTNweDsiPiDigJQgRmlsZSBzb3VyY2UgKHtwYXRoTGlzdH0pPC9zcGFuPicKICAgICAgICAgICAgICAgICAgICBlbGlmIHBhcnRpdGlvbi5pc0RhdGFiYXNlOgogICAgICAgICAgICAgICAgICAgICAgICBkYnMgPSBzZXQoKQogICAgICAgICAgICAgICAgICAgICAgICBkYkxpc3QgPSAiIgogICAgICAgICAgICAgICAgICAgICAgICBmb3IgZCBpbiBwYXJ0aXRpb24uR2V0QWxsRGVwZW5kZW5jaWVzKCk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGQsIENvbm5lY3Rvcik6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZGIgPSBkLmdldFByb3BlcnR5KCJkYXRhYmFzZVR5cGUiKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIG5hbWUgPSBkLmdldFByb3BlcnR5KCJuYW1lIikKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBkYiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZGJzLmFkZChmIntkYn06IHtuYW1lfSIpCiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihkYnMpID4gMDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGRiTGlzdCA9ICIsICIuam9pbihsaXN0KGRicykpCiAgICAgICAgICAgICAgICAgICAgICAgIHNvdXJjZV9pbmZvID0gZic8c3BhbiBzdHlsZT0iZm9udC13ZWlnaHQ6IDQwMDsgY29sb3I6ICM2YjcyODA7IGZvbnQtc2l6ZTogMTNweDsiPiDigJQgZGF0YWJhc2UgKHtkYkxpc3R9KSA8Yj5Nb2RlOiB7cGFydGl0aW9uLm1vZGV9PC9iPjwvc3Bhbj4nCiAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgc3QubWFya2Rvd24oZiIiIgogICAgICAgICAgICAgICAgICAgIDxkaXYgc3R5bGU9ImJhY2tncm91bmQ6ICNmOWZhZmI7IHBhZGRpbmc6IDEycHggMTZweDsgYm9yZGVyLXJhZGl1czogNnB4OyAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBtYXJnaW4tYm90dG9tOiAxNnB4OyBib3JkZXItbGVmdDogM3B4IHNvbGlkICM2NjdlZWE7Ij4KICAgICAgICAgICAgICAgICAgICAgICAgPHNwYW4gc3R5bGU9ImZvbnQtc2l6ZTogMTVweDsgZm9udC13ZWlnaHQ6IDYwMDsgY29sb3I6ICMzNzQxNTE7Ij5QYXJ0aXRpb24ge2krMX08L3NwYW4+e3NvdXJjZV9pbmZvfQogICAgICAgICAgICAgICAgICAgIDwvZGl2PgogICAgICAgICAgICAgICAgICAgICIiIiwgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAjIEFsbCBwYXJ0aXRpb24gY29udGVudCBpbnNpZGUgdGhlIHNhbWUgYm9yZGVyCiAgICAgICAgICAgICAgICAgICAgVGFibGVQYXJ0aXRpb25WaWV3KHBhcnRpdGlvbiwgcGFydGl0aW9uSW5kZXg9aSwga2V5PWtleSkKICAgICAgICAKICAgICAgICBzdC5tYXJrZG93bigiPGJyPiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgCiAgICAgICAgIyBTRUNUSU9OIDI6IFNFTUFOVElDIFRBQkxFIEFDVElPTlMgKE5vdyBTZWNvbmQhKQogICAgICAgIHN0Lm1hcmtkb3duKCIiIgogICAgICAgIDxkaXYgc3R5bGU9InBhZGRpbmctbGVmdDogMHB4OyBtYXJnaW4tYm90dG9tOiAxNnB4OyI+CiAgICAgICAgICAgIDxzcGFuIHN0eWxlPSJmb250LXNpemU6IDIwcHg7IGZvbnQtd2VpZ2h0OiA3MDA7IGNvbG9yOiAjMzc0MTUxOyI+4pqZ77iPIFNlbWFudGljIFRhYmxlIEFjdGlvbnMgKGZvciBTZW1hbnRpYyBWaWV3KTwvc3Bhbj4KICAgICAgICA8L2Rpdj4KICAgICAgICAiIiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICAgICAgCiAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICBuZXdDcmVhdGVWYWx1ZSA9IHN0LmNoZWNrYm94KAogICAgICAgICAgICAgICAgIkNyZWF0ZSBWaWV3IGluIFNub3dmbGFrZSIsIAogICAgICAgICAgICAgICAgdmFsdWU9dGFibGUuQ3JlYXRlU25vd2ZsYWtlT2JqZWN0LCAKICAgICAgICAgICAgICAgIGtleT1mIntrZXl9X2NyZWF0ZV9vYmpfY2tfYngiLAogICAgICAgICAgICAgICAgZGlzYWJsZWQ9dGFibGUuSGFzSXNzdWVzLAogICAgICAgICAgICAgICAgaGVscD0iQ3JlYXRlIGEgU25vd2ZsYWtlIHZpZXcgZm9yIHRoaXMgc2VtYW50aWMgdGFibGUuIgogICAgICAgICAgICApIAogICAgICAgICAgICBpZiBuZXdDcmVhdGVWYWx1ZSAhPSB0YWJsZS5DcmVhdGVTbm93Zmxha2VPYmplY3Q6CiAgICAgICAgICAgICAgICB0YWJsZS5DcmVhdGVTbm93Zmxha2VPYmplY3QgPSBuZXdDcmVhdGVWYWx1ZQogICAgICAgICAgICAgICAgc3QucmVydW4oKQoKICAgICAgICAgICAgaWYgdGFibGUuQ3JlYXRlU25vd2ZsYWtlT2JqZWN0OgogICAgICAgICAgICAgICAgc3QubWFya2Rvd24oIi0tLSIpCiAgICAgICAgICAgICAgICBjMSwgYzIsIGMzID0gc3QuY29sdW1ucyhbMywgMTAsIDVdKQogICAgICAgICAgICAgICAgd2l0aCBjMToKICAgICAgICAgICAgICAgICAgICBmcm9tIHV0aWwuc25vd2ZsYWtlT2JqZWN0VHlwZSBpbXBvcnQgU25vd2ZsYWtlT2JqZWN0VHlwZQogICAgICAgICAgICAgICAgICAgIG9iamVjdFR5cGVzID0gW1Nub3dmbGFrZU9iamVjdFR5cGUuVklFVywgU25vd2ZsYWtlT2JqZWN0VHlwZS5NQVRFUklBTElaRURfVklFV10KICAgICAgICAgICAgICAgICAgICBpbmRleCA9IG9iamVjdFR5cGVzLmluZGV4KHRhYmxlLlNub3dmbGFrZU9iamVjdC50eXBlKQogICAgICAgICAgICAgICAgICAgIHNlbGVjdGVkX3R5cGUgPSBzdC5zZWxlY3Rib3goCiAgICAgICAgICAgICAgICAgICAgICAgICJUeXBlIiwKICAgICAgICAgICAgICAgICAgICAgICAgb3B0aW9ucz1vYmplY3RUeXBlcywKICAgICAgICAgICAgICAgICAgICAgICAgaW5kZXg9aW5kZXgsCiAgICAgICAgICAgICAgICAgICAgICAgIGZvcm1hdF9mdW5jPWxhbWJkYSB4OiBzdHIoeCkudGl0bGUoKSwKICAgICAgICAgICAgICAgICAgICAgICAga2V5PWYie2tleX1fc25vd2ZsYWtlX29iamVjdF90eXBlIiwKICAgICAgICAgICAgICAgICAgICAgICAgaGVscD0iU2VsZWN0IHRoZSB0eXBlIG9mIFNub3dmbGFrZSB2aWV3IHRvIGNyZWF0ZSBmb3IgdGhpcyBzZW1hbnRpYyB0YWJsZS4gQSBtYXRlcmlhbGl6ZWQgdmlldyBpcyBtb3JlIGVmZmljaWVudCBmb3IgY29tcGxleCBxdWVyaWVzLCBob3dldmVyIGl0IGNhbm5vdCBiZSBjcmVhdGVkIG9uIHRvcCBvZiBhbm90aGVyIHZpZXcuIiwKICAgICAgICAgICAgICAgICAgICApCiAgICAgICAgICAgICAgICAgICAgdGFibGUuU25vd2ZsYWtlT2JqZWN0LnR5cGUgPSBzZWxlY3RlZF90eXBlCiAgICAgICAgICAgICAgICB3aXRoIGMyOgogICAgICAgICAgICAgICAgICAgIHRhYmxlLlNub3dmbGFrZU9iamVjdC5uYW1lID0gc3QudGV4dF9pbnB1dCgKICAgICAgICAgICAgICAgICAgICAgICAgIk5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICB2YWx1ZT10YWJsZS5Tbm93Zmxha2VPYmplY3QubmFtZSwKICAgICAgICAgICAgICAgICAgICAgICAga2V5PWYie2tleX1fc25vd2ZsYWtlX29iamVjdF9uYW1lIiwKICAgICAgICAgICAgICAgICAgICAgICAgbWF4X2NoYXJzPTEwMCwKICAgICAgICAgICAgICAgICAgICAgICAgcGxhY2Vob2xkZXI9ImUuZy4sIERJTV9EQVRFX1ZJRVciCiAgICAgICAgICAgICAgICAgICAgKQogICAgICAgICAgICAgICAgd2l0aCBjMzoKICAgICAgICAgICAgICAgICAgICAjIENyZWF0ZSBpbnZpc2libGUgbGFiZWwgdG8gbWF0Y2ggaW5wdXQgYWxpZ25tZW50CiAgICAgICAgICAgICAgICAgICAgc3QubWFya2Rvd24oJzxsYWJlbCBzdHlsZT0iZm9udC1zaXplOiAxNHB4OyBmb250LXdlaWdodDogNDAwOyB2aXNpYmlsaXR5OiBoaWRkZW47IGhlaWdodDogMjFweDsgZGlzcGxheTogYmxvY2s7IG1hcmdpbi1ib3R0b206IDhweDsiPkhpZGRlbjwvbGFiZWw+JywgdW5zYWZlX2FsbG93X2h0bWw9VHJ1ZSkKICAgICAgICAgICAgICAgICAgICBpZiBzdC5idXR0b24oCiAgICAgICAgICAgICAgICAgICAgICAgICLinKggQ3JlYXRlIiwgCiAgICAgICAgICAgICAgICAgICAgICAgIGtleT1mIntrZXl9X3Nub3dmbGFrZV9vYmplY3RfY3JlYXRlX2J0biIsIAogICAgICAgICAgICAgICAgICAgICAgICB0eXBlPSJwcmltYXJ5IiwKICAgICAgICAgICAgICAgICAgICAgICAgdXNlX2NvbnRhaW5lcl93aWR0aD1UcnVlLAogICAgICAgICAgICAgICAgICAgICAgICBoZWxwPSJJZiB5b3UgcGxhbiB0byBjcmVhdGUgU25vd2ZsYWtlIHZpZXdzIGZvciBwYXJ0aXRpb25zIGluIHRoaXMgdGFibGUsIGNyZWF0ZSB0aG9zZSBmaXJzdC4iCiAgICAgICAgICAgICAgICAgICAgKToKICAgICAgICAgICAgICAgICAgICAgICAgZGRsID0gdGFibGUuR2V0T2JqZWN0Q3JlYXRlRERMKCkKICAgICAgICAgICAgICAgICAgICAgICAgc2ZPYmpOYW1lID0gIi4iLmpvaW4oW3RhYmxlLlNub3dmbGFrZU9iamVjdC5zY2hlbWEsIHRhYmxlLlNub3dmbGFrZU9iamVjdC5uYW1lXSkKICAgICAgICAgICAgICAgICAgICAgICAgd2l0aCBzdC5zcGlubmVyKGYiQ3JlYXRpbmcge3RhYmxlLlNub3dmbGFrZU9iamVjdC50eXBlfSB7c2ZPYmpOYW1lfS4uLiIpOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHJlc3VsdCA9IHRhYmxlLk1vZGVsLlNlc3Npb24uc3FsKGRkbCkuY29sbGVjdCgpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgc3Quc3VjY2VzcyhmIuKchSB7c2ZPYmpOYW1lfSBjcmVhdGVkIHN1Y2Nlc3NmdWxseSEiKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LmVycm9yKGYi4p2MIEVycm9yIGNyZWF0aW5nIHtzZk9iak5hbWV9OiB7ZX0iKQogICAgICAgIAogICAgICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKGYiVmlldyBTUUwgZm9yIHt0YWJsZS5uYW1lfSIsIGV4cGFuZGVkPUZhbHNlKToKICAgICAgICAgICAgICAgIHNxbCA9IHRhYmxlLmdldFNxbFZhbHVlKCkKICAgICAgICAgICAgICAgIFF1ZXJ5VmlldyhzcWw9c3FsLCBrZXk9a2V5KQoKICAgICAgICBzdC5tYXJrZG93bigiPGJyPiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/modelView.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsLCBBbnksIENhbGxhYmxlCmltcG9ydCBzdHJlYW1saXQgYXMgc3QKZnJvbSBzdHJlYW1saXQuZGVsdGFfZ2VuZXJhdG9yIGltcG9ydCBEZWx0YUdlbmVyYXRvcgpmcm9tIGJpLnJlYWRlcnMucGJpLm1vZGVsIGltcG9ydCBNb2RlbApmcm9tIGJpLnJlYWRlcnMucGJpLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUKCmNsYXNzIE1vZGVsVmlldygpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIAogICAgICAgICAgICAgICAgIG1vZGVsOiBNb2RlbCwgCiAgICAgICAgICAgICAgICAga2V5OiBPcHRpb25hbFtzdHJdID0gTm9uZSwKICAgICAgICAgICAgICAgICBjb2xvcjogT3B0aW9uYWxbc3RyXSA9ICIjN2Y3ZjdmIiwKICAgICAgICAgICAgICAgICBiYWNrZ3JvdW5kX2NvbG9yOiBPcHRpb25hbFtzdHJdID0gIndoaXRlIiwKICAgICAgICAgICAgICAgICBob3Zlcl9jb2xvcjogT3B0aW9uYWxbc3RyXSA9ICIjN2Y3ZjdmIiwKICAgICAgICAgICAgICAgICBob3Zlcl9iYWNrZ3JvdW5kX2NvbG9yOiBPcHRpb25hbFtzdHJdID0gIiNmMWYxZjEiLAogICAgICAgICAgICAgICAgIHNlbGVjdGVkX2NvbG9yOiBPcHRpb25hbFtzdHJdID0gIndoaXRlIiwKICAgICAgICAgICAgICAgICBzZWxlY3RlZF9iYWNrZ3JvdW5kX2NvbG9yOiBPcHRpb25hbFtzdHJdID0gImJsdWUiLAogICAgICAgICAgICAgICAgIG9uX3NlbGVjdGlvbl9jaGFuZ2VkOiBPcHRpb25hbFtDYWxsYWJsZV0gPSBOb25lKToKICAgICAgICAKICAgICAgICBzZWxmLl9fbW9kZWwgPSBtb2RlbAogICAgICAgIGlmIGtleSBpcyBOb25lIG9yIGtleSA9PSAiIjoKICAgICAgICAgICAgc2VsZi5fX2tleSA9IGYibW9kZWwiCiAgICAgICAgZWxzZToKICAgICAgICAgICAgc2VsZi5fX2tleSA9IGYie2tleX1fbW9kZWwiCiAgICAgICAgICAgIAogICAgICAgIHNlbGYub25fc2VsZWN0aW9uX2NoYW5nZWQgPSBvbl9zZWxlY3Rpb25fY2hhbmdlZAogICAgICAgIHNlbGYuY29sb3IgPSBjb2xvcgogICAgICAgIHNlbGYuYmFja2dyb3VuZF9jb2xvciA9IGJhY2tncm91bmRfY29sb3IKICAgICAgICBzZWxmLmhvdmVyX2NvbG9yID0gaG92ZXJfY29sb3IKICAgICAgICBzZWxmLmhvdmVyX2JhY2tncm91bmRfY29sb3IgPSBob3Zlcl9iYWNrZ3JvdW5kX2NvbG9yCiAgICAgICAgc2VsZi5zZWxlY3RlZF9jb2xvciA9IHNlbGVjdGVkX2NvbG9yCiAgICAgICAgc2VsZi5zZWxlY3RlZF9iYWNrZ3JvdW5kX2NvbG9yID0gc2VsZWN0ZWRfYmFja2dyb3VuZF9jb2xvcgoKICAgICAgICBzZWxmLl9fb25fc2VsZWN0aW9uX2NoYW5nZWQoKSAjZmlyZSBzZWxlY3Rpb24gY2hhbmdlZCBpZiBpdCBoYXMgY2hhbmdlZCAtIG5lZWRzIHRvIGhhcHBlbiBpbiB0aGUgX19pbml0X18gc2luY2Ugc3RyZWFtbGl0IHdpbGwgcmVydW4KCiAgICBAcHJvcGVydHkKICAgIGRlZiBNb2RlbChzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX21vZGVsCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIEtleShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2tleQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIFNob3dIaWRkZW5UYWJsZXMoc2VsZikgLT4gYm9vbDoKICAgICAgICBpZiBub3QgZiJ7c2VsZi5LZXl9X3Nob3dfaGlkZGVuX3RhYmxlcyIgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2hvd19oaWRkZW5fdGFibGVzIl0gPSBGYWxzZQogICAgICAgIHJldHVybiBzdC5zZXNzaW9uX3N0YXRlW2Yie3NlbGYuS2V5fV9zaG93X2hpZGRlbl90YWJsZXMiXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBfX3NlbGVjdGVkSXRlbShzZWxmKToKICAgICAgICBpZiBmIntzZWxmLktleX1fc2VsZWN0ZWRJdGVtIiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgICAgICByZXR1cm4gc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2VsZWN0ZWRJdGVtIl0gCiAgICAgICAgcmV0dXJuIE5vbmUKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgU2VsZWN0ZWRJdGVtKHNlbGYpOgogICAgICAgIGlmIHNlbGYuX19zZWxlY3RlZEl0ZW0gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLk1vZGVsLmdldFRhYmxlKHNlbGYuX19zZWxlY3RlZEl0ZW0pCgogICAgQHByb3BlcnR5CiAgICBkZWYgU3R5bGUoc2VsZik6CiAgICAgICAgcmV0dXJuIGYiIiIKPHN0eWxlPgpkaXZbY2xhc3MqPSJzdC1rZXkte3NlbGYuS2V5fV90YWJsZV8iXSA+IGRpdltjbGFzcz0ic3RCdXR0b24iXXt7CiAgICBtYXgtaGVpZ2h0OjE2cHg7CiAgICBwYWRkaW5nOjBweDsKICAgIG1hcmdpbjowcHg7Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbnt7CiAgICBqdXN0aWZ5LWNvbnRlbnQ6ZmxleC1zdGFydDsKICAgIHBhZGRpbmc6MHB4OwogICAgcGFkZGluZy1sZWZ0OjNweDsKICAgIGJvcmRlcjpub25lOwogICAgYm9yZGVyLXJhZGl1czowcHg7CiAgICB3aWR0aDoxMDAlOwogICAgb3ZlcmZsb3c6aGlkZGVuOwogICAgbWFyZ2luOjBweDsKICAgIG1heC1oZWlnaHQ6MTZweDsKICAgIG1pbi1oZWlnaHQ6MTZweDsKfX0KZGl2W2NsYXNzKj0ic3Qta2V5LXtzZWxmLktleX1fdGFibGVfIl0gYnV0dG9uW2tpbmQ9c2Vjb25kYXJ5XSB7ewogICAgYmFja2dyb3VuZC1jb2xvcjoge3NlbGYuYmFja2dyb3VuZF9jb2xvcn07CiAgICBjb2xvcjoge3NlbGYuY29sb3J9Owp9fQpkaXZbY2xhc3MqPSJzdC1rZXkte3NlbGYuS2V5fV90YWJsZV8iXSBidXR0b25ba2luZD1zZWNvbmRhcnldOmhvdmVyIHt7CiAgICBiYWNrZ3JvdW5kLWNvbG9yOiB7c2VsZi5iYWNrZ3JvdW5kX2NvbG9yfTsKICAgIGNvbG9yOiB7c2VsZi5jb2xvcn07Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbltraW5kPXNlY29uZGFyeV06YWN0aXZlIHt7CiAgICBiYWNrZ3JvdW5kLWNvbG9yOiB7c2VsZi5iYWNrZ3JvdW5kX2NvbG9yfTsKICAgIGNvbG9yOiB7c2VsZi5jb2xvcn07Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbltraW5kPXNlY29uZGFyeV06Zm9jdXM6bm90KDphY3RpdmUpIHt7CiAgICBiYWNrZ3JvdW5kLWNvbG9yOiB7c2VsZi5iYWNrZ3JvdW5kX2NvbG9yfTsKICAgIGNvbG9yOiB7c2VsZi5jb2xvcn07Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbltraW5kPXByaW1hcnldIHt7CiAgICBiYWNrZ3JvdW5kLWNvbG9yOiB7c2VsZi5zZWxlY3RlZF9iYWNrZ3JvdW5kX2NvbG9yfTsKICAgIGNvbG9yOiB7c2VsZi5zZWxlY3RlZF9jb2xvcn07Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbltraW5kPXByaW1hcnldOmhvdmVyIHt7CiAgICBiYWNrZ3JvdW5kLWNvbG9yOiB7c2VsZi5zZWxlY3RlZF9iYWNrZ3JvdW5kX2NvbG9yfTsKICAgIGNvbG9yOiB7c2VsZi5zZWxlY3RlZF9jb2xvcn07Cn19CmRpdltjbGFzcyo9InN0LWtleS17c2VsZi5LZXl9X3RhYmxlXyJdIGJ1dHRvbltraW5kPXByaW1hcnldOmFjdGl2ZSB7ewogICAgYmFja2dyb3VuZC1jb2xvcjoge3NlbGYuc2VsZWN0ZWRfYmFja2dyb3VuZF9jb2xvcn07CiAgICBjb2xvcjoge3NlbGYuc2VsZWN0ZWRfY29sb3J9Owp9fQpkaXZbY2xhc3MqPSJzdC1rZXkte3NlbGYuS2V5fV90YWJsZV8iXSBidXR0b25ba2luZD1wcmltYXJ5XTpmb2N1czpub3QoOmFjdGl2ZSkge3sKICAgIGJhY2tncm91bmQtY29sb3I6IHtzZWxmLnNlbGVjdGVkX2JhY2tncm91bmRfY29sb3J9OwogICAgY29sb3I6IHtzZWxmLnNlbGVjdGVkX2NvbG9yfTsKfX0KZGl2W2NsYXNzKj0ic3Qta2V5LXtzZWxmLktleX1fdGFibGVfIl0gYnV0dG9uIHB7ewogICAgZm9udC1zaXplOiAxNHB4OwogICAgbWFyZ2luOiAwcHg7CiAgICBwYWRkaW5nOiAwcHg7CiAgICB0ZXh0LW92ZXJmbG93OiBlbGxpcHNpczsKICAgIHdoaXRlLXNwYWNlOiBub3dyYXA7CiAgICBvdmVyZmxvdzogaGlkZGVuOwp9fQpkaXZbY2xhc3MqPSJzdC1rZXkte3NlbGYuS2V5fV90YWJsZV8iXSBidXR0b24gcDpob3Zlcnt7CiAgICBvdmVyZmxvdzogdmlzaWJsZTsKfX0KPC9zdHlsZT4KIiIiCiAgICAKCiAgICAKICAgICAgICAKICAgCiAgICBkZWYgR2V0VGFibGVOYW1lcyhzZWxmKSAtPiBkaWN0OgogICAgICAgIGlmIGYie3NlbGYuS2V5fV90YWJsZV9uYW1lcyIgbm90IGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgICAgIHRhYmxlTmFtZXMgPSB7fSAgICAKICAgICAgICAgICAgc25vd2ZsYWtlVGFibGVzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgICAgICBleHRlcm5hbFRhYmxlczogTGlzdFtzdHJdID0gW10KICAgICAgICAgICAgZmlsZVRhYmxlczogTGlzdFtzdHJdID0gW10KICAgICAgICAgICAgZW1iZWRkZWRUYWJsZXM6IExpc3Rbc3RyXSA9IFtdCiAgICAgICAgICAgIG90aGVyVGFibGVzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgICAgICBoaWRkZW5Tbm93Zmxha2VUYWJsZXM6IExpc3Rbc3RyXSA9IFtdCiAgICAgICAgICAgIGhpZGRlbkV4dGVybmFsVGFibGVzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgICAgICBoaWRkZW5GaWxlVGFibGVzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgICAgICBoaWRkZW5FbWJlZGRlZFRhYmxlczogTGlzdFtzdHJdID0gW10KICAgICAgICAgICAgaGlkZGVuT3RoZXJUYWJsZXM6IExpc3Rbc3RyXSA9IFtdCgogICAgICAgICAgICBmb3IgdGFibGUgaW4gc2VsZi5Nb2RlbC5UYWJsZXM6CiAgICAgICAgICAgICAgICBpZiB0YWJsZS5pc1Nub3dmbGFrZToKICAgICAgICAgICAgICAgICAgICBpZiB0YWJsZS5pc0hpZGRlbjoKICAgICAgICAgICAgICAgICAgICAgICAgaGlkZGVuU25vd2ZsYWtlVGFibGVzLmFwcGVuZCh0YWJsZS5uYW1lKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIHNub3dmbGFrZVRhYmxlcy5hcHBlbmQodGFibGUubmFtZSkKICAgICAgICAgICAgICAgIGVsaWYgdGFibGUuaXNDYWNoZWQ6CiAgICAgICAgICAgICAgICAgICAgaWYgdGFibGUuaXNIaWRkZW46CiAgICAgICAgICAgICAgICAgICAgICAgIGhpZGRlbkVtYmVkZGVkVGFibGVzLmFwcGVuZCh0YWJsZS5uYW1lKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIGVtYmVkZGVkVGFibGVzLmFwcGVuZCh0YWJsZS5uYW1lKQogICAgICAgICAgICAgICAgZWxpZiB0YWJsZS5pc0RhdGFiYXNlOgogICAgICAgICAgICAgICAgICAgIGlmIHRhYmxlLmlzSGlkZGVuOgogICAgICAgICAgICAgICAgICAgICAgICBoaWRkZW5FeHRlcm5hbFRhYmxlcy5hcHBlbmQodGFibGUubmFtZSkKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBleHRlcm5hbFRhYmxlcy5hcHBlbmQodGFibGUubmFtZSkKICAgICAgICAgICAgICAgIGVsaWYgdGFibGUuaXNGaWxlOgogICAgICAgICAgICAgICAgICAgIGlmIHRhYmxlLmlzSGlkZGVuOgogICAgICAgICAgICAgICAgICAgICAgICBoaWRkZW5GaWxlVGFibGVzLmFwcGVuZCh0YWJsZS5uYW1lKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIGZpbGVUYWJsZXMuYXBwZW5kKHRhYmxlLm5hbWUpCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIGlmIHRhYmxlLmlzSGlkZGVuOgogICAgICAgICAgICAgICAgICAgICAgICBoaWRkZW5PdGhlclRhYmxlcy5hcHBlbmQodGFibGUubmFtZSkKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBvdGhlclRhYmxlcy5hcHBlbmQodGFibGUubmFtZSkKCiAgICAgICAgICAgIHRhYmxlTmFtZXNbInNub3dmbGFrZSJdID0gc25vd2ZsYWtlVGFibGVzCiAgICAgICAgICAgIHRhYmxlTmFtZXNbImV4dGVybmFsIl0gPSBleHRlcm5hbFRhYmxlcwogICAgICAgICAgICB0YWJsZU5hbWVzWyJmaWxlIl0gPSBmaWxlVGFibGVzCiAgICAgICAgICAgIHRhYmxlTmFtZXNbImVtYmVkZGVkIl0gPSBlbWJlZGRlZFRhYmxlcwogICAgICAgICAgICB0YWJsZU5hbWVzWyJvdGhlciJdID0gb3RoZXJUYWJsZXMKICAgICAgICAgICAgdGFibGVOYW1lc1siaGlkZGVuU25vd2ZsYWtlIl0gPSBoaWRkZW5Tbm93Zmxha2VUYWJsZXMKICAgICAgICAgICAgdGFibGVOYW1lc1siaGlkZGVuRXh0ZXJuYWwiXSA9IGhpZGRlbkV4dGVybmFsVGFibGVzCiAgICAgICAgICAgIHRhYmxlTmFtZXNbImhpZGRlbkZpbGUiXSA9IGhpZGRlbkZpbGVUYWJsZXMKICAgICAgICAgICAgdGFibGVOYW1lc1siaGlkZGVuRW1iZWRkZWQiXSA9IGhpZGRlbkVtYmVkZGVkVGFibGVzCiAgICAgICAgICAgIHRhYmxlTmFtZXNbImhpZGRlbk90aGVyIl0gPSBoaWRkZW5PdGhlclRhYmxlcwoKICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fdGFibGVfbmFtZXMiXSA9IHRhYmxlTmFtZXMKCiAgICAgICAgcmV0dXJuIHN0LnNlc3Npb25fc3RhdGVbZiJ7c2VsZi5LZXl9X3RhYmxlX25hbWVzIl0KCiAgICAKICAgIGRlZiBSZW5kZXIoc2VsZik6CiAgICAgICAgc3QuaHRtbChzZWxmLlN0eWxlKQogICAgICAgIAogICAgICAgIHRhYmxlTmFtZXMgPSBzZWxmLkdldFRhYmxlTmFtZXMoKQogICAgICAgIHNub3dmbGFrZVRhYmxlczogTGlzdFtzdHJdID0gdGFibGVOYW1lcy5nZXQoInNub3dmbGFrZSIpCiAgICAgICAgIyBBbHdheXMgc2hvdyBoaWRkZW4gU25vd2ZsYWtlIHRhYmxlcwogICAgICAgIHNub3dmbGFrZVRhYmxlcy5leHRlbmQodGFibGVOYW1lcy5nZXQoImhpZGRlblNub3dmbGFrZSIpKQogICAgICAgICAgICAKICAgICAgICBlbWJlZGRlZFRhYmxlcyA9IHRhYmxlTmFtZXMuZ2V0KCJlbWJlZGRlZCIpCiAgICAgICAgaWYgc2VsZi5TaG93SGlkZGVuVGFibGVzOgogICAgICAgICAgICBlbWJlZGRlZFRhYmxlcy5leHRlbmQodGFibGVOYW1lcy5nZXQoImhpZGRlbkVtYmVkZGVkIikpCiAgICAgICAgCiAgICAgICAgZXh0ZXJuYWxUYWJsZXMgPSB0YWJsZU5hbWVzLmdldCgiZXh0ZXJuYWwiKQogICAgICAgIGlmIHNlbGYuU2hvd0hpZGRlblRhYmxlczoKICAgICAgICAgICAgZXh0ZXJuYWxUYWJsZXMuZXh0ZW5kKHRhYmxlTmFtZXMuZ2V0KCJoaWRkZW5FeHRlcm5hbCIpKQogICAgICAgIAogICAgICAgIGZpbGVUYWJsZXMgPSB0YWJsZU5hbWVzLmdldCgiZmlsZSIpCiAgICAgICAgaWYgc2VsZi5TaG93SGlkZGVuVGFibGVzOgogICAgICAgICAgICBmaWxlVGFibGVzLmV4dGVuZCh0YWJsZU5hbWVzLmdldCgiaGlkZGVuRmlsZSIpKQogICAgICAgIAogICAgICAgIG90aGVyVGFibGVzID0gdGFibGVOYW1lcy5nZXQoIm90aGVyIikKICAgICAgICBpZiBzZWxmLlNob3dIaWRkZW5UYWJsZXM6CiAgICAgICAgICAgIG90aGVyVGFibGVzLmV4dGVuZCh0YWJsZU5hbWVzLmdldCgiaGlkZGVuT3RoZXIiKSkKICAgICAgICAKCiAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICBpZiBsZW4oc25vd2ZsYWtlVGFibGVzKSA+IDA6CiAgICAgICAgICAgICAgICBzbm93Zmxha2VUYWJsZXMuc29ydCgpCiAgICAgICAgICAgICAgICB3aXRoIHN0LmNvbnRhaW5lcihib3JkZXI9VHJ1ZSk6CiAgICAgICAgICAgICAgICAgICAgc3Quc3ViaGVhZGVyKCJTbm93Zmxha2UgVGFibGVzIikKICAgICAgICAgICAgICAgICAgICBmb3IgdGFibGUgaW4gc25vd2ZsYWtlVGFibGVzOgogICAgICAgICAgICAgICAgICAgICAgICBzZWxmLlNob3dUYWJsZUJ1dHRvbih0YWJsZSkgICAKICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIGxlbihlbWJlZGRlZFRhYmxlcykgPiAwOgogICAgICAgICAgICAgICAgZW1iZWRkZWRUYWJsZXMuc29ydCgpCiAgICAgICAgICAgICAgICB3aXRoIHN0LmNvbnRhaW5lcihib3JkZXI9VHJ1ZSk6CiAgICAgICAgICAgICAgICAgICAgc3Quc3ViaGVhZGVyKCJFbWJlZGRlZCBUYWJsZXMiKQogICAgICAgICAgICAgICAgICAgIGZvciB0YWJsZSBpbiBlbWJlZGRlZFRhYmxlczoKICAgICAgICAgICAgICAgICAgICAgICAgc2VsZi5TaG93VGFibGVCdXR0b24odGFibGUpICAgCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBsZW4oZXh0ZXJuYWxUYWJsZXMpID4gMDoKICAgICAgICAgICAgICAgIGV4dGVybmFsVGFibGVzLnNvcnQoKQogICAgICAgICAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICAgICAgICAgIHN0LnN1YmhlYWRlcigiRXh0ZXJuYWwgVGFibGVzIikKICAgICAgICAgICAgICAgICAgICBmb3IgdGFibGUgaW4gZXh0ZXJuYWxUYWJsZXM6CiAgICAgICAgICAgICAgICAgICAgICAgIHNlbGYuU2hvd1RhYmxlQnV0dG9uKHRhYmxlKSAKCiAgICAgICAgICAgIGlmIGxlbihmaWxlVGFibGVzKSA+IDA6CiAgICAgICAgICAgICAgICBmaWxlVGFibGVzLnNvcnQoKQogICAgICAgICAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICAgICAgICAgIHN0LnN1YmhlYWRlcigiRmlsZSBUYWJsZXMiKQogICAgICAgICAgICAgICAgICAgIGZvciB0YWJsZSBpbiBmaWxlVGFibGVzOgogICAgICAgICAgICAgICAgICAgICAgICBzZWxmLlNob3dUYWJsZUJ1dHRvbih0YWJsZSkgCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBsZW4ob3RoZXJUYWJsZXMpID4gMDoKICAgICAgICAgICAgICAgIG90aGVyVGFibGVzLnNvcnQoKQogICAgICAgICAgICAgICAgd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgICAgICAgICAgICAgICAgIHN0LnN1YmhlYWRlcigiT3RoZXIgVGFibGVzIikKICAgICAgICAgICAgICAgICAgICBmb3IgdGFibGUgaW4gb3RoZXJUYWJsZXM6CiAgICAgICAgICAgICAgICAgICAgICAgIHNlbGYuU2hvd1RhYmxlQnV0dG9uKHRhYmxlKSAKCiAgICBkZWYgU2hvd1RhYmxlQnV0dG9uKHNlbGYsIHRhYmxlTmFtZTogc3RyKToKICAgICAgICBpZiB0YWJsZU5hbWUgPT0gc2VsZi5fX3NlbGVjdGVkSXRlbToKICAgICAgICAgICAgc3QuYnV0dG9uKHRhYmxlTmFtZSwga2V5PWYie3NlbGYuS2V5fV90YWJsZV97dGFibGVOYW1lfSIsIHR5cGU9InByaW1hcnkiICkKICAgICAgICBlbHNlOgogICAgICAgICAgICBzdC5idXR0b24odGFibGVOYW1lLCBrZXk9ZiJ7c2VsZi5LZXl9X3RhYmxlX3t0YWJsZU5hbWV9Iiwgb25fY2xpY2s9c2VsZi5TZXRTZWxlY3Rpb24sIGFyZ3M9KHRhYmxlTmFtZSwpICkKICAgICAgICAKCiAgICBkZWYgU2V0U2VsZWN0aW9uKHNlbGYsIHNlbGVjdGVkSXRlbTogT3B0aW9uYWxbQW55XSk6CiAgICAgICAgIGlmIHNlbGVjdGVkSXRlbSAhPSBzZWxmLlNlbGVjdGVkSXRlbToKICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2VsZWN0ZWRJdGVtIl0gPSBzZWxlY3RlZEl0ZW0KICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2VsZWN0aW9uQ2hhbmdlZCJdID0gc2VsZWN0ZWRJdGVtCgogICAgZGVmIF9fb25fc2VsZWN0aW9uX2NoYW5nZWQoc2VsZik6CiAgICAgICAgaWYgc2VsZi5vbl9zZWxlY3Rpb25fY2hhbmdlZCBpcyBub3QgTm9uZSBhbmQgZiJ7c2VsZi5LZXl9X3NlbGVjdGlvbkNoYW5nZWQiIGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgICAgIHNlbGYub25fc2VsZWN0aW9uX2NoYW5nZWQoc2VsZiwgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2VsZWN0aW9uQ2hhbmdlZCJdKQogICAgICAgICAgICBkZWwgc3Quc2Vzc2lvbl9zdGF0ZVtmIntzZWxmLktleX1fc2VsZWN0aW9uQ2hhbmdlZCJd'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/relationshipView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkuY29yZS5yZWxhdGlvbnNoaXAgaW1wb3J0IFJlbGF0aW9uc2hpcApmcm9tIC50YWJsZUl0ZW1WaWV3IGltcG9ydCBUYWJsZUl0ZW1WaWV3CgpkZWYgUmVsYXRpb25zaGlwVmlldyhyZWxhdGlvbnNoaXA6IFJlbGF0aW9uc2hpcCwgaW5jbHVkZUhlYWRlcjogYm9vbCA9IFRydWUsIGtleTogc3RyID0gIiIpOgogICAga2V5ID0gZiJ7a2V5fV97cmVsYXRpb25zaGlwLl9faGFzaF9fKCl9IgogICAgd2l0aCBzdC5jb250YWluZXIoa2V5PWYicmVsYXRpb25zaGlwX3tyZWxhdGlvbnNoaXAuX19oYXNoX18oKX1fe2tleX0iKToKICAgICAgICBpZiBpbmNsdWRlSGVhZGVyOgogICAgICAgICAgICBzdC5zdWJoZWFkZXIoZiJSZWxhdGlvbnNoaXA6IHtyZWxhdGlvbnNoaXAubmFtZX0iKQogICAgICAgIGZyb21Db2wsIGFycm93Q29sLCB0b0NvbCA9IHN0LmNvbHVtbnMoWzEwLDEsMTBdKQogICAgCiAgICAgICAgd2l0aCBmcm9tQ29sOgogICAgICAgICAgICBUYWJsZUl0ZW1WaWV3KHRhYmxlSXRlbT1yZWxhdGlvbnNoaXAuZnJvbVRhYmxlSXRlbSwga2V5PWYiZnJvbV97a2V5fSIsIGRpc3BsYXlGdWxsTmFtZT1UcnVlKSAgIAoKICAgICAgICB3aXRoIGFycm93Q29sOgogICAgICAgICAgICBzdC5odG1sKCI8cCBzdHlsZT1cInRleHQtYWxpZ246IGNlbnRlclwiPiAtLSZndDsgPC9wPiIpIAoKICAgICAgICB3aXRoIHRvQ29sOgogICAgICAgICAgICBUYWJsZUl0ZW1WaWV3KHRhYmxlSXRlbT1yZWxhdGlvbnNoaXAudG9UYWJsZUl0ZW0sIGtleT1mInRvX3trZXl9IiwgZGlzcGxheUZ1bGxOYW1lPVRydWUpCiAgICAgICAgICAgIAo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/queryView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIHN0cmVhbWxpdC5kZWx0YV9nZW5lcmF0b3IgaW1wb3J0IERlbHRhR2VuZXJhdG9yCmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gLmV4cHJlc3Npb25WaWV3IGltcG9ydCBFeHByZXNzaW9uVmlldwpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KZnJvbSBzbm93Zmxha2Uuc25vd3BhcmsgaW1wb3J0IFNlc3Npb24KZnJvbSB1dGlsLnNlc3Npb24gaW1wb3J0IGdldF9hY3RpdmVfc2Vzc2lvbgoKZGVmIFF1ZXJ5VmlldyhzcWw6IHN0ciwgZWRpdGFibGU6IGJvb2wgPSBGYWxzZSwga2V5OiBzdHIgPSAiIik6CiAgICBrZXk9ZiJxdWVyeV97a2V5fSIKICAgIAogICAgaWYgZiJ7a2V5fV9zcWwiIG5vdCBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgIHN0LnNlc3Npb25fc3RhdGVbZiJ7a2V5fV9zcWwiXSA9IHNxbAogICAgZWxzZToKICAgICAgICBzcWwgPSBzdC5zZXNzaW9uX3N0YXRlW2Yie2tleX1fc3FsIl0KCiAgICBzZXNzaW9uID0gZ2V0X2FjdGl2ZV9zZXNzaW9uKCkKICAgIGlmIGVkaXRhYmxlID09IFRydWU6CiAgICAgICAgaWYgZiJ7a2V5fV9lZGl0IiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgICAgICBlZGl0ID0gVHJ1ZQogICAgICAgICAgICBsYWJlbCA9ICJTYXZlIgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIGVkaXQgPSBGYWxzZQogICAgICAgICAgICBsYWJlbCA9ICJFZGl0IgogICAgZWxzZToKICAgICAgICBlZGl0ID0gRmFsc2UKCiAgICBjb2wxLCBjb2wyID0gc3QuY29sdW1ucyhbMTMsMV0pICAgICAgICAKICAgIHdpdGggY29sMToKICAgICAgICBpZiBzdC5idXR0b24oIlZhbGlkYXRlIFF1ZXJ5Iiwga2V5PWYie2tleX1fZXhlY3V0ZV9idXR0b24iLCBoZWxwPSJFeGVjdXRlIHRoZSBxdWVyeSB0byBlbnN1cmUgaXQgcmV0dXJucyB0aGUgY29ycmVjdCBkYXRhIik6ICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIGYie2tleX1fcXVlcnlfcmVzdWx0cyIgbm90IGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgICAgICAgICB3aXRoIHN0LnNwaW5uZXIoIkV4ZWN1dGluZyBRdWVyeS4uLiIpOgogICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgcmVzdWx0REYgPSBzZXNzaW9uLnNxbChzcWwpLmNvbGxlY3QoKQogICAgICAgICAgICAgICAgICAgICAgICBzdC5zZXNzaW9uX3N0YXRlW2Yie2tleX1fcXVlcnlfcmVzdWx0cyJdID0gcmVzdWx0REYKICAgICAgICAgICAgICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgICAgICAgICAgICAgIHN0LmVycm9yKGYiRXJyb3IgZXhlY3V0aW5nIFF1ZXJ5OiB7ZX0iKQogICAgd2l0aCBjb2wyOgogICAgICAgIGlmIGVkaXRhYmxlID09IFRydWU6CiAgICAgICAgICAgIGlmIHN0LmJ1dHRvbihsYWJlbCxrZXk9ZiJ7a2V5fV9lZGl0X2J0biIsIHVzZV9jb250YWluZXJfd2lkdGg9VHJ1ZSk6CiAgICAgICAgICAgICAgICBpZiBlZGl0ID09IFRydWU6CiAgICAgICAgICAgICAgICAgICAgZGVsIHN0LnNlc3Npb25fc3RhdGVbZiJ7a2V5fV9lZGl0Il0KICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVtmIntrZXl9X2VkaXQiXSA9IFRydWUKICAgICAgICAgICAgICAgIHN0LnJlcnVuKCkKCiAgICBpZiBmIntrZXl9X3F1ZXJ5X3Jlc3VsdHMiIGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgcmVzdWx0ID0gc3Quc2Vzc2lvbl9zdGF0ZVtmIntrZXl9X3F1ZXJ5X3Jlc3VsdHMiXQogICAgICAgIGlmIHJlc3VsdCBpcyBub3QgTm9uZToKICAgICAgICAgICAgcm93Q291bnQgPSBsZW4ocmVzdWx0KQogICAgICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKCJSZXN1bHRzIik6CiAgICAgICAgICAgICAgICBzdC53cml0ZShmIntyb3dDb3VudH0gUm93cyByZXR1cm5lZCIpCiAgICAgICAgICAgICAgICBzdC5kYXRhZnJhbWUocmVzdWx0KQogICAgCiAgICB3aXRoIHN0LmNvbnRhaW5lcihrZXk9a2V5LCBib3JkZXI9VHJ1ZSk6CiAgICAgICAgaWYgZWRpdCA9PSBUcnVlOgogICAgICAgICAgICBzdC5zZXNzaW9uX3N0YXRlW2Yie2tleX1fc3FsIl0gPSBzdC50ZXh0X2FyZWEobGFiZWw9IlNRTCIsIGxhYmVsX3Zpc2liaWxpdHk9ImNvbGxhcHNlZCIsIHZhbHVlPXNxbCwga2V5PWYie2tleX1fdGV4dF9hcmVhIikKICAgICAgICBlbHNlOgogICAgICAgICAgICBzdC5jb2RlKHNxbCwgbGFuZ3VhZ2U9InNxbCIsIHdyYXBfbGluZXM9VHJ1ZSkKICAgIAogICAgcmV0dXJuIHNxbAogICAgICAgIAogICAgICAgIAoKCiAgICAgICAgICAgICAgICAK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/issue_detector.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/expressionView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbiwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkuY29yZS5zY3JpcHQgaW1wb3J0IFNjcmlwdApmcm9tIGJpLmNvcmUub3BlcmF0b3IgaW1wb3J0IE9wZXJhdG9yCmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25Db2RlVHlwZSBpbXBvcnQgRXhwcmVzc2lvbkNvZGVUeXBlCmltcG9ydCBwYW5kYXMgYXMgcGQKCmRlZiBFeHByZXNzaW9uVmlldyhleHByZXNzaW9uOiBFeHByZXNzaW9uLCBpbmNsdWRlSGVhZGVyOiBib29sID0gVHJ1ZSwga2V5OiBzdHIgPSAiIik6CiAgICBmcm9tIC50YWJsZV92aWV3IGltcG9ydCBUYWJsZVZpZXcKICAgIGZyb20gLnRhYmxlSXRlbVZpZXcgaW1wb3J0IFRhYmxlSXRlbVZpZXcKICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWE9wZXJhdG9yRXhwcmVzc2lvbiBpbXBvcnQgREFYT3BlcmF0b3JFeHByZXNzaW9uCgogICAgY29kZVR5cGUgPSAiIgogICAgaWYgZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICB0cnk6CiAgICAgICAgICAgIGtleSA9IGYie2tleX1fZXtoYXNoKGV4cHJlc3Npb24uc291cmNlU3RyaW5nKX0iCiAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICBwcmludChmIkV4cHJlc3Npb24gVmlldyBDcmVhdGlvbiBFcnJvciB7ZX0iKQogICAgICAgIGlmIGV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIG1hdGNoIGV4cHJlc3Npb24uY29kZVR5cGU6CiAgICAgICAgICAgICAgICBjYXNlIEV4cHJlc3Npb25Db2RlVHlwZS5EQVg6CiAgICAgICAgICAgICAgICAgICAgY29kZVR5cGUgPSAiREFYIgogICAgICAgICAgICAgICAgY2FzZSBFeHByZXNzaW9uQ29kZVR5cGUuTToKICAgICAgICAgICAgICAgICAgICBjb2RlVHlwZSA9ICJQb3dlclF1ZXJ5IgogICAgICAgICAgICAgICAgY2FzZSBFeHByZXNzaW9uQ29kZVR5cGUuU1FMOgogICAgICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gIlNRTCIKCiAgICAgICAgICAgIAogICAgICAgICAgICB3aXRoIHN0LmNvbnRhaW5lcihib3JkZXI9VHJ1ZSk6CiAgICAgICAgICAgICAgICBpZiBpbmNsdWRlSGVhZGVyOgogICAgICAgICAgICAgICAgICAgIHN0LnN1YmhlYWRlcihmIntjb2RlVHlwZX06IHtleHByZXNzaW9uLnR5cGUubmFtZX0gRXhwcmVzc2lvbiIpCiAgICAgICAgICAgICAgICBhbGxJc3N1ZXMgPSBleHByZXNzaW9uLkdldEFsbElzc3VlcygpCiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgIGlzc3VlQ291bnQgPSBsZW4oYWxsSXNzdWVzKQogICAgICAgICAgICAgICAgaWYgaXNzdWVDb3VudCA+IDA6CiAgICAgICAgICAgICAgICAgICAgc3Qud3JpdGUoZiJ7aXNzdWVDb3VudH0gSXNzdWVzIEZvdW5kIikKCiAgICAgICAgICAgICAgICAgICAgd2l0aCBzdC5leHBhbmRlcigiRXhwcmVzc2lvbiBJc3N1ZXMiKToKICAgICAgICAgICAgICAgICAgICAgICAgZm9yIGlzc3VlIGluIGFsbElzc3VlczoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGlzc3VlIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnRleHQoZiJ7aXNzdWUudHlwZS5uYW1lfToge2lzc3VlLmRlc2NyaXB0aW9ufSIpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZGV0YWlscyA9IGV4cHJlc3Npb24uR2V0SXNzdWVEZXRhaWxzKGlzc3VlLCBpbmNsdWRlQ2hpbGRyZW49RmFsc2UpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgaWYgZGV0YWlscyBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZm9yIGUsZCBpbiBkZXRhaWxzLml0ZW1zKCk6ICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dChmIiB7ZX0gLSB7ZH0iKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgIHdpdGggc3QuY29udGFpbmVyKCk6CgogICAgICAgICAgICAgICAgICAgIHN0LmNvZGUoZXhwcmVzc2lvbi5zb3VyY2VTdHJpbmcsIHdyYXBfbGluZXM9VHJ1ZSwgbGFuZ3VhZ2U9Y29kZVR5cGUubG93ZXIoKSkKCiAgICAgICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShleHByZXNzaW9uLCBTY3JpcHQpOgogICAgICAgICAgICAgICAgICAgICAgICBmb3Igayx2IGluIGV4cHJlc3Npb24uZXhwcmVzc2lvbnMuaXRlbXMoKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHdpdGggc3QuZXhwYW5kZXIoayk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgRXhwcmVzc2lvblZpZXcodiwga2V5PWtleSkgICAgCiAgICAgICAgICAgICAgICAgICAgZWxpZiBpc2luc3RhbmNlKGV4cHJlc3Npb24sIEZ1bmN0aW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgaWYgZXhwcmVzc2lvbi5oZWxwVXJsIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QuaHRtbChmIjxhIHRhcmdldD1cIl9ibGFua1wiIHJlbD1cIm5vb3BlbmVyIG5vcmVmZXJyZXJcIiBocmVmPVwie2V4cHJlc3Npb24uaGVscFVybH1cIj57ZXhwcmVzc2lvbi5mdW5jdGlvbk5hbWV9PC9hPiIpCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KGYie2V4cHJlc3Npb24uZnVuY3Rpb25OYW1lfSIpCiAgICAgICAgICAgICAgICAgICAgICAgIHdpdGggc3QuZXhwYW5kZXIoIlBhcmFtZXRlcnMiKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGZvciBpLCBwIGluIGVudW1lcmF0ZShleHByZXNzaW9uLnBhcmFtZXRlcnMpOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHdpdGggc3QuY29udGFpbmVyKGJvcmRlcj1UcnVlKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgdHlwZVN0ciA9ICIiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UocCwgVGFibGUpOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgdHlwZVN0ciA9ICJUYWJsZSBSZWZlcmVuY2UiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZShwLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gcC5jb2RlVHlwZS5uYW1lCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBtYXRjaCBwLmNvZGVUeXBlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLk06CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gIlBvd2VyIFF1ZXJ5IgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgdHlwZVN0ciA9IGYie2NvZGVUeXBlfSB7cC50eXBlfSBFeHByZXNzaW9uIiAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHR5cGVTdHIgPSB0eXBlKHApCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnRleHQoZiJQYXJhbWV0ZXI6IHtpKzF9OiB7dHlwZVN0cn0iKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHAsIFRhYmxlKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFRhYmxlVmlldyhwLCBrZXk9a2V5KQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2UocCwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBFeHByZXNzaW9uVmlldyhwLCBpbmNsdWRlSGVhZGVyPUZhbHNlLCBrZXk9a2V5KSAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KHApCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgcHJpbnQoZiJFeHByZXNzaW9uIFZpZXc6IGVycm9yIHNob3dpbmcgcGFyYW1ldGVyIHtwfToge2V9IikKICAgICAgICAgICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbiwgT3BlcmF0b3IpOgogICAgICAgICAgICAgICAgICAgICAgICBmb3IgaSwgb3BlcmFuZCBpbiBlbnVtZXJhdGUoZXhwcmVzc2lvbi5vcGVyYW5kcyk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBFeHByZXNzaW9uVmlldyhvcGVyYW5kLCBpbmNsdWRlSGVhZGVyPUZhbHNlLCBrZXk9a2V5KQogICAgICAgICAgICAgICAgICAgICAgICAgICAgaWYgbGVuKGV4cHJlc3Npb24ub3BlcmF0b3JzKSA+IGk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dChleHByZXNzaW9uLm9wZXJhdG9yc1tpXSkKICAgICAgICAgICAgICAgICAgICBlbGlmIGV4cHJlc3Npb24udHlwZSA9PSBFeHByZXNzaW9uVHlwZS5FYWNoOgogICAgICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KCJFYWNoIEV4cHJlc3Npb24iKQogICAgICAgICAgICAgICAgICAgICAgICBFeHByZXNzaW9uVmlldyhleHByZXNzaW9uPWV4cHJlc3Npb24uZXhwcmVzc2lvbiwgaW5jbHVkZUhlYWRlcj1GYWxzZSwga2V5PWtleSkKICAgICAgICAgICAgICAgICAgICBlbGlmIGV4cHJlc3Npb24udHlwZSA9PSBFeHByZXNzaW9uVHlwZS5Db25kaXRpb25hbDoKICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dCgiQ29uZGl0aW9uIikKICAgICAgICAgICAgICAgICAgICAgICAgRXhwcmVzc2lvblZpZXcoZXhwcmVzc2lvbj1leHByZXNzaW9uLmNvbmRpdGlvbiwgaW5jbHVkZUhlYWRlcj1GYWxzZSwga2V5PWtleSkKICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dCgiVHJ1ZSBFeHByZXNzaW9uIikKICAgICAgICAgICAgICAgICAgICAgICAgRXhwcmVzc2lvblZpZXcoZXhwcmVzc2lvbj1nZXRhdHRyKGV4cHJlc3Npb24sICd0cnVlVmFsdWUnLCBOb25lKSwgaW5jbHVkZUhlYWRlcj1GYWxzZSwga2V5PWtleSkKICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dCgiRmFsc2UgRXhwcmVzc2lvbiIpCiAgICAgICAgICAgICAgICAgICAgICAgIEV4cHJlc3Npb25WaWV3KGV4cHJlc3Npb249Z2V0YXR0cihleHByZXNzaW9uLCAnZmFsc2VWYWx1ZScsIE5vbmUpLCBpbmNsdWRlSGVhZGVyPUZhbHNlLCBrZXk9a2V5KQoKICAgICAgICAgICAgICAgICAgICBlbGlmIGV4cHJlc3Npb24udHlwZSA9PSBFeHByZXNzaW9uVHlwZS5Mb29rdXA6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGV4cHJlc3Npb24ubG9va3VwRXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGxvb2t1cEV4cHJlc3Npb25WYWx1ZSA9ICIiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGV4cHJlc3Npb24ubG9va3VwRXhwcmVzc2lvbiwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgbG9va3VwRXhwcmVzc2lvblZhbHVlID0gZXhwcmVzc2lvbi5sb29rdXBFeHByZXNzaW9uLnZhbHVlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGxvb2t1cEV4cHJlc3Npb25WYWx1ZSA9IGV4cHJlc3Npb24ubG9va3VwRXhwcmVzc2lvbgoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnRleHQoZiJMb29rdXAgaXRlbSBtYXRjaGluZyB7ZXhwcmVzc2lvbi5sb29rdXBGaWx0ZXJ9IGluIHtsb29rdXBFeHByZXNzaW9uVmFsdWV9IikKICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgd2l0aCBzdC5jb250YWluZXIoKToKICAgICAgICAgICAgICAgICAgICBzdC5zdWJoZWFkZXIoIlJlc3VsdCIpCiAgICAgICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShleHByZXNzaW9uLnZhbHVlLCBUYWJsZSk6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbi52YWx1ZSwgRXhwcmVzc2lvbikgYW5kIGV4cHJlc3Npb24udmFsdWUuZGF0YUlzQXZhaWxhYmxlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgd2l0aCBzdC5wb3BvdmVyKCJTaG93IFRhYmxlIERhdGEiKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdC5kYXRhZnJhbWUoZXhwcmVzc2lvbi52YWx1ZS5nZXREYXRhKCkpCiAgICAgICAgICAgICAgICAgICAgICAgIFRhYmxlVmlldyhleHByZXNzaW9uLnZhbHVlLCBrZXk9a2V5KQogICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbi52YWx1ZSwgcGQuRGF0YUZyYW1lKToKICAgICAgICAgICAgICAgICAgICAgICAgc3QuY29kZShzdHIoZXhwcmVzc2lvbiksIGxhbmd1YWdlPSJzcWwiKQogICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgd2l0aCBzdC5wb3BvdmVyKCJTaG93IFRhYmxlIERhdGEiKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LmRhdGFmcmFtZShleHByZXNzaW9uLnZhbHVlKQoKICAgICAgICAgICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbi52YWx1ZSwgVGFibGVJdGVtKToKICAgICAgICAgICAgICAgICAgICAgICAgVGFibGVJdGVtVmlldyhleHByZXNzaW9uLnZhbHVlLCBkaXNwbGF5RnVsbE5hbWU9KGlzaW5zdGFuY2UoZXhwcmVzc2lvbi52YWx1ZS50YWJsZSwgU2VtYW50aWNUYWJsZSkpLCBrZXk9a2V5KQogICAgICAgICAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZShleHByZXNzaW9uLnZhbHVlLCBsaXN0KToKICAgICAgICAgICAgICAgICAgICAgICAgZm9yIGksdiBpbiBlbnVtZXJhdGUoZXhwcmVzc2lvbi52YWx1ZSk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiB2IGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UodiwgRXhwcmVzc2lvbikgYW5kIG5vdCB2LmlzVmFsdWU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIEV4cHJlc3Npb25WaWV3KHYsIGtleT1rZXkpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QudGV4dChmIntbaV19IHt2fSIpCiAgICAgICAgICAgICAgICAgICAgZWxpZiBoYXNhdHRyKGV4cHJlc3Npb24sImdldFNxbFZhbHVlIik6CiAgICAgICAgICAgICAgICAgICAgICAgIHN0LnRleHQoZXhwcmVzc2lvbi5nZXRTcWxWYWx1ZSgpKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIHN0LnRleHQoZXhwcmVzc2lvbi52YWx1ZSkKICAgIGVsc2U6CiAgICAgICAgcGFzcwogICAgICAgICAgICAK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/columnView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIHN0cmVhbWxpdC5kZWx0YV9nZW5lcmF0b3IgaW1wb3J0IERlbHRhR2VuZXJhdG9yCmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gdWkuZXhwcmVzc2lvblZpZXcgaW1wb3J0IEV4cHJlc3Npb25WaWV3CmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQoKZGVmIENvbHVtblZpZXcodGFibGVJdGVtOiBUYWJsZUl0ZW0sIGtleTogc3RyID0gIiIsIGRpc3BsYXlGdWxsTmFtZTogYm9vbCA9IFRydWUsIGRpc3BsYXlXaW5kb3c6IERlbHRhR2VuZXJhdG9yID0gTm9uZSk6CiAgICBrZXk9ZiJjb2x1bW5fe3RhYmxlSXRlbS5fX2hhc2hfXygpfV97a2V5fSIucmVwbGFjZSgiLSIsIl8iKQogICAgCiAgICAjIEFkZCBsaWdodGVyIGdyZWVuIHN0eWxpbmcgZm9yIGV4cHJlc3Npb24gY29sdW1ucyAtIG1vcmUgc3BlY2lmaWMgc2VsZWN0b3JzCiAgICBzdC5tYXJrZG93bigiIiIKICAgIDxzdHlsZT4KICAgIGJ1dHRvbltraW5kPSJwcmltYXJ5Il1bZGF0YS10ZXN0aWQqPSJjb2x1bW4iXSB7CiAgICAgICAgYmFja2dyb3VuZDogI0QxRkFFNSAhaW1wb3J0YW50OwogICAgICAgIGJhY2tncm91bmQtY29sb3I6ICNEMUZBRTUgIWltcG9ydGFudDsKICAgICAgICBjb2xvcjogIzA2NUY0NiAhaW1wb3J0YW50OwogICAgICAgIGJvcmRlcjogMXB4IHNvbGlkICNBN0YzRDAgIWltcG9ydGFudDsKICAgIH0KICAgIGJ1dHRvbltraW5kPSJwcmltYXJ5Il1bZGF0YS10ZXN0aWQqPSJjb2x1bW4iXTpob3ZlciB7CiAgICAgICAgYmFja2dyb3VuZDogI0E3RjNEMCAhaW1wb3J0YW50OwogICAgICAgIGJhY2tncm91bmQtY29sb3I6ICNBN0YzRDAgIWltcG9ydGFudDsKICAgICAgICBjb2xvcjogIzA2NEUzQiAhaW1wb3J0YW50OwogICAgfQogICAgLyogQWxzbyB0YXJnZXQgYnkgdGhlIGNvbnRhaW5lciBjb250ZXh0ICovCiAgICBkaXZbZGF0YS10ZXN0aWQ9InN0QnV0dG9uIl06aGFzKGJ1dHRvbltraW5kPSJwcmltYXJ5Il0pIGJ1dHRvbiB7CiAgICAgICAgYmFja2dyb3VuZDogI0QxRkFFNSAhaW1wb3J0YW50OwogICAgICAgIGJhY2tncm91bmQtY29sb3I6ICNEMUZBRTUgIWltcG9ydGFudDsKICAgICAgICBjb2xvcjogIzA2NUY0NiAhaW1wb3J0YW50OwogICAgICAgIGJvcmRlcjogMXB4IHNvbGlkICNBN0YzRDAgIWltcG9ydGFudDsKICAgIH0KICAgIGRpdltkYXRhLXRlc3RpZD0ic3RCdXR0b24iXTpoYXMoYnV0dG9uW2tpbmQ9InByaW1hcnkiXSkgYnV0dG9uOmhvdmVyIHsKICAgICAgICBiYWNrZ3JvdW5kOiAjQTdGM0QwICFpbXBvcnRhbnQ7CiAgICAgICAgYmFja2dyb3VuZC1jb2xvcjogI0E3RjNEMCAhaW1wb3J0YW50OwogICAgICAgIGNvbG9yOiAjMDY0RTNCICFpbXBvcnRhbnQ7CiAgICB9CiAgICA8L3N0eWxlPgogICAgIiIiLCB1bnNhZmVfYWxsb3dfaHRtbD1UcnVlKQogICAgCiAgICBkYXRhVHlwZUljb24gPSB0YWJsZUl0ZW0udHlwZS50b0ljb24oKQogICAgCiAgICAjIENoZWNrIGZvciBpc3N1ZXMgYW5kIHJlcGxhY2UgaWNvbiB3aXRoIHdhcm5pbmcgaWYgaXNzdWVzIGV4aXN0CiAgICBhbGxJc3N1ZXMgPSB0YWJsZUl0ZW0uR2V0QWxsSXNzdWVzKCkKICAgIGlmIGxlbihhbGxJc3N1ZXMpID4gMDoKICAgICAgICBkYXRhVHlwZUljb24gPSAiOm1hdGVyaWFsL2Vycm9yOiIKICAgIGVsaWYgZGF0YVR5cGVJY29uID09ICIiOgogICAgICAgIHBhc3MKICAgIAogICAgaWYgZGlzcGxheUZ1bGxOYW1lID09IFRydWU6CiAgICAgICAgbmFtZSA9IHRhYmxlSXRlbS5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPUZhbHNlLCBpbmNsdWRlVGFibGVBbGlhcz1UcnVlLCBpbmNsdWRlVGFibGVOYW1lPVRydWUpCiAgICBlbHNlOgogICAgICAgIG5hbWUgPSB0YWJsZUl0ZW0uZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1GYWxzZSkKICAgIAogICAgIyBBZGQgaXNzdWUgY291bnQgdG8gbmFtZSBpZiBpc3N1ZXMgZXhpc3QKICAgIGlmIGxlbihhbGxJc3N1ZXMpID4gMDoKICAgICAgICBpZiBsZW4oYWxsSXNzdWVzKSA+IDE6CiAgICAgICAgICAgIG5hbWUgPSBmIntuYW1lfSAoe2xlbihhbGxJc3N1ZXMpfSBJc3N1ZXMpIgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIG5hbWUgPSBmIntuYW1lfSAoMSBJc3N1ZSkiCgogICAgd2l0aCBzdC5jb250YWluZXIoa2V5PWtleSk6IAogICAgICAgIGlmIHRhYmxlSXRlbS5leHByZXNzaW9uIGlzIG5vdCBOb25lIG9yIHRhYmxlSXRlbS5leHByZXNzaW9uU3RyaW5nIGlzIG5vdCBOb25lOiAKICAgICAgICAgICAgY29sMSwgY29sMiA9IHN0LmNvbHVtbnMoWzUsIDNdKQogICAgICAgICAgICB3aXRoIGNvbDE6CiAgICAgICAgICAgICAgICBzdC5idXR0b24obmFtZSwga2V5PWYiY29sdW1uX3tuYW1lfSIsIHR5cGU9InByaW1hcnkiLCBpY29uPWRhdGFUeXBlSWNvbiwgdXNlX2NvbnRhaW5lcl93aWR0aD1UcnVlKQogICAgICAgICAgICB3aXRoIGNvbDI6CiAgICAgICAgICAgICAgICB3aXRoIHN0LnBvcG92ZXIoIkV4cHJlc3Npb24gRGV0YWlscyIsIHVzZV9jb250YWluZXJfd2lkdGg9VHJ1ZSk6CiAgICAgICAgICAgICAgICAgICAgIyBTaG93IGlzc3VlcyBmaXJzdCBpZiB0aGV5IGV4aXN0CiAgICAgICAgICAgICAgICAgICAgaWYgbGVuKGFsbElzc3VlcykgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICBzdC53cml0ZSgiKipJc3N1ZXM6KioiKQogICAgICAgICAgICAgICAgICAgICAgICBmb3IgaXNzdWUgaW4gYWxsSXNzdWVzOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QuZXJyb3IoZiJ7aXNzdWUudHlwZS5uYW1lfToge2lzc3VlLmRlc2NyaXB0aW9ufSIpCiAgICAgICAgICAgICAgICAgICAgICAgIHN0LmRpdmlkZXIoKQogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgIGlmIHRhYmxlSXRlbS5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICBzdC5jb2RlKHRhYmxlSXRlbS5leHByZXNzaW9uLnNvdXJjZVN0cmluZykKICAgICAgICAgICAgICAgICAgICAgICAgaWYgc3QuYnV0dG9uKCJEZXRhaWxzIiwga2V5PWYie2tleX1fZXhwX2RldGFpbHMiKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnNlc3Npb25fc3RhdGVbImV4cHJlc3Npb25Ub0VkaXQiXSA9IHRhYmxlSXRlbS5leHByZXNzaW9uCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAjIE5hdmlnYXRlIHRvIHRoZSBFeHByZXNzaW9uIEVkaXRvciBwYWdlIHdoaWNoIHJlbmRlcnMgRXhwcmVzc2lvblZpZXcKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnNlc3Npb25fc3RhdGVbImN1cnJlbnRfcGFnZSJdID0gImV4cHJlc3Npb25fZWRpdG9yIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QucmVydW4oKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIHN0LmNvZGUodGFibGVJdGVtLmV4cHJlc3Npb25TdHJpbmcpCgogICAgICAgICAgICAgICAgICAgIGlmIHRhYmxlSXRlbS5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICBzdC53cml0ZSgiR2VuZXJhdGVkIFNRTCIpCiAgICAgICAgICAgICAgICAgICAgICAgIHN0LmNvZGUodGFibGVJdGVtLmV4cHJlc3Npb24udmFsdWUsIGxhbmd1YWdlPSJzcWwiKQogICAgICAgICAgICAgICAgICAgIAogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHN0LmJ1dHRvbihuYW1lLCBrZXk9ZiJjb2x1bW5fe25hbWV9IiwgdHlwZT0ic2Vjb25kYXJ5IiwgaWNvbj1kYXRhVHlwZUljb24gKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/tablePartitionView.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsCmltcG9ydCBzdHJlYW1saXQgYXMgc3QKZnJvbSBiaS5yZWFkZXJzLnBiaS50YWJsZVBhcnRpdGlvbiBpbXBvcnQgVGFibGVQYXJ0aXRpb24KZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQpmcm9tIC5xdWVyeVZpZXcgaW1wb3J0IFF1ZXJ5Vmlldwpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbkNvZGVUeXBlIGltcG9ydCBFeHByZXNzaW9uQ29kZVR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLmNvbm5lY3RvckV4cHJlc3Npb24gaW1wb3J0IEZpbGVDb25uZWN0b3JFeHByZXNzaW9uLCBEYXRhYmFzZUNvbm5lY3RvckV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmNvbm5lY3RvciBpbXBvcnQgQ29ubmVjdG9yLCBDb25uZWN0b3JUeXBlCgpAc3QuZnJhZ21lbnQKZGVmIFRhYmxlUGFydGl0aW9uVmlldyhwYXJ0aXRpb246IFRhYmxlUGFydGl0aW9uLCBwYXJ0aXRpb25JbmRleDogaW50ID0gMCwga2V5OiBzdHIgPSAiIik6CiAgICBmcm9tIGJpLnNxbC5TUUxFeHByZXNzaW9uIGltcG9ydCBTUUxFeHByZXNzaW9uCgogICAga2V5ID0gZiJ7a2V5fV9wYXJ0aXRpb25fe3BhcnRpdGlvbkluZGV4ICsgMX0iCgogICAgIyBFeGl0IGlmIHRoZSBwYXJ0aXRpb24gaXMgbm90IHN1cHBvcnRlZAogICAgaWYgcGFydGl0aW9uLmV4cHJlc3Npb24gaXMgTm9uZToKICAgICAgICBzdC53cml0ZShmIlRoaXMgcGFydGl0aW9uIGlzIGN1cnJlbnRseSBub3Qgc3VwcG9ydGVkIikKICAgICAgICByZXR1cm4KCiAgICAjIERpc3BsYXkgdGFibGUgZGVwZW5kZW5jaWVzIChpZiBhbnkpCiAgICB0cnk6CiAgICAgICAgcmVmX3RhYmxlcyA9IHBhcnRpdGlvbi5HZXRSZWZlcmVuY2VUYWJsZXMoKQogICAgICAgIGlmIHJlZl90YWJsZXMgYW5kIGxlbihyZWZfdGFibGVzKSA+IDA6CiAgICAgICAgICAgIHN0Lm1hcmtkb3duKCIiIgogICAgICAgICAgICA8ZGl2IHN0eWxlPSJiYWNrZ3JvdW5kOiAjRkVGM0M3OyBib3JkZXItbGVmdDogNHB4IHNvbGlkICNGNTlFMEI7IHBhZGRpbmc6IDEycHggMTZweDsgCiAgICAgICAgICAgICAgICAgICAgICAgIGJvcmRlci1yYWRpdXM6IDZweDsgbWFyZ2luLWJvdHRvbTogMTZweDsiPgogICAgICAgICAgICAgICAgPGRpdiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IGNlbnRlcjsgZ2FwOiA4cHg7IG1hcmdpbi1ib3R0b206IDhweDsiPgogICAgICAgICAgICAgICAgICAgIDxzcGFuIHN0eWxlPSJmb250LXNpemU6IDIwcHg7Ij7imqDvuI88L3NwYW4+CiAgICAgICAgICAgICAgICAgICAgPHNwYW4gc3R5bGU9ImZvbnQtd2VpZ2h0OiA2MDA7IGNvbG9yOiAjOTI0MDBFOyBmb250LXNpemU6IDE0cHg7Ij5EZXBlbmRlbmNpZXMgRGV0ZWN0ZWQ8L3NwYW4+CiAgICAgICAgICAgICAgICA8L2Rpdj4KICAgICAgICAgICAgICAgIDxkaXYgc3R5bGU9ImNvbG9yOiAjNzgzNTBGOyBmb250LXNpemU6IDEzcHg7IGxpbmUtaGVpZ2h0OiAxLjU7Ij4KICAgICAgICAgICAgICAgICAgICBUaGlzIHBhcnRpdGlvbiBkZXBlbmRzIG9uIHRoZSBmb2xsb3dpbmcgdGFibGVzLiA8c3Ryb25nPkNyZWF0ZSB0aGVtIGZpcnN0PC9zdHJvbmc+IGJlZm9yZSBjcmVhdGluZyB0aGlzIHBhcnRpdGlvbjoKICAgICAgICAgICAgICAgIDwvZGl2PgogICAgICAgICAgICA8L2Rpdj4KICAgICAgICAgICAgIiIiLCB1bnNhZmVfYWxsb3dfaHRtbD1UcnVlKQoKICAgICAgICAgICAgIyBEaXNwbGF5IGVhY2ggZGVwZW5kZW5jeSBhcyBhIGNoaXAKICAgICAgICAgICAgZGVwX2NvbHMgPSBzdC5jb2x1bW5zKG1pbihsZW4ocmVmX3RhYmxlcyksIDMpKQogICAgICAgICAgICBmb3IgaWR4LCByZWZfdGFibGUgaW4gZW51bWVyYXRlKHNvcnRlZChyZWZfdGFibGVzLCBrZXk9bGFtYmRhIHQ6IHQubmFtZSkpOgogICAgICAgICAgICAgICAgd2l0aCBkZXBfY29sc1tpZHggJSBsZW4oZGVwX2NvbHMpXToKICAgICAgICAgICAgICAgICAgICBzdC5tYXJrZG93bihmIiIiCiAgICAgICAgICAgICAgICAgICAgPGRpdiBzdHlsZT0iYmFja2dyb3VuZDogd2hpdGU7IGJvcmRlcjogMXB4IHNvbGlkICNGREIwMjI7IHBhZGRpbmc6IDhweCAxMnB4OyAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBib3JkZXItcmFkaXVzOiA4cHg7IHRleHQtYWxpZ246IGNlbnRlcjsgbWFyZ2luLWJvdHRvbTogOHB4OyI+CiAgICAgICAgICAgICAgICAgICAgICAgIDxzcGFuIHN0eWxlPSJmb250LXNpemU6IDE2cHg7Ij7inYTvuI88L3NwYW4+CiAgICAgICAgICAgICAgICAgICAgICAgIDxkaXYgc3R5bGU9ImZvbnQtc2l6ZTogMTNweDsgZm9udC13ZWlnaHQ6IDYwMDsgY29sb3I6ICMzNzQxNTE7IG1hcmdpbi10b3A6IDRweDsiPgogICAgICAgICAgICAgICAgICAgICAgICAgICAge3JlZl90YWJsZS5uYW1lfQogICAgICAgICAgICAgICAgICAgICAgICA8L2Rpdj4KICAgICAgICAgICAgICAgICAgICA8L2Rpdj4KICAgICAgICAgICAgICAgICAgICAiIiIsIHVuc2FmZV9hbGxvd19odG1sPVRydWUpCiAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgIyBTaWxlbnRseSBmYWlsIGlmIEdldFJlZmVyZW5jZVRhYmxlcyBoYXMgaXNzdWVzCiAgICAgICAgcGFzcwoKICAgIGlmIHBhcnRpdGlvbi5pc1Nub3dmbGFrZToKICAgICAgICBzdC5zdWJoZWFkZXIoZiJQYXJ0aXRpb24ge3BhcnRpdGlvbkluZGV4ICsgMX0gQWN0aW9ucyIpCiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5wYXJ0aXRpb25Nb2RlIGltcG9ydCBQYXJ0aXRpb25Nb2RlCgogICAgICAgICMgUGFydGl0aW9uIE1vZGUgUmFkaW8gQnV0dG9ucwogICAgICAgIHBhcnRpdGlvbi5tb2RlID0gc3QucmFkaW8oCiAgICAgICAgICAgICJQYXJ0aXRpb24gTW9kZSIsCiAgICAgICAgICAgIG9wdGlvbnM9W1BhcnRpdGlvbk1vZGUuSU1QT1JULCBQYXJ0aXRpb25Nb2RlLkRJUkVDVF9RVUVSWV0sCiAgICAgICAgICAgIGZvcm1hdF9mdW5jPWxhbWJkYSB4OiBzdHIoeCkucmVwbGFjZSgnaW1wb3J0JywgJ0ltcG9ydCcpLnJlcGxhY2UoJ2RpcmVjdFF1ZXJ5JywgJ0RpcmVjdCBRdWVyeScpLAogICAgICAgICAgICBpbmRleD0wIGlmIHBhcnRpdGlvbi5tb2RlID09IFBhcnRpdGlvbk1vZGUuSU1QT1JUIGVsc2UgMSwKICAgICAgICAgICAga2V5PWYie2tleX1fcGFydGl0aW9uX3twYXJ0aXRpb25JbmRleH1fbW9kZV9hY3Rpb24iLAogICAgICAgICAgICBob3Jpem9udGFsPVRydWUKICAgICAgICApCgogICAgICAgICMgTW92ZSBsb2dpYyBpbnRvIFNub3dmbGFrZSBjaGVja2JveAogICAgICAgIHBhcnRpdGlvbi5Nb3ZlTG9naWNJbnRvU25vd2ZsYWtlID0gc3QuY2hlY2tib3goCiAgICAgICAgICAgICJDb252ZXJ0IHRvIE5hdGl2ZSBRdWVyeSAoTW92ZSBsb2dpYyBpbnRvIFNub3dmbGFrZSkiLCAKICAgICAgICAgICAgdmFsdWU9cGFydGl0aW9uLk1vdmVMb2dpY0ludG9Tbm93Zmxha2UsIAogICAgICAgICAgICBkaXNhYmxlZD1wYXJ0aXRpb24uR2V0Q29udmVydGVkRXhwcmVzc2lvbigpIGlzIE5vbmUgb3IgcGFydGl0aW9uLkhhc0lzc3VlcywKICAgICAgICAgICAga2V5PWYie2tleX1fbW92ZV9pbnRvX3NmX2NiIgogICAgICAgICkKCiAgICAgICAgbmV3Q3JlYXRlVmFsdWUgPSBzdC5jaGVja2JveCgKICAgICAgICAgICAgIkNyZWF0ZSBWaWV3IGluIFNub3dmbGFrZSIsCiAgICAgICAgICAgIHZhbHVlPXBhcnRpdGlvbi5DcmVhdGVTbm93Zmxha2VPYmplY3QsCiAgICAgICAgICAgIGRpc2FibGVkPXBhcnRpdGlvbi5IYXNJc3N1ZXMgb3IgcGFydGl0aW9uLkdldE9iamVjdENyZWF0ZURETCgpIGlzIE5vbmUsCiAgICAgICAgICAgIGtleT1mIntrZXl9X2NyZWF0ZV9zZl9vYmpfY2IiLAogICAgICAgICAgICBoZWxwPSJDcmVhdGUgYSBTbm93Zmxha2UgdmlldyBmb3IgdGhpcyBwYXJ0aXRpb24uIiwKICAgICAgICApCgogICAgICAgIGlmIG5ld0NyZWF0ZVZhbHVlICE9IHBhcnRpdGlvbi5DcmVhdGVTbm93Zmxha2VPYmplY3Q6CiAgICAgICAgICAgIHBhcnRpdGlvbi5DcmVhdGVTbm93Zmxha2VPYmplY3QgPSBuZXdDcmVhdGVWYWx1ZQogICAgICAgICAgICBzdC5yZXJ1bigpCgogICAgICAgIGlmIHBhcnRpdGlvbi5DcmVhdGVTbm93Zmxha2VPYmplY3QgPT0gVHJ1ZToKICAgICAgICAgICAgYzEsIGMyLCBjMyA9IHN0LmNvbHVtbnMoWzMsMTAsNV0pCiAgICAgICAgICAgIHdpdGggYzE6CiAgICAgICAgICAgICAgICBmcm9tIHV0aWwuc25vd2ZsYWtlT2JqZWN0VHlwZSBpbXBvcnQgU25vd2ZsYWtlT2JqZWN0VHlwZQogICAgICAgICAgICAgICAgb2JqZWN0VHlwZXMgPSBbU25vd2ZsYWtlT2JqZWN0VHlwZS5WSUVXLCBTbm93Zmxha2VPYmplY3RUeXBlLk1BVEVSSUFMSVpFRF9WSUVXXQoKICAgICAgICAgICAgICAgIGluZGV4ID0gb2JqZWN0VHlwZXMuaW5kZXgocGFydGl0aW9uLlNub3dmbGFrZU9iamVjdC50eXBlKQogICAgICAgICAgICAgICAgc2VsZWN0ZWRfdHlwZSA9IHN0LnNlbGVjdGJveCgKICAgICAgICAgICAgICAgICAgICAiVHlwZSIsCiAgICAgICAgICAgICAgICAgICAgb3B0aW9ucz1vYmplY3RUeXBlcywKICAgICAgICAgICAgICAgICAgICBpbmRleD1pbmRleCwKICAgICAgICAgICAgICAgICAgICBmb3JtYXRfZnVuYz1sYW1iZGEgeDogc3RyKHgpLnRpdGxlKCksCiAgICAgICAgICAgICAgICAgICAga2V5PWYie2tleX1fc25vd2ZsYWtlX29iamVjdF90eXBlIiwKICAgICAgICAgICAgICAgICAgICBoZWxwPSJTZWxlY3QgdGhlIHR5cGUgb2YgU25vd2ZsYWtlIHZpZXcgdG8gY3JlYXRlIGZvciB0aGlzIHBhcnRpdGlvbi4gQSBtYXRlcmlhbGl6ZWQgdmlldyBpcyBtb3JlIGVmZmljaWVudCBmb3IgY29tcGxleCBxdWVyaWVzLCBob3dldmVyIGl0IGNhbm5vdCBiZSBjcmVhdGVkIG9uIHRvcCBvZiBhbm90aGVyIHZpZXcuIiwKICAgICAgICAgICAgICAgICkKICAgICAgICAgICAgICAgIHBhcnRpdGlvbi5Tbm93Zmxha2VPYmplY3QudHlwZSA9IHNlbGVjdGVkX3R5cGUKCiAgICAgICAgICAgIHdpdGggYzI6CiAgICAgICAgICAgICAgICBwYXJ0aXRpb24uU25vd2ZsYWtlT2JqZWN0Lm5hbWUgPSBzdC50ZXh0X2lucHV0KAogICAgICAgICAgICAgICAgICAgICJOYW1lIiwKICAgICAgICAgICAgICAgICAgICB2YWx1ZT1wYXJ0aXRpb24uU25vd2ZsYWtlT2JqZWN0Lm5hbWUsCiAgICAgICAgICAgICAgICAgICAga2V5PWYie2tleX1fc25vd2ZsYWtlX29iamVjdF9uYW1lIiwKICAgICAgICAgICAgICAgICAgICBtYXhfY2hhcnM9MTAwCiAgICAgICAgICAgICAgICApCiAgICAgICAgICAgIHdpdGggYzM6CiAgICAgICAgICAgICAgICAjIENyZWF0ZSBpbnZpc2libGUgbGFiZWwgdG8gbWF0Y2ggaW5wdXQgYWxpZ25tZW50CiAgICAgICAgICAgICAgICBzdC5tYXJrZG93bignPGxhYmVsIHN0eWxlPSJmb250LXNpemU6IDE0cHg7IGZvbnQtd2VpZ2h0OiA0MDA7IHZpc2liaWxpdHk6IGhpZGRlbjsgaGVpZ2h0OiAyMXB4OyBkaXNwbGF5OiBibG9jazsgbWFyZ2luLWJvdHRvbTogOHB4OyI+SGlkZGVuPC9sYWJlbD4nLCB1bnNhZmVfYWxsb3dfaHRtbD1UcnVlKQogICAgICAgICAgICAgICAgaWYgc3QuYnV0dG9uKCJDcmVhdGUiLCBrZXk9ZiJ7a2V5fV9zbm93Zmxha2Vfb2JqZWN0X2NyZWF0ZV9idG4iLCB0eXBlPSJwcmltYXJ5IiwgdXNlX2NvbnRhaW5lcl93aWR0aD1UcnVlKToKICAgICAgICAgICAgICAgICAgICBkZGwgPSBwYXJ0aXRpb24uR2V0T2JqZWN0Q3JlYXRlRERMKCkKICAgICAgICAgICAgICAgICAgICBzZk9iak5hbWUgPSAiLiIuam9pbihbcGFydGl0aW9uLlNub3dmbGFrZU9iamVjdC5zY2hlbWEsIHBhcnRpdGlvbi5Tbm93Zmxha2VPYmplY3QubmFtZV0pICAgIAogICAgICAgICAgICAgICAgICAgIHdpdGggc3Quc3Bpbm5lcihmIkNyZWF0aW5nIHtwYXJ0aXRpb24uU25vd2ZsYWtlT2JqZWN0LnR5cGV9IHtzZk9iak5hbWV9Li4uIik6CiAgICAgICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHJlc3VsdCA9IHBhcnRpdGlvbi5Nb2RlbC5TZXNzaW9uLnNxbChkZGwpLmNvbGxlY3QoKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3QudG9hc3QoZiJTdWNjZXNzOiB7c2ZPYmpOYW1lfSBDcmVhdGVkIikKICAgICAgICAgICAgICAgICAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3Qud2FybmluZyhmIkVycm9yIGNyZWF0aW5nIHtzZk9iak5hbWV9OiB7ZX0iKQoKICAgIGVsaWYgcGFydGl0aW9uLmlzRmlsZToKICAgICAgICBmaWxlcyA9IFtdCiAgICAgICAgZm9yIGQgaW4gcGFydGl0aW9uLkdldEFsbERlcGVuZGVuY2llcygpOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKGQsIENvbm5lY3RvcikgYW5kIGQuc291cmNlVHlwZSA9PSBDb25uZWN0b3JUeXBlLkZpbGU6CiAgICAgICAgICAgICAgICBmaWxlcy5hcHBlbmQoZC5nZXRQcm9wZXJ0eSgicGF0aCIpKQogICAgICAgIHN0LndyaXRlKGYiVGhpcyBwYXJ0aXRpb24gaXMgc291cmNlZCBmcm9tIGEgZmlsZTogeycsJy5qb2luKGZpbGVzKX0iKQogICAgZWxpZiBwYXJ0aXRpb24uaXNEYXRhYmFzZToKICAgICAgICBkYnMgPSBzZXQoKQogICAgICAgIGZvciBkIGluIHBhcnRpdGlvbi5HZXRBbGxEZXBlbmRlbmNpZXMoKToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShkLCBDb25uZWN0b3IpIGFuZCBkLnNvdXJjZVR5cGUgPT0gQ29ubmVjdG9yVHlwZS5EYXRhYmFzZToKICAgICAgICAgICAgICAgIGRicy5hZGQoZC5nZXRQcm9wZXJ0eSgiZGF0YWJhc2VUeXBlIikpCiAgICAgICAgc3Qud3JpdGUoZiJUaGlzIHBhcnRpdGlvbiBpcyBzb3VyY2VkIGZyb20gYW4gZXh0ZXJuYWwgZGF0YWJhc2UgY29ubmVjdGlvbjogeycsICcuam9pbihkYnMpfSIpCiAgICBlbHNlOgogICAgICAgIGlmIGlzaW5zdGFuY2UocGFydGl0aW9uLmV4cHJlc3Npb24sIFNRTEV4cHJlc3Npb24pIGFuZCAocGFydGl0aW9uLmV4cHJlc3Npb24uRGF0YVNvdXJjZSBpcyBub3QgTm9uZSk6CiAgICAgICAgICAgIGRhdGFTb3VyY2UgPSBwYXJ0aXRpb24uZXhwcmVzc2lvbi5EYXRhU291cmNlCiAgICAgICAgICAgIHN0LndyaXRlKGYiVGhpcyBwYXJ0aXRpb24gaXMgc291cmNlZCBmcm9tIGFuIGV4dGVybmFsIERhdGEgU291cmNlOiB7ZGF0YVNvdXJjZS5OYW1lfSIpCiAgICAgICAgICAgIHN0LmNvZGUoZGF0YVNvdXJjZS5Db25uZWN0aW9uU3RyaW5nLCB3cmFwX2xpbmVzPVRydWUpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgc3Qud3JpdGUocGFydGl0aW9uLmV4cHJlc3Npb24udmFsdWUpCgogICAgIyBEaXNwbGF5IHRlc3QgU1FMIGJ1dHRvbiBmb3IgdGhpcyBwYXJ0aXRpb24uCiAgICBzcWwgPSBwYXJ0aXRpb24uZ2V0U3FsKCkKICAgIGlmIHNxbCBpcyBub3QgTm9uZSBhbmQgcGFydGl0aW9uLkNyZWF0ZVNub3dmbGFrZU9iamVjdDoKICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKGYiR2VuZXJhdGVkIFNRTCBmb3IgUGFydGl0aW9uIHtwYXJ0aXRpb25JbmRleCArIDF9IiwgZXhwYW5kZWQ9RmFsc2UpOgogICAgICAgICAgICBRdWVyeVZpZXcoc3FsPXNxbCwga2V5PWtleSkKCiAgICB3aXRoIHN0LmV4cGFuZGVyKCJFeHByZXNzaW9uIENvbnZlcnNpb24gRGV0YWlscyIsIGV4cGFuZGVkPUZhbHNlKToKICAgICAgICBpZiBzdC5idXR0b24oIkRldGFpbHMiLCBrZXk9ZiJ7a2V5fV9leHBfZGV0YWlscyIpOgogICAgICAgICAgICBzdC5zZXNzaW9uX3N0YXRlWyJleHByZXNzaW9uVG9FZGl0Il0gPSBwYXJ0aXRpb24ucm9vdEV4cHJlc3Npb24KICAgICAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVsiY3VycmVudF9wYWdlIl0gPSAiZXhwcmVzc2lvbl9lZGl0b3IiCiAgICAgICAgICAgIHN0LnJlcnVuKCkKICAgICAgICBjb2RlVHlwZSA9IE5vbmUKICAgICAgICBtYXRjaCBwYXJ0aXRpb24ucm9vdEV4cHJlc3Npb24uY29kZVR5cGU6CiAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLkRBWDoKICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gImRheCIKICAgICAgICAgICAgY2FzZSBFeHByZXNzaW9uQ29kZVR5cGUuTToKICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gInBvd2VycXVlcnkiCiAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLlNRTDoKICAgICAgICAgICAgICAgIGNvZGVUeXBlID0gInNxbCIKICAgICAgICBvcmlnaW5hbFRhYiwgY29udmVydGVkVGFiLCBkZGxUYWIgPSBzdC50YWJzKFsiT3JpZ2luYWwiLCAiQ29udmVydGVkIE5hdGl2ZSBRdWVyeSIsICJWaWV3IERETCJdKQogICAgICAgIHdpdGggb3JpZ2luYWxUYWI6CiAgICAgICAgICAgIHN0LmNvZGUocGFydGl0aW9uLnJvb3RFeHByZXNzaW9uLnNvdXJjZVN0cmluZywgd3JhcF9saW5lcz1UcnVlLCBsYW5ndWFnZT1jb2RlVHlwZSkKICAgICAgICB3aXRoIGNvbnZlcnRlZFRhYjoKICAgICAgICAgICAgY29udmVydGVkID0gcGFydGl0aW9uLkdldENvbnZlcnRlZEV4cHJlc3Npb24oKQogICAgICAgICAgICBzdC5jb2RlKGNvbnZlcnRlZCwgd3JhcF9saW5lcz1UcnVlLCBsYW5ndWFnZT1jb2RlVHlwZSkKICAgICAgICAjIGlmIHBhcnRpdGlvbi5DcmVhdGVTbm93Zmxha2VPYmplY3Q6CiAgICAgICAgd2l0aCBkZGxUYWI6CiAgICAgICAgICAgIHN0LmNvZGUocGFydGl0aW9uLkdldE9iamVjdENyZWF0ZURETCgpLCB3cmFwX2xpbmVzPVRydWUsIGxhbmd1YWdlPSJzcWwiKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/startPage.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/table_view.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsCmltcG9ydCBzdHJlYW1saXQgYXMgc3QKZnJvbSBzdHJlYW1saXQuZGVsdGFfZ2VuZXJhdG9yIGltcG9ydCBEZWx0YUdlbmVyYXRvcgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUKZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4gYXMgQmFzZUNvbHVtbgpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gLmV4cHJlc3Npb25WaWV3IGltcG9ydCBFeHByZXNzaW9uVmlldwpmcm9tIC50YWJsZUl0ZW1WaWV3IGltcG9ydCBUYWJsZUl0ZW1WaWV3CmZyb20gYmkucmVhZGVycy5wYmkudG1kbC5uYW1lZEl0ZW0gaW1wb3J0IE5hbWVkSXRlbQoKZGVmIFRhYmxlVmlldyh0YWJsZTogVGFibGUsIGRpc3BsYXlXaW5kb3c6IERlbHRhR2VuZXJhdG9yID0gTm9uZSwga2V5OiBzdHIgPSAiIik6CiAgICAKCiAgICBpY29uID0gIuKchSIKCiAgICBleHByZXNzaW9uczpMaXN0W0V4cHJlc3Npb25dID0gW10KICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIFNlbWFudGljVGFibGUpOgogICAgICAgIGZvciBwIGluIHRhYmxlLnBhcnRpdGlvbnM6CiAgICAgICAgICAgIGlmIHAucm9vdEV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBleHByZXNzaW9ucy5hcHBlbmQocC5yb290RXhwcmVzc2lvbikKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIGljb24gPSAi4p2MIgoKICAgIGVsaWYgaXNpbnN0YW5jZSh0YWJsZSwgRXhwcmVzc2lvbik6CiAgICAgICAgZXhwcmVzc2lvbnMuYXBwZW5kKHRhYmxlKQoKICAgIGZvciBlIGluIGV4cHJlc3Npb25zOgogICAgICAgIGUuVXBkYXRlKGZvcmNlPUZhbHNlLCB1cGRhdGVBbGxDaGlsZHJlbj1UcnVlKSAgICAgICAgICAgICAgICAgICAgICAgIAogICAgCiAgICAKICAgIGlzc3VlcyA9IHRhYmxlLkdldEFsbElzc3VlcygpCiAgICBpZiBsZW4oaXNzdWVzKSA+IDA6CiAgICAgICAgaWNvbiA9ICLinYwiCgogICAgbmFtZSA9ICIiCiAgICB0YWJsZVR5cGUgPSB0eXBlKHRhYmxlKS5fX25hbWVfXwogICAga2V5ID0gZiJ0YWJsZV97dGFibGUuX19oYXNoX18oKX1fe2tleX0iCgogICAgaWYgaXNpbnN0YW5jZSh0YWJsZSwgTmFtZWRJdGVtKToKICAgICAgICBuYW1lID0gdGFibGUuZnVsbHlRdWFsaWZpZWROYW1lCiAgICBlbGlmIGlzaW5zdGFuY2UodGFibGUsIFNlbWFudGljVGFibGUpOgogICAgICAgIG5hbWUgPSB0YWJsZS5uYW1lCgogICAgd2l0aCBzdC5leHBhbmRlcihmIntuYW1lfSAoe3RhYmxlVHlwZX0pIiwgZXhwYW5kZWQ9RmFsc2UsIGljb249aWNvbik6CiAgICAgICAgd2l0aCBzdC5jb250YWluZXIoa2V5PWtleSk6CiAgICAgICAgICAgIGlmIGlzc3VlcyBpcyBub3QgTm9uZSBhbmQgbGVuKGlzc3VlcykgPiAwOgogICAgICAgICAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlIGltcG9ydCBJc3N1ZQogICAgICAgICAgICAgICAgd2l0aCBzdC5leHBhbmRlcigiSXNzdWVzIik6CiAgICAgICAgICAgICAgICAgICAgZm9yIGlzc3VlIGluIGlzc3VlczoKICAgICAgICAgICAgICAgICAgICAgICAgaWYgaXNzdWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdC50ZXh0KGYie2lzc3VlLnR5cGUubmFtZX06IHtpc3N1ZS5kZXNjcmlwdGlvbn0iKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIGRpc3BsYXlXaW5kb3cgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBTZW1hbnRpY1RhYmxlKToKICAgICAgICAgICAgICAgICAgICBmb3IgaSwgcCBpbiBlbnVtZXJhdGUodGFibGUucGFydGl0aW9ucyk6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIHAucm9vdEV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdC5idXR0b24oZiJTaG93IHt0YWJsZS5uYW1lfSBUYWJsZSBQYXJ0aXRpb24ge2krMX0gRXhwcmVzc2lvbiBEZXRhaWxzIiwga2V5PWYie3RhYmxlLm5hbWV9X3BhcnRpdGlvbl97aX1fc2hvd19leHByZXNzaW9uX2J0biIsIG9uX2NsaWNrPXNob3dFeHByZXNzaW9uRGV0YWlscywgYXJncz0ocC5yb290RXhwcmVzc2lvbiwgZGlzcGxheVdpbmRvdywgZiJQYXJ0aXRpb24ge2krMX0gRXhwcmVzc2lvbiBEZXRhaWxzIikpCiAgICAgICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2UodGFibGUsIEV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgIHN0LmJ1dHRvbihmIlNob3cgRXhwcmVzc2lvbiBEZXRhaWxzIiwga2V5PWYie3RhYmxlLl9faGFzaF9fKCl9X3Nob3dfZXhwcmVzc2lvbl9idG4iLCBvbl9jbGljaz1zaG93RXhwcmVzc2lvbkRldGFpbHMsIGFyZ3M9KHRhYmxlLCBkaXNwbGF5V2luZG93LCBmIkV4cHJlc3Npb24gRGV0YWlscyIpKQoKICAgICAgICAgICAgbm9uQ29sdW1uSXRlbXMgPSBbXQogICAgICAgICAgICAgICAgCiAgICAgICAgICAgIHdpdGggc3QuZXhwYW5kZXIoIkNvbHVtbnMiLCBleHBhbmRlZD1UcnVlKToKICAgICAgICAgICAgICAgIGZvciBjb2wgaW4gdGFibGUuYWxsSXRlbXM6CiAgICAgICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb2wsIENvbHVtbik6CiAgICAgICAgICAgICAgICAgICAgICAgIFRhYmxlSXRlbVZpZXcoY29sLCBkaXNwbGF5V2luZG93PWRpc3BsYXlXaW5kb3csIGRpc3BsYXlGdWxsTmFtZT1GYWxzZSwga2V5PWtleSkKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBub25Db2x1bW5JdGVtcy5hcHBlbmQoY29sKQoKICAgICAgICAgICAgaWYgbGVuKG5vbkNvbHVtbkl0ZW1zKSA+IDA6CiAgICAgICAgICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKCJNZWFzdXJlcyIsIGV4cGFuZGVkPUZhbHNlKToKICAgICAgICAgICAgICAgICAgICBmb3IgbWVhc3VyZSBpbiBub25Db2x1bW5JdGVtczoKICAgICAgICAgICAgICAgICAgICAgICAgVGFibGVJdGVtVmlldyhtZWFzdXJlLCBkaXNwbGF5V2luZG93PWRpc3BsYXlXaW5kb3csIGRpc3BsYXlGdWxsTmFtZT1GYWxzZSwga2V5PWtleSkKICAgICAgICAgICAgCiAgICAgICAgICAgIHdpdGggc3QuZXhwYW5kZXIoIkdlbmVyYXRlZCBTUUwiLCBleHBhbmRlZD1GYWxzZSk6CiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZSk6CiAgICAgICAgICAgICAgICAgICAgc3QuY29kZSh0YWJsZS5nZXRTcWxWYWx1ZSgpKQogICAgICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIHRhYmxlLmRhdGFJc0F2YWlsYWJsZToKICAgICAgICAgICAgICAgIGlmIGRpc3BsYXlXaW5kb3cgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgaWYgc3QuYnV0dG9uKCJTaG93IERhdGEiLCBmInNob3dfZGF0YV97dGFibGUubmFtZX0iKToKICAgICAgICAgICAgICAgICAgICAgICAgZGlzcGxheVdpbmRvdy5lbXB0eSgpCiAgICAgICAgICAgICAgICAgICAgICAgIHdpdGggZGlzcGxheVdpbmRvdzoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LnN1YmhlYWRlcihmIkRhdGEgZm9yIFRhYmxlICd7dGFibGUubmFtZX0nIikKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0LmRhdGFmcmFtZSh0YWJsZS5nZXREYXRhKCkpCiAgICAgICAgICAgIGVsaWYgdGFibGUuaXNGaWxlOgogICAgICAgICAgICAgICAgZnJvbSBiaS5jb3JlLmNvbm5lY3RvciBpbXBvcnQgQ29ubmVjdG9yLCBDb25uZWN0b3JUeXBlCiAgICAgICAgICAgICAgICBmaWxlcyA9IFtdCiAgICAgICAgICAgICAgICBmb3IgZCBpbiB0YWJsZS5HZXRBbGxEZXBlbmRlbmNpZXMoKToKICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGQsIENvbm5lY3RvcikgYW5kIGQuc291cmNlVHlwZSA9PSBDb25uZWN0b3JUeXBlLkZpbGU6CiAgICAgICAgICAgICAgICAgICAgICAgIGZpbGVzLmFwcGVuZChkLmdldFByb3BlcnR5KCJwYXRoIikpCgogICAgICAgICAgICAgICAgc3QudGV4dChmIkltcG9ydCB7dGFibGUubmFtZX0gdG8gU25vd2ZsYWtlOiBVcGxvYWQ6IHsnLCAnLmpvaW4oZmlsZXMpfSIpCgpkZWYgc2hvd0V4cHJlc3Npb25EZXRhaWxzKGV4cHJlc3Npb246IEV4cHJlc3Npb24sIGRpc3BsYXlXaW5kb3c6IERlbHRhR2VuZXJhdG9yLCB0aXRsZTogT3B0aW9uYWxbc3RyXSA9IE5vbmUpOgogICAgaWYgdGl0bGUgaXMgTm9uZToKICAgICAgICB0aXRsZSA9ICJFeHByZXNzaW9uIERldGFpbHMiCiAgICBkaXNwbGF5V2luZG93LmVtcHR5KCkKICAgIHdpdGggZGlzcGxheVdpbmRvdzoKICAgICAgICBzdC5zdWJoZWFkZXIodGl0bGUpCiAgICAgICAgd2l0aCBzdC5jb250YWluZXIoKToKICAgICAgICAgICAgRXhwcmVzc2lvblZpZXcoZXhwcmVzc2lvbikKCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/parameterView.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbiwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS5tb2RlbCBpbXBvcnQgTW9kZWwKCmRlZiBQYXJhbWV0ZXJWaWV3KG1vZGVsOiBNb2RlbCwgcGFyYW1ldGVyTmFtZTogc3RyLCBrZXk6IHN0ciA9ICIiKToKICAgIGtleSA9IGYie2tleX1fcGFyYW1ldGVyX3twYXJhbWV0ZXJOYW1lfSIKICAgIHBhcmFtZXRlckluZm86IGRpY3QgPSBtb2RlbC5QYXJhbWV0ZXJzLmdldChwYXJhbWV0ZXJOYW1lKQogICAgcGFyYW1ldGVyVHlwZTogRXhwcmVzc2lvblR5cGUgPSBwYXJhbWV0ZXJJbmZvLmdldCgidHlwZSIpCiAgICBwYXJhbWV0ZXJWYWx1ZXM6IGxpc3QgPSBwYXJhbWV0ZXJJbmZvLmdldCgibGlzdCIpCiAgICBkZWZhdWx0VmFsdWUgPSBwYXJhbWV0ZXJJbmZvLmdldCgiZGVmYXVsdFZhbHVlIikKICAgIHZhbHVlRXhwcmVzc2lvbiA9IG1vZGVsLkdldFBhcmFtZXRlclZhbHVlKHBhcmFtZXRlck5hbWUpCiAgICByZXF1aXJlZCA9IHBhcmFtZXRlckluZm8uZ2V0KCJyZXF1aXJlZCIsIEZhbHNlKQoKICAgIGlmIHZhbHVlRXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICB2YWx1ZSA9IHZhbHVlRXhwcmVzc2lvbi52YWx1ZQogICAgZWxzZToKICAgICAgICB2YWx1ZSA9IE5vbmUKCiAgICBpZiByZXF1aXJlZCA9PSBUcnVlOgogICAgICAgIGxhYmVsID0gZiJ7cGFyYW1ldGVyTmFtZX0gKHJlcXVpcmVkKSIKICAgIGVsc2U6CiAgICAgICAgbGFiZWwgPSBwYXJhbWV0ZXJOYW1lCgogICAgZGVmIHVwZGF0ZV9wYXJhbWV0ZXIobmFtZTogc3RyLCBrZXk6IHN0cik6CiAgICAgICAgbmV3VmFsdWUgPSBzdC5zZXNzaW9uX3N0YXRlW2tleV0KICAgICAgICBtb2RlbC5TZXRQYXJhbWV0ZXJWYWx1ZShuYW1lLCBuZXdWYWx1ZSkKCiAgICB3aXRoIHN0LmNvbnRhaW5lcigpOgogICAgICAgIGlmIHBhcmFtZXRlclZhbHVlcyBpcyBub3QgTm9uZToKICAgICAgICAgICAgaWYgdmFsdWUgaXMgTm9uZSBhbmQgZGVmYXVsdFZhbHVlIGlzIE5vbmU6CiAgICAgICAgICAgICAgICBzZWxlY3RlZF9pbmRleCA9IDAKICAgICAgICAgICAgICAgIG1vZGVsLlNldFBhcmFtZXRlclZhbHVlKHBhcmFtZXRlck5hbWUsIHBhcmFtZXRlclZhbHVlc1swXSkKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHNlbGVjdGVkX2luZGV4ID0gcGFyYW1ldGVyVmFsdWVzLmluZGV4KHZhbHVlIG9yIGRlZmF1bHRWYWx1ZSkKICAgICAgICAgICAgICAgIAogICAgICAgICAgICBzdC5zZWxlY3Rib3gobGFiZWw9bGFiZWwsIG9wdGlvbnM9cGFyYW1ldGVyVmFsdWVzLCBpbmRleD1zZWxlY3RlZF9pbmRleCwga2V5PWtleSwgb25fY2hhbmdlPXVwZGF0ZV9wYXJhbWV0ZXIsIGFyZ3M9KHBhcmFtZXRlck5hbWUsIGtleSwpKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIG5ld1ZhbHVlID0gc3QudGV4dF9pbnB1dChsYWJlbD1sYWJlbCwgdmFsdWU9dmFsdWUsIGtleT1rZXkpICAgICAgICAKICAgICAgICAgICAgaWYgbmV3VmFsdWUgIT0gdmFsdWU6CiAgICAgICAgICAgICAgICBtb2RlbC5TZXRQYXJhbWV0ZXJWYWx1ZShwYXJhbWV0ZXJOYW1lLCBuZXdWYWx1ZSkKICAgIAogICAg'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/ui/file_upload.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IE9wdGlvbmFsLCBMaXN0CmltcG9ydCBpbwppbXBvcnQgdGltZQppbXBvcnQgc3RyZWFtbGl0IGFzIHN0CmZyb20gc25vd2ZsYWtlLnNub3dwYXJrIGltcG9ydCBTZXNzaW9uCmZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IFNub3dmbGFrZU9iamVjdApmcm9tIHV0aWwuc2Vzc2lvbl91dGlscyBpbXBvcnQgZ2V0X2N1cnJlbnRfdXNlcgoKY2xhc3MgRmlsZUluZm86CiAgICBkZWYgX19pbml0X18oCiAgICAgICAgc2VsZiwKICAgICAgICBzdGFnZTogU25vd2ZsYWtlT2JqZWN0LAogICAgICAgIHBhdGg6IHN0ciA9ICIiLAogICAgICAgIGZpbGVzOiBMaXN0W3N0cl0gPSBbXSwKICAgICAgICBuZXdGaWxlczogTGlzdFtzdHJdID0gW10KICAgICk6CiAgICAgICAgc2VsZi5zdGFnZSA9IHN0YWdlCiAgICAgICAgc2VsZi5wYXRoID0gcGF0aAogICAgICAgIHNlbGYuZmlsZXMgPSBmaWxlcwogICAgICAgIHNlbGYuZmlsZVVwbG9hZGVkID0gRmFsc2UKICAgICAgICBzZWxmLm5ld0ZpbGVzID0gbmV3RmlsZXMKCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYuc3RhZ2V9L3tzZWxmLnBhdGh9ICh7JywnLmpvaW4oc2VsZi5maWxlcyl9KSIKICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCB2YWx1ZSk6CiAgICAgICAgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgRmlsZUluZm8pOgogICAgICAgICAgICByZXR1cm4gKHNlbGYuc3RhZ2UgPT0gdmFsdWUuc3RhZ2UgYW5kIHNlbGYucGF0aCA9PSB2YWx1ZS5wYXRoIGFuZCBzZWxmLmZpbGVzID09IHZhbHVlLmZpbGVzKQogICAgICAgIHJldHVybiBGYWxzZQogICAgICAgIAoKZGVmIGZpbGVVcGxvYWRDb250cm9sKHNlc3Npb246IFNlc3Npb24sIAogICAgICAgICAgICAgICAgICAgICAgZmlsZUluZm86IEZpbGVJbmZvLAogICAgICAgICAgICAgICAgICAgICAga2V5OiBzdHIgPSAiIikgLT4gRmlsZUluZm86CiAgICBjb250YWluZXIgPSBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpCgogICAgd2l0aCBjb250YWluZXI6CiAgICAgICAgc3QuaGVhZGVyKCJGaWxlIFVwbG9hZCIpCiAgICAgICAgCiAgICAgICAgdXBsb2FkZWRfZmlsZXMgPSBzdC5maWxlX3VwbG9hZGVyKCJDaG9vc2UgYSBmaWxlIG9yIGZpbGVzIHRvIHVwbG9hZCIsIGFjY2VwdF9tdWx0aXBsZV9maWxlcz1UcnVlLCBoZWxwPSJQaWNrIG9uZSBvciBtb3JlIGZpbGVzIHRvIHVwbG9hZCIsIGtleT1mIntrZXl9X3NvdXJjZV91cGxvYWRfYnRuIikKICAgICAgICAKICAgICAgICBpZiB1cGxvYWRlZF9maWxlcyBpcyBub3QgTm9uZSBhbmQgbGVuKHVwbG9hZGVkX2ZpbGVzKSA+IDA6CiAgICAgICAgICAgICNtYWtlIHN1cmUgdGhlIHN0YWdlIGFjdHVhbGx5IGV4aXN0cwogICAgICAgICAgICBpZiBub3QgZmlsZUluZm8uc3RhZ2UuY3JlYXRlX2lmX25vdF9leGlzdChzZXNzaW9uKToKICAgICAgICAgICAgICAgICMgY2hlY2sgdG8gc2VlIGlmIHRoZSBzdGFnZSBhbHJlYWR5IGV4aXN0cyAoYnV0IHRoZSB1c2VyIGRpZG4ndCBoYXZlIENSRUFURSBTVEFHRSBwcml2cykKICAgICAgICAgICAgICAgIGlmIG5vdCBmaWxlSW5mby5zdGFnZS5jaGVja19pZl9leGlzdHMoc2Vzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgc3QuZXJyb3IoZiJFcnJvciBjcmVhdGluZyBmaWxlIHVwbG9hZCBzdGFnZSB7ZmlsZUluZm8uc3RhZ2UuZ2V0X2ZxbigpfSIpCiAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICMgc2V0IHRoZSBwYXRoCiAgICAgICAgICAgIGlmIGZpbGVJbmZvLnBhdGggaXMgTm9uZSBvciBsZW4oZmlsZUluZm8ucGF0aCkgPT0gMDoKICAgICAgICAgICAgICAgIGN1cnJlbnRfdXNlciA9IGdldF9jdXJyZW50X3VzZXIoc2Vzc2lvbikKICAgICAgICAgICAgICAgIGN1cnJlbnRfdGltZSA9IHRpbWUudGltZSgpCiAgICAgICAgICAgICAgICBmaWxlSW5mby5wYXRoID0gZiJ7Y3VycmVudF91c2VyfS97Y3VycmVudF90aW1lfSIKICAgICAgICAgICAgCiAgICAgICAgICAgICMgaW5pdGlhbGl6ZSB0aGUgZmlsZXMgYXJyYXkKICAgICAgICAgICAgaWYgZmlsZUluZm8uZmlsZXMgaXMgTm9uZToKICAgICAgICAgICAgICAgIGZpbGVJbmZvLmZpbGVzID0gW10KCiAgICAgICAgICAgICMgY2xlYXIgdGhlIG5ldyBmaWxlcwogICAgICAgICAgICBmaWxlSW5mby5uZXdGaWxlcy5jbGVhcigpCiAgICAgICAgICAgIGZvciB1cGxvYWRlZF9maWxlIGluIHVwbG9hZGVkX2ZpbGVzOgogICAgICAgICAgICAgICAgIyBBbHdheXMgdXBsb2FkIGZpbGVzIHdpdGggb3ZlcndyaXRlPVRydWUsIGV2ZW4gaWYgdGhleSB3ZXJlIHVwbG9hZGVkIGJlZm9yZQogICAgICAgICAgICAgICAgd2l0aCBzdC5zcGlubmVyKGYiVXBsb2FkaW5nIHt1cGxvYWRlZF9maWxlLm5hbWV9Li4uIik6CiAgICAgICAgICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgICAgICAgICAjIENyZWF0ZSBmaWxlIHN0cmVhbSB1c2luZyBCeXRlc0lPIGFuZCB1cGxvYWQKICAgICAgICAgICAgICAgICAgICAgICAgZmlsZV9zdHJlYW0gPSBpby5CeXRlc0lPKHVwbG9hZGVkX2ZpbGUuZ2V0dmFsdWUoKSkKICAgICAgICAgICAgICAgICAgICAgICAgc2Vzc2lvbi5maWxlLnB1dF9zdHJlYW0oCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBmaWxlX3N0cmVhbSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGYiQHtmaWxlSW5mby5zdGFnZS5nZXRfZnFuKCl9L3tmaWxlSW5mby5wYXRofS97dXBsb2FkZWRfZmlsZS5uYW1lfSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBhdXRvX2NvbXByZXNzPUZhbHNlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgb3ZlcndyaXRlPVRydWUKICAgICAgICAgICAgICAgICAgICAgICAgKQogICAgICAgICAgICAgICAgICAgICAgICAjIE9ubHkgYWRkIHRvIGZpbGVzIGxpc3QgaWYgbm90IGFscmVhZHkgdGhlcmUKICAgICAgICAgICAgICAgICAgICAgICAgaWYgdXBsb2FkZWRfZmlsZS5uYW1lIG5vdCBpbiBmaWxlSW5mby5maWxlczoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGVJbmZvLmZpbGVzLmFwcGVuZCh1cGxvYWRlZF9maWxlLm5hbWUpCiAgICAgICAgICAgICAgICAgICAgICAgICMgQWx3YXlzIGFkZCB0byBuZXdGaWxlcyB0byB0cmlnZ2VyIHByb2Nlc3NpbmcKICAgICAgICAgICAgICAgICAgICAgICAgZmlsZUluZm8ubmV3RmlsZXMuYXBwZW5kKHVwbG9hZGVkX2ZpbGUubmFtZSkKICAgICAgICAgICAgICAgICAgICAgICAgZmlsZUluZm8uZmlsZVVwbG9hZGVkID0gVHJ1ZQogICAgICAgICAgICAgICAgICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZToKICAgICAgICAgICAgICAgICAgICAgICAgc3QuZXJyb3IoZiJFcnJvciBvY2N1cnJlZCB3aGlsZSB1cGxvYWRpbmcgZmlsZSB7dXBsb2FkZWRfZmlsZS5uYW1lfS4gUGxlYXNlIG1ha2Ugc3VyZSB0aGUgY3VycmVudCByb2xlIGhhcyB0aGUgVVNBR0UgYW5kIFdSSVRFIHByaXZpbGVnZXMgb24gdGhlIHN0YWdlIHtmaWxlSW5mby5zdGFnZS5nZXRfZnFuKCl9OiB7c3RyKGUpfSIpCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgIHJldHVybiBmaWxlSW5mbwoKCgoKCgoKCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/environment.yml FROM (SELECT BASE64_DECODE_STRING('IyBUaGlzIGZpbGUgaXMgdXNlZCB0byBpbnN0YWxsIHBhY2thZ2VzIHVzZWQgYnkgdGhlIFN0cmVhbWxpdCBBcHAuCiMgRm9yIG1vcmUgZGV0YWlscywgcmVmZXIgdG8gaHR0cHM6Ly9kb2NzLnNub3dmbGFrZS5jb20vZW4vZGV2ZWxvcGVyLWd1aWRlL3N0cmVhbWxpdC9jcmVhdGUtc3RyZWFtbGl0LXNxbCNsYWJlbC1zdHJlYW1saXQtaW5zdGFsbC1wYWNrYWdlcy1tYW51YWwKCmNoYW5uZWxzOgogIC0gc25vd2ZsYWtlCmRlcGVuZGVuY2llczoKICAtIHN0cmVhbWxpdAogIC0gcmVnZXgK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/util/session_utils.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApmcm9tIHR5cGluZyBpbXBvcnQgT3B0aW9uYWwKZnJvbSBzbm93Zmxha2Uuc25vd3BhcmsgaW1wb3J0IFNlc3Npb24KZnJvbSAuc2Vzc2lvbiBpbXBvcnQgc2Vzc2lvbgoKQHN0LmNhY2hlX3Jlc291cmNlCmRlZiBjb25uZWN0X3RvX3Nub3dmbGFrZSgpIC0+IE9wdGlvbmFsW1Nlc3Npb25dOgogICAgdHJ5OgogICAgICAgIHJldHVybiBzZXNzaW9uKCkKICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZToKICAgICAgICBzdC53YXJuaW5nKGYiRXJyb3IgY29ubmVjdGluZyB0byBTbm93Zmxha2U6IHtlfSIpCiAgICAgICAgcmV0dXJuIE5vbmUKCmRlZiBnZXRfY3VycmVudF9hY2NvdW50X2lkZW50aWZpZXIoc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIGlmICJjdXJyZW50X2FjY291bnRfaWRlbnRpZmllciIgbm90IGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgY3VycmVudF9hY2NvdW50X2lkZW50aWZpZXIgPSBfZ2V0X2N1cnJlbnRfYWNjb3VudF9pZGVudGlmaWVyKHNlc3Npb24pCiAgICAgICAgc3Quc2Vzc2lvbl9zdGF0ZVsiY3VycmVudF9hY2NvdW50X2lkZW50aWZpZXIiXSA9IGN1cnJlbnRfYWNjb3VudF9pZGVudGlmaWVyCgogICAgaWYgImN1cnJlbnRfYWNjb3VudF9pZGVudGlmaWVyIiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgIHJldHVybiBzdC5zZXNzaW9uX3N0YXRlWyJjdXJyZW50X2FjY291bnRfaWRlbnRpZmllciJdCiAgICBlbHNlOgogICAgICAgIHJldHVybiAiIgoKZGVmIGdldF9jdXJyZW50X3VzZXIoc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIGlmICJjdXJyZW50X3VzZXIiIGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgY3VycmVudF91c2VyOiBPcHRpb25hbFtzdHJdID0gc3Quc2Vzc2lvbl9zdGF0ZVsiY3VycmVudF91c2VyIl0KICAgIGVsc2U6CiAgICAgICAgY3VycmVudF91c2VyID0gTm9uZQogICAgICAgIAogICAgaWYgY3VycmVudF91c2VyIGlzIE5vbmU6CiAgICAgICAgY3VycmVudF91c2VyID0gX2dldF9jdXJyZW50X3VzZXIoc2Vzc2lvbikKICAgICAgICBzdC5zZXNzaW9uX3N0YXRlWyJjdXJyZW50X3VzZXIiXSA9IGN1cnJlbnRfdXNlcgogICAgcmV0dXJuIGN1cnJlbnRfdXNlcgoKZGVmIGdldF9jdXJyZW50X2RhdGFiYXNlKHNlc3Npb246IFNlc3Npb24pIC0+IE9wdGlvbmFsW3N0cl06CiAgICBpZiAiY3VycmVudF9kYXRhYmFzZSIgbm90IGluIHN0LnNlc3Npb25fc3RhdGU6CiAgICAgICAgY3VycmVudF9kYXRhYmFzZSA9IF9nZXRfY3VycmVudF9kYXRhYmFzZShzZXNzaW9uKQogICAgICAgIHN0LnNlc3Npb25fc3RhdGVbImN1cnJlbnRfZGF0YWJhc2UiXSA9IGN1cnJlbnRfZGF0YWJhc2UKICAgICAgICAKICAgIGlmICJjdXJyZW50X2RhdGFiYXNlIiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgIHJldHVybiBzdC5zZXNzaW9uX3N0YXRlWyJjdXJyZW50X2RhdGFiYXNlIl0KICAgIGVsc2U6CiAgICAgICAgcmV0dXJuICIiCgoKZGVmIGdldF9jdXJyZW50X3dhcmVob3VzZShzZXNzaW9uOiBTZXNzaW9uKSAtPiBPcHRpb25hbFtzdHJdOgogICAgaWYgImN1cnJlbnRfd2FyZWhvdXNlIiBub3QgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgICAgICBjdXJyZW50X3dhcmVob3VzZSA9IF9nZXRfY3VycmVudF93YXJlaG91c2Uoc2Vzc2lvbikKICAgICAgICBzdC5zZXNzaW9uX3N0YXRlWyJjdXJyZW50X3dhcmVob3VzZSJdID0gY3VycmVudF93YXJlaG91c2UKICAgIGlmICJjdXJyZW50X3dhcmVob3VzZSIgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgICAgICByZXR1cm4gc3Quc2Vzc2lvbl9zdGF0ZVsiY3VycmVudF93YXJlaG91c2UiXQogICAgZWxzZToKICAgICAgICByZXR1cm4gIiIKCmRlZiBnZXRfY3VycmVudF9hY2NvdW50KHNlc3Npb246IFNlc3Npb24pIC0+IE9wdGlvbmFsW3N0cl06CiAgICBpZiAiY3VycmVudF9hY2NvdW50IiBub3QgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgICAgICBjdXJyZW50X2FjY291bnQgPSBfZ2V0X2N1cnJlbnRfYWNjb3VudChzZXNzaW9uKQogICAgICAgIHN0LnNlc3Npb25fc3RhdGVbImN1cnJlbnRfYWNjb3VudCJdID0gY3VycmVudF9hY2NvdW50CiAgICBpZiAiY3VycmVudF9hY2NvdW50IiBpbiBzdC5zZXNzaW9uX3N0YXRlOgogICAgICAgIHJldHVybiBzdC5zZXNzaW9uX3N0YXRlWyJjdXJyZW50X2FjY291bnQiXQogICAgZWxzZToKICAgICAgICByZXR1cm4gIiIKCkBzdC5jYWNoZV9kYXRhCmRlZiBfZ2V0X2N1cnJlbnRfYWNjb3VudF9pZGVudGlmaWVyKF9zZXNzaW9uOiBTZXNzaW9uKSAtPiBPcHRpb25hbFtzdHJdOgogICAgdmFsdWUgPSAiIgogICAgdHJ5OgogICAgICAgICMgU0YgYWNjb3VudCBpZGVudGlmaWVyOiBvcmdfbmFtZS1hY2NvdW50X25hbWUKICAgICAgICByZXN1bHQgPSBfc2Vzc2lvbi5zcWwoIlNFTEVDVCBDVVJSRU5UX09SR0FOSVpBVElPTl9OQU1FKCkgfHwgJy0nIHx8IENVUlJFTlRfQUNDT1VOVF9OQU1FKCk7IikuY29sbGVjdCgpCiAgICAgICAgdmFsdWUgPSByZXN1bHRbMF1bMF0KICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZToKICAgICAgICBzdC5lcnJvcigiRXJyb3IgZ2V0dGluZyBjdXJyZW50IGFjY291bnQgaWRlbnRpZmllciIpCiAgICAgICAgc3QuZXJyb3IoZiJEZXRhaWxzOntlfSIpCiAgICByZXR1cm4gdmFsdWUgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKSBlbHNlIE5vbmUKCkBzdC5jYWNoZV9kYXRhCmRlZiBfZ2V0X2N1cnJlbnRfdXNlcihfc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIHZhbHVlID0gIiIKICAgIHRyeToKICAgICAgICByZXN1bHQgPSBfc2Vzc2lvbi5zcWwoInNlbGVjdCBjdXJyZW50X3VzZXIoKSIpLmNvbGxlY3QoKQogICAgICAgIHZhbHVlID0gcmVzdWx0WzBdWzBdCiAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgc3QuZXJyb3IoIkVycm9yIGdldHRpbmcgY3VycmVudCB1c2VyIikKICAgICAgICBzdC5lcnJvcihmIkRldGFpbHM6e2V9IikKICAgIHJldHVybiB2YWx1ZSBpZiBpc2luc3RhbmNlKHZhbHVlLCBzdHIpIGVsc2UgTm9uZQoKQHN0LmNhY2hlX2RhdGEKZGVmIF9nZXRfY3VycmVudF9kYXRhYmFzZShfc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIHZhbHVlID0gIiIKICAgIHRyeToKICAgICAgICByZXN1bHQgPSBfc2Vzc2lvbi5zcWwoInNlbGVjdCBjdXJyZW50X2RhdGFiYXNlKCkiKS5jb2xsZWN0KCkKICAgICAgICB2YWx1ZSA9IHJlc3VsdFswXVswXQogICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgIHN0LmVycm9yKCJFcnJvciBnZXR0aW5nIGN1cnJlbnQgZGF0YWJhc2UiKQogICAgICAgIHN0LmVycm9yKGYiRGV0YWlsczp7ZX0iKQogICAgcmV0dXJuIHZhbHVlIGlmIGlzaW5zdGFuY2UodmFsdWUsIHN0cikgZWxzZSBOb25lCgpAc3QuY2FjaGVfZGF0YQpkZWYgX2dldF9jdXJyZW50X3dhcmVob3VzZShfc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIHZhbHVlID0gIiIKICAgIHRyeToKICAgICAgICByZXN1bHQgPSBfc2Vzc2lvbi5zcWwoInNlbGVjdCBjdXJyZW50X3dhcmVob3VzZSgpIikuY29sbGVjdCgpCiAgICAgICAgdmFsdWUgPSByZXN1bHRbMF1bMF0KICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZToKICAgICAgICBzdC5lcnJvcigiRXJyb3IgZ2V0dGluZyBjdXJyZW50IHdhcmVob3VzZSIpCiAgICAgICAgc3QuZXJyb3IoZiJEZXRhaWxzOntlfSIpCiAgICByZXR1cm4gdmFsdWUgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKSBlbHNlIE5vbmUKCkBzdC5jYWNoZV9kYXRhCmRlZiBfZ2V0X2N1cnJlbnRfYWNjb3VudChfc2Vzc2lvbjogU2Vzc2lvbikgLT4gT3B0aW9uYWxbc3RyXToKICAgIHZhbHVlID0gIiIKICAgIHRyeToKICAgICAgICByZXN1bHQgPSBfc2Vzc2lvbi5zcWwoIlNFTEVDVCBDVVJSRU5UX0FDQ09VTlQoKSIpLmNvbGxlY3QoKQogICAgICAgIHZhbHVlID0gcmVzdWx0WzBdWzBdCiAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgc3QuZXJyb3IoIkVycm9yIGdldHRpbmcgY3VycmVudCBhY2NvdW50IikKICAgICAgICBzdC5lcnJvcihmIkRldGFpbHM6e2V9IikKICAgIHJldHVybiB2YWx1ZSBpZiBpc2luc3RhbmNlKHZhbHVlLCBzdHIpIGVsc2UgTm9uZQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/util/session.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IG9zCgppbXBvcnQgc25vd2ZsYWtlLmNvbm5lY3Rvcgpmcm9tIHNub3dmbGFrZS5zbm93cGFyayBpbXBvcnQgU2Vzc2lvbgpmcm9tIHNub3dmbGFrZS5zbm93cGFyay5jb250ZXh0IGltcG9ydCBnZXRfYWN0aXZlX3Nlc3Npb24KCmRlZiBnZXRfbG9naW5fdG9rZW4oKSAtPiBzdHI6CiAgICBpZiBub3QgaXNfdG9rZW4oKTogcmV0dXJuICcnCiAgICB3aXRoIG9wZW4oJy9zbm93Zmxha2Uvc2Vzc2lvbi90b2tlbicsICdyJykgYXMgZjoKICAgICAgICByZXR1cm4gZi5yZWFkKCkKCmRlZiByZWFkX2xvY2FsX2NyZWRlbnRpYWxzKCk6CiAgICBpbXBvcnQganNvbgogICAgd2l0aCBvcGVuKGdldF9sb2NhbF9jb25uX2ZpbGVfYWJzcGF0aCgpKSBhcyBjb25uX2Y6CiAgICAgICAgY29uZmlnID0ganNvbi5sb2FkKGNvbm5fZikKICAgICAgICAKICAgICAgICAjIEhhbmRsZSB1bmlmaWVkIGNvbmZpZyBmb3JtYXQKICAgICAgICBpZiAncnVudGltZScgaW4gY29uZmlnOgogICAgICAgICAgICAjIE5ldyB1bmlmaWVkIGZvcm1hdCAtIG1lcmdlIHJ1bnRpbWUgc2V0dGluZ3Mgd2l0aCBjb25uZWN0aW9uIHNldHRpbmdzCiAgICAgICAgICAgIHJ1bnRpbWVfY29uZmlnID0gY29uZmlnWydydW50aW1lJ10uY29weSgpCiAgICAgICAgICAgIGZvciBrZXkgaW4gWydhY2NvdW50JywgJ3VzZXInLCAncGFzc3dvcmQnLCAncHJpdmF0ZV9rZXlfcGF0aCcsICdwcml2YXRlX2tleV9wYXNzcGhyYXNlJ106CiAgICAgICAgICAgICAgICBpZiBrZXkgaW4gY29uZmlnOgogICAgICAgICAgICAgICAgICAgIHJ1bnRpbWVfY29uZmlnW2tleV0gPSBjb25maWdba2V5XQogICAgICAgICAgICByZXR1cm4gcnVudGltZV9jb25maWcKICAgICAgICBlbHNlOgogICAgICAgICAgICAjIExlZ2FjeSBmb3JtYXQgLSByZXR1cm4gYXMtaXMKICAgICAgICAgICAgcmV0dXJuIGNvbmZpZwoKZGVmIGNvbm5lY3Rpb24oKSAtPiBzbm93Zmxha2UuY29ubmVjdG9yLlNub3dmbGFrZUNvbm5lY3Rpb246CiAgICBpZiBpc19sb2NhbCgpOgogICAgICAgIHNlc3Npb24gPSByZWFkX2xvY2FsX2NyZWRlbnRpYWxzKCkKICAgICAgICBjcmVkcyA9IHsKICAgICAgICAgICAgJ2FjY291bnQnOiBzZXNzaW9uWydhY2NvdW50J10sCiAgICAgICAgICAgICd1c2VyJzogc2Vzc2lvblsndXNlciddLAogICAgICAgICAgICAnd2FyZWhvdXNlJzogc2Vzc2lvblsnd2FyZWhvdXNlJ10sCiAgICAgICAgICAgICdkYXRhYmFzZSc6IHNlc3Npb25bJ2RhdGFiYXNlJ10sCiAgICAgICAgICAgICdzY2hlbWEnOiBzZXNzaW9uWydzY2hlbWEnXSwKICAgICAgICAgICAgJ3JvbGUnOiBzZXNzaW9uWydyb2xlJ10sCiAgICAgICAgICAgICdjbGllbnRfc2Vzc2lvbl9rZWVwX2FsaXZlJzogVHJ1ZQogICAgICAgIH0KICAgICAgICAKICAgICAgICAjIEhhbmRsZSBkaWZmZXJlbnQgYXV0aGVudGljYXRpb24gbWV0aG9kcwogICAgICAgIGlmICdwcml2YXRlX2tleV9wYXRoJyBpbiBzZXNzaW9uOgogICAgICAgICAgICAjIFByaXZhdGUga2V5IGF1dGhlbnRpY2F0aW9uCiAgICAgICAgICAgIGltcG9ydCBvcwogICAgICAgICAgICBmcm9tIGNyeXB0b2dyYXBoeS5oYXptYXQucHJpbWl0aXZlcyBpbXBvcnQgc2VyaWFsaXphdGlvbgogICAgICAgICAgICAKICAgICAgICAgICAgcHJpdmF0ZV9rZXlfcGF0aCA9IHNlc3Npb25bJ3ByaXZhdGVfa2V5X3BhdGgnXQogICAgICAgICAgICBpZiBvcy5wYXRoLmV4aXN0cyhwcml2YXRlX2tleV9wYXRoKToKICAgICAgICAgICAgICAgICMgZ2V0IHBhc3NwaHJhc2UgZnJvbSBlbnZpcm9ubWVudCB2YXJpYWJsZSBvciBjb25maWcKICAgICAgICAgICAgICAgIHBhc3NwaHJhc2UgPSAoCiAgICAgICAgICAgICAgICAgICAgc2Vzc2lvbi5nZXQoInByaXZhdGVfa2V5X3Bhc3NwaHJhc2UiKQogICAgICAgICAgICAgICAgICAgIGlmIHNlc3Npb24uZ2V0KCJwcml2YXRlX2tleV9wYXNzcGhyYXNlIikKICAgICAgICAgICAgICAgICAgICBlbHNlIG9zLmdldGVudigiU05PV1NRTF9QUklWQVRFX0tFWV9QQVNTUEhSQVNFIiwgTm9uZSkKICAgICAgICAgICAgICAgICkKICAgICAgICAgICAgICAgIHdpdGggb3Blbihwcml2YXRlX2tleV9wYXRoLCAicmIiKSBhcyBrZXlfZmlsZToKICAgICAgICAgICAgICAgICAgICBwcml2YXRlX2tleSA9IHNlcmlhbGl6YXRpb24ubG9hZF9wZW1fcHJpdmF0ZV9rZXkoCiAgICAgICAgICAgICAgICAgICAgICAgIGtleV9maWxlLnJlYWQoKSwKICAgICAgICAgICAgICAgICAgICAgICAgcGFzc3dvcmQ9cGFzc3BocmFzZS5lbmNvZGUoKSBpZiBwYXNzcGhyYXNlIGVsc2UgTm9uZQogICAgICAgICAgICAgICAgICAgICkKICAgICAgICAgICAgICAgIGNyZWRzWydwcml2YXRlX2tleSddID0gcHJpdmF0ZV9rZXkKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHJhaXNlIEZpbGVOb3RGb3VuZEVycm9yKGYiUHJpdmF0ZSBrZXkgZmlsZSBub3QgZm91bmQ6IHtwcml2YXRlX2tleV9wYXRofSIpCiAgICAgICAgZWxpZiAncGFzc3dvcmQnIGluIHNlc3Npb246CiAgICAgICAgICAgICMgUGFzc3dvcmQgYXV0aGVudGljYXRpb24KICAgICAgICAgICAgY3JlZHNbJ3Bhc3N3b3JkJ10gPSBzZXNzaW9uWydwYXNzd29yZCddCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiRWl0aGVyICdwcml2YXRlX2tleV9wYXRoJyBvciAncGFzc3dvcmQnIG11c3QgYmUgcHJvdmlkZWQgaW4gY29uZmlndXJhdGlvbiIpCiAgICBlbGlmIGlzX2RvY2tlcigpOgogICAgICAgIGNyZWRzID0gewogICAgICAgICAgICAnYWNjb3VudCc6IG9zLmdldGVudignU05PV0ZMQUtFX0FDQ09VTlQnKSwKICAgICAgICAgICAgJ3VzZXInOiBvcy5nZXRlbnYoJ1NOT1dGTEFLRV9VU0VSJyksCiAgICAgICAgICAgICdwYXNzd29yZCc6IG9zLmdldGVudignU05PV0ZMQUtFX1BBU1NXT1JEJyksCiAgICAgICAgICAgICd3YXJlaG91c2UnOiBvcy5nZXRlbnYoJ1NOT1dGTEFLRV9XQVJFSE9VU0UnKSwKICAgICAgICAgICAgJ2RhdGFiYXNlJzogb3MuZ2V0ZW52KCdTTk9XRkxBS0VfREFUQUJBU0UnKSwKICAgICAgICAgICAgJ3NjaGVtYSc6IG9zLmdldGVudignU05PV0ZMQUtFX1NDSEVNQScpLAogICAgICAgICAgICAncm9sZSc6IG9zLmdldGVudignU05PV0ZMQUtFX1JPTEUnKSwKICAgICAgICAgICAgJ2NsaWVudF9zZXNzaW9uX2tlZXBfYWxpdmUnOiBUcnVlCiAgICAgICAgfQogICAgZWxzZToKICAgICAgICBjcmVkcyA9IHsKICAgICAgICAgICAgJ3Byb3RvY29sJzogImh0dHBzIiwKICAgICAgICAgICAgJ2F1dGhlbnRpY2F0b3InOiAib2F1dGgiLAogICAgICAgICAgICAnaG9zdCc6IG9zLmdldGVudignU05PV0ZMQUtFX0hPU1QnKSwKICAgICAgICAgICAgJ3BvcnQnOiBvcy5nZXRlbnYoJ1NOT1dGTEFLRV9QT1JUJyksCiAgICAgICAgICAgICdhY2NvdW50Jzogb3MuZ2V0ZW52KCdTTk9XRkxBS0VfQUNDT1VOVCcpLAogICAgICAgICAgICAnd2FyZWhvdXNlJzogb3MuZ2V0ZW52KCdTTk9XRkxBS0VfV0FSRUhPVVNFJyksCiAgICAgICAgICAgICdkYXRhYmFzZSc6IG9zLmdldGVudignU05PV0ZMQUtFX0RBVEFCQVNFJyksCiAgICAgICAgICAgICdzY2hlbWEnOiBvcy5nZXRlbnYoJ1NOT1dGTEFLRV9TQ0hFTUEnKSwKICAgICAgICAgICAgJ3Rva2VuJzogZ2V0X2xvZ2luX3Rva2VuKCksCiAgICAgICAgICAgICdjbGllbnRfc2Vzc2lvbl9rZWVwX2FsaXZlJzogVHJ1ZQogICAgICAgIH0KCiAgICBzbm93Zmxha2UuY29ubmVjdG9yLnBhcmFtc3R5bGU9J251bWVyaWMnCiAgICBjb25uZWN0aW9uID0gc25vd2ZsYWtlLmNvbm5lY3Rvci5jb25uZWN0KCoqY3JlZHMpCiAgICByZXR1cm4gY29ubmVjdGlvbgoKZGVmIHNlc3Npb24oKSAtPiBTZXNzaW9uOgogICAgdHJ5OgogICAgICAgIHJldHVybiBnZXRfYWN0aXZlX3Nlc3Npb24oKQogICAgZXhjZXB0IEV4Y2VwdGlvbjoKICAgICAgICByZXR1cm4gU2Vzc2lvbi5idWlsZGVyLmNvbmZpZ3MoeyJjb25uZWN0aW9uIjogY29ubmVjdGlvbigpfSkuZ2V0T3JDcmVhdGUoKQoKZGVmIGdldF9sb2NhbF9jb25uX2ZpbGVfYWJzcGF0aCgpOgogICAgIyBUcnkgdW5pZmllZCBjb25maWcgZmlyc3QgKGluIHByb2plY3Qgcm9vdCkKICAgIHVuaWZpZWRfY29uZmlnX3BhdGggPSAnLi4vLi4vY29ubmVjdGlvbl9jb25maWcuanNvbicKICAgIHVuaWZpZWRfY29uZmlnX2Fic3BhdGggPSBvcy5wYXRoLmFic3BhdGgob3MucGF0aC5qb2luKG9zLnBhdGguZGlybmFtZShfX2ZpbGVfXyksIHVuaWZpZWRfY29uZmlnX3BhdGgpKQogICAgCiAgICBpZiBvcy5wYXRoLmlzZmlsZSh1bmlmaWVkX2NvbmZpZ19hYnNwYXRoKToKICAgICAgICByZXR1cm4gdW5pZmllZF9jb25maWdfYWJzcGF0aAogICAgCiAgICAjIEZhbGwgYmFjayB0byBsZWdhY3kgY29ubmVjdGlvbl9jb25maWcuanNvbiBpbiBzcmMvCiAgICBsZWdhY3lfY29uZmlnX3BhdGggPSAnLi4vY29ubmVjdGlvbl9jb25maWcuanNvbicKICAgIHJldHVybiBvcy5wYXRoLmFic3BhdGgob3MucGF0aC5qb2luKG9zLnBhdGguZGlybmFtZShfX2ZpbGVfXyksIGxlZ2FjeV9jb25maWdfcGF0aCkpCgojIFJldHVybnMgaWYgdGhlIGN1cnJlbnQgc2Vzc2lvbiBpcyBydW5uaW5nIGluIGxvY2FsIG1hY2hpbmUKZGVmIGlzX2xvY2FsKCkgLT4gYm9vbDoKICAgIGNvbm5fZmlsZV9hYnNwYXRoID0gZ2V0X2xvY2FsX2Nvbm5fZmlsZV9hYnNwYXRoKCkKICAgIHJldHVybiBvcy5wYXRoLmlzZmlsZShjb25uX2ZpbGVfYWJzcGF0aCkKCmRlZiBpc190b2tlbigpIC0+IGJvb2w6CiAgICByZXR1cm4gb3MucGF0aC5pc2ZpbGUoIi9zbm93Zmxha2Uvc2Vzc2lvbi90b2tlbiIpCgpkZWYgaXNfZG9ja2VyKCkgLT4gYm9vbDoKICAgIHJldHVybiBvcy5nZXRlbnYoJ0NSRURFTlRJQUxTX0RPQ0tFUicpIGlzIG5vdCBOb25lCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/util/snowflakeObject.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IGpzb24KZnJvbSB0eXBpbmcgaW1wb3J0IE9wdGlvbmFsCmZyb20gdXRpbC5zbm93Zmxha2VPYmplY3RUeXBlIGltcG9ydCBTbm93Zmxha2VPYmplY3RUeXBlCmZyb20gc25vd2ZsYWtlLnNub3dwYXJrLnNlc3Npb24gaW1wb3J0IFNlc3Npb24KCgpjbGFzcyBTbm93Zmxha2VPYmplY3QoKToKICAgIGRlZiBfX2luaXRfXygKICAgICAgICBzZWxmLAogICAgICAgIHR5cGU6IFNub3dmbGFrZU9iamVjdFR5cGUsCiAgICAgICAgbmFtZTogT3B0aW9uYWxbc3RyXSwKICAgICAgICBzY2hlbWE6IE9wdGlvbmFsW3N0cl0gPSBOb25lLAogICAgICAgIGRhdGFiYXNlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwKICAgICAgICBjYXNlX2luc2Vuc2l0aXZlOiBib29sID0gVHJ1ZQogICAgKToKICAgICAgICBzZWxmLnR5cGUgPSB0eXBlCiAgICAgICAgc2VsZi5uYW1lID0gbmFtZQogICAgICAgIHNlbGYuc2NoZW1hID0gc2NoZW1hCiAgICAgICAgc2VsZi5kYXRhYmFzZSA9IGRhdGFiYXNlCiAgICAgICAgc2VsZi5jYXNlX2luc2Vuc2l0aXZlOiBib29sID0gY2FzZV9pbnNlbnNpdGl2ZQoKICAgIGRlZiBnZXRfdHlwZV9uYW1lKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gc2VsZi50eXBlLnZhbHVlLmNhcGl0YWxpemUoKQogICAgCiAgICBkZWYgdG9fZGljdChzZWxmKSAtPiBkaWN0OgogICAgICAgIHJldHVybiB7CiAgICAgICAgICAgICJuYW1lIjogc2VsZi5uYW1lLAogICAgICAgICAgICAic2NoZW1hIjogc2VsZi5zY2hlbWEsCiAgICAgICAgICAgICJkYXRhYmFzZSI6IHNlbGYuZGF0YWJhc2UsCiAgICAgICAgICAgICJ0eXBlIjogc2VsZi50eXBlCiAgICAgICAgfQogICAgCiAgICBkZWYgY3JlYXRlX2lmX25vdF9leGlzdChzZWxmLCBzZXNzaW9uOiBTZXNzaW9uKSAtPiBib29sOgogICAgICAgIGNtZCA9IGYiQ1JFQVRFIHtzZWxmLnR5cGV9IElGIE5PVCBFWElTVFMge3NlbGYuZ2V0X2ZxbigpfSIKICAgICAgICB0cnk6CiAgICAgICAgICAgIHNlc3Npb24uc3FsKGNtZCkuY29sbGVjdCgpCiAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICByZXR1cm4gRmFsc2UKICAgICAgICByZXR1cm4gVHJ1ZQoKICAgIGRlZiBjaGVja19pZl9leGlzdHMoc2VsZiwgc2Vzc2lvbjogU2Vzc2lvbikgLT4gYm9vbDoKICAgICAgICBjbWQgPSBmIkRFU0Mge3NlbGYudHlwZX0ge3NlbGYuZ2V0X2ZxbigpfSIKICAgICAgICB0cnk6CiAgICAgICAgICAgIHNlc3Npb24uc3FsKGNtZCkuY29sbGVjdCgpCiAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICByZXR1cm4gRmFsc2UKICAgICAgICByZXR1cm4gVHJ1ZQogICAgICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCB2YWx1ZTogb2JqZWN0KSAtPiBib29sOgogICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIFNub3dmbGFrZU9iamVjdCk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLm5hbWUgPT0gdmFsdWUubmFtZSBhbmQgc2VsZi5zY2hlbWEgPT0gdmFsdWUuc2NoZW1hIGFuZCBzZWxmLmRhdGFiYXNlID09IHZhbHVlLmRhdGFiYXNlIGFuZCBzZWxmLnR5cGUgPT0gdmFsdWUudHlwZQogICAgICAgIHJldHVybiBGYWxzZQoKICAgIGRlZiBnZXRfaGFzaF9rZXkoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBmIntzZWxmLmdldFR5cGVOYW1lKCl9X3tzZWxmLmRhdGFiYXNlfV97c2VsZi5zY2hlbWF9X3tzZWxmLm5hbWV9IgogICAgCiAgICAKICAgIGRlZiBnZXRfZnFuKHNlbGYpIC0+IHN0cjoKICAgICAgICBmcm9tIGNvbGxlY3Rpb25zIGltcG9ydCBkZXF1ZQogICAgICAgIGZxbl9kZXF1ZSA9IGRlcXVlKCkKICAgICAgICBpZiBzZWxmLm5hbWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGZxbl9kZXF1ZS5hcHBlbmQoc2VsZi5uYW1lKQogICAgICAgICAgICBpZiBzZWxmLnNjaGVtYSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIGZxbl9kZXF1ZS5hcHBlbmRsZWZ0KHNlbGYuc2NoZW1hKQogICAgICAgICAgICAgICAgaWYgc2VsZi5kYXRhYmFzZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBmcW5fZGVxdWUuYXBwZW5kbGVmdChzZWxmLmRhdGFiYXNlKQoKICAgICAgICAjIE5vIG5lZWQgdG8gcXVvdGUgdGhlIGlkZW50aWZpZXIgaWYgaXQgaXMgY2FzZSBpbnNlbnNpdGl2ZS4KICAgICAgICBpZiBzZWxmLmNhc2VfaW5zZW5zaXRpdmU6CiAgICAgICAgICAgIHJldHVybiAiLiIuam9pbihmcW5fZGVxdWUpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuICIuIi5qb2luKGdldFF1b3RlZElkZW50aWZpZXIoaSkgZm9yIGkgaW4gZnFuX2RlcXVlKQoKICAgIGRlZiBoYXNfZGJfc2NoZW1hX25hbWUoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gKHNlbGYuZGF0YWJhc2UgaXMgbm90IE5vbmUgYW5kIGxlbihzZWxmLmRhdGFiYXNlKSA+IDAgYW5kIAogICAgICAgICAgICAgICAgc2VsZi5zY2hlbWEgaXMgbm90IE5vbmUgYW5kIGxlbihzZWxmLnNjaGVtYSkgPiAwIGFuZCAKICAgICAgICAgICAgICAgIHNlbGYubmFtZSBpcyBub3QgTm9uZSBhbmQgbGVuKHNlbGYubmFtZSkgPiAwKQogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYuZ2V0X3R5cGVfbmFtZSgpfToge3NlbGYuZ2V0X2ZxbigpfSIKCgpkZWYgZ2V0UXVvdGVkSWRlbnRpZmllcihpZGVudGlmaWVyOiBzdHIpIC0+IHN0cjoKICAgICIiIgogICAgUXVvdGUgU25vd2ZsYWtlIGlkZW50aWZpZXJzIHdoZW4gbmVjZXNzYXJ5LgogICAgCiAgICBUaGlzIGZ1bmN0aW9uIHF1b3RlcyBpZGVudGlmaWVycyBmb3IgU25vd2ZsYWtlIE9iamVjdCBOYW1lcyBiYXNlZCBvbiBTUUwgc3RhbmRhcmRzLgogICAgSWRlbnRpZmllcnMgbmVlZCBxdW90aW5nIGlmIHRoZXk6CiAgICAtIERvbid0IHN0YXJ0IHdpdGggYSBsZXR0ZXIgb3IgdW5kZXJzY29yZQogICAgLSBDb250YWluIGludmFsaWQgY2hhcmFjdGVycyAobm90IEEtWiwgYS16LCAwLTksIF8sICQpCiAgICAtIEhhdmUgbWl4ZWQgY2FzZSAobm90IGFsbCB1cHBlcmNhc2UpCiAgICAKICAgIEFyZ3M6CiAgICAgICAgaWRlbnRpZmllcjogVGhlIGlkZW50aWZpZXIgdG8gcG90ZW50aWFsbHkgcXVvdGUKICAgICAgICAKICAgIFJldHVybnM6CiAgICAgICAgVGhlIGlkZW50aWZpZXIsIHF1b3RlZCBpZiBuZWNlc3NhcnkKICAgICIiIgogICAgaWYgbm90IGlkZW50aWZpZXI6CiAgICAgICAgcmV0dXJuICIiCgogICAgaW1wb3J0IHJlCgogICAgIyBDaGVjayBpZiBpZGVudGlmaWVyIG5lZWRzIHF1b3RpbmcgYmFzZWQgb24gU1FMIHN0YW5kYXJkcwogICAgcmVxX3F1b3RlcyA9IEZhbHNlCgogICAgIyBWYWxpZCBTbm93Zmxha2UgT2JqZWN0IE5hbWVzIG5lZWQgdG8gc3RhcnQgd2l0aCBhIGxldHRlciBvciB1bmRlcnNjb3JlLgogICAgIyBDaGVjayBpZiBpZGVudGlmaWVyIHN0YXJ0cyB3aXRoIGludmFsaWQgY2hhcmFjdGVyCiAgICBmaXJzdF9jaGFyID0gaWRlbnRpZmllclswXQoKICAgICMgVmFsaWQgQ2hhcmFjdGVycyBpbiBTbm93Zmxha2UgT2JqZWN0IE5hbWVzOiBsZXR0ZXJzIChBLVosIGEteiksIHVuZGVyc2NvcmUgKF8pLCBkaWdpdHMgKDAtOSksIGRvbGxhciBzaWduICgkKQogICAgcGF0dGVybiA9IHInW15hLXpBLVowLTlfJF0nCgogICAgaWYgaWRlbnRpZmllci5yZXBsYWNlKCciJywgIiIpICE9ICIqIiBhbmQgKCAgIyAnKicgYnkgaXRzZWxmIGlzIGEgc3BlY2lhbCBjYXNlLCBlLmcuIFQxLiosIGl0IHNob3VsZCBub3QgYmUgcXVvdGVkCiAgICAgICAgbm90IChmaXJzdF9jaGFyLmlzYWxwaGEoKSBvciBmaXJzdF9jaGFyID09ICJfIikKICAgICAgICBvciByZS5zZWFyY2gocGF0dGVybiwgaWRlbnRpZmllcikKICAgICAgICBvciBpZGVudGlmaWVyICE9IGlkZW50aWZpZXIudXBwZXIoKSAgIyBNaXggQ2FzZSBOYW1lIHJlcXVpcmVzIFF1b3RpbmcKICAgICk6CiAgICAgICAgcmVxX3F1b3RlcyA9IFRydWUKCiAgICBpZiByZXFfcXVvdGVzOgogICAgICAgIGlkZW50aWZpZXIgPSBmJyJ7aWRlbnRpZmllcn0iJwoKICAgIHJldHVybiBpZGVudGlmaWVyCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/util/snowflakeObjectType.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBlbnVtIGltcG9ydCBFbnVtCgpjbGFzcyBTbm93Zmxha2VPYmplY3RUeXBlKEVudW0pOgogICAgVEFCTEUgPSAidGFibGUiCiAgICBWSUVXID0gInZpZXciCiAgICBTVFJFQU0gPSAic3RyZWFtIgogICAgU1RBR0UgPSAic3RhZ2UiCiAgICBNQVRFUklBTElaRURfVklFVyA9ICJtYXRlcmlhbGl6ZWQgdmlldyIKCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlOiBvYmplY3QpIC0+IGJvb2w6CiAgICAgICAgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgU25vd2ZsYWtlT2JqZWN0VHlwZSk6CiAgICAgICAgICAgIHJldHVybiB2YWx1ZS52YWx1ZSA9PSBzZWxmLnZhbHVlCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIHZhbHVlIGlzIG5vdCBOb25lIGFuZCBzdHIodmFsdWUpLmxvd2VyKCkgPT0gc2VsZi52YWx1ZQogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKSAtPiBzdHI6CiAgICAgICAgcmV0dXJuIHNlbGYudmFsdWU='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/semantic_view_generator.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/app.py FROM (SELECT BASE64_DECODE_STRING('aW1wb3J0IHN0cmVhbWxpdCBhcyBzdApzdC5zZXRfcGFnZV9jb25maWcobGF5b3V0PSJ3aWRlIikKIyAgICAgICAgIHBhZ2VfdGl0bGU9IlN0ZWxsYXIgQkkiLAojICAgICAgICAgcGFnZV9pY29uPSLinYTvuI8iLAojICAgICAgICAgbGF5b3V0PSJ3aWRlIiwKIyAgICAgICAgIG1lbnVfaXRlbXM9ewojICAgICAgICAgICAgICJHZXQgSGVscCI6ICJodHRwczovL3F1aWNrc3RhcnRzLnNub3dmbGFrZS5jb20vIiwgICMgVE9ETyBSRVBMQUNFIEZPUiB0aGUgYWN0dWFsIGRvY3VtZW50YXRpb24gbGluawojICAgICAgICAgICAgICJSZXBvcnQgYSBidWciOiAiaHR0cHM6Ly9jb21tdW5pdHkuc25vd2ZsYWtlLmNvbS9zL2ZvcnVtIiwKIyAgICAgICAgICAgICAiQWJvdXQiOiAiIiIjIyBCSSBUb29sLgojICAgICAgICAgIFZlcnNpb24gMS4wLjAKIyAgICBTb2x1dGlvbiBJbm5vdmF0aW9uIFRlYW0KIyAgICAgICAgICAgICAiIiIsCiMgICAgICAgICB9LAojICAgICApCgpmcm9tIHR5cGluZyBpbXBvcnQgT3B0aW9uYWwKZnJvbSBzbm93Zmxha2Uuc25vd3BhcmsgaW1wb3J0IFNlc3Npb24KZnJvbSB1dGlsLnNlc3Npb25fdXRpbHMgaW1wb3J0IGNvbm5lY3RfdG9fc25vd2ZsYWtlCmZyb20gdWkuc3RhcnRQYWdlIGltcG9ydCBzdGFydFBhZ2UKZnJvbSBiaS5jb3JlLnJ1bnRpbWUgaW1wb3J0IFJ1bnRpbWUKZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYRXhwcmVzc2lvblBhcnNlciBpbXBvcnQgREFYRXhwcmVzc2lvblBhcnNlcgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb25QYXJzZXIgaW1wb3J0IE1FeHByZXNzaW9uUGFyc2VyCmZyb20gYmkuc3FsLlNRTEV4cHJlc3Npb25QYXJzZXIgaW1wb3J0IFNRTEV4cHJlc3Npb25QYXJzZXIKCnNlc3Npb24gPSBjb25uZWN0X3RvX3Nub3dmbGFrZSgpCnJ1bnRpbWUgPSBSdW50aW1lKCkKcnVudGltZS5Mb2FkRXhwcmVzc2lvblBhcnNlcihEQVhFeHByZXNzaW9uUGFyc2VyKCkpCnJ1bnRpbWUuTG9hZEV4cHJlc3Npb25QYXJzZXIoTUV4cHJlc3Npb25QYXJzZXIoKSkKcnVudGltZS5Mb2FkRXhwcmVzc2lvblBhcnNlcihTUUxFeHByZXNzaW9uUGFyc2VyKCkpCgpzdGFydFBhZ2UgPSBzdGFydFBhZ2UocnVudGltZT1ydW50aW1lLCBzZXNzaW9uPXNlc3Npb24pCgpzdGFydFBhZ2UuUnVuKCkKI25hdiA9ICBzdC5uYXZpZ2F0aW9uKHN0YXJ0UGFnZS5HZW5lcmF0ZU1lbnUoKSwgZXhwYW5kZWQ9VHJ1ZSkKI25hdi5ydW4oKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/tableItem.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSAuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQsIEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIC5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCgpjbGFzcyBUYWJsZUl0ZW0oRXhwcmVzc2lvblBhcmVudCk6CgogICAgZGVmIF9faW5pdF9fKHNlbGYsIAogICAgICAgICAgICAgICAgIG5hbWU6IHN0ciwgCiAgICAgICAgICAgICAgICAgdHlwZTogRXhwcmVzc2lvblR5cGUgPSBFeHByZXNzaW9uVHlwZS5BbnksIAogICAgICAgICAgICAgICAgIGV4cHJlc3Npb246IE9wdGlvbmFsW1VuaW9uW3N0cnxFeHByZXNzaW9uXV0gPSBOb25lLCAKICAgICAgICAgICAgICAgICBleHByZXNzaW9uQ29kZVR5cGU6IE9wdGlvbmFsW0V4cHJlc3Npb25Db2RlVHlwZV0gPSBFeHByZXNzaW9uQ29kZVR5cGUuU1FMLAogICAgICAgICAgICAgICAgIHRhYmxlQWxpYXM6IE9wdGlvbmFsW3N0cl0gPSBOb25lLAogICAgICAgICAgICAgICAgIHRhYmxlID0gTm9uZSwKICAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzOiBkaWN0ID0ge30sCiAgICAgICAgICAgICAgICAgKToKICAgICAgICBmcm9tIC50YWJsZSBpbXBvcnQgVGFibGUKICAgICAgICBmcm9tIC5yZWxhdGlvbnNoaXAgaW1wb3J0IFJlbGF0aW9uc2hpcAogICAgICAgIEV4cHJlc3Npb25QYXJlbnQuX19pbml0X18oc2VsZikKICAgICAgICBpZiBuYW1lIGlzIG5vdCBOb25lOgogICAgICAgICAgICBuYW1lID0gbmFtZS5zdHJpcCgiXCIiKQogICAgICAgIHNlbGYuX190YWJsZTogVGFibGUgPSB0YWJsZQogICAgICAgIHNlbGYuX19uYW1lOiBzdHIgPSBuYW1lCiAgICAgICAgc2VsZi5fX3RhYmxlQWxpYXM6IE9wdGlvbmFsW3N0cl0gPSB0YWJsZUFsaWFzCiAgICAgICAgc2VsZi5fX3R5cGU6IEV4cHJlc3Npb25UeXBlID0gdHlwZQogICAgICAgIHNlbGYuX19leHByZXNzaW9uOiBFeHByZXNzaW9uID0gTm9uZQogICAgICAgIHNlbGYuX19leHByZXNzaW9uU3RyOiBzdHIgPSBOb25lCiAgICAgICAgc2VsZi5fX2NoaWxkRXhwcmVzc2lvbkNvZGVUeXBlOiBFeHByZXNzaW9uQ29kZVR5cGUgPSBleHByZXNzaW9uQ29kZVR5cGUKCiAgICAgICAgaWYgaXNpbnN0YW5jZShleHByZXNzaW9uLCBsaXN0KSBhbmQgYWxsKGlzaW5zdGFuY2UobCwgc3RyKSBmb3IgbCBpbiBleHByZXNzaW9uKToKICAgICAgICAgICAgZSA9ICIiCiAgICAgICAgICAgIGZvciBsIGluIGV4cHJlc3Npb246CiAgICAgICAgICAgICAgICBlICs9IGYie2x9XG4iCiAgICAgICAgICAgIGV4cHJlc3Npb24gPSBlCgogICAgICAgIGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbiwgc3RyKToKICAgICAgICAgICAgc2VsZi5fX2V4cHJlc3Npb25TdHIgPSBleHByZXNzaW9uCiAgICAgICAgICAgIHNlbGYuX19leHByZXNzaW9uID0gc2VsZi51cGRhdGVFeHByZXNzaW9uKCkKICAgICAgICAgICAgaWYgc2VsZi5fX2V4cHJlc3Npb24gaXMgTm9uZToKICAgICAgICAgICAgICAgIHByaW50KGYiRXJyb3IgUGFyc2luZzoge2V4cHJlc3Npb259IikKCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKGV4cHJlc3Npb24sIEV4cHJlc3Npb24pOgogICAgICAgICAgICBzZWxmLl9fZXhwcmVzc2lvbiA9IGV4cHJlc3Npb24KICAgICAgICAgICAgc2VsZi5fX2V4cHJlc3Npb25TdHIgPSBleHByZXNzaW9uLnNvdXJjZVN0cmluZyAgICAKICAgICAgICAgICAgc2VsZi5fX2NoaWxkRXhwcmVzc2lvbkNvZGVUeXBlID0gZXhwcmVzc2lvbi5jb2RlVHlwZQoKICAgICAgICBzZWxmLl9fcHJvcGVydGllcyA9IHByb3BlcnRpZXMKCiAgICAgICAgc2VsZi5fX3JlbGF0aW9uc2hpcHM6IExpc3RbUmVsYXRpb25zaGlwXSA9IFtdCgogICAgQHByb3BlcnR5CiAgICBkZWYgdGFibGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX190YWJsZQoKICAgIEBwcm9wZXJ0eQogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgaXNDb2x1bW4oc2VsZikgLT4gYm9vbDoKICAgICAgICBwYXNzCgogICAgQHByb3BlcnR5CiAgICBkZWYgdGFibGVBbGlhcyhzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgIHJldHVybiBzZWxmLl9fdGFibGVBbGlhcyAgCgogICAgQHRhYmxlQWxpYXMuc2V0dGVyCiAgICBkZWYgdGFibGVBbGlhcyhzZWxmLCB0YWJsZUFsaWFzOiBzdHIpOgogICAgICAgIGlmIHRhYmxlQWxpYXMgPT0gIiI6CiAgICAgICAgICAgIHRhYmxlQWxpYXMgPSBOb25lCiAgICAgICAgc2VsZi5fX3RhYmxlQWxpYXMgPSB0YWJsZUFsaWFzCgogICAgQHByb3BlcnR5CiAgICBkZWYgY2hpbGRFeHByZXNzaW9uQ29kZVR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19jaGlsZEV4cHJlc3Npb25Db2RlVHlwZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIE1vZGVsKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLnRhYmxlLk1vZGVsCgogICAgQHByb3BlcnR5CiAgICBkZWYgcHJvcGVydGllcyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3Byb3BlcnRpZXMKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgbmFtZShzZWxmKSAtPiBzdHI6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IGdldFF1b3RlZElkZW50aWZpZXIKICAgICAgICByZXR1cm4gZ2V0UXVvdGVkSWRlbnRpZmllcihzZWxmLl9fbmFtZSkKCiAgICBAbmFtZS5zZXR0ZXIKICAgIGRlZiBuYW1lKHNlbGYsIG5hbWU6IHN0cik6CiAgICAgICAgaWYgbmFtZSBpcyBub3QgTm9uZSBhbmQgbmFtZSAhPSAiIjoKICAgICAgICAgICAgc2VsZi5fX25hbWUgPSBuYW1lCgogICAgQHByb3BlcnR5CiAgICBkZWYgSGFzSXNzdWVzKHNlbGYpOgogICAgICAgIHJldHVybiBzdXBlcigpLkhhc0lzc3VlcyBvciAoc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lIGFuZCBzZWxmLmV4cHJlc3Npb24uSGFzSXNzdWVzKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX190eXBlCgogICAgQHByb3BlcnR5CiAgICBkZWYgZnVsbE5hbWUoc2VsZik6CiAgICAgICAgdGFibGUgPSBOb25lCiAgICAgICAgaWYgc2VsZi50YWJsZUFsaWFzIGlzIG5vdCBOb25lOgogICAgICAgICAgICB0YWJsZSA9IHNlbGYudGFibGVBbGlhcwogICAgICAgIGVsaWYgc2VsZi50YWJsZSBpcyBub3QgTm9uZSBhbmQgaGFzYXR0cihzZWxmLnRhYmxlLCAibmFtZSIpOgogICAgICAgICAgICB0YWJsZSA9IHNlbGYudGFibGUubmFtZQoKICAgICAgICBpZiB0YWJsZSBpcyBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5uYW1lCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIGYie3RhYmxlfS57c2VsZi5uYW1lfSIgICAgCgogICAgQHR5cGUuc2V0dGVyCiAgICBkZWYgdHlwZShzZWxmLCB0eXBlOiBFeHByZXNzaW9uVHlwZSk6CiAgICAgICAgaWYgdHlwZSAhPSBzZWxmLl9fdHlwZToKICAgICAgICAgICAgc2VsZi5fX3R5cGUgPSB0eXBlCgogICAgZGVmIEdldEFsbElzc3VlcyhzZWxmKToKICAgICAgICBpc3N1ZXMgPSBzdXBlcigpLkdldEFsbElzc3VlcygpCgogICAgICAgIGlmIHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgaXNzdWVzID0gaXNzdWVzLnVuaW9uKHNlbGYuZXhwcmVzc2lvbi5HZXRBbGxJc3N1ZXMoKSkKICAgICAgICByZXR1cm4gaXNzdWVzCgogICAgQHByb3BlcnR5CiAgICBkZWYgZXhwcmVzc2lvbihzZWxmKToKICAgICAgICBpZiBzZWxmLl9fZXhwcmVzc2lvbiBpcyBOb25lIGFuZCBzZWxmLl9fZXhwcmVzc2lvblN0ciBpcyBub3QgTm9uZToKICAgICAgICAgICAgc2VsZi5fX2V4cHJlc3Npb24gPSBzZWxmLnVwZGF0ZUV4cHJlc3Npb24oKQogICAgICAgIGVsaWYgc2VsZi5fX2V4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHNlbGYuX19leHByZXNzaW9uLlVwZGF0ZSgpCiAgICAgICAgcmV0dXJuIHNlbGYuX19leHByZXNzaW9uCgogICAgQHByb3BlcnR5CiAgICBkZWYgZXhwcmVzc2lvblN0cmluZyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2V4cHJlc3Npb25TdHIKCiAgICBAcHJvcGVydHkKICAgIGRlZiByZWxhdGlvbnNoaXBzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fcmVsYXRpb25zaGlwcwoKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIG5hbWUpOgogICAgICAgIHJldHVybiBOb25lCgogICAgZGVmIHNldFRhYmxlKHNlbGYsIHRhYmxlKToKICAgICAgICBmcm9tIC50YWJsZSBpbXBvcnQgVGFibGUKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZSk6CiAgICAgICAgICAgIGlmIHRhYmxlICE9IHNlbGYuX190YWJsZToKICAgICAgICAgICAgICAgIHNlbGYuX190YWJsZSA9IHRhYmxlCiAgICAgICAgICAgICAgICBzZWxmLnVwZGF0ZUV4cHJlc3Npb24oKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoInRhYmxlIG11c3QgYmUgVGFibGUgdHlwZSIpCgogICAgZGVmIGFkZFJlbGF0aW9uc2hpcChzZWxmLCByZWxhdGlvbnNoaXApOgogICAgICAgIHNlbGYuX19yZWxhdGlvbnNoaXBzLmFwcGVuZChyZWxhdGlvbnNoaXApCgogICAgZGVmIHVwZGF0ZUV4cHJlc3Npb24oc2VsZik6CiAgICAgICAgZXhwcmVzc2lvbjogRXhwcmVzc2lvbiA9IE5vbmUKICAgICAgICBpZiBzZWxmLl9fZXhwcmVzc2lvblN0ciBpcyBub3QgTm9uZSBhbmQgc2VsZi5SdW50aW1lIGlzIG5vdCBOb25lOgogICAgICAgICAgICBpZiBzZWxmLmNoaWxkRXhwcmVzc2lvbkNvZGVUeXBlIGlzIE5vbmU6CiAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICBleHByZXNzaW9uID0gc2VsZi5BZGRFeHByZXNzaW9uKHNlbGYuX19leHByZXNzaW9uU3RyLnN0cmlwKCksIHByb3BlcnRpZXM9e30pCiAgICAgICAgICAgICAgICBpZiBleHByZXNzaW9uIGlzIE5vbmU6CiAgICAgICAgICAgICAgICAgICAgcGFzcwogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHBhc3MKICAgICAgICByZXR1cm4gZXhwcmVzc2lvbgoKICAgIGRlZiBzZXRFeHByZXNzaW9uKHNlbGYsIGV4cHJlc3Npb246IEV4cHJlc3Npb24pOgogICAgICAgIHNlbGYuX19leHByZXNzaW9uID0gZXhwcmVzc2lvbgogICAgICAgIAogICAgI2RlZiBzZXRFeHByZXNzaW9uKHNlbGYsIGV4cHJlc3Npb25TdHI6IHN0ciwgZXhwcmVzc2lvbkNvZGVUeXBlOiBFeHByZXNzaW9uQ29kZVR5cGUpOgogICAgIyAgICBzZWxmLl9fZXhwcmVzc2lvblN0ciA9IGV4cHJlc3Npb25TdHIKICAgICMgICAgc2VsZi5fX2NoaWxkRXhwcmVzc2lvbkNvZGVUeXBlID0gZXhwcmVzc2lvbkNvZGVUeXBlCiAgICAjICAgIHNlbGYudXBkYXRlRXhwcmVzc2lvbigpCgogICAgZGVmIGdldElkZW50aWZpZXIoc2VsZiwgcXVvdGVNaXhlZENhc2U6IGJvb2wgPSBGYWxzZSwgaW5jbHVkZVRhYmxlQWxpYXM6IGJvb2wgPSBGYWxzZSwgaW5jbHVkZVRhYmxlTmFtZTogYm9vbCA9IFRydWUpIC0+IHN0cjoKICAgICAgICBmcm9tIHV0aWwuc25vd2ZsYWtlT2JqZWN0IGltcG9ydCBnZXRRdW90ZWRJZGVudGlmaWVyCgogICAgICAgIGlkZW50aWZpZXIgPSBzZWxmLl9fbmFtZQogICAgICAgIGlmIGlkZW50aWZpZXIgaXMgbm90IE5vbmUgYW5kIHF1b3RlTWl4ZWRDYXNlOgogICAgICAgICAgICBpZGVudGlmaWVyID0gZ2V0UXVvdGVkSWRlbnRpZmllcihpZGVudGlmaWVyKQogICAgICAgIGlmIGlkZW50aWZpZXIgaXMgbm90IE5vbmUgYW5kIGluY2x1ZGVUYWJsZUFsaWFzIGFuZCBzZWxmLnRhYmxlQWxpYXMgaXMgbm90IE5vbmUgYW5kIGxlbihzZWxmLnRhYmxlQWxpYXMpID4gMDoKICAgICAgICAgICAgaWRlbnRpZmllciA9IGYie2dldFF1b3RlZElkZW50aWZpZXIoc2VsZi50YWJsZUFsaWFzKX0ue2lkZW50aWZpZXJ9IgogICAgICAgIGVsaWYgaWRlbnRpZmllciBpcyBub3QgTm9uZSBhbmQgaW5jbHVkZVRhYmxlTmFtZSBhbmQgaGFzYXR0cihzZWxmLnRhYmxlLCAibmFtZSIpOgogICAgICAgICAgICBpZGVudGlmaWVyID0gZiJ7c2VsZi50YWJsZS5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPXF1b3RlTWl4ZWRDYXNlKX0ue2lkZW50aWZpZXJ9IgogICAgICAgIHJldHVybiBpZGVudGlmaWVyCgogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19yZXByX18oKQoKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c2VsZi5uYW1lfSAoe3NlbGYudHlwZX0pIgoKICAgIGRlZiBfX2VxX18oc2VsZiwgdmFsdWUpOgogICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIFRhYmxlSXRlbSk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLm5hbWUgPT0gdmFsdWUubmFtZSBhbmQgc2VsZi50eXBlID09IHZhbHVlLnR5cGUgYW5kIHNlbGYuX19leHByZXNzaW9uID09IHZhbHVlLl9fZXhwcmVzc2lvbgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuX19uYW1lLnVwcGVyKCkgPT0gdmFsdWUudXBwZXIoKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBGYWxzZQoKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaChzZWxmLmZ1bGxOYW1lKQoKI0V4cHJlc3Npb25QYXJlbnQucmVnaXN0ZXIoVGFibGVJdGVtKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/functionEvaluator.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbCwgTGlzdCwgVW5pb24sIFNldApmcm9tIGFiYyBpbXBvcnQgQUJDCmZyb20gLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIC5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlcgpmcm9tIC5leHByZXNzaW9uUGFyZW50IGltcG9ydCBFeHByZXNzaW9uQ29kZVR5cGUKaW1wb3J0IG9zCmltcG9ydCBpbXBvcnRsaWIKaW1wb3J0IGluc3BlY3QKCmRlZiBsb2FkX2NsYXNzZXNfZnJvbV9kaXJlY3RvcnkoZGlyZWN0b3J5X3BhdGgsIG1vZHVsZV9wYXRoKToKICAgIGNsYXNzZXMgPSB7fQogICAgZm9yIGZpbGVuYW1lIGluIG9zLmxpc3RkaXIoZGlyZWN0b3J5X3BhdGgpOgogICAgICAgICNwcmludChmIkxvYWRpbmcgSGFuZGxlciBmcm9tIHtmaWxlbmFtZX0iKQogICAgICAgIGlmIGZpbGVuYW1lLmVuZHN3aXRoKCIucHkiKSBhbmQgZmlsZW5hbWUgIT0gIl9faW5pdF9fLnB5IjoKICAgICAgICAgICAgbW9kdWxlX25hbWUgPSBmaWxlbmFtZVs6LTNdCiAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgIG1vZHVsZSA9IGltcG9ydGxpYi5pbXBvcnRfbW9kdWxlKGYiLnttb2R1bGVfbmFtZX0iLCBtb2R1bGVfcGF0aCkKICAgICAgICAgICAgICAgIGZvciBuYW1lLCBvYmogaW4gaW5zcGVjdC5nZXRtZW1iZXJzKG1vZHVsZSwgaW5zcGVjdC5pc2NsYXNzKToKICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICBpZiBpc3N1YmNsYXNzKG9iaiwgRnVuY3Rpb25IYW5kbGVyKToKICAgICAgICAgICAgICAgICAgICAgICAgaWYgbm90IGhhc2F0dHIob2JqLCAiX19hYnN0cmFjdG1ldGhvZHNfXyIpIG9yIG9iai5fX21vZHVsZV9fLnN0YXJ0c3dpdGgobW9kdWxlX3BhdGgpOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgY2xzID0gb2JqLl9fY2FsbF9fKCkKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNsYXNzZXNbY2xzLm5hbWVdID0gY2xzCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgIGV4Y2VwdCBJbXBvcnRFcnJvciBhcyBlOgogICAgICAgICAgICAgICAgcHJpbnQoZiJFcnJvciBpbXBvcnRpbmcge2ZpbGVuYW1lfToge2V9IikKICAgICAgICAgICAgICAgIGNvbnRpbnVlCiAgICByZXR1cm4gY2xhc3NlcwoKIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMKIyAKIyBDbGFzcyB3aGljaCBwcm92aWRlcyBmdW5jdGlvbiBldmFsdWF0aW9uIGZvciBFeHByZXNzaW9uIEZ1bmN0aW9ucyAoREFYIGFuZCBNL1Bvd2VyIFF1ZXJ5KQojIHRoaXMgY2xhc3MgbG9hZHMgYWxsIG9mIHRoZSBGdW5jdGlvbkhhbmRsZXIgaW1wbGVtZW50YXRpb25zLCB0aGVuIGZvcndhcmRzIHRoZSBFdmFsdWF0ZSBjYWxsIAojIHRvIHRoZSBjb3JyZWN0IG9uZSBmb3IgdGhlIGdpdmVuIGZ1bmN0aW9uCiMKIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMKY2xhc3MgRnVuY3Rpb25FdmFsdWF0b3IoKToKICAgIGZyb20gLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgogICAgICAgIAogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgICMgY2hlY2sgdG8gc2VlIGlmIHdlIGFyZSBydW5uaW5nIGxvY2FsIG9yIHN5cwogICAgICAgIGlmIG9zLmdldGN3ZCgpLnN0YXJ0c3dpdGgoIi90bXAvYXBwUm9vdCIpOgogICAgICAgICAgICByb290UGF0aCA9ICJiaSIKICAgICAgICBlbHNlOgogICAgICAgICAgICByb290UGF0aCA9ICJzcmMvYmkiCiAgICAgICAgICAgIAogICAgICAgIHNlbGYubV9oYW5kbGVycyA9IGxvYWRfY2xhc3Nlc19mcm9tX2RpcmVjdG9yeShmIntyb290UGF0aH0vcmVhZGVycy9wYmkvdG1kbC9mdW5jdGlvbnMiLCAiYmkucmVhZGVycy5wYmkudG1kbC5mdW5jdGlvbnMiKQogICAgICAgIHNlbGYuZGF4X2hhbmRsZXJzID0gbG9hZF9jbGFzc2VzX2Zyb21fZGlyZWN0b3J5KGYie3Jvb3RQYXRofS9yZWFkZXJzL3BiaS9kYXgvZnVuY3Rpb25zIiwgImJpLnJlYWRlcnMucGJpLmRheC5mdW5jdGlvbnMiKQoKICAgIGRlZiBfX2dldEhhbmRsZXIoc2VsZiwgZnVuY3Rpb246IEZ1bmN0aW9uKSAtPiBGdW5jdGlvbkhhbmRsZXI6CiAgICAgICAgZnJvbSAuaXNzdWUgaW1wb3J0IElzc3VlVHlwZQogICAgICAgIGZyb20gLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgogICAgICAgIGhhbmRsZXI6IEZ1bmN0aW9uSGFuZGxlciA9IE5vbmUKCiAgICAgICAgIyBjaGVjayBmb3IgYW5vbiBmdW5jdGlvbiAod2hpY2ggaXMgaXRzIG93biBoYW5kbGVyKQogICAgICAgIGlmIGlzaW5zdGFuY2UoZnVuY3Rpb24sIEZ1bmN0aW9uSGFuZGxlcikgYW5kIChmdW5jdGlvbi5mdW5jdGlvbk5hbWUgaXMgTm9uZSBvciBmdW5jdGlvbi5mdW5jdGlvbk5hbWUgPT0gIiIpOgogICAgICAgICAgICBoYW5kbGVyID0gZnVuY3Rpb24KICAgICAgICBlbHNlOgogICAgICAgICAgICAjIGxvb2sgZm9yIGFuIGFub24gZnVuY3Rpb24gd2l0aCB0aGUgbmFtZQogICAgICAgICAgICBhbm9uRnVuY3Rpb24gPSBmdW5jdGlvbi5leHByZXNzaW9uLmdldEV4cHJlc3Npb24oZnVuY3Rpb24uZnVuY3Rpb25OYW1lKQogICAgICAgICAgICBpZiBhbm9uRnVuY3Rpb24gaXMgbm90IE5vbmUgYW5kIGlzaW5zdGFuY2UoYW5vbkZ1bmN0aW9uLCBGdW5jdGlvbkhhbmRsZXIpOgogICAgICAgICAgICAgICAgaGFuZGxlciA9IGFub25GdW5jdGlvbgogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgbWF0Y2ggZnVuY3Rpb24uY29kZVR5cGU6CiAgICAgICAgICAgICAgICAgICAgY2FzZSBFeHByZXNzaW9uQ29kZVR5cGUuTToKICAgICAgICAgICAgICAgICAgICAgICAgaGFuZGxlciA9IHNlbGYubV9oYW5kbGVycy5nZXQoZnVuY3Rpb24uZnVuY3Rpb25OYW1lKQogICAgICAgICAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLkRBWDoKICAgICAgICAgICAgICAgICAgICAgICAgaGFuZGxlciA9IHNlbGYuZGF4X2hhbmRsZXJzLmdldChmdW5jdGlvbi5mdW5jdGlvbk5hbWUudXBwZXIoKSkKICAgICAgICAgICAgCiAgICAgICAgaWYgaGFuZGxlciBpcyBub3QgTm9uZToKICAgICAgICAgICAgaGFuZGxlci5TZXRGdW5jdGlvbihmdW5jdGlvbikKICAgICAgICBlbHNlOgogICAgICAgICAgICBmdW5jdGlvbi5leHByZXNzaW9uLkFkZElzc3VlKHR5cGU9SXNzdWVUeXBlLk1pc3NpbmdGdW5jdGlvbkhhbmRsZXIsIGRlc2NyaXB0aW9uPWYie2Z1bmN0aW9uLmV4cHJlc3Npb24uY29kZVR5cGUubmFtZX0ge2Z1bmN0aW9uLmZ1bmN0aW9uTmFtZX0iKQoKICAgICAgICByZXR1cm4gaGFuZGxlcgogICAgCiAgICBkZWYgRXZhbHVhdGUoc2VsZiwgZnVuY3Rpb246IEZ1bmN0aW9uKToKICAgICAgICBmcm9tIC5pc3N1ZSBpbXBvcnQgSXNzdWUsIElzc3VlVHlwZQogICAgICAgIGZyb20gLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KICAgICAgICBpZiBmdW5jdGlvbi5leHByZXNzaW9uIGlzIE5vbmU6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkZ1bmN0aW9uIGlzIG5vdCBzdWJjbGFzcyBvZiBFeHByZXNzaW9uIikKICAgICAgICAKICAgICAgICByZXN1bHQ6IEV4cHJlc3Npb24gPSBOb25lCiAgICAgICAgaGFuZGxlciA9IHNlbGYuX19nZXRIYW5kbGVyKGZ1bmN0aW9uPWZ1bmN0aW9uKQoKICAgICAgICBpZiBoYW5kbGVyIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXN1bHQgPSBoYW5kbGVyLkV2YWx1YXRlKCkKICAgICAgICAgICAKICAgICAgICByZXR1cm4gcmVzdWx0CiAgICAKICAgIGRlZiBnZXRSZXR1cm5UeXBlKHNlbGYsIGZ1bmN0aW9uOiBGdW5jdGlvbikgLT4gRXhwcmVzc2lvblR5cGU6CiAgICAgICAgZnJvbSAuaXNzdWUgaW1wb3J0IElzc3VlLCBJc3N1ZVR5cGUKICAgICAgICByZXR1cm5UeXBlID0gRXhwcmVzc2lvblR5cGUuVW5rbm93bgogICAgICAgIGhhbmRsZXIgPSBzZWxmLl9fZ2V0SGFuZGxlcihmdW5jdGlvbikKICAgICAgICBpZiBoYW5kbGVyIGlzIG5vdCBOb25lOgogICAgICAgICAgICB0cnk6CiAgICAgICAgICAgICAgICByZXR1cm5UeXBlID0gaGFuZGxlci5nZXRSZXR1cm5UeXBlKCkKICAgICAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICAgICAgZGV0YWlscyA9IGYie2Z1bmN0aW9uLmV4cHJlc3Npb24uY29kZVR5cGUubmFtZX0ge2Z1bmN0aW9uLmZ1bmN0aW9uTmFtZX0iCiAgICAgICAgICAgICAgICBmdW5jdGlvbi5leHByZXNzaW9uLkFkZElzc3VlKHR5cGU9SXNzdWVUeXBlLkV4Y2VwdGlvblJ1bm5pbmdGdW5jdGlvbiwgZGVzY3JpcHRpb249ZiJFcnJvciBnZXR0aW5nIHJldHVybiB0eXBlIixkZXRhaWxzPWRldGFpbHMsIGV4Y2VwdGlvbj1lKQogICAgICAgICAgICAKCiAgICAgICAgcmV0dXJuIHJldHVyblR5cGUKCiAgICAgICAgCgogICAgICAgICAgICAKICAgICAgICAgICAgCiAgICAgICAgICAgIAoKICAgICAgICAKICAgICAgICAK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/issueType.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGVudW0gaW1wb3J0IEVudW0KCmNsYXNzIElzc3VlVHlwZShFbnVtKToKICAgIE1pc3NpbmdGdW5jdGlvbkhhbmRsZXIgPSAxLAogICAgVW5zdXBwb3J0ZWRGZWF0dXJlID0gMiwKICAgIEV4Y2VwdGlvblJ1bm5pbmdGdW5jdGlvbiA9IDMsCiAgICBQYXJzaW5nRXJyb3IgPSA0LAogICAgSW52YWxpZFJlZmVyZW5jZSA9IDUKCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYudmFsdWUuX19oYXNoX18oKQogICAgCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlKToKICAgICAgICBpZiBpc2luc3RhbmNlKHZhbHVlLCBJc3N1ZVR5cGUpOgogICAgICAgICAgICByZXR1cm4gc2VsZi52YWx1ZSA9PSB2YWx1ZS52YWx1ZQogICAgICAgIHJldHVybiBGYWxzZQoKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLm5hbWUKCgoKICAgIAogICAg'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/issueContainer.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgT3B0aW9uYWwsIExpc3QsIFNldCwgRGljdCwgY2FzdAoKZnJvbSBiaS5jb3JlLmlzc3VlVHlwZSBpbXBvcnQgSXNzdWVUeXBlCmZyb20gYmkuY29yZS5pc3N1ZURldGFpbHMgaW1wb3J0IElzc3VlRGV0YWlsCgoKIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIwojCiMgCiMKIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIwpjbGFzcyBJc3N1ZUNvbnRhaW5lcigpOgogICAgZnJvbSBiaS5jb3JlLmlzc3VlIGltcG9ydCBJc3N1ZQogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIGZyb20gYmkuY29yZS5pc3N1ZSBpbXBvcnQgSXNzdWUKICAgICAgICBzZWxmLl9faXNzdWVzOiBEaWN0W0lzc3VlLCBTZXRbSXNzdWVEZXRhaWxdXSA9IHt9CiAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgSXNzdWVzKHNlbGYpOgogICAgICAgIHJldHVybiBzZXQoc2VsZi5fX2lzc3Vlcy5rZXlzKCkpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIEhhc0lzc3VlcyhzZWxmKToKICAgICAgICByZXR1cm4gbGVuKHNlbGYuX19pc3N1ZXMpID4gMAogICAgCiAgICBkZWYgR2V0SXNzdWVEZXRhaWxzKHNlbGYsIGlzc3VlOiBJc3N1ZSk6CiAgICAgICAgcmV0dXJuIHNlbGYuX19pc3N1ZXMuZ2V0KGlzc3VlKSBvciBzZXQoKQoKICAgIGRlZiBBZGRJc3N1ZShzZWxmLCB0eXBlOiBJc3N1ZVR5cGUsIGRlc2NyaXB0aW9uOiBzdHIsICBkZXRhaWxzOiBPcHRpb25hbFtzdHJdPSIiLCBzb3VyY2U6IE9wdGlvbmFsW3N0cl0gPSAiIiwgZXhjZXB0aW9uOiBPcHRpb25hbFtFeGNlcHRpb25dID0gTm9uZSk6CiAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlIGltcG9ydCBJc3N1ZQoKICAgICAgICAjcHJpbnQoZiJBZGRJc3N1ZToge3R5cGV9IHtkZXNjcmlwdGlvbn0ge3NvdXJjZX0ge2RldGFpbHN9IikKICAgICAgICBpc3N1ZSA9IElzc3VlKHBhcmVudD1zZWxmLCB0eXBlPXR5cGUsIGRlc2NyaXB0aW9uPWRlc2NyaXB0aW9uKQogICAgICAgIGRldGFpbCA9IElzc3VlRGV0YWlsKHNvdXJjZT1zb3VyY2UsIGRlc2NyaXB0aW9uPWRldGFpbHMsIGV4Y2VwdGlvbj1leGNlcHRpb24pCiAgICAgICAgdHJ5OgogICAgICAgICAgICBpZiBpc3N1ZSBub3QgaW4gc2VsZi5fX2lzc3VlczoKICAgICAgICAgICAgICAgIHNlbGYuX19pc3N1ZXNbaXNzdWVdID0gc2V0KCkKICAgICAgICAgICAgc2VsZi5fX2lzc3Vlc1tpc3N1ZV0uYWRkKGRldGFpbCkKICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgIHByaW50KGYiRXJyb3IgYWRkaW5nIElzc3VlIHtkZXRhaWxzfSBvbiB7c2VsZn06IHtlfSIpCgoKICAgICAgICAKCiAgICAKICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/relationship.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSAudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0KCmNsYXNzIFJlbGF0aW9uc2hpcCgpOiAgICAKICAgIGRlZiBfX2luaXRfXyhzZWxmLCAKICAgICAgICAgICAgICAgICBuYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIGZyb21UYWJsZUl0ZW06IFRhYmxlSXRlbSwgCiAgICAgICAgICAgICAgICAgdG9UYWJsZUl0ZW06IFRhYmxlSXRlbSwgCiAgICAgICAgICAgICAgICAgcHJvcGVydGllczogT3B0aW9uYWxbZGljdF0gPSB7fSk6CiAgICAgICAgc2VsZi5fX25hbWUgPSBuYW1lCiAgICAgICAgc2VsZi5fX2Zyb21UYWJsZUl0ZW0gPSBmcm9tVGFibGVJdGVtCiAgICAgICAgZnJvbVRhYmxlSXRlbS5hZGRSZWxhdGlvbnNoaXAoc2VsZikKICAgICAgICBzZWxmLl9fdG9UYWJsZUl0ZW0gPSB0b1RhYmxlSXRlbQogICAgICAgIHRvVGFibGVJdGVtLmFkZFJlbGF0aW9uc2hpcChzZWxmKQogICAgICAgIHNlbGYuX19wcm9wZXJ0aWVzID0gcHJvcGVydGllcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fbmFtZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwcm9wZXJ0aWVzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fcHJvcGVydGllcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBmcm9tVGFibGVJdGVtKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fZnJvbVRhYmxlSXRlbQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiB0b1RhYmxlSXRlbShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3RvVGFibGVJdGVtCgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzZWxmLmZyb21UYWJsZUl0ZW0uZnVsbE5hbWV9LT57c2VsZi50b1RhYmxlSXRlbS5mdWxsTmFtZX0gKHtzZWxmLm5hbWV9KSIKICAgIAogICAgZGVmIF9faGFzaF9fKHNlbGYpOgogICAgICAgIHJldHVybiBoYXNoKChzZWxmLmZyb21UYWJsZUl0ZW0sIHNlbGYubmFtZSwgc2VsZi50b1RhYmxlSXRlbSkpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/issueDetails.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgT3B0aW9uYWwKCmNsYXNzIElzc3VlRGV0YWlsKCk6CiAgICBkZWYgX19pbml0X18oc2VsZiwgc291cmNlOiBzdHIsIGRlc2NyaXB0aW9uOiBPcHRpb25hbFtzdHJdID0gIiIsIGV4Y2VwdGlvbjogT3B0aW9uYWxbRXhjZXB0aW9uXSA9IE5vbmUpOgogICAgICAgIHNlbGYuc291cmNlID0gc291cmNlCiAgICAgICAgc2VsZi5kZXNjcmlwdGlvbiA9IGRlc2NyaXB0aW9uCiAgICAgICAgc2VsZi5leGNlcHRpb24gPSBleGNlcHRpb24KICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCB2YWx1ZSk6CiAgICAgICAgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgSXNzdWVEZXRhaWwpOgogICAgICAgICAgICByZXR1cm4gc2VsZi5zb3VyY2UgPT0gdmFsdWUuc291cmNlIGFuZCBzZWxmLmRlc2NyaXB0aW9uID09IHZhbHVlLmRlc2NyaXB0aW9uIGFuZCBzZWxmLmV4Y2VwdGlvbiA9PSB2YWx1ZS5leGNlcHRpb24KICAgICAgICByZXR1cm4gRmFsc2UKICAgIAogICAgZGVmIF9faGFzaF9fKHNlbGYpOgogICAgICAgIHJldHVybiBoYXNoKChzZWxmLnNvdXJjZSwgc2VsZi5kZXNjcmlwdGlvbiwgc2VsZi5leGNlcHRpb24pKQogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c2VsZi5zb3VyY2V9OiB7c2VsZi5kZXNjcmlwdGlvbn0iCiAgICAKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3N0cl9fKCkKCiAgICAKICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/functionHandler.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbCwgTGlzdCwgVW5pb24sIFNldCwgY2FzdApmcm9tIC5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIC5mdW5jdGlvbiBpbXBvcnQgRnVuY3Rpb24KZnJvbSAuc2NyaXB0IGltcG9ydCBTY3JpcHQKZnJvbSBiaS5zcWwuU1FMRXhwcmVzc2lvbiBpbXBvcnQgU1FMRXhwcmVzc2lvbgoKY2xhc3MgRnVuY3Rpb25QYXJhbWV0ZXIoKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBuYW1lOiBzdHIsIHR5cGU6IEV4cHJlc3Npb25UeXBlLCBpc051bGxhYmxlOiBib29sID0gVHJ1ZSwgZGVmYXVsdFZhbHVlOiBBbnkgPSBOb25lKToKICAgICAgICBzZWxmLm5hbWUgPSBuYW1lCiAgICAgICAgc2VsZi50eXBlID0gdHlwZQogICAgICAgIHNlbGYuaXNOdWxsYWJsZSA9IGlzTnVsbGFibGUKICAgICAgICBzZWxmLmRlZmF1bHRWYWx1ZSA9IGRlZmF1bHRWYWx1ZQogICAgICAgIAogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYubmFtZX0gKHtzZWxmLnR5cGV9KSIKCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c2VsZi5uYW1lfSBhcyB7c2VsZi50eXBlfSIKICAgICAgICAKY2xhc3MgRnVuY3Rpb25IYW5kbGVyKEFCQyk6CiAgICAKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBuYW1lOiBzdHIsIHJldHVyblR5cGU6IEV4cHJlc3Npb25UeXBlLCBuYW1lZFBhcmFtZXRlclJlY29yZEluZGV4OiBpbnQgPSAtMSk6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygpCiAgICAgICAgc2VsZi5fX25hbWUgPSBuYW1lCiAgICAgICAgc2VsZi5fX3JldHVyblR5cGUgPSByZXR1cm5UeXBlCiAgICAgICAgc2VsZi5uYW1lZFBhcmFtZXRlclJlY29yZEluZGV4ID0gbmFtZWRQYXJhbWV0ZXJSZWNvcmRJbmRleAogICAgICAgIHNlbGYuX19mdW5jdGlvbjogRnVuY3Rpb24gPSBOb25lCiAgICAgICAgc2VsZi5fX3BhcmFtZXRlclZhbHVlcyA9IHt9CiAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnMgPSB7fQogICAgICAgIAoKICAgIEBwcm9wZXJ0eQogICAgZGVmIG5hbWUoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBzZWxmLl9fbmFtZQogICAgCiAgICBkZWYgZ2V0UmV0dXJuVHlwZShzZWxmKSAtPiBFeHByZXNzaW9uVHlwZToKICAgICAgICByZXR1cm4gc2VsZi5fX3JldHVyblR5cGUKICAgIAogICAgQHByb3BlcnR5CiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICBwYXNzCgogICAgQHByb3BlcnR5CiAgICBkZWYgbmFtZWRQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBoYXNOYW1lZFBhcmFtZXRlcnMoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gKHNlbGYubmFtZWRQYXJhbWV0ZXJSZWNvcmRJbmRleCA+PSAwKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBmdW5jdGlvbkV4cHJlc3Npb24oc2VsZik6CiAgICAgICAgaWYgc2VsZi5fX2Z1bmN0aW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5fX2Z1bmN0aW9uLmV4cHJlc3Npb24KCiAgICBkZWYgU2V0RnVuY3Rpb24oc2VsZiwgZnVuY3Rpb246IEZ1bmN0aW9uKToKICAgICAgICBzZWxmLl9fZnVuY3Rpb24gPSBmdW5jdGlvbgogICAgICAgIGlmIGZ1bmN0aW9uLmV4cHJlc3Npb24gaXMgTm9uZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiZnVuY3Rpb24gaXMgbm90IEV4cHJlc3Npb24iKQogICAgICAgIAogICAgICAgIHNlbGYuc2V0UGFyYW1ldGVyVmFsdWVzKGZ1bmN0aW9uLnBhcmFtZXRlcnMpCgogICAgZGVmIEV2YWx1YXRlKHNlbGYpIC0+IEV4cHJlc3Npb246CiAgICAgICAgaWYgc2VsZi5fX2Z1bmN0aW9uIGlzIE5vbmU6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkZ1bmN0aW9uIGlzIG5vdCBzZXQiKQogICAgICAgIGZyb20gLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZSAgCiAgICAgICAgZnJvbSAuaXNzdWUgaW1wb3J0IElzc3VlLCBJc3N1ZVR5cGUKICAgICAgICBmcm9tIC5pc3N1ZUNvbnRhaW5lciBpbXBvcnQgSXNzdWVDb250YWluZXIKICAgICAgICByZXN1bHQ6IEV4cHJlc3Npb24gPSBOb25lCiAgICAgICAgcmVzdWx0VmFsdWUgPSBOb25lCiAgICAgICAgdHJ5OgogICAgICAgICAgICByZXN1bHRWYWx1ZSA9IHNlbGYuZXZhbHVhdGUoKQogICAgICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZXhjZXB0aW9uOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYuX19mdW5jdGlvbiwgSXNzdWVDb250YWluZXIpOgogICAgICAgICAgICAgICAgaWMgPSBjYXN0KElzc3VlQ29udGFpbmVyLCBzZWxmLl9fZnVuY3Rpb24pCiAgICAgICAgICAgICAgICBpYy5BZGRJc3N1ZSh0eXBlPUlzc3VlVHlwZS5FeGNlcHRpb25SdW5uaW5nRnVuY3Rpb24sIGRlc2NyaXB0aW9uPWYie3NlbGYuZnVuY3Rpb25FeHByZXNzaW9uLmNvZGVUeXBlLm5hbWV9IHtzZWxmLl9fZnVuY3Rpb24uZnVuY3Rpb25OYW1lfSIsIGRldGFpbHM9c3RyKGV4Y2VwdGlvbiksIGV4Y2VwdGlvbj1leGNlcHRpb24pCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgICMgcmV0dXJuIGEgbnVsbCBleHByZXNzaW9uIHNvIHdlIGRvbid0IHJlcnVuIHRoZSBmYWlsdXJlIGV2ZXJ5IHNpbmdsZSB0aW1lCiAgICAgICAgICAgIHJldHVybiBzZWxmLl9fZnVuY3Rpb24uZXhwcmVzc2lvbi5wYXJzZXIuQ3JlYXRlRnJvbVZhbHVlKHBhcmVudD1zZWxmLl9fZnVuY3Rpb24uZXhwcmVzc2lvbiwgdmFsdWU9Tm9uZSwgcHJvcGVydGllcz17ImVycm9yIjogZXhjZXB0aW9ufSkKCiAgICAgICAgbWF0Y2ggc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24uY29kZVR5cGU6CiAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLk06CiAgICAgICAgICAgICAgICBpZiBub3QgaXNpbnN0YW5jZShyZXN1bHRWYWx1ZSwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgcmVzdWx0ID0gc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24uQWRkRXhwcmVzc2lvbkZyb21WYWx1ZSh2YWx1ZT1yZXN1bHRWYWx1ZSwgcHJvcGVydGllcz17ICJvcmlnaW5hbCI6IHNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnNvdXJjZVN0cmluZyB9ICkKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgcmVzdWx0ID0gcmVzdWx0VmFsdWUKCiAgICAgICAgICAgIGNhc2UgRXhwcmVzc2lvbkNvZGVUeXBlLkRBWDoKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UocmVzdWx0VmFsdWUsIHN0cik6CiAgICAgICAgICAgICAgICAgICAgIyB0aGUgZGVmYXVsdCBmb3IgYSBEQVggZnVuY3Rpb24gaXMgdG8gcmV0dXJuIGEgU1FMIGV4cHJlc3Npb24KICAgICAgICAgICAgICAgICAgICByZXN1bHQgPSBTUUxFeHByZXNzaW9uKHNxbD1yZXN1bHRWYWx1ZSwgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCByZXR1cm5UeXBlPXNlbGYuX19mdW5jdGlvbi5yZXR1cm5UeXBlLCBwcm9wZXJ0aWVzPXsgIm9yaWdpbmFsIjogc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24uc291cmNlU3RyaW5nIH0pCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHJlc3VsdCA9IHJlc3VsdFZhbHVlCgogICAgICAgICMgYXQgdGhpcyBwb2ludCB0aGUgcmVzdWx0IHNob3VsZCBiZSBhbiBleHByZXNzaW9uLCBpZiBpdCdzIG5vdCB0aGVuIHRoZXJlJ3MgYW4gaXNzdWUgd2l0aCB0aGUgaGFuZGxlcgogICAgICAgIGlmIGlzaW5zdGFuY2UocmVzdWx0LCBFeHByZXNzaW9uKToKICAgICAgICAgICAgaWYgcmVzdWx0LnNvdXJjZVN0cmluZyBpcyBOb25lIG9yIGxlbihyZXN1bHQuc291cmNlU3RyaW5nKSA9PSAwOgogICAgICAgICAgICAgICAgcmVzdWx0LnByb3BlcnRpZXNbIm9yaWdpbmFsIl0gPSBzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbi5zb3VyY2VTdHJpbmcKICAgICAgICAgICAgaWYgcmVzdWx0LmlzUmVmZXJlbmNlOgogICAgICAgICAgICAgICAgc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24uQWRkRGVwZW5kZW5jeShyZXN1bHQpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24uQWRkSXNzdWUodHlwZT1Jc3N1ZVR5cGUuRXhjZXB0aW9uUnVubmluZ0Z1bmN0aW9uLCBkZXNjcmlwdGlvbj1zZWxmLl9fZnVuY3Rpb24uZnVuY3Rpb25OYW1lLCBkZXRhaWxzPWYicmVzdWx0IGZyb20gZnVuY3Rpb24gJ3tyZXN1bHRWYWx1ZX0nIHdhcyB1bmV4cGVjdGVkIHR5cGUiKQogICAgICAgICAgICAjcHJpbnQoZiJFdmFsdWF0aW9uIEVycm9yIGluIHtzZWxmLm5hbWV9IEhhbmRsZXI6IHJlc3VsdCB2YWx1ZSB1bmV4cGVjdGVkOiB7cmVzdWx0VmFsdWV9IikKICAgICAgICByZXR1cm4gcmVzdWx0CiAgICAKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBwYXNzCiAgICAKICAgIGRlZiBnZXRQYXJhbWV0ZXJJbmRleFZhbHVlKHNlbGYsIGluZGV4OiBpbnQsIHJldHVyblZhbHVlOiBib29sID0gVHJ1ZSkgLT4gQW55OgogICAgICAgIGZyb20gLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KICAgICAgICB2YWx1ZSA9IE5vbmUKICAgICAgICBpZiBpbmRleCA+PSAwIGFuZCBpbmRleCA8IGxlbihzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbi5wYXJhbWV0ZXJzKToKICAgICAgICAgICAgcGFyYW1ldGVyID0gc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24ucGFyYW1ldGVyc1tpbmRleF0KICAgICAgICAgICAgaWYgcmV0dXJuVmFsdWUgPT0gVHJ1ZToKICAgICAgICAgICAgICAgIHZhbHVlID0gcGFyYW1ldGVyLnZhbHVlCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICB2YWx1ZSA9IHBhcmFtZXRlcgogICAgICAgIHJldHVybiB2YWx1ZQogICAgCiAgICBkZWYgZ2V0UGFyYW1ldGVyVmFsdWUoc2VsZiwgbmFtZSwgZ2V0SW5uZXJWYWx1ZTogYm9vbCA9IFRydWUpIC0+IEFueToKICAgICAgICBmcm9tIC5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCgogICAgICAgIHZhbHVlID0gc2VsZi5fX3BhcmFtZXRlclZhbHVlcy5nZXQobmFtZS5sb3dlcigpKQogICAgICAgIGlmIGdldElubmVyVmFsdWUgYW5kIGlzaW5zdGFuY2UodmFsdWUsIEV4cHJlc3Npb24pIGFuZCAodmFsdWUuaXNWYWx1ZSBvciBpc2luc3RhbmNlKHZhbHVlLCBGdW5jdGlvbikgb3IgaXNpbnN0YW5jZSh2YWx1ZSwgU2NyaXB0KSBvciB2YWx1ZS5pc1JlZmVyZW5jZSk6CiAgICAgICAgICAgIHZhbHVlID0gdmFsdWUudmFsdWUKCiAgICAgICAgcmV0dXJuIHZhbHVlCiAgICAKICAgIGRlZiBnZXRQYXJhbWV0ZXIoc2VsZiwgbmFtZSkgLT4gQW55OgogICAgICAgIGZyb20gLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KCiAgICAgICAgcmV0dXJuIHNlbGYuX19wYXJhbWV0ZXJzLmdldChuYW1lLmxvd2VyKCkpCiAgICAKICAgIGRlZiBzZXRQYXJhbWV0ZXJWYWx1ZXMoc2VsZiwgcGFyYW1ldGVyczogTGlzdFtFeHByZXNzaW9uXSk6CiAgICAgICAgZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgoKICAgICAgICBzZWxmLl9fcGFyYW1ldGVyVmFsdWVzID0ge30KICAgICAgICBuYW1lZFBhcmFtZXRlclJlY29yZDogT3B0aW9uYWxbRXhwcmVzc2lvbl0gPSBOb25lCiAgICAgICAgICAgICAgICAKICAgICAgICBpZiBzZWxmLmhhc05hbWVkUGFyYW1ldGVyczoKICAgICAgICAgICAgaWYgbGVuKHBhcmFtZXRlcnMpID4gc2VsZi5uYW1lZFBhcmFtZXRlclJlY29yZEluZGV4IGFuZCBwYXJhbWV0ZXJzW3NlbGYubmFtZWRQYXJhbWV0ZXJSZWNvcmRJbmRleF0gaXMgbm90IE5vbmUgYW5kIHBhcmFtZXRlcnNbc2VsZi5uYW1lZFBhcmFtZXRlclJlY29yZEluZGV4XS50eXBlID09IEV4cHJlc3Npb25UeXBlLlJlY29yZDoKICAgICAgICAgICAgICAgIG5hbWVkUGFyYW1ldGVyUmVjb3JkID0gcGFyYW1ldGVyc1tzZWxmLm5hbWVkUGFyYW1ldGVyUmVjb3JkSW5kZXhdCiAgICAgICAgICAgICAgICAjIGNoZWNrIHRvIHNlZSBpZiB0aGVyZSBhcmUgYW55IGFkZGl0aW9uYWwgcGFyYW1ldGVycyBzZXQKICAgICAgICAgICAgICAgIGlmIGxlbihwYXJhbWV0ZXJzKSA+IHNlbGYubmFtZWRQYXJhbWV0ZXJSZWNvcmRJbmRleCArIDE6CiAgICAgICAgICAgICAgICAgICAgZm9yIGkgaW4gcmFuZ2Uoc2VsZi5uYW1lZFBhcmFtZXRlclJlY29yZEluZGV4ICsgMSwgbGVuKHBhcmFtZXRlcnMpKToKICAgICAgICAgICAgICAgICAgICAgICAgaWYgcGFyYW1ldGVyc1tpXS52YWx1ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIG5hbWVkUGFyYW1ldGVyUmVjb3JkID0gTm9uZQogICAgICAgICAgICAgICAgICAgICAgICAgICAgYnJlYWsKICAgICAgICAgICAgCiAgICAgICAgaWYgbmFtZWRQYXJhbWV0ZXJSZWNvcmQgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGZvciBpIGluIHJhbmdlKDAsIHNlbGYubmFtZWRQYXJhbWV0ZXJSZWNvcmRJbmRleCk6CiAgICAgICAgICAgICAgICBwYXJhbVZhbHVlID0gTm9uZQogICAgICAgICAgICAgICAgcGFyYW1ldGVyRXhwcmVzc2lvbjogRXhwcmVzc2lvbiA9IE5vbmUKICAgICAgICAgICAgICAgIGlmIGxlbihwYXJhbWV0ZXJzKSA+IGk6CiAgICAgICAgICAgICAgICAgICAgcGFyYW1ldGVyRXhwcmVzc2lvbiA9IHBhcmFtZXRlcnNbaV0KICAgICAgICAgICAgICAgICAgICBpZiBwYXJhbWV0ZXJFeHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHBhcmFtZXRlckV4cHJlc3Npb24sIEV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgcGFyYW1WYWx1ZSA9IHBhcmFtZXRlckV4cHJlc3Npb24udmFsdWUKICAgICAgICAgICAgICAgICAgICAgICAgZWxpZiBpc2luc3RhbmNlKHBhcmFtZXRlckV4cHJlc3Npb24sIGRpY3QpOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgcGFyYW1WYWx1ZSA9IHBhcmFtZXRlckV4cHJlc3Npb24uZ2V0KCJ2YWx1ZSIpCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXJhbVZhbHVlID0gcGFyYW1ldGVyRXhwcmVzc2lvbgogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgcGFyYW0gPSBzZWxmLnBvc2l0aW9uYWxQYXJhbWV0ZXJzW2ldCgogICAgICAgICAgICAgICAgaWYgcGFyYW1WYWx1ZSBpcyBOb25lOgogICAgICAgICAgICAgICAgICAgIHBhcmFtVmFsdWUgPSBwYXJhbS5kZWZhdWx0VmFsdWUKICAgICAgICAgICAgICAgIHNlbGYuX19wYXJhbWV0ZXJWYWx1ZXNbcGFyYW0ubmFtZS5sb3dlcigpXSA9IHBhcmFtVmFsdWUKICAgICAgICAgICAgICAgIGlmIHBhcmFtZXRlckV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnNbcGFyYW0ubmFtZS5sb3dlcigpXSA9IHBhcmFtZXRlckV4cHJlc3Npb24KICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnNbcGFyYW0ubmFtZS5sb3dlcigpXSA9IHBhcmFtVmFsdWUKICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIG5hbWVkUGFyYW1ldGVyUmVjb3JkLnZhbHVlIGlzIE5vbmU6CiAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgIGZvciBrLHYgaW4gbmFtZWRQYXJhbWV0ZXJSZWNvcmQudmFsdWUuaXRlbXMoKToKICAgICAgICAgICAgICAgIHNlbGYuX19wYXJhbWV0ZXJWYWx1ZXNbay5sb3dlcigpXSA9IHYKICAgICAgICAgICAgICAgIHNlbGYuX19wYXJhbWV0ZXJzW2subG93ZXIoKV0gPSB2CiAgICAgICAgZWxzZToKICAgICAgICAgICAgZm9yIGkgaW4gcmFuZ2UoMCwgbGVuKHNlbGYucG9zaXRpb25hbFBhcmFtZXRlcnMpKToKICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgcGFyYW0gPSBzZWxmLnBvc2l0aW9uYWxQYXJhbWV0ZXJzW2ldCiAgICAgICAgICAgICAgICBwYXJhbWV0ZXJFeHByZXNzaW9uOiBFeHByZXNzaW9uID0gTm9uZQogICAgICAgICAgICAgICAgcGFyYW1WYWx1ZSA9IE5vbmUKICAgICAgICAgICAgICAgIGlmIGxlbihwYXJhbWV0ZXJzKSA+IGk6CiAgICAgICAgICAgICAgICAgICAgcGFyYW1ldGVyRXhwcmVzc2lvbiA9IHBhcmFtZXRlcnNbaV0KICAgICAgICAgICAgICAgICAgICB0cnk6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIHBhcmFtZXRlckV4cHJlc3Npb24gaXMgbm90IE5vbmUgYW5kIGlzaW5zdGFuY2UocGFyYW1ldGVyRXhwcmVzc2lvbiwgRXhwcmVzc2lvbikgYW5kIHBhcmFtZXRlckV4cHJlc3Npb24udmFsdWUgaXMgbm90IE5vbmUgYW5kIHBhcmFtZXRlckV4cHJlc3Npb24uaXNWYWx1ZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHBhcmFtVmFsdWUgPSBwYXJhbWV0ZXJzW2ldLnZhbHVlCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXJhbVZhbHVlID0gcGFyYW1ldGVyRXhwcmVzc2lvbgogICAgICAgICAgICAgICAgICAgIGV4Y2VwdDoKICAgICAgICAgICAgICAgICAgICAgICAgcGFzcyAgICAKICAgICAgICAgICAgICAgIGlmIHBhcmFtVmFsdWUgaXMgTm9uZSBhbmQgcGFyYW0uZGVmYXVsdFZhbHVlIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgIHBhcmFtVmFsdWUgPSBwYXJhbS5kZWZhdWx0VmFsdWUKCiAgICAgICAgICAgICAgICBzZWxmLl9fcGFyYW1ldGVyVmFsdWVzW3BhcmFtLm5hbWUubG93ZXIoKV0gPSBwYXJhbVZhbHVlCiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgIGlmIHBhcmFtZXRlckV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnNbcGFyYW0ubmFtZS5sb3dlcigpXSA9IHBhcmFtZXRlckV4cHJlc3Npb24KICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnNbcGFyYW0ubmFtZS5sb3dlcigpXSA9IHBhcmFtVmFsdWUKICAgICAgICAgICAgCiAgICAgICAgICAgIAogICAgICAgICAgICAKICAgICAgICAgICAgCgogICAgICAgIAogICAgICAgIAo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/expressionParser.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgVW5pb24sIEFueSwgT3B0aW9uYWwKCgoKCmNsYXNzIEV4cHJlc3Npb25QYXJzZXIoQUJDKToKICAgIGZyb20gLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKICAgIGZyb20gLmV4cHJlc3Npb25Db2RlVHlwZSBpbXBvcnQgRXhwcmVzc2lvbkNvZGVUeXBlCiAgICBmcm9tIC5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKICAgIAogICAgZGVmIF9faW5pdF9fKHNlbGYsIGNvZGVUeXBlOiBFeHByZXNzaW9uQ29kZVR5cGUpOgogICAgICAgIHNlbGYuX19jb2RlVHlwZSA9IGNvZGVUeXBlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIENvZGVUeXBlKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fY29kZVR5cGUKCiAgICBAcHJvcGVydHkKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIE9wZXJhdG9ycyhzZWxmKSAtPiBMaXN0W3N0cl06CiAgICAgICAgcGFzcwoKICAgIEBwcm9wZXJ0eQogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgQm91bmRhcnkoc2VsZikgLT4gTGlzdFtzdHJdOgogICAgICAgIHBhc3MKCiAgICBAcHJvcGVydHkKICAgIGRlZiBQYXJhbWV0ZXJTZXBhcmF0b3Ioc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiAiLCIKCiAgICBAcHJvcGVydHkKICAgIGRlZiBTaW5nbGVMaW5lQ29tbWVudChzZWxmKSAtPiBMaXN0W3N0cl06CiAgICAgICAgcmV0dXJuIFsiLy8iXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBNdWx0aUxpbmVDb21tZW50U3RhcnQoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiAiLyoiCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIE11bHRpTGluZUNvbW1lbnRFbmQoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiAiKi8iCiAgICAKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIENyZWF0ZUZyb21TdHJpbmcoc2VsZiwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBleHByZXNzaW9uU3RyaW5nOiBVbmlvbltMaXN0W3N0cl18c3RyXSwgcmV0dXJuVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvblR5cGVdID0gRXhwcmVzc2lvblR5cGUuQW55LCBwcm9wZXJ0aWVzOiBkaWN0PXt9KToKICAgICAgICBwYXNzCgogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgQ3JlYXRlRnJvbVZhbHVlKHNlbGYsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgdmFsdWU6IEFueSwgcHJvcGVydGllczogZGljdD17fSk6CiAgICAgICAgcGFzcwoKICAgIGRlZiBDcmVhdGVDaGlsZEV4cHJlc3Npb25zRnJvbVN0cmluZyhzZWxmLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIGNoaWxkRXhwcmVzc2lvblN0cmluZzogc3RyKToKICAgICAgICBmcm9tIC5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCiAgICAgICAgZnJvbSAuaXNzdWUgaW1wb3J0IElzc3VlLCBJc3N1ZVR5cGUKICAgICAgICBjaGlsZEV4cHJlc3Npb25zOiBMaXN0W0V4cHJlc3Npb25dID0gW10KICAgICAgICBmb3IgZXhwcmVzc2lvblN0cmluZyBpbiBzZWxmLlBhcnNlRXhwcmVzc2lvblN0cmluZyhjaGlsZEV4cHJlc3Npb25TdHJpbmcpOgogICAgICAgICAgICBjaGlsZEV4cHJlc3Npb24gPSBzZWxmLkNyZWF0ZUZyb21TdHJpbmcocGFyZW50PXBhcmVudCwgZXhwcmVzc2lvblN0cmluZz1leHByZXNzaW9uU3RyaW5nKQoKICAgICAgICAgICAgaWYgY2hpbGRFeHByZXNzaW9uIGlzIE5vbmU6CiAgICAgICAgICAgICAgICBwYXJlbnQuQWRkSXNzdWUodHlwZT1Jc3N1ZVR5cGUuUGFyc2luZ0Vycm9yLCBkZXNjcmlwdGlvbj1mIkNoaWxkIGV4cHJlc3Npb24gZmFpbGVkIHRvIHBhcnNlIiwgZGV0YWlscz1leHByZXNzaW9uU3RyaW5nICkKICAgICAgICAgICAgY2hpbGRFeHByZXNzaW9ucy5hcHBlbmQoY2hpbGRFeHByZXNzaW9uKQoKICAgICAgICByZXR1cm4gY2hpbGRFeHByZXNzaW9ucwoKICAgIGRlZiBSZW1vdmVPdXRlclBhcmVudGhlc2lzKHNlbGYsIGV4cHJlc3Npb25TdHJpbmc6IHN0cik6CiAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9IGV4cHJlc3Npb25TdHJpbmcuc3RyaXAoKQogICAgICAgIG9wZXJhdG9ySW5mbyA9IHNlbGYuUGFyc2VFeHByZXNzaW9uT3BlcmF0aW9ucyhleHByZXNzaW9uU3RyaW5nKQogICAgICAgIGlmIGxlbihvcGVyYXRvckluZm8uZ2V0KCJvcGVyYXRvcnMiKSkgPT0gMCBhbmQgZXhwcmVzc2lvblN0cmluZy5zdGFydHN3aXRoKCIoIikgYW5kIGV4cHJlc3Npb25TdHJpbmcuZW5kc3dpdGgoIikiKToKICAgICAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9IGV4cHJlc3Npb25TdHJpbmdbMTpsZW4oZXhwcmVzc2lvblN0cmluZyktMV0KICAgICAgICAgICAgcmV0dXJuIHNlbGYuUmVtb3ZlT3V0ZXJQYXJlbnRoZXNpcyhleHByZXNzaW9uU3RyaW5nKQogICAgICAgIHJldHVybiBleHByZXNzaW9uU3RyaW5nCiAgICAKICAgIGRlZiBQYXJzZUV4cHJlc3Npb25PcGVyYXRpb25zKHNlbGYsIGV4cHJlc3Npb246IHN0ciwgb3BlcmF0b3JzOiBMaXN0W3N0cl0gPSBOb25lLCBtYXhPcGVyYXRvckNvdW50OiBpbnQgPSBOb25lKSAtPiBkaWN0OgogICAgICAgIGZyb20gY29sbGVjdGlvbnMgaW1wb3J0IGRlcXVlCiAgICAgICAgaWYgb3BlcmF0b3JzIGlzIE5vbmU6CiAgICAgICAgICAgIG9wZXJhdG9ycyA9IHNlbGYuT3BlcmF0b3JzCgogICAgICAgIG9wZXJhbmRzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgIGJvdW5kYXJ5ID0gc2VsZi5Cb3VuZGFyeQogICAgICAgIHN0YWNrID0gZGVxdWUoKQogICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9ICIiCiAgICAgICAgcHJldmlvdXNDaGFyID0gIiIKICAgICAgICBuZXh0Q2hhciA9ICIiCiAgICAgICAgc3RhcnQgPSAwCiAgICAgICAgaW5UZXh0U3RyaW5nID0gRmFsc2UKICAgICAgICBjdXJyZW50T3BlcmF0b3IgPSAiIgogICAgICAgIGZvdW5kT3BlcmF0b3JzID0gW10KICAgICAgICAKICAgICAgICBleHByZXNzaW9uID0gc2VsZi5SZW1vdmVDb21tZW50cyhleHByZXNzaW9uKQoKICAgICAgICBmb3IgaSwgYyBpbiBlbnVtZXJhdGUoZXhwcmVzc2lvbik6CiAgICAgICAgICAgIGlmIGkgPCBsZW4oZXhwcmVzc2lvbikgLSAxOgogICAgICAgICAgICAgICAgbmV4dENoYXIgPSBleHByZXNzaW9uW2krMV0KICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIG5leHRDaGFyID0gTm9uZQogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICBpZiBwcmV2aW91c0NoYXIgIT0gIlxcIjoKICAgICAgICAgICAgICAgIGlmIGMgPT0gJyInOgogICAgICAgICAgICAgICAgICAgIGlmIGN1cnJlbnRCb3VuZGFyeSA9PSAnIic6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIG5leHRDaGFyID09ICciJzoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNvbnRpbnVlCiAgICAgICAgICAgICAgICAgICAgICAgIHN0YWNrLnBvcCgpICAgIAogICAgICAgICAgICAgICAgICAgICAgICBpZiBsZW4oc3RhY2spID4gMDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9IHN0YWNrW2xlbihzdGFjaykgLSAxXQogICAgICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgY3VycmVudEJvdW5kYXJ5ID0gIiIKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBzdGFjay5hcHBlbmQoYykKICAgICAgICAgICAgICAgICAgICAgICAgY3VycmVudEJvdW5kYXJ5ID0gYwogICAgICAgICAgICAgICAgZWxpZiBjIGluIGJvdW5kYXJ5OgogICAgICAgICAgICAgICAgICAgIHN0YWNrLmFwcGVuZChjKQogICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9IGMKICAgICAgICAgICAgICAgIGVsaWYgYyA9PSBib3VuZGFyeS5nZXQoY3VycmVudEJvdW5kYXJ5KToKICAgICAgICAgICAgICAgICAgICAjIGNsb3NlIHRoZSBncm91cAogICAgICAgICAgICAgICAgICAgIHN0YWNrLnBvcCgpCiAgICAgICAgICAgICAgICAgICAgaWYgbGVuKHN0YWNrKSA+IDA6CiAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9IHN0YWNrW2xlbihzdGFjaykgLSAxXQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9ICIiCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIGlmIGMgPT0gJyInOgogICAgICAgICAgICAgICAgICAgICAgICBpblRleHRTdHJpbmcgPSBub3QgaW5UZXh0U3RyaW5nCgogICAgICAgICAgICBpZiBjdXJyZW50Qm91bmRhcnkgPT0gIiIgYW5kIG5vdCBpblRleHRTdHJpbmc6CiAgICAgICAgICAgICAgICBpZiBuZXh0Q2hhciBpcyBOb25lOgogICAgICAgICAgICAgICAgICAgIG9wZXJhbmRzLmFwcGVuZChleHByZXNzaW9uW3N0YXJ0Ol0uc3RyaXAoKSkKICAgICAgICAgICAgICAgIGVsaWYgbWF4T3BlcmF0b3JDb3VudCBpcyBOb25lIG9yIGxlbihmb3VuZE9wZXJhdG9ycykgPCBtYXhPcGVyYXRvckNvdW50OgogICAgICAgICAgICAgICAgICAgIGN1cnJlbnRPcGVyYXRvciArPSBjCiAgICAgICAgICAgICAgICAgICAgcGFydGlhbE9wZXJhdG9yID0gRmFsc2UKICAgICAgICAgICAgICAgICAgICBmb3Igb3BlcmF0b3IgaW4gb3BlcmF0b3JzOgogICAgICAgICAgICAgICAgICAgICAgICBpZiBvcGVyYXRvci51cHBlcigpLnN0YXJ0c3dpdGgoY3VycmVudE9wZXJhdG9yLnVwcGVyKCkgKyBuZXh0Q2hhci51cHBlcigpKToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHBhcnRpYWxPcGVyYXRvciA9IFRydWUKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGJyZWFrCiAgICAgICAgICAgICAgICAgICAgaWYgbm90IHBhcnRpYWxPcGVyYXRvcjoKICAgICAgICAgICAgICAgICAgICAgICAgaWYgY3VycmVudE9wZXJhdG9yIGluIG9wZXJhdG9yczoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRfb3BlcmFuZCA9IGV4cHJlc3Npb25bc3RhcnQ6aS0obGVuKGN1cnJlbnRPcGVyYXRvciktMSldLnN0cmlwKCkKCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAjIENoZWNrIGlmIHRoaXMgaXMgYSB1bmFyeSBtaW51cwogICAgICAgICAgICAgICAgICAgICAgICAgICAgIyBJdCdzIHVuYXJ5IGlmIHRoZXJlJ3Mgbm8gdmFsaWQgb3BlcmFuZCBiZWZvcmUgaXQKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGN1cnJlbnRPcGVyYXRvciA9PSAiLSIgYW5kIGN1cnJlbnRfb3BlcmFuZCA9PSAiIjoKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAjIFRoaXMgaXMgYSB1bmFyeSBtaW51cywgZG9uJ3QgdHJlYXQgYXMgb3BlcmF0b3IKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50T3BlcmF0b3IgPSAiIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGNvbnRpbnVlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGZvdW5kT3BlcmF0b3JzLmFwcGVuZChjdXJyZW50T3BlcmF0b3IudXBwZXIoKSkgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBvcGVyYW5kcy5hcHBlbmQoY3VycmVudF9vcGVyYW5kKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3RhcnQgPSBpICsgMQogICAgICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgY3VycmVudE9wZXJhdG9yID0gIiIKICAgICAgICAgICAgcHJldmlvdXNDaGFyID0gYwogICAgCiAgICAgICAgcmV0dXJuIHsib3JpZ2luYWwiOiBleHByZXNzaW9uLCAib3BlcmFuZHMiIDogb3BlcmFuZHMsICJvcGVyYXRvcnMiOiBmb3VuZE9wZXJhdG9yc30KICAgIAogICAgZGVmIFJlbW92ZUNvbW1lbnRzKHNlbGYsIGV4cHJlc3Npb246IHN0cikgLT4gc3RyOgogICAgICAgIGZyb20gY29sbGVjdGlvbnMgaW1wb3J0IGRlcXVlCiAgICAgICAgYm91bmRhcnkgPSBzZWxmLkJvdW5kYXJ5CiAgICAgICAgc3RhY2sgPSBkZXF1ZSgpCiAgICAgICAgY3VycmVudEJvdW5kYXJ5ID0gIiIKICAgICAgICBwcmV2aW91c0NoYXIgPSAiIgogICAgICAgIG5leHRDaGFyID0gIiIKICAgICAgICAKICAgICAgICBzdGFydCA9IDAKICAgICAgICBpblRleHRTdHJpbmcgPSBGYWxzZQogICAgICAgIGluU2luZ2xlTGluZUNvbW1lbnQgPSBGYWxzZQogICAgICAgIGluTXVsdGlMaW5lQ29tbWVudCA9IEZhbHNlCiAgICAgICAgbmV3RXhwcmVzc2lvbiA9ICIiCgogICAgICAgIGZvciBpLCBjIGluIGVudW1lcmF0ZShleHByZXNzaW9uKToKICAgICAgICAgICAgaWYgaSA8IGxlbihleHByZXNzaW9uKSAtIDE6CiAgICAgICAgICAgICAgICBuZXh0Q2hhciA9IGV4cHJlc3Npb25baSsxXQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgbmV4dENoYXIgPSBOb25lCgogICAgICAgICAgICBpZiBpblNpbmdsZUxpbmVDb21tZW50ID09IFRydWU6CiAgICAgICAgICAgICAgICBpZiBuZXh0Q2hhciA9PSAiXG4iIG9yIG5leHRDaGFyIGlzIE5vbmU6CiAgICAgICAgICAgICAgICAgICAgaW5TaW5nbGVMaW5lQ29tbWVudCA9IEZhbHNlCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIG5ld0V4cHJlc3Npb24gKz0gIiAiCiAgICAgICAgICAgIGVsaWYgaW5NdWx0aUxpbmVDb21tZW50ID09IFRydWU6CiAgICAgICAgICAgICAgICBpZiBleHByZXNzaW9uW2ktKGxlbihzZWxmLk11bHRpTGluZUNvbW1lbnRFbmQpLTEpOmkrMV0gPT0gc2VsZi5NdWx0aUxpbmVDb21tZW50RW5kOgogICAgICAgICAgICAgICAgICAgIGluTXVsdGlMaW5lQ29tbWVudCA9IEZhbHNlCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIG5ld0V4cHJlc3Npb24gKz0gIiAiCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICBpZiBub3QgaW5UZXh0U3RyaW5nOgogICAgICAgICAgICAgICAgICAgIGZvciBzaW5nbGVMaW5lQ29tbWVudCBpbiBzZWxmLlNpbmdsZUxpbmVDb21tZW50OgogICAgICAgICAgICAgICAgICAgICAgICBpZiBleHByZXNzaW9uW2k6aStsZW4oc2luZ2xlTGluZUNvbW1lbnQpXSA9PSBzaW5nbGVMaW5lQ29tbWVudDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGluU2luZ2xlTGluZUNvbW1lbnQgPSBUcnVlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBicmVhawogICAgICAgICAgICAgICAgICAgIGlmIGluU2luZ2xlTGluZUNvbW1lbnQgPT0gRmFsc2UgYW5kIGV4cHJlc3Npb25baTppK2xlbihzZWxmLk11bHRpTGluZUNvbW1lbnRTdGFydCldID09IHNlbGYuTXVsdGlMaW5lQ29tbWVudFN0YXJ0OgogICAgICAgICAgICAgICAgICAgICAgICBpbk11bHRpTGluZUNvbW1lbnQgPSBUcnVlCiAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgaWYgaW5TaW5nbGVMaW5lQ29tbWVudCA9PSBGYWxzZSBhbmQgaW5NdWx0aUxpbmVDb21tZW50ID09IEZhbHNlOgogICAgICAgICAgICAgICAgICAgIG5ld0V4cHJlc3Npb24gKz0gYwogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgIGlmIHByZXZpb3VzQ2hhciAhPSAiXFwiOgogICAgICAgICAgICAgICAgICAgICAgICBpZiBjID09ICciJzoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGluVGV4dFN0cmluZyA9IG5vdCBpblRleHRTdHJpbmcKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGN1cnJlbnRCb3VuZGFyeSA9PSAnIic6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgaWYgbmV4dENoYXIgPT0gJyInOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjb250aW51ZQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0YWNrLnBvcCgpICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihzdGFjaykgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSBzdGFja1tsZW4oc3RhY2spIC0gMV0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSAiIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBzdGFjay5hcHBlbmQoYykKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSBjCiAgICAgICAgICAgICAgICAgICAgICAgIGVsaWYgYyBpbiBib3VuZGFyeToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0YWNrLmFwcGVuZChjKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgY3VycmVudEJvdW5kYXJ5ID0gYwogICAgICAgICAgICAgICAgICAgICAgICBlbGlmIGMgPT0gYm91bmRhcnkuZ2V0KGN1cnJlbnRCb3VuZGFyeSk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAjIGNsb3NlIHRoZSBncm91cAogICAgICAgICAgICAgICAgICAgICAgICAgICAgc3RhY2sucG9wKCkKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihzdGFjaykgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9IHN0YWNrW2xlbihzdGFjaykgLSAxXQogICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSAiIgoKICAgICAgICAgICAgcHJldmlvdXNDaGFyID0gYwogICAgCiAgICAgICAgcmV0dXJuIG5ld0V4cHJlc3Npb24KICAgIAogICAgIyB0YWtlcyBhIHN0cmluZyBhbmQgcGFyc2VzIGl0IGludG8gMSBvciBtb3JlIGVzcHJlc3Npb24gc3RyaW5ncyAoYmFzZWQgb24gdGhlIGJvdW5kYXJpZXMgYW5kIHBhcmFtZXRlciBzZXBhcmF0b3IpCiAgICBkZWYgUGFyc2VFeHByZXNzaW9uU3RyaW5nKHNlbGYsIGV4cHJlc3Npb246IHN0cikgLT4gTGlzdFtzdHJdOgogICAgICAgIGZyb20gY29sbGVjdGlvbnMgaW1wb3J0IGRlcXVlCgogICAgICAgIGV4cHJlc3Npb24gPSBzZWxmLlJlbW92ZUNvbW1lbnRzKGV4cHJlc3Npb24pCgogICAgICAgIGJvdW5kYXJ5ID0gc2VsZi5Cb3VuZGFyeQogICAgICAgIGV4cHJlc3Npb25TdHJpbmdzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgIHN0YWNrID0gZGVxdWUoKQogICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9ICIiCiAgICAgICAgcHJldmlvdXNDaGFyID0gIiIKICAgICAgICBuZXh0Q2hhciA9ICIiCiAgICAgICAgc3RhcnQgPSAwCiAgICAgICAgaW5UZXh0U3RyaW5nID0gRmFsc2UKICAgICAgICAKICAgICAgICBmb3IgaSwgYyBpbiBlbnVtZXJhdGUoZXhwcmVzc2lvbik6CiAgICAgICAgICAgIGlmIGkgPCBsZW4oZXhwcmVzc2lvbikgLSAxOgogICAgICAgICAgICAgICAgbmV4dENoYXIgPSBleHByZXNzaW9uW2krMV0KICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIG5leHRDaGFyID0gTm9uZQogICAgICAgICAgICAKICAgICAgICAgICAgaWYgcHJldmlvdXNDaGFyICE9ICJcXCI6CiAgICAgICAgICAgICAgICBpZiBjID09ICciJzoKICAgICAgICAgICAgICAgICAgICBpZiBjdXJyZW50Qm91bmRhcnkgPT0gJyInOgogICAgICAgICAgICAgICAgICAgICAgICBpZiBuZXh0Q2hhciA9PSAnIic6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBjb250aW51ZQogICAgICAgICAgICAgICAgICAgICAgICBzdGFjay5wb3AoKSAgICAKICAgICAgICAgICAgICAgICAgICAgICAgaWYgbGVuKHN0YWNrKSA+IDA6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSBzdGFja1tsZW4oc3RhY2spIC0gMV0KICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9ICIiCiAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgc3RhY2suYXBwZW5kKGMpCiAgICAgICAgICAgICAgICAgICAgICAgIGN1cnJlbnRCb3VuZGFyeSA9IGMKICAgICAgICAgICAgICAgIGVsaWYgYyBpbiBib3VuZGFyeToKICAgICAgICAgICAgICAgICAgICBzdGFjay5hcHBlbmQoYykKICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSBjCiAgICAgICAgICAgICAgICBlbGlmIGMgPT0gYm91bmRhcnkuZ2V0KGN1cnJlbnRCb3VuZGFyeSk6CiAgICAgICAgICAgICAgICAgICAgIyBjbG9zZSB0aGUgZ3JvdXAKICAgICAgICAgICAgICAgICAgICBzdGFjay5wb3AoKQogICAgICAgICAgICAgICAgICAgIGlmIGxlbihzdGFjaykgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSBzdGFja1tsZW4oc3RhY2spIC0gMV0KICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBjdXJyZW50Qm91bmRhcnkgPSAiIgogICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICBpZiBjID09ICciJzoKICAgICAgICAgICAgICAgICAgICAgICAgaW5UZXh0U3RyaW5nID0gbm90IGluVGV4dFN0cmluZwoKICAgICAgICAgICAgaWYgY3VycmVudEJvdW5kYXJ5ID09ICIiIGFuZCBub3QgaW5UZXh0U3RyaW5nIDoKICAgICAgICAgICAgICAgIGlmIChjID09IHNlbGYuUGFyYW1ldGVyU2VwYXJhdG9yIG9yIG5leHRDaGFyIGlzIE5vbmUpOgogICAgICAgICAgICAgICAgICAgIGlmIGMgPT0gc2VsZi5QYXJhbWV0ZXJTZXBhcmF0b3I6CiAgICAgICAgICAgICAgICAgICAgICAgIGV4cHJlc3Npb25TdHJpbmcgPSBleHByZXNzaW9uW3N0YXJ0OmldCiAgICAgICAgICAgICAgICAgICAgICAgIHN0YXJ0ID0gaSArIDEKICAgICAgICAgICAgICAgICAgICBlbGlmIG5leHRDaGFyIGlzIE5vbmU6CiAgICAgICAgICAgICAgICAgICAgICAgIGV4cHJlc3Npb25TdHJpbmcgPSBleHByZXNzaW9uW3N0YXJ0Ok5vbmVdCgogICAgICAgICAgICAgICAgICAgIGlmIGV4cHJlc3Npb25TdHJpbmcuc3RyaXAoKSAhPSAiIjoKICAgICAgICAgICAgICAgICAgICAgICAgZXhwcmVzc2lvblN0cmluZ3MuYXBwZW5kKGV4cHJlc3Npb25TdHJpbmcuc3RyaXAoKSkKICAgICAgICAgICAgcHJldmlvdXNDaGFyID0gYyAgICAgICAgICAgIAogICAgICAgIHJldHVybiBleHByZXNzaW9uU3RyaW5ncw=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/operator.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKCmNsYXNzIE9wZXJhdG9yKEFCQyk6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygpCiAgICAgICAgCiAgICBAcHJvcGVydHkKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIG9wZXJhbmRzKHNlbGYpIC0+IExpc3RbRXhwcmVzc2lvbl06CiAgICAgICAgcGFzcwoKICAgIEBwcm9wZXJ0eQogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgb3BlcmF0b3JzKHNlbGYpIC0+IExpc3Rbc3RyXToKICAgICAgICBwYXNzCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgaWYgYW55KG8gaXMgbm90IE5vbmUgYW5kIG8ucmV0dXJuVHlwZSA9PSBFeHByZXNzaW9uVHlwZS5UZXh0IGZvciBvIGluIHNlbGYub3BlcmFuZHMpOgogICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuVGV4dAogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHR5cGVzID0gc2V0KCkKICAgICAgICAgICAgZm9yIG8gaW4gc2VsZi5vcGVyYW5kczoKICAgICAgICAgICAgICAgIGlmIG8gaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgdHlwZXMuYWRkKG8ucmV0dXJuVHlwZSkKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgcGFzcwogICAgICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIGxlbih0eXBlcykgPiAxIG9yIGxlbih0eXBlcykgPT0gMDoKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5Vbmtub3duCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICByZXR1cm4gdHlwZXMucG9wKCkKICAgIAogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIGl0ZW1zID0gW10KICAgICAgICBmb3IgaSwgdiBpbiBlbnVtZXJhdGUoc2VsZi5vcGVyYW5kcyk6CiAgICAgICAgICAgIGl0ZW1zLmFwcGVuZChFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHYpKQogICAgICAgICAgICBpZiBpIDwgbGVuKHNlbGYub3BlcmF0b3JzKToKICAgICAgICAgICAgICAgIG9wZXJhdG9yID0gc2VsZi5vcGVyYXRvcnNbaV0udXBwZXIoKS5zdHJpcCgpCiAgICAgICAgICAgICAgICBtYXRjaChvcGVyYXRvcik6CiAgICAgICAgICAgICAgICAgICAgY2FzZSAiPSI6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihzZWxmLm9wZXJhbmRzKSA+IGkrMSBhbmQgc2VsZi5vcGVyYW5kc1tpKzFdLmdldFNxbFZhbHVlKCkubG93ZXIoKSA9PSAibnVsbCI6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQoIklTIikKICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGl0ZW1zLmFwcGVuZChvcGVyYXRvcikKICAgICAgICAgICAgICAgICAgICBjYXNlICI8PiI6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGxlbihzZWxmLm9wZXJhbmRzKSA+IGkrMSBhbmQgc2VsZi5vcGVyYW5kc1tpKzFdLmdldFNxbFZhbHVlKCkubG93ZXIoKSA9PSAibnVsbCI6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQoIklTIE5PVCIpCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQob3BlcmF0b3IpCiAgICAgICAgICAgICAgICAgICAgY2FzZSAiKyI6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIHNlbGYucmV0dXJuVHlwZSA9PSBFeHByZXNzaW9uVHlwZS5UZXh0OgogICAgICAgICAgICAgICAgICAgICAgICAgICAgaXRlbXMuYXBwZW5kKCJ8fCIpCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQob3BlcmF0b3IpCiAgICAgICAgICAgICAgICAgICAgY2FzZSAiJiI6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIHNlbGYucmV0dXJuVHlwZSA9PSBFeHByZXNzaW9uVHlwZS5UZXh0OgogICAgICAgICAgICAgICAgICAgICAgICAgICAgaXRlbXMuYXBwZW5kKCJ8fCIpCiAgICAgICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQob3BlcmF0b3IpCiAgICAgICAgICAgICAgICAgICAgY2FzZSAiJiYiOgogICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQoIkFORCIpCiAgICAgICAgICAgICAgICAgICAgY2FzZSAifHwiOgogICAgICAgICAgICAgICAgICAgICAgICBpdGVtcy5hcHBlbmQoIk9SIikKICAgICAgICAgICAgICAgICAgICBjYXNlIF86CiAgICAgICAgICAgICAgICAgICAgICAgIGl0ZW1zLmFwcGVuZChvcGVyYXRvcikgICAgICAgICAgCgogICAgICAgIHJldHVybiAnICcuam9pbihpdGVtcykKICAgIApBQkMucmVnaXN0ZXIoT3BlcmF0b3Ip'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/__init__.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuY29sdW1uIGltcG9ydCBDb2x1bW4KZnJvbSAuY29ubmVjdG9yIGltcG9ydCBDb25uZWN0b3IsIENvbm5lY3RvclR5cGUKZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIC5leHByZXNzaW9uQ29kZVR5cGUgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIC5leHByZXNzaW9uUGFyZW50IGltcG9ydCBFeHByZXNzaW9uUGFyZW50CmZyb20gLmV4cHJlc3Npb25QYXJzZXIgaW1wb3J0IEV4cHJlc3Npb25QYXJzZXIKZnJvbSAuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIC5mdW5jdGlvbkV2YWx1YXRvciBpbXBvcnQgRnVuY3Rpb25FdmFsdWF0b3IKZnJvbSAuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIKZnJvbSAuaXNzdWUgaW1wb3J0IElzc3VlCmZyb20gLmlzc3VlQ29udGFpbmVyIGltcG9ydCBJc3N1ZUNvbnRhaW5lcgpmcm9tIC5pc3N1ZURldGFpbHMgaW1wb3J0IElzc3VlRGV0YWlsCmZyb20gLmlzc3VlVHlwZSBpbXBvcnQgSXNzdWVUeXBlCmZyb20gLnJlbGF0aW9uc2hpcCBpbXBvcnQgUmVsYXRpb25zaGlwCmZyb20gLnRhYmxlIGltcG9ydCBUYWJsZQpmcm9tIC50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/expressionType.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55CmZyb20gZW51bSBpbXBvcnQgRW51bQoKY2xhc3MgRXhwcmVzc2lvblR5cGUoRW51bSk6ICMgc2VlOiBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3R5cGUtY29udmVyc2lvbgogICAgVW5rbm93biA9IC0xLAogICAgQW55ID0gMCwgIwlUaGUgYW55IGRhdGEgdHlwZSBpcyB0aGUgc3RhdHVzIGdpdmVuIHdoZW4gYSB2YWx1ZSBkb2Vzbid0IGhhdmUgYW4gZXhwbGljaXQgZGF0YSB0eXBlIGRlZmluaXRpb24uIFRoZSBhbnkgdHlwZSBpcyB0aGUgZGF0YSB0eXBlIHRoYXQgY2xhc3NpZmllcyBhbGwgdmFsdWVzLgogICAgQmluYXJ5ID0gMSwgIwlUaGUgYmluYXJ5IGRhdGEgdHlwZSBjYW4gYmUgdXNlZCB0byByZXByZXNlbnQgYW55IG90aGVyIGRhdGEgd2l0aCBhIGJpbmFyeSBmb3JtYXQuCiAgICBUeXBlID0gMiwgIwlBIHZhbHVlIHRoYXQgY2xhc3NpZmllcyBvdGhlciB2YWx1ZXMuIEZvciBtb3JlIGluZm9ybWF0aW9uLCBnbyB0byBUeXBlcy4KICAgIE51bGwgPSAzLCAjCVJlcHJlc2VudHMgdGhlIGFic2VuY2Ugb2YgYSB2YWx1ZSwgb3IgYSB2YWx1ZSBvZiBpbmRldGVybWluYXRlIG9yIHVua25vd24gc3RhdGUuCiAgICBEYXRlID0gNCwgIwlSZXByZXNlbnRzIGp1c3QgYSBkYXRlIChubyB0aW1lIHBvcnRpb24pLgogICAgVGltZSA9IDUsICMJUmVwcmVzZW50cyBqdXN0IHRpbWUgKG5vIGRhdGUgcG9ydGlvbikuCiAgICBEYXRlVGltZSA9IDYsICMJUmVwcmVzZW50cyBib3RoIGEgZGF0ZSBhbmQgdGltZSB2YWx1ZS4gVGhlIHRpbWUgcG9ydGlvbiBvZiBhIGRhdGUgaXMgc3RvcmVkIGFzIGEgZnJhY3Rpb24gdG8gd2hvbGUgbXVsdGlwbGVzIG9mIDEvMzAwIHNlY29uZHMgKDMuMzMgbXMpLiBEYXRlcyBiZXR3ZWVuIHRoZSB5ZWFycyAxOTAwIGFuZCA5OTk5IGFyZSBzdXBwb3J0ZWQuCiAgICBEYXRlVGltZVpvbmUgPSA3LCAjCVJlcHJlc2VudHMgYSBVVEMgZGF0ZSBhbmQgdGltZSB3aXRoIGEgdGltZS16b25lIG9mZnNldC4KICAgIER1cmF0aW9uID0gOCwgIwlSZXByZXNlbnRzIGEgbGVuZ3RoIG9mIHRpbWUuIFRoaXMgdHlwZSBjYW4gYmUgYWRkZWQgb3Igc3VidHJhY3RlZCBmcm9tIGEgZGF0ZXRpbWUgZmllbGQgd2l0aCBjb3JyZWN0IHJlc3VsdHMuIEZvciBtb3JlIGluZm9ybWF0aW9uLCBnbyB0byBEdXJhdGlvbi4KICAgIFRleHQgPSA5LCAjCUEgVW5pY29kZSBjaGFyYWN0ZXIgZGF0YSBzdHJpbmcuIENhbiBiZSBzdHJpbmdzLCBudW1iZXJzLCBvciBkYXRlcyByZXByZXNlbnRlZCBpbiBhIHRleHQgZm9ybWF0LiBNYXhpbXVtIHN0cmluZyBsZW5ndGggaXMgMjY4LDQzNSw0NTYgVW5pY29kZSBjaGFyYWN0ZXJzICh3aGVyZSBlYWNoIFVuaWNvZGUgY2hhcmFjdGVyIGlzIHR3byBieXRlcykgb3IgNTM2LDg3MCw5MTIgYnl0ZXMuCiAgICBMb2dpY2FsID0gMTAsICMJQSBCb29sZWFuIHZhbHVlIG9mIGVpdGhlciB0cnVlIG9yIGZhbHNlLgogICAgTGlzdCA9IDExLCAjCUEgdmFsdWUgd2hpY2ggcHJvZHVjZXMgYSBzZXF1ZW5jZSBvZiB2YWx1ZXMgd2hlbiBlbnVtZXJhdGVkLiBGb3IgbW9yZSBpbmZvcm1hdGlvbiwgZ28gdG8gTGlzdCB0eXBlcyBhbmQgTGlzdCB2YWx1ZXMuCiAgICBSZWNvcmQgPSAxMiwgIwlBbiBvcmRlcmVkIHNlcXVlbmNlIG9mIGZpZWxkcy4gRWFjaCBmaWVsZCBjb250YWlucyBhIGZpZWxkIG5hbWUgYW5kIGZpZWxkIHZhbHVlLiBGb3IgbW9yZSBpbmZvcm1hdGlvbiwgZ28gdG8gUmVjb3JkIHR5cGVzIGFuZCBSZWNvcmQgdmFsdWVzLgogICAgVGFibGUgPSAxMywgIwlBbiBvcmRlcmVkIHNlcXVlbmNlIG9mIHJvd3MgZGl2aWRlZCBpbnRvIGNvbHVtbnMuIEZvciBtb3JlIGluZm9ybWF0aW9uLCBnbyB0byBUYWJsZSB0eXBlcyBhbmQgVGFibGUgdmFsdWVzLgogICAgRnVuY3Rpb24gPSAxNCwgIwlBIHZhbHVlIHRoYXQgbWFwcyBhIHNldCBvZiBhcmd1bWVudHMgdG8gYSBzaW5nbGUgdmFsdWUuIEZvciBtb3JlIGluZm9ybWF0aW9uLCBnbyB0byBGdW5jdGlvbnMgYW5kIEZ1bmN0aW9uIHR5cGVzLgogICAgTnVtYmVyID0gMTUsICMJUmVwcmVzZW50cyBhbnkgbnVtYmVyIHVzZWQgZm9yIG51bWVyaWMgYW5kIGFyaXRobWV0aWMgb3BlcmF0aW9ucy4gRm9yIG1vcmUgaW5mb3JtYXRpb24sIGdvIHRvIE51bWJlci4KICAgIE5vVmFsdWUgPSAxNiwgIwlUaGUgZGF0YSB0eXBlIHRoYXQgY2xhc3NpZmllcyBubyB2YWx1ZXMuCiAgICBMZXQgPSAxNywKICAgIExvb2t1cCA9IDE4LAogICAgRWFjaCA9IDE5LAogICAgUm9vdCA9IDIwLAogICAgQ29ubmVjdG9yID0gMjEsCiAgICBOYW1lZEl0ZW0gPSAyMiwKICAgIE1lYXN1cmUgPSAyMywKICAgIENvbHVtbiA9IDI0LAogICAgT3BlcmF0b3IgPSAyNSwKICAgIENvbmRpdGlvbmFsID0gMjYKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNTY2FsYXIoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYubmFtZSBpbiBbIk51bWJlciIsIlRleHQiLCJMb2dpY2FsIiwiRGF0ZSIsIlRpbWUiLCJEYXRlVGltZSIsIkRhdGVUaW1lWm9uZSIsIlRpbWUiXQogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYubmFtZQogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5uYW1lCiAgICAKICAgIGRlZiBfX2VxX18oc2VsZiwgb3RoZXIpOgogICAgICAgIHJldHVybiBzZWxmIGlzIG90aGVyIG9yIHNlbGYudmFsdWUgPT0gb3RoZXIKICAgIAogICAgZGVmIF9faGFzaF9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLnZhbHVlLl9faGFzaF9fKCkKICAgIAogICAgZGVmIHRvSWNvbihzZWxmKToKICAgICAgICBtYXRjaChzZWxmLm5hbWUpOgogICAgICAgICAgICBjYXNlICJBbnkiOgogICAgICAgICAgICAgICAgcmV0dXJuICIq77iP4oOjIgogICAgICAgICAgICBjYXNlICJCaW5hcnkiOgogICAgICAgICAgICAgICAgcmV0dXJuICLwn5SfIgogICAgICAgICAgICBjYXNlICJUZXh0IjoKICAgICAgICAgICAgICAgIHJldHVybiAi8J+UoSIKICAgICAgICAgICAgY2FzZSAiRGF0ZSI6CiAgICAgICAgICAgICAgICByZXR1cm4gIvCfk4UiCiAgICAgICAgICAgIGNhc2UgIlRpbWUiOgogICAgICAgICAgICAgICAgcmV0dXJuICLwn5WSIgogICAgICAgICAgICBjYXNlICJEYXRlVGltZSI6CiAgICAgICAgICAgICAgICByZXR1cm4gIvCfk4YiCiAgICAgICAgICAgIGNhc2UgIkRhdGVUaW1lWm9uZSI6CiAgICAgICAgICAgICAgICByZXR1cm4gIvCfk4YiCiAgICAgICAgICAgIGNhc2UgIkxvZ2ljYWwiOgogICAgICAgICAgICAgICAgcmV0dXJuICLwn4aXIgogICAgICAgICAgICBjYXNlICJOdW1iZXIiOgogICAgICAgICAgICAgICAgcmV0dXJuICLwn5SiIgogICAgICAgICAgICBjYXNlICJVbmtub3duIjoKICAgICAgICAgICAgICAgIHJldHVybiAiOm1hdGVyaWFsL3F1ZXN0aW9uX21hcms6IgogICAgICAgICAgICBjYXNlIF86CiAgICAgICAgICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBAc3RhdGljbWV0aG9kCiAgICBkZWYgRnJvbVZhbHVlKHZhbHVlOiBBbnkpIC0+IEV4cHJlc3Npb25UeXBlOgogICAgICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCiAgICAgICAgaW1wb3J0IGJpLnJlYWRlcnMucGJpLnRtZGwuZXhwcmVzc2lvbkluZm8KICAgICAgICBmcm9tIGRhdGV0aW1lIGltcG9ydCBkYXRlLCBkYXRldGltZSwgdHppbmZvCiAgICAgICAgaW1wb3J0IHBhbmRhcyBhcyBwZAogICAgICAgIHR5cGU6IEV4cHJlc3Npb25UeXBlID0gRXhwcmVzc2lvblR5cGUuVW5rbm93bgogICAgICAgIGlmIHZhbHVlIGlzIE5vbmU6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5OdWxsCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgdHlwZSA9IHZhbHVlLnR5cGUKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGJpLnJlYWRlcnMucGJpLnRtZGwuZXhwcmVzc2lvbkluZm8uRXhwcmVzc2lvbkluZm8pOgogICAgICAgICAgICB0eXBlID0gdmFsdWUudHlwZQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKToKICAgICAgICAgICAgdHlwZSA9IEV4cHJlc3Npb25UeXBlLlRleHQKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGludCk6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5OdW1iZXIKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGZsb2F0KToKICAgICAgICAgICAgdHlwZSA9IEV4cHJlc3Npb25UeXBlLk51bWJlcgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgYm9vbCk6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBieXRlcykgb3IgaXNpbnN0YW5jZSh2YWx1ZSwgYnl0ZWFycmF5KToKICAgICAgICAgICAgdHlwZSA9IEV4cHJlc3Npb25UeXBlLkJpbmFyeQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgbGlzdCk6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5MaXN0CiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBkYXRlKToKICAgICAgICAgICAgdHlwZSA9IEV4cHJlc3Npb25UeXBlLkRhdGUKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGRhdGV0aW1lKToKICAgICAgICAgICAgaWYgdmFsdWUudHppbmZvIGlzIE5vbmUgb3IgdmFsdWUudHppbmZvLnV0Y29mZnNldCh2YWx1ZSkgaXMgTm9uZToKICAgICAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5EYXRlVGltZQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgdHlwZSA9IEV4cHJlc3Npb25UeXBlLkRhdGVUaW1lWm9uZQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgZGljdCk6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5SZWNvcmQKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIHBkLkRhdGFGcmFtZSk6CiAgICAgICAgICAgIHR5cGUgPSBFeHByZXNzaW9uVHlwZS5UYWJsZQogICAgICAgIHJldHVybiB0eXBlCiAgICAgICAgICAgICAgICAKICAgIAogICAgQHN0YXRpY21ldGhvZAogICAgZGVmIEZyb21TdHJpbmcodmFsdWU6IHN0cikgLT4gRXhwcmVzc2lvblR5cGU6CiAgICAgICAgaWYgdmFsdWUgaXMgTm9uZToKICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLk5vVmFsdWUKICAgICAgICAKICAgICAgICBpZiB2YWx1ZS5lbmRzd2l0aCgiLlR5cGUiKToKICAgICAgICAgICAgdmFsdWUgPSB2YWx1ZVswOmxlbih2YWx1ZSktNV0KICAgICAgICAKICAgICAgICBtYXRjaCB2YWx1ZS5sb3dlcigpOgogICAgICAgICAgICBjYXNlICJudW1iZXIiOiAgICAKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5OdW1iZXIKICAgICAgICAgICAgY2FzZSAiZGVjaW1hbCI6CiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTnVtYmVyCiAgICAgICAgICAgIGNhc2UgInZhcmNoYXIiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLlRleHQKICAgICAgICAgICAgY2FzZSAibG9naWNhbCI6CiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTG9naWNhbAogICAgICAgICAgICBjYXNlICJib29sIjoKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsCiAgICAgICAgICAgIGNhc2UgImJvb2xlYW4iOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkxvZ2ljYWwKICAgICAgICAgICAgY2FzZSAiZG91YmxlIjogICAgCiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTnVtYmVyCiAgICAgICAgICAgIGNhc2UgImZsb2F0IjogICAgCiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTnVtYmVyCiAgICAgICAgICAgIGNhc2UgImludCI6ICAgIAogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLk51bWJlcgogICAgICAgICAgICBjYXNlICJzaG9ydCI6ICAgIAogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLk51bWJlcgogICAgICAgICAgICBjYXNlICJsb25nIjogICAgCiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTnVtYmVyCiAgICAgICAgICAgIGNhc2UgImludDMyIjogICAgCiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTnVtYmVyCiAgICAgICAgICAgIGNhc2UgImludDY0IjoKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5OdW1iZXIKICAgICAgICAgICAgY2FzZSAic3RyaW5nIjoKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5UZXh0CiAgICAgICAgICAgIGNhc2UgInZhcmlhbnQiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLlJlY29yZAogICAgICAgICAgICBjYXNlICJhcnJheSI6CiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuTGlzdAogICAgICAgICAgICBjYXNlICJ0aW1lc3RhbXAiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkRhdGVUaW1lCiAgICAgICAgICAgIGNhc2UgInRpbWVzdGFtcF9udHoiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkRhdGVUaW1lCiAgICAgICAgICAgIGNhc2UgInRpbWVzdGFtcF9sdHoiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkRhdGVUaW1lWm9uZQogICAgICAgICAgICBjYXNlICJ0aW1lc3RhbXBfdHoiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkRhdGVUaW1lWm9uZQogICAgICAgIGZvciB0IGluIEV4cHJlc3Npb25UeXBlOgogICAgICAgICAgICBpZiB0Lm5hbWUubG93ZXIoKSA9PSB2YWx1ZS5zdHJpcCgpLmxvd2VyKCk6CiAgICAgICAgICAgICAgICByZXR1cm4gdAogICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5Vbmtub3duCiAgICAKICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/runtime.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgRGljdAoKY2xhc3MgUnVudGltZSgpOgogICAgCiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgZnJvbSAuZXhwcmVzc2lvbkNvZGVUeXBlIGltcG9ydCBFeHByZXNzaW9uQ29kZVR5cGUKICAgICAgICBmcm9tIC5leHByZXNzaW9uUGFyc2VyIGltcG9ydCBFeHByZXNzaW9uUGFyc2VyCiAgICAgICAgc2VsZi5fX3BhcnNlcnM6IERpY3RbRXhwcmVzc2lvbkNvZGVUeXBlLCBFeHByZXNzaW9uUGFyc2VyXSA9IHt9CgogICAgZGVmIExvYWRFeHByZXNzaW9uUGFyc2VyKHNlbGYsIHBhcnNlcik6CiAgICAgICAgc2VsZi5fX3BhcnNlcnNbcGFyc2VyLkNvZGVUeXBlXSA9IHBhcnNlcgoKICAgIGRlZiBHZXRFeHByZXNzaW9uUGFyc2VyKHNlbGYsIGNvZGVUeXBlKToKICAgICAgICByZXR1cm4gc2VsZi5fX3BhcnNlcnMuZ2V0KGNvZGVUeXBlKQoKICAgIAogICAg'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/connector.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgT3B0aW9uYWwsIEFueSwgRGljdCwgU2V0Cgpmcm9tIGVudW0gaW1wb3J0IEVudW0KY2xhc3MgQ29ubmVjdG9yVHlwZShFbnVtKToKICAgIFVua25vd24gPSAwLAogICAgRGF0YWJhc2UgPSAxLAogICAgRmlsZSA9IDIsCiAgICBDYWNoZWQgPSAzLAogICAgTXVsdGlwbGUgPSA0CgpjbGFzcyBDb25uZWN0b3IoQUJDKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB0eXBlOiBDb25uZWN0b3JUeXBlLCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIHNlbGYuX19zb3VyY2V0eXBlID0gdHlwZQogICAgICAgIHNlbGYuX19jb25uZWN0b3JQcm9wZXJ0aWVzID0gcHJvcGVydGllcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lKHNlbGYpIC0+IE9wdGlvbmFsW3N0cl06CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0UHJvcGVydHkoIm5hbWUiKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBzb3VyY2VUeXBlKHNlbGYpIC0+IENvbm5lY3RvclR5cGU6CiAgICAgICAgcmV0dXJuIHNlbGYuX19zb3VyY2V0eXBlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRmlsZShzZWxmKSAtPiBib29sOgogICAgICAgIHJldHVybiAoc2VsZi5zb3VyY2VUeXBlID09IENvbm5lY3RvclR5cGUuRmlsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNDYWNoZWQoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gKHNlbGYuc291cmNlVHlwZSA9PSBDb25uZWN0b3JUeXBlLkNhY2hlZCkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNEYXRhYmFzZShzZWxmKSAtPiBib29sOgogICAgICAgIHJldHVybiAoc2VsZi5zb3VyY2VUeXBlID09IENvbm5lY3RvclR5cGUuRGF0YWJhc2UpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzTXVsdGlwbGUoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gKHNlbGYuc291cmNlVHlwZSA9PSBDb25uZWN0b3JUeXBlLk11bHRpcGxlKQogICAgCiAgICBkZWYgZ2V0UHJvcGVydHkoc2VsZiwgbmFtZTogc3RyLCBkZWZhdWx0VmFsdWU6IE9wdGlvbmFsW0FueV0gPSBOb25lKToKICAgICAgICByZXR1cm4gc2VsZi5fX2Nvbm5lY3RvclByb3BlcnRpZXMuZ2V0KG5hbWUsIGRlZmF1bHRWYWx1ZSkgCiAgICAKICAgIGRlZiBzZXRQcm9wZXJ0eShzZWxmLCBuYW1lOiBzdHIsIHZhbHVlOiBBbnkpOgogICAgICAgIHNlbGYuX19jb25uZWN0b3JQcm9wZXJ0aWVzW25hbWVdID0gdmFsdWUKICAgIAogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzZWxmLnNvdXJjZVR5cGUubmFtZX0iCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fcmVwcl9fKCkK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/expressionParent.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgVW5pb24sIEFueSwgRGljdCwgU2V0LCBPcHRpb25hbApmcm9tIC5leHByZXNzaW9uQ29kZVR5cGUgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIC5ydW50aW1lIGltcG9ydCBSdW50aW1lCmZyb20gLmlzc3VlIGltcG9ydCBJc3N1ZQpmcm9tIC5pc3N1ZUNvbnRhaW5lciBpbXBvcnQgSXNzdWVDb250YWluZXIsIElzc3VlRGV0YWlsCmZyb20gYmkuY29yZS5jb25uZWN0b3IgaW1wb3J0IENvbm5lY3RvciwgQ29ubmVjdG9yVHlwZQoKY2xhc3MgRXhwcmVzc2lvblBhcmVudChJc3N1ZUNvbnRhaW5lciwgQUJDKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLm1vZGVsSXRlbSBpbXBvcnQgTW9kZWxJdGVtCiAgICAgICAgZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgoKICAgICAgICBzZWxmLl9fY2hpbGRyZW46IFNldFtFeHByZXNzaW9uXSA9IHNldCgpCiAgICAgICAgc2VsZi5fX2RlcGVuZGVuY2llczogU2V0W01vZGVsSXRlbV0gPSBzZXQoKQogICAgICAgIElzc3VlQ29udGFpbmVyLl9faW5pdF9fKHNlbGYpCgogICAgQHByb3BlcnR5CiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiBSdW50aW1lKHNlbGYpIC0+IFJ1bnRpbWU6CiAgICAgICAgcGFzcwogICAgCiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiBjaGlsZEV4cHJlc3Npb25Db2RlVHlwZShzZWxmKSAtPiBFeHByZXNzaW9uQ29kZVR5cGU6CiAgICAgICAgcGFzcwoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGNoaWxkRXhwcmVzc2lvbnMoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19jaGlsZHJlbgogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBIYXNJc3N1ZXMoc2VsZik6CiAgICAgICAgIyBUcmFjayBvYmplY3RzIGN1cnJlbnRseSBiZWluZyBjaGVja2VkIHRvIHByZXZlbnQgaW5maW5pdGUgcmVjdXJzaW9uCiAgICAgICAgIyBVc2UgYSBjbGFzcyB2YXJpYWJsZSB0byBtYWludGFpbiBzdGF0ZSBhY3Jvc3MgdGhlIGNhbGwgc3RhY2sKICAgICAgICBpZiBub3QgaGFzYXR0cihFeHByZXNzaW9uUGFyZW50LCAnX2NoZWNraW5nX2hhc19pc3N1ZXMnKToKICAgICAgICAgICAgRXhwcmVzc2lvblBhcmVudC5fY2hlY2tpbmdfaGFzX2lzc3VlcyA9IHNldCgpCiAgICAgICAgCiAgICAgICAgIyBJZiB3ZSdyZSBhbHJlYWR5IGNoZWNraW5nIHRoaXMgb2JqZWN0LCByZXR1cm4gRmFsc2UgdG8gYnJlYWsgdGhlIGN5Y2xlCiAgICAgICAgb2JqX2lkID0gaWQoc2VsZikKICAgICAgICBpZiBvYmpfaWQgaW4gRXhwcmVzc2lvblBhcmVudC5fY2hlY2tpbmdfaGFzX2lzc3VlczoKICAgICAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAgICAgCiAgICAgICAgIyBNYXJrIHRoaXMgb2JqZWN0IGFzIGJlaW5nIGNoZWNrZWQKICAgICAgICBFeHByZXNzaW9uUGFyZW50Ll9jaGVja2luZ19oYXNfaXNzdWVzLmFkZChvYmpfaWQpCiAgICAgICAgdHJ5OgogICAgICAgICAgICByZXR1cm4gc3VwZXIoKS5IYXNJc3N1ZXMgb3IgYW55KGMuSGFzSXNzdWVzIGZvciBjIGluIHNlbGYuY2hpbGRFeHByZXNzaW9ucykKICAgICAgICBmaW5hbGx5OgogICAgICAgICAgICAjIEFsd2F5cyByZW1vdmUgZnJvbSB0aGUgc2V0IHdoZW4gZG9uZQogICAgICAgICAgICBFeHByZXNzaW9uUGFyZW50Ll9jaGVja2luZ19oYXNfaXNzdWVzLmRpc2NhcmQob2JqX2lkKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzQ2FjaGVkKHNlbGYpOgogICAgICAgIHJldHVybiBsZW4oc2VsZi5nZXRTb3VyY2VzKHR5cGUgPSBDb25uZWN0b3JUeXBlLkNhY2hlZCkpID4gMAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0ZpbGUoc2VsZik6CiAgICAgICAgcmV0dXJuIGxlbihzZWxmLmdldFNvdXJjZXModHlwZSA9IENvbm5lY3RvclR5cGUuRmlsZSkpID4gMAoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRGF0YWJhc2Uoc2VsZik6CiAgICAgICAgcmV0dXJuIGxlbihzZWxmLmdldFNvdXJjZXModHlwZSA9IENvbm5lY3RvclR5cGUuRGF0YWJhc2UpKSA+IDAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNTbm93Zmxha2Uoc2VsZik6CiAgICAgICAgZGF0YWJhc2VzID0gc2VsZi5nZXRTb3VyY2VzKHR5cGUgPSBDb25uZWN0b3JUeXBlLkRhdGFiYXNlKQogICAgICAgIHJldHVybiBhbnkoZGIuZ2V0UHJvcGVydHkoImRhdGFiYXNlVHlwZSIpID09ICJzbm93Zmxha2UiIGZvciBkYiBpbiBkYXRhYmFzZXMpCgogICAgZGVmIGdldFNvdXJjZXMoc2VsZiwgdHlwZTogQ29ubmVjdG9yVHlwZSA9IE5vbmUpOgogICAgICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCiAgICAgICAgc291cmNlczogU2V0W0Nvbm5lY3Rvcl0gPSBzZXQoKQogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZiwgQ29ubmVjdG9yKToKICAgICAgICAgICAgc291cmNlcy5hZGQoc2VsZikKCiAgICAgICAgZm9yIHNvdXJjZSBpbiBzZWxmLkdldEFsbERlcGVuZGVuY2llcygpOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNvdXJjZSwgQ29ubmVjdG9yKSBhbmQgKHR5cGUgaXMgTm9uZSBvciBzb3VyY2Uuc291cmNlVHlwZSA9PSB0eXBlKToKICAgICAgICAgICAgICAgIHNvdXJjZXMuYWRkKHNvdXJjZSkKICAgICAgICAgICAgZWxpZiBpc2luc3RhbmNlKHNvdXJjZSwgRXhwcmVzc2lvbikgYW5kIGlzaW5zdGFuY2Uoc291cmNlLnZhbHVlLCBDb25uZWN0b3IpIGFuZCAodHlwZSBpcyBOb25lIG9yIHNvdXJjZS52YWx1ZS5zb3VyY2VUeXBlID09IHR5cGUpOgogICAgICAgICAgICAgICAgc291cmNlcy5hZGQoc291cmNlLnZhbHVlKQogICAgICAgIHJldHVybiBzb3VyY2VzCiAgICAKICAgIGRlZiBnZXRTb3VyY2VQcm9wZXJ0aWVzKHNlbGYsIHNvdXJjZVR5cGU6IENvbm5lY3RvclR5cGUsIHByb3BlcnR5TmFtZTogc3RyKToKICAgICAgICBwcm9wZXJ0aWVzID0gc2V0KCkKICAgICAgICBmb3Igc291cmNlIGluIHNlbGYuZ2V0U291cmNlcyhzb3VyY2VUeXBlKToKICAgICAgICAgICAgcCA9IHNvdXJjZS5nZXRQcm9wZXJ0eShwcm9wZXJ0eU5hbWUpCiAgICAgICAgICAgIGlmIHAgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzLmFkZChwKQogICAgICAgIHJldHVybiBwcm9wZXJ0aWVzCiAgICAKICAgIGRlZiBHZXRBbGxJc3N1ZXMoc2VsZik6CiAgICAgICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KICAgICAgICBhbGxJc3N1ZXMgPSBzZXQoc2VsZi5Jc3N1ZXMpCgogICAgICAgIGZvciBjIGluIHNlbGYuY2hpbGRFeHByZXNzaW9uczoKICAgICAgICAgICAgaWYgYyBpcyBub3QgTm9uZSBhbmQgaXNpbnN0YW5jZShjLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIGFsbElzc3VlcyA9IGFsbElzc3Vlcy51bmlvbihjLkdldEFsbElzc3VlcygpKQoKICAgICAgICByZXR1cm4gYWxsSXNzdWVzCiAgICAKICAgIGRlZiBHZXRBbGxEZXBlbmRlbmNpZXMoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19kZXBlbmRlbmNpZXMKICAgICAgICAjIGFsbERlcGVuZGVuY2llcyA9IHNlbGYuX19kZXBlbmRlbmNpZXMKICAgICAgICAjIGZvciBjIGluIHNlbGYuY2hpbGRFeHByZXNzaW9uczoKICAgICAgICAjICAgICBpZiBjIGlzIG5vdCBOb25lOgogICAgICAgICMgICAgICAgICBpZiBjLmlzUmVmZXJlbmNlOgogICAgICAgICMgICAgICAgICAgICAgYWxsRGVwZW5kZW5jaWVzLmFkZChjKQogICAgICAgICMgICAgICAgICBjaGlsZERlcGVuZGVuY2llcyA9IGMuR2V0QWxsRGVwZW5kZW5jaWVzKCkKICAgICAgICAjICAgICAgICAgYWxsRGVwZW5kZW5jaWVzID0gYWxsRGVwZW5kZW5jaWVzLnVuaW9uKGNoaWxkRGVwZW5kZW5jaWVzKQogICAgICAgICMgcmV0dXJuIGFsbERlcGVuZGVuY2llcwoKICAgIGRlZiBHZXRJc3N1ZURldGFpbHMoc2VsZiwgaXNzdWU6IElzc3VlLCBpbmNsdWRlQ2hpbGRyZW46IGJvb2wgPSBGYWxzZSk6CiAgICAgICAgZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIHJlc3VsdHM6IERpY3RbRXhwcmVzc2lvbiwgU2V0W0lzc3VlRGV0YWlsXV0gPSB7fQogICAgICAgIAogICAgICAgIGRldGFpbHMgPSBzdXBlcigpLkdldElzc3VlRGV0YWlscyhpc3N1ZSkgCiAgICAgICAgaWYgbGVuKGRldGFpbHMpID4gMDoKICAgICAgICAgICAgcmVzdWx0c1tzZWxmXSA9IGRldGFpbHMKCiAgICAgICAgaWYgaW5jbHVkZUNoaWxkcmVuID09IFRydWU6CiAgICAgICAgICAgIGZvciBjIGluIHNlbGYuY2hpbGRFeHByZXNzaW9uczoKICAgICAgICAgICAgICAgIGNoaWxkSXNzdWVEZXRhaWxzID0gYy5HZXRJc3N1ZURldGFpbHMoaXNzdWUsIGluY2x1ZGVDaGlsZHJlbj1pbmNsdWRlQ2hpbGRyZW4pCiAgICAgICAgICAgICAgICBpZiBjaGlsZElzc3VlRGV0YWlscyBpcyBub3QgTm9uZSBhbmQgbGVuKGNoaWxkSXNzdWVEZXRhaWxzKSA+IDA6ICAgCiAgICAgICAgICAgICAgICAgICAgZm9yIGssdiBpbiBjaGlsZElzc3VlRGV0YWlscy5pdGVtcygpOgogICAgICAgICAgICAgICAgICAgICAgICByZXN1bHRzW2tdID0gdgoKICAgICAgICByZXR1cm4gcmVzdWx0cwoKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIGdldEV4cHJlc3Npb24oc2VsZiwgbmFtZTogc3RyKToKICAgICAgICBwYXNzCgogICAgZGVmIEFkZEV4cHJlc3Npb24oc2VsZiwgZXhwcmVzc2lvblN0cmluZzogVW5pb25bc3RyfGxpc3RdLCBjb2RlVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvbkNvZGVUeXBlXSA9IE5vbmUsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgZnJvbSAuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIGZyb20gLmV4cHJlc3Npb25QYXJzZXIgaW1wb3J0IEV4cHJlc3Npb25QYXJzZXIKICAgICAgICBleHByZXNzaW9uOiBFeHByZXNzaW9uID0gTm9uZQogICAgICAgIHBhcnNlcjogRXhwcmVzc2lvblBhcnNlciA9IE5vbmUKCiAgICAgICAgaWYgIm9yaWdpbmFsIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgcHJvcGVydGllc1sib3JpZ2luYWwiXSA9IGV4cHJlc3Npb25TdHJpbmcKCiAgICAgICAgIyBnZXQgdGhlIGV4cHJlc3Npb24gcGFyc2VyCiAgICAgICAgaWYgY29kZVR5cGUgaXMgTm9uZToKICAgICAgICAgICAgY29kZVR5cGUgPSBzZWxmLmNoaWxkRXhwcmVzc2lvbkNvZGVUeXBlCgogICAgICAgIGlmIGNvZGVUeXBlIGlzIG5vdCBOb25lOgogICAgICAgICAgICBpZiBzZWxmLlJ1bnRpbWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBwYXJzZXIgPSBzZWxmLlJ1bnRpbWUuR2V0RXhwcmVzc2lvblBhcnNlcihjb2RlVHlwZT1jb2RlVHlwZSkKICAgICAgICAKICAgICAgICBpZiBwYXJzZXIgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHRyeTogICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgZXhwcmVzc2lvbiA9IHBhcnNlci5DcmVhdGVGcm9tU3RyaW5nKHBhcmVudD1zZWxmLCBleHByZXNzaW9uU3RyaW5nPWV4cHJlc3Npb25TdHJpbmcsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICBwYXJlbnRFeHByZXNzaW9uID0gc2VsZi5zb3VyY2VTdHJpbmcKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgcGFyZW50RXhwcmVzc2lvbiA9IHN0cihzZWxmKQoKICAgICAgICAgICAgICAgIHByaW50KGYiRXJyb3IgY3JlYXRpbmcge2NvZGVUeXBlfSBFeHByZXNzaW9uXG57ZXhwcmVzc2lvblN0cmluZ31cbmZyb20gcGFyZW50Olxue3BhcmVudEV4cHJlc3Npb259XG57ZX0iKQogICAgICAgICAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlVHlwZSBpbXBvcnQgSXNzdWVUeXBlCiAgICAgICAgICAgICAgICBzZWxmLkFkZElzc3VlKHR5cGU9SXNzdWVUeXBlLlVuc3VwcG9ydGVkRmVhdHVyZSwgZGVzY3JpcHRpb249c3RyKGUpLCBkZXRhaWxzPWV4cHJlc3Npb25TdHJpbmcsIGV4Y2VwdGlvbj1lKQogICAgICAgICAgICAgICAgCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIk5vIHBhcnNlciBhdmFpbGFibGUgZm9yIHtjb2RlVHlwZX0iKQogICAgICAgIAogICAgICAgIGlmIGV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHNlbGYuX19jaGlsZHJlbi5hZGQoZXhwcmVzc2lvbikKICAgICAgICAgICAgaWYgZXhwcmVzc2lvbi5pc1JlZmVyZW5jZToKICAgICAgICAgICAgICAgIHNlbGYuQWRkRGVwZW5kZW5jeShleHByZXNzaW9uKQogICAgICAgICAgICAKICAgICAgICByZXR1cm4gZXhwcmVzc2lvbgoKICAgIGRlZiBBZGREZXBlbmRlbmN5KHNlbGYsIGRlcGVuZGVuY3kpOgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkubW9kZWxJdGVtIGltcG9ydCBNb2RlbEl0ZW0KICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIGlmIGRlcGVuZGVuY3kgaXMgbm90IE5vbmUgYW5kIGlzaW5zdGFuY2UoZGVwZW5kZW5jeSwgTW9kZWxJdGVtKSBhbmQgZGVwZW5kZW5jeSAhPSBzZWxmOgogICAgICAgICAgICBzZWxmLl9fZGVwZW5kZW5jaWVzLmFkZChkZXBlbmRlbmN5KQogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYsIEV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgc2VsZi5wYXJlbnQuQWRkRGVwZW5kZW5jeShkZXBlbmRlbmN5KQoKICAgIGRlZiBBZGRFeHByZXNzaW9uRnJvbVZhbHVlKHNlbGYsIHZhbHVlOiBBbnksIGNvZGVUeXBlOiBPcHRpb25hbFtFeHByZXNzaW9uQ29kZVR5cGVdID0gTm9uZSwgcHJvcGVydGllczogZGljdD17fSk6CiAgICAgICAgaWYgY29kZVR5cGUgaXMgTm9uZToKICAgICAgICAgICAgY29kZVR5cGUgPSBzZWxmLmNoaWxkRXhwcmVzc2lvbkNvZGVUeXBlCiAgICAgICAgcGFyc2VyID0gc2VsZi5SdW50aW1lLkdldEV4cHJlc3Npb25QYXJzZXIoY29kZVR5cGU9Y29kZVR5cGUpCiAgICAgICAgZXhwcmVzc2lvbiA9IHBhcnNlci5DcmVhdGVGcm9tVmFsdWUocGFyZW50PXNlbGYsIHZhbHVlPXZhbHVlLCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgcmV0dXJuIGV4cHJlc3Npb24KICAgIAogICAg'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/table.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSAuaXNzdWVDb250YWluZXIgaW1wb3J0IElzc3VlQ29udGFpbmVyCgpjbGFzcyBUYWJsZShJc3N1ZUNvbnRhaW5lciwgQUJDKToKICAgIGZyb20gLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICBkZWYgX19pbml0X18oc2VsZiwgbmFtZTogT3B0aW9uYWxbc3RyXSA9IE5vbmUpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oKQogICAgICAgIGZyb20gIC50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIHNlbGYuX19pdGVtczogTGlzdFtUYWJsZUl0ZW1dID0gW10KICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgZnJvbVN0cmluZyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSh3cmFwU3ViUXVlcnk9VHJ1ZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgY29sdW1ucyhzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KICAgICAgICBjb2x1bW5zOiBMaXN0W0NvbHVtbl0gPSBbXQogICAgICAgIGZvciBpIGluIHNlbGYuX19pdGVtczoKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShpLCBDb2x1bW4pOgogICAgICAgICAgICAgICAgY29sdW1ucy5hcHBlbmQoaSkKCiAgICAgICAgcmV0dXJuIGNvbHVtbnMKCiAgICBkZWYgYWRkSXRlbShzZWxmLCBpdGVtOiBUYWJsZUl0ZW0pOgogICAgICAgIHNlbGYuX19pdGVtcy5hcHBlbmQoaXRlbSkKICAgICAgICBpdGVtLnNldFRhYmxlKHNlbGYpCgogICAgZGVmIGdldEl0ZW0oc2VsZiwgbmFtZTogc3RyKToKICAgICAgICBmb3IgaXRlbSBpbiBzZWxmLl9faXRlbXM6CiAgICAgICAgICAgIGlmIGl0ZW0gPT0gbmFtZToKICAgICAgICAgICAgICAgIHJldHVybiBpdGVtCiAgICAgICAgcmV0dXJuIE5vbmUKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgYWxsSXRlbXMoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19pdGVtcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiByZWxhdGlvbnNoaXBzKHNlbGYpOgogICAgICAgIGZyb20gLnJlbGF0aW9uc2hpcCBpbXBvcnQgUmVsYXRpb25zaGlwCiAgICAgICAgcmVsYXRpb25zaGlwczogU2V0W1JlbGF0aW9uc2hpcF0gPSBzZXQoKQogICAgICAgIGZvciBpdGVtIGluIHNlbGYuX19pdGVtczoKICAgICAgICAgICAgZm9yIHIgaW4gaXRlbS5yZWxhdGlvbnNoaXBzOgogICAgICAgICAgICAgICAgcmVsYXRpb25zaGlwcy5hZGQocikKICAgICAgICByZXR1cm4gcmVsYXRpb25zaGlwcwogICAgCiAgICBAcHJvcGVydHkKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIGRhdGFJc0F2YWlsYWJsZShzZWxmKSAtPiBib29sOgogICAgICAgIHBhc3MKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNTcWxRdWVyeShzZWxmKToKICAgICAgICByZXR1cm4gVHJ1ZQoKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICBwYXNzCgogICAgZGVmIGdldFNxbFZhbHVlKHNlbGYsIHdyYXBTdWJRdWVyeTogYm9vbCA9IEZhbHNlKSAtPiBzdHI6CiAgICAgICAgcXVlcnkgPSBzZWxmLmdldFNxbCgpCiAgICAgICAgaWYgd3JhcFN1YlF1ZXJ5ID09IFRydWUgYW5kIHNlbGYuaXNTcWxRdWVyeToKICAgICAgICAgICAgcXVlcnkgPSBmIih7cXVlcnl9KSIKICAgICAgICByZXR1cm4gcXVlcnkKICAgIAoKICAgIGRlZiBnZXRJZGVudGlmaWVyKHNlbGYsIHF1b3RlTWl4ZWRDYXNlOiBib29sID0gRmFsc2UpIC0+IHN0cjoKICAgICAgICBmcm9tIHV0aWwuc25vd2ZsYWtlT2JqZWN0IGltcG9ydCBnZXRRdW90ZWRJZGVudGlmaWVyCiAgICAgICAgaWRlbnRpZmllciA9IHNlbGYubmFtZQogICAgICAgIGlmIGlkZW50aWZpZXIgaXMgbm90IE5vbmUgYW5kIHF1b3RlTWl4ZWRDYXNlOgogICAgICAgICAgICBpZGVudGlmaWVyID0gZ2V0UXVvdGVkSWRlbnRpZmllcihpZGVudGlmaWVyKQogICAgICAgIHJldHVybiBpZGVudGlmaWVyCgogICAgZGVmIGdldENvbHVtbihzZWxmLCBuYW1lOiBzdHIpOgogICAgICAgIGZyb20gYmkuY29yZS5jb2x1bW4gaW1wb3J0IENvbHVtbgogICAgICAgIGl0ZW0gPSBzZWxmLmdldEl0ZW0obmFtZSkKICAgICAgICBpZiBpc2luc3RhbmNlKGl0ZW0sIENvbHVtbik6CiAgICAgICAgICAgIHJldHVybiBpdGVtCiAgICAgICAgcmV0dXJuIE5vbmUKCiAgICBkZWYgZ2V0Q29sdW1uTmFtZXMoc2VsZiwgcXVvdGVNaXhlZENhc2U6IGJvb2wgPSBUcnVlLCBpbmNsdWRlVGFibGVBbGlhczogYm9vbCA9IEZhbHNlLCBpbmNsdWRlVGFibGVOYW1lOiBib29sID0gRmFsc2UsIHNob3dFeHByZXNzaW9uOiBib29sID0gVHJ1ZSkgLT4gTGlzdFtzdHJdOgogICAgICAgIGlmIGxlbihzZWxmLmNvbHVtbnMpID09IDA6CiAgICAgICAgICAgIHJldHVybiBbIioiXQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBbYy5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPXF1b3RlTWl4ZWRDYXNlLCBpbmNsdWRlVGFibGVBbGlhcz1pbmNsdWRlVGFibGVBbGlhcywgaW5jbHVkZVRhYmxlTmFtZT1pbmNsdWRlVGFibGVOYW1lLCBzaG93RXhwcmVzc2lvbj1zaG93RXhwcmVzc2lvbikgZm9yIGMgaW4gc2VsZi5jb2x1bW5zXQoKICAgCiAgICAK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/expressionCodeType.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGVudW0gaW1wb3J0IEVudW0KCmNsYXNzIEV4cHJlc3Npb25Db2RlVHlwZShFbnVtKToKICAgIE0gPSAxLAogICAgREFYID0gMiwKICAgIFNRTCA9IDMKCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYubmFtZQogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5uYW1lCiAgICAKICAgIEBzdGF0aWNtZXRob2QKICAgIGRlZiBGcm9tU3RyaW5nKHZhbHVlOiBzdHIpIC0+IEV4cHJlc3Npb25Db2RlVHlwZToKICAgICAgICBtYXRjaCB2YWx1ZS5sb3dlcigpOgogICAgICAgICAgICBjYXNlICJtIjogICAgCiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvbkNvZGVUeXBlLk0KICAgICAgICAgICAgY2FzZSAicG93ZXJxdWVyeSI6ICAgIAogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25Db2RlVHlwZS5NCiAgICAgICAgICAgIGNhc2UgImRheCI6CiAgICAgICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvbkNvZGVUeXBlLkRBWAogICAgICAgICAgICBjYXNlICJzcWwiOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25Db2RlVHlwZS5TUUwKICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCBvdGhlcik6CiAgICAgICAgaWYgaXNpbnN0YW5jZShvdGhlciwgRXhwcmVzc2lvbkNvZGVUeXBlKToKICAgICAgICAgICAgcmV0dXJuIG90aGVyLnZhbHVlID09IHNlbGYudmFsdWUKICAgICAgICBlbGlmIGlzaW5zdGFuY2Uob3RoZXIsIHN0cik6CiAgICAgICAgICAgIHJldHVybiBvdGhlci51cHBlcigpID09IHNlbGYubmFtZS51cHBlcigpCiAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi52YWx1ZS5fX2hhc2hfXygp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/conditional.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbiwgRXhwcmVzc2lvblR5cGUKZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsLCBEaWN0CgoKY2xhc3MgQ29uZGl0aW9uYWwoQUJDKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBjb25kaXRpb24sIHRydWVWYWx1ZSwgZmFsc2VWYWx1ZSk6CiAgICAgICAgc2VsZi5fX2NvbmRpdGlvbiA9IGNvbmRpdGlvbgogICAgICAgIHNlbGYuX190cnVlVmFsdWUgPSB0cnVlVmFsdWUKICAgICAgICBzZWxmLl9fZmFsc2VWYWx1ZSA9IGZhbHNlVmFsdWUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBjb25kaXRpb24oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19jb25kaXRpb24KICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgdHJ1ZVZhbHVlKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fdHJ1ZVZhbHVlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGZhbHNlVmFsdWUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19mYWxzZVZhbHVlCgogICAg'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/issue.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIC5pc3N1ZVR5cGUgaW1wb3J0IElzc3VlVHlwZQoKY2xhc3MgSXNzdWUoKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBwYXJlbnQsIHR5cGU6IElzc3VlVHlwZSwgZGVzY3JpcHRpb246IHN0cik6CiAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlQ29udGFpbmVyIGltcG9ydCBJc3N1ZUNvbnRhaW5lcgogICAgICAgIHNlbGYuX19wYXJlbnQ6IElzc3VlQ29udGFpbmVyID0gcGFyZW50CiAgICAgICAgc2VsZi50eXBlID0gdHlwZQogICAgICAgIHNlbGYuZGVzY3JpcHRpb24gPSBkZXNjcmlwdGlvbgogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBJc3N1ZUNvbnRhaW5lcihzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3BhcmVudAogICAgCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlKToKICAgICAgICBpZiBpc2luc3RhbmNlKHZhbHVlLCBJc3N1ZSk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLnR5cGUgPT0gdmFsdWUudHlwZSBhbmQgc2VsZi5kZXNjcmlwdGlvbiA9PSB2YWx1ZS5kZXNjcmlwdGlvbgogICAgICAgIHJldHVybiBGYWxzZQogICAgCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGhhc2goKHNlbGYudHlwZS5uYW1lLCBzZWxmLmRlc2NyaXB0aW9uKSkKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYudHlwZX0ge3NlbGYuZGVzY3JpcHRpb259IgogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19zdHJfXygpCgogICAgCiAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/script.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbiwgRXhwcmVzc2lvblR5cGUKZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsLCBEaWN0CgoKY2xhc3MgU2NyaXB0KEFCQyk6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc2VsZi5fX2V4cHJlc3Npb25zOiBEaWN0W3N0cixFeHByZXNzaW9uXSA9IHt9CiAgICAgICAgc2VsZi5fX3ZhbHVlOiBFeHByZXNzaW9uID0gTm9uZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBleHByZXNzaW9ucyhzZWxmKSAtPiBEaWN0W3N0cixFeHByZXNzaW9uXToKICAgICAgICByZXR1cm4gc2VsZi5fX2V4cHJlc3Npb25zCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHZhcmlhYmxlcyhzZWxmKSAtPiBMaXN0W3N0cl06CiAgICAgICAgcmV0dXJuIGxpc3Qoc2VsZi5leHByZXNzaW9ucy5rZXlzKCkpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgaWYgc2VsZi5fX3ZhbHVlIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5fX3ZhbHVlLnJldHVyblR5cGUKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuVW5rbm93bgogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiB2YWx1ZUV4cHJlc3Npb24oc2VsZikgLT4gRXhwcmVzc2lvbjoKICAgICAgICByZXR1cm4gc2VsZi5fX3ZhbHVlCiAgICAKICAgIGRlZiBBZGRWYXJpYWJsZUV4cHJlc3Npb24oc2VsZiwgdmFyaWFibGVOYW1lOiBzdHIsIGV4cHJlc3Npb246IEV4cHJlc3Npb24pOgogICAgICAgIHNlbGYuX19leHByZXNzaW9uc1t2YXJpYWJsZU5hbWVdID0gZXhwcmVzc2lvbgoKICAgIGRlZiBTZXRWYWx1ZShzZWxmLCB2YWx1ZTogRXhwcmVzc2lvbik6CiAgICAgICAgc2VsZi5fX3ZhbHVlID0gdmFsdWUKCiAgICBkZWYgZ2V0VmFyaWFibGUoc2VsZiwgdmFyaWFibGVOYW1lKSAtPiBPcHRpb25hbFtFeHByZXNzaW9uXToKICAgICAgICByZXR1cm4gc2VsZi5fX2V4cHJlc3Npb25zLmdldCh2YXJpYWJsZU5hbWUp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/expression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgT3B0aW9uYWwsIEFueSwgTGlzdCwgVW5pb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudCwgRXhwcmVzc2lvbkNvZGVUeXBlCmZyb20gYmkuY29yZS5jb25uZWN0b3IgaW1wb3J0IENvbm5lY3RvciwgQ29ubmVjdG9yVHlwZQpmcm9tIGRhdGV0aW1lIGltcG9ydCBkYXRlLCB0aW1lLCBkYXRldGltZQoKY2xhc3MgRXhwcmVzc2lvbihFeHByZXNzaW9uUGFyZW50LCBBQkMpOgogICAgQHN0YXRpY21ldGhvZAogICAgZGVmIENvbnZlcnRUb1NxbFZhbHVlKHZhbHVlOiBVbmlvbltFeHByZXNzaW9uLHN0cixpbnQsZmxvYXQsYm9vbCxkYXRlLHRpbWUsZGF0ZXRpbWUsbGlzdCxkaWN0XSk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgICAgIGZyb20gYmkuY29yZS5vcGVyYXRvciBpbXBvcnQgT3BlcmF0b3IKICAgICAgICBpZiB2YWx1ZSBpcyBOb25lOgogICAgICAgICAgICByZXR1cm4gIk5VTEwiCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgcmV0dXJuIHZhbHVlLmdldFNxbFZhbHVlKHdyYXBTdWJRdWVyeT12YWx1ZS53cmFwU3FsT3V0cHV0KQogICAgICAgIGVsaWYgaGFzYXR0cih2YWx1ZSwgImdldFNxbFZhbHVlIik6CiAgICAgICAgICAgIHJldHVybiB2YWx1ZS5nZXRTcWxWYWx1ZSgpCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBUYWJsZSk6CiAgICAgICAgICAgIHJldHVybiB2YWx1ZS5nZXRTcWxWYWx1ZSgpCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBUYWJsZUl0ZW0pOgogICAgICAgICAgICByZXR1cm4gdmFsdWUuZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1UcnVlLCBpbmNsdWRlVGFibGVBbGlhcz1UcnVlLCBpbmNsdWRlVGFibGVOYW1lPUZhbHNlKQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKToKICAgICAgICAgICAgcmV0dXJuIGYiJ3t2YWx1ZX0nIgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgaW50KSBvciBpc2luc3RhbmNlKHZhbHVlLCBmbG9hdCk6CiAgICAgICAgICAgIHJldHVybiBmInt2YWx1ZX0iCiAgICAgICAgZWxpZiB2YWx1ZSA9PSAwOgogICAgICAgICAgICByZXR1cm4gIjAiCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBib29sKToKICAgICAgICAgICAgaWYgdmFsdWUgPT0gVHJ1ZToKICAgICAgICAgICAgICAgIHJldHVybiAiVFJVRSIKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHJldHVybiAiRkFMU0UiCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBkYXRlKToKICAgICAgICAgICAgcmV0dXJuIGYiREFURV9GUk9NX1BBUlRTKHt2YWx1ZS55ZWFyfSwge3ZhbHVlLm1vbnRofSwge3ZhbHVlLmRheX0pIgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgdGltZSk6CiAgICAgICAgICAgIHJldHVybiBmIlRPX1RJTUUoe3ZhbHVlLmhvdXJ9LCB7dmFsdWUubWludXRlfSwge3ZhbHVlLnNlY29uZH0sIHt2YWx1ZS5taWNyb3NlY29uZH0pIgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgZGF0ZXRpbWUpOgogICAgICAgICAgICBpZiB2YWx1ZS50emluZm8gaXMgTm9uZToKICAgICAgICAgICAgICAgIHJldHVybiBmIlRJTUVTUEFOX05UWl9GUk9NX1BBUlRTKHt2YWx1ZS55ZWFyfSwge3ZhbHVlLm1vbnRofSwge3ZhbHVlLmRheX0sIHt2YWx1ZS5ob3VyfSwge3ZhbHVlLm1pbnV0ZX0sIHt2YWx1ZS5zZWNvbmR9LCB7dmFsdWUubWljcm9zZWNvbmR9KSIKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHJldHVybiBmIlRJTUVTUEFOX1RaX0ZST01fUEFSVFMoe3ZhbHVlLnllYXJ9LCB7dmFsdWUubW9udGh9LCB7dmFsdWUuZGF5fSwge3ZhbHVlLmhvdXJ9LCB7dmFsdWUubWludXRlfSwge3ZhbHVlLnNlY29uZH0sIHt2YWx1ZS5taWNyb3NlY29uZH0sIHt2YWx1ZS50emluZm8udHpuYW1lfSkiCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBsaXN0KToKICAgICAgICAgICAgdmFsdWVzID0gW10KICAgICAgICAgICAgZm9yIHYgaW4gdmFsdWU6CiAgICAgICAgICAgICAgICB2YWx1ZXMuYXBwZW5kKEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUodikpCiAgICAgICAgICAgIHJldHVybiBmIkFSUkFZX0NPTlNUUlVDVCh7JywnLmpvaW4odmFsdWVzKX0pIgogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc2V0KToKICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUobGlzdCh2YWx1ZSkpCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBkaWN0KToKICAgICAgICAgICAgbWVtYmVycyA9IFtdCiAgICAgICAgICAgIGZvciBrLHYgaW4gdmFsdWUuaXRlbXMoKToKICAgICAgICAgICAgICAgIG1lbWJlcnMuYXBwZW5kKGYiJ3trfSciKQogICAgICAgICAgICAgICAgbWVtYmVycy5hcHBlbmQoRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZSh2KSkKICAgICAgICAgICAgcmV0dXJuIGYiT0JKRUNUX0NPTlNUUlVDVCh7JywnLmpvaW4obWVtYmVycyl9KSIKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGJ5dGVzKToKICAgICAgICAgICAgcmV0dXJuIHZhbHVlLmRlY29kZSgibGF0aW4iKQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgQ29ubmVjdG9yKToKICAgICAgICAgICAgcmV0dXJuICIiCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIkNhbid0IGNvbnZlcnQge3R5cGUodmFsdWUpfSBpbnRvIFNRTCB2YWx1ZSIpCiAgICAgICAgCiAgICBmcm9tIGJpLmNvcmUuaXNzdWVUeXBlIGltcG9ydCBJc3N1ZVR5cGUKCiAgICBkZWYgX19pbml0X18oc2VsZiwgY29kZVR5cGU6IEV4cHJlc3Npb25Db2RlVHlwZSwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIEV4cHJlc3Npb25QYXJlbnQuX19pbml0X18oc2VsZikKCiAgICAgICAgaWYgcHJvcGVydGllcyBpcyBOb25lOgogICAgICAgICAgICBwcm9wZXJ0aWVzID0ge30KICAgICAgICAKICAgICAgICBzZWxmLl9fcHJvcGVydGllcyA9IHByb3BlcnRpZXMKICAgICAgICBzZWxmLl9fcGFyZW50ID0gcGFyZW50CiAgICAgICAgc2VsZi5fX2NvZGVUeXBlID0gY29kZVR5cGUKICAgICAgICBzZWxmLl9fdmFsdWUgPSBOb25lCiAgICAgICAgc291cmNlU3RyaW5nOiBzdHIgPSBzZWxmLnByb3BlcnRpZXMuZ2V0KCJvcmlnaW5hbCIsIiIpCiAgICAgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZShzb3VyY2VTdHJpbmcsIGxpc3QpOgogICAgICAgICAgICBzb3VyY2VTdHJpbmcgPSAiXG4iLmpvaW4oc291cmNlU3RyaW5nKQogICAgICAgIHNlbGYuX19zb3VyY2VTdHJpbmcgPSBzb3VyY2VTdHJpbmcKICAgICAgICAKCiAgICBAcHJvcGVydHkKICAgIGRlZiBjb2RlVHlwZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2NvZGVUeXBlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgdHlwZShzZWxmKSAtPiBFeHByZXNzaW9uVHlwZToKICAgICAgICBwYXNzCgogICAgQHByb3BlcnR5CiAgICBkZWYgTWV0YWRhdGEoc2VsZikgLT4gZGljdDoKICAgICAgICByZXR1cm4ge30KCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0Z1bmN0aW9uKHNlbGYpIC0+IGJvb2w6CiAgICAgICAgZnJvbSAuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCiAgICAgICAgcmV0dXJuIGlzaW5zdGFuY2Uoc2VsZiwgRnVuY3Rpb24pCiAgICAKICAgIEBwcm9wZXJ0eQogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgaXNTY3JpcHQoc2VsZikgLT4gYm9vbDoKICAgICAgICBwYXNzCgogICAgQHByb3BlcnR5CiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiBpc1ZhbHVlKHNlbGYpIC0+IGJvb2w6CiAgICAgICAgcGFzcwoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBhcnNlcihzZWxmKToKICAgICAgICBpZiBzZWxmLlJ1bnRpbWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLlJ1bnRpbWUuR2V0RXhwcmVzc2lvblBhcnNlcihzZWxmLmNvZGVUeXBlKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzUmVmZXJlbmNlKHNlbGYpIC0+IGJvb2w6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIGZyb20gYmkuY29yZS5jb25uZWN0b3IgaW1wb3J0IENvbm5lY3RvcgogICAgICAgIGZyb20gYmkuY29yZS5mdW5jdGlvbiBpbXBvcnQgRnVuY3Rpb24KICAgICAgICByZXR1cm4gKGlzaW5zdGFuY2Uoc2VsZiwgVGFibGUpIG9yIAogICAgICAgICAgICAgICAgaXNpbnN0YW5jZShzZWxmLCBUYWJsZUl0ZW0pIG9yIAogICAgICAgICAgICAgICAgaXNpbnN0YW5jZShzZWxmLCBDb25uZWN0b3IpIG9yIAogICAgICAgICAgICAgICAgKGlzaW5zdGFuY2Uoc2VsZiwgRnVuY3Rpb24pIGFuZCBpc2luc3RhbmNlKHNlbGYudmFsdWUsIEV4cHJlc3Npb24pIGFuZCBzZWxmLnZhbHVlLmlzUmVmZXJlbmNlKSBvcgogICAgICAgICAgICAgICAgKGlzaW5zdGFuY2Uoc2VsZi52YWx1ZSwgRXhwcmVzc2lvbikgYW5kIHNlbGYuaXNTY3JpcHQgYW5kICBzZWxmLnZhbHVlLmlzUmVmZXJlbmNlKQogICAgICAgICAgICAgICAgKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHNvdXJjZVN0cmluZyhzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgIHJldHVybiBzZWxmLl9fc291cmNlU3RyaW5nCiAgICAgICAgICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcHJvcGVydGllcyhzZWxmKSAtPiBkaWN0OgogICAgICAgIHJldHVybiBzZWxmLl9fcHJvcGVydGllcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwYXJlbnQoc2VsZikgLT4gRXhwcmVzc2lvblBhcmVudDoKICAgICAgICByZXR1cm4gc2VsZi5fX3BhcmVudAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBNb2RlbChzZWxmKToKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLm1vZGVsIGltcG9ydCBNb2RlbAogICAgICAgIG1vZGVsOiBNb2RlbCA9IE5vbmUKICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYucGFyZW50LCBNb2RlbCk6CiAgICAgICAgICAgIG1vZGVsID0gc2VsZi5wYXJlbnQKICAgICAgICBlbGlmIHNlbGYucGFyZW50IGlzIG5vdCBOb25lIGFuZCBoYXNhdHRyKHNlbGYucGFyZW50LCAiTW9kZWwiKToKICAgICAgICAgICAgbW9kZWwgPSBzZWxmLnBhcmVudC5Nb2RlbAogICAgICAgIHJldHVybiBtb2RlbAogICAgCiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgcGFzcwoKICAgIGRlZiBVcGRhdGUoc2VsZiwgZm9yY2U6IGJvb2wgPSBGYWxzZSwgdXBkYXRlQWxsQ2hpbGRyZW46IGJvb2wgPSBGYWxzZSk6CiAgICAgICAgdmFsdWUgPSBzZWxmLl9fdmFsdWUKICAgICAgICBpZiB2YWx1ZSBpcyBOb25lIG9yIGZvcmNlID09IFRydWU6CiAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgIHZhbHVlID0gc2VsZi51cGRhdGUoKQogICAgICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgICAgICB2YWx1ZSA9IHNlbGYucGFyc2VyLkNyZWF0ZUZyb21WYWx1ZShwYXJlbnQ9c2VsZiwgdmFsdWU9Tm9uZSkKICAgICAgICAgICAgICAgIHByaW50KGYiRXJyb3IgY2FsbGluZyB1cGRhdGUgb24ge3NlbGZ9OiB7ZX0iKQogICAgICAgIGlmIHVwZGF0ZUFsbENoaWxkcmVuID09IFRydWU6CiAgICAgICAgICAgIGZvciBjIGluIHNlbGYuY2hpbGRFeHByZXNzaW9uczoKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoYywgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgYy5VcGRhdGUoZm9yY2U9Zm9yY2UsIHVwZGF0ZUFsbENoaWxkcmVuPXVwZGF0ZUFsbENoaWxkcmVuKQogICAgICAgIHJldHVybiB2YWx1ZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBPd25lcihzZWxmKSAtPiBBbnk6CiAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgIHJldHVybiBzZWxmLnBhcmVudC5Pd25lcgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBzZWxmLnBhcmVudAogICAgICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgdmFsdWUoc2VsZikgLT4gQW55OgogICAgICAgIGlmIHNlbGYuX192YWx1ZSBpcyBOb25lOgogICAgICAgICAgICBzZWxmLl9fdmFsdWUgPSBzZWxmLlVwZGF0ZSgpCgogICAgICAgIHJldHVybiBzZWxmLl9fdmFsdWUKICAgIAogICAgQHByb3BlcnR5CiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiByZXR1cm5UeXBlKHNlbGYpIC0+IEV4cHJlc3Npb25UeXBlOgogICAgICAgcGFzcwoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHdyYXBTcWxPdXRwdXQoc2VsZikgLT4gYm9vbDoKICAgICAgICBmcm9tIGJpLmNvcmUub3BlcmF0b3IgaW1wb3J0IE9wZXJhdG9yCiAgICAgICAgcmV0dXJuIHNlbGYucmV0dXJuVHlwZSA9PSBFeHByZXNzaW9uVHlwZS5UYWJsZSBvciBpc2luc3RhbmNlKHNlbGYsIE9wZXJhdG9yKQoKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaCgoc2VsZi5fX3NvdXJjZVN0cmluZywgc2VsZi5fX2NvZGVUeXBlLnZhbHVlLCBzZWxmLnR5cGUudmFsdWUpKQogICAgICAgIAogICAgIyB1c2VkIHRvIHJldHVybiBwcm9wZXJseSBmb3JtYXRlZCBTbm93Zmxha2UgU1FMIGZvciB0aGUgZ2l2ZW4gZXhwcmVzc2lvbgogICAgQGFic3RyYWN0bWV0aG9kCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIHBhc3MKCiAgICBkZWYgZ2V0U3FsVmFsdWUoc2VsZiwgd3JhcFN1YlF1ZXJ5OiBib29sID0gRmFsc2UpIC0+IHN0cjoKICAgICAgICBxdWVyeSA9IHNlbGYuZ2V0U3FsKCkKICAgICAgICBpZiB3cmFwU3ViUXVlcnkgPT0gVHJ1ZToKICAgICAgICAgICAgcXVlcnkgPSBmIih7cXVlcnl9KSIKICAgICAgICByZXR1cm4gcXVlcnkKICAgIAogICAgZGVmIEdldEFsbERlcGVuZGVuY2llcyhzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCiAgICAgICAgZnJvbSBiaS5jb3JlLnNjcmlwdCBpbXBvcnQgU2NyaXB0CiAgICAgICAgZGVwZW5kZW5jaWVzID0gc3VwZXIoKS5HZXRBbGxEZXBlbmRlbmNpZXMoKQogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZiwgRnVuY3Rpb24pIG9yIGlzaW5zdGFuY2Uoc2VsZiwgU2NyaXB0KToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnZhbHVlLCBFeHByZXNzaW9uKSBhbmQgc2VsZi52YWx1ZS5pc1JlZmVyZW5jZToKICAgICAgICAgICAgICAgIGRlcGVuZGVuY2llcy5hZGQoc2VsZi52YWx1ZSkKICAgICAgICByZXR1cm4gZGVwZW5kZW5jaWVzCiAgICAKICAgICMgcmV0dXJuIHRoZSBleHByZXNzaW9uIGZvciB0aGUgZ2l2ZW4gdmFyaWFibGUgb3IgTm9uZSBpZiBpdCBkb2Vzbid0IGV4aXN0CiAgICAjIHRoaXMgd2lsbCB0cmF2ZXJzZSB0aGUgZW52aXJvbm1lbnQgKHVwIHRoZSBwYXJlbnQgdHJlZSkKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIG5hbWU6IHN0cikgLT4gT3B0aW9uYWxbRXhwcmVzc2lvbl06CiAgICAgICAgZXhwcmVzc2lvbiA9IE5vbmUKICAgICAgICBpZiBzZWxmLnBhcmVudCBpcyBub3QgTm9uZToKICAgICAgICAgICAgZXhwcmVzc2lvbiA9IHNlbGYucGFyZW50LmdldEV4cHJlc3Npb24obmFtZSkKICAgICAgICBpZiBleHByZXNzaW9uIGlzIE5vbmUgYW5kIHNlbGYuTW9kZWwgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGV4cHJlc3Npb24gPSBzZWxmLk1vZGVsLkdldFBhcmFtZXRlclZhbHVlKG5hbWUpCiAgICAgICAgcmV0dXJuIGV4cHJlc3Npb24KICAgIAogICAgZGVmIGdldEhpc3Rvcnkoc2VsZik6CiAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgIGhpc3RvcnkgPSBzZWxmLnBhcmVudC5nZXRIaXN0b3J5KCkKICAgICAgICBlbHNlOgogICAgICAgICAgICBoaXN0b3J5OiBMaXN0W0V4cHJlc3Npb25dID0gW10KCiAgICAgICAgaGlzdG9yeS5hcHBlbmQoc2VsZikKICAgICAgICByZXR1cm4gaGlzdG9yeQoKICAgIGRlZiBBZGRJc3N1ZShzZWxmLCB0eXBlOiBJc3N1ZVR5cGUsIGRlc2NyaXB0aW9uOiBzdHIsIGRldGFpbHM6IE9wdGlvbmFsW3N0cl09IiIsIHNvdXJjZTogT3B0aW9uYWxbc3RyXSA9IE5vbmUsIGV4Y2VwdGlvbjogT3B0aW9uYWxbRXhjZXB0aW9uXSA9IE5vbmUpOgogICAgICAgIGlmIHNvdXJjZSBpcyBOb25lIG9yIHNvdXJjZSA9PSAiIjoKICAgICAgICAgICAgc291cmNlID0gc2VsZi5zb3VyY2VTdHJpbmcKCiAgICAgICAgcmV0dXJuIHN1cGVyKCkuQWRkSXNzdWUodHlwZT10eXBlLCBkZXNjcmlwdGlvbj1kZXNjcmlwdGlvbiwgc291cmNlPXNvdXJjZSwgZGV0YWlscz1kZXRhaWxzLCBleGNlcHRpb249ZXhjZXB0aW9uKQoKRXhwcmVzc2lvblBhcmVudC5yZWdpc3RlcihFeHByZXNzaW9uKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/function.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQsIGNhc3QKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZSAgICAKCmNsYXNzIEZ1bmN0aW9uKEFCQyk6CiAgICBkZWYgX19pbml0X18oc2VsZiwgCiAgICAgICAgICAgICAgICAgZnVuY3Rpb25OYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIHBhcmFtZXRlclN0cmluZzogc3RyCiAgICAgICAgICAgICAgICAgKToKICAgICAgICBzZWxmLl9fZnVuY3Rpb25OYW1lID0gZnVuY3Rpb25OYW1lCiAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnM6IExpc3RbRXhwcmVzc2lvbl0gPSBOb25lCiAgICAgICAgc2VsZi5fX3JldHVyblR5cGU6IEV4cHJlc3Npb25UeXBlID0gTm9uZQogICAgICAgIHNlbGYuX19yZXN1bHQ6IEV4cHJlc3Npb24gPSBOb25lCgogICAgICAgIGlmIHBhcmFtZXRlclN0cmluZyBpcyBub3QgTm9uZSBhbmQgcGFyYW1ldGVyU3RyaW5nLnN0cmlwKCkgIT0gIiI6CiAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgIHNlbGYuX19wYXJhbWV0ZXJzID0gc2VsZi5DcmVhdGVQYXJhbWV0ZXJzRnJvbVN0cmluZyhwYXJhbWV0ZXJTdHJpbmcpCiAgICAgICAgICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZToKICAgICAgICAgICAgICAgIHByaW50KGYiRXJyb3IgY3JlYXRpbmcgcGFyYW1ldGVyIGZyb20gc3RyaW5nIHtwYXJhbWV0ZXJTdHJpbmd9IGluIEZ1bmN0aW9uIHtmdW5jdGlvbk5hbWV9OiB7ZX0iKQogICAgICAgICAgICAgICAgaWYgc2VsZi5fX3BhcmFtZXRlcnMgaXMgTm9uZToKICAgICAgICAgICAgICAgICAgICBzZWxmLl9fcGFyYW1ldGVycyA9IFtdCiAgICAgICAgZWxzZToKICAgICAgICAgICAgc2VsZi5fX3BhcmFtZXRlcnMgPSBbXQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGZ1bmN0aW9uTmFtZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2Z1bmN0aW9uTmFtZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBleHByZXNzaW9uKHNlbGYpOgogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZiwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgIHJldHVybiBjYXN0KEV4cHJlc3Npb24sIHNlbGYpCiAgICAgICAgcmV0dXJuIE5vbmUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBwYXJhbWV0ZXJzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fcGFyYW1ldGVycyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgTW9kZWwoc2VsZik6CiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5leHByZXNzaW9uLk1vZGVsIAogICAgICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgRnVuY3Rpb25FdmFsdWF0b3Ioc2VsZik6CiAgICAgICAgaWYgc2VsZi5Nb2RlbCBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuTW9kZWwuRnVuY3Rpb25FdmFsdWF0b3IKCiAgICBAcHJvcGVydHkKICAgIGRlZiByZXR1cm5UeXBlKHNlbGYpIC0+IEV4cHJlc3Npb25UeXBlOgogICAgICAgIGlmIHNlbGYuX19yZXR1cm5UeXBlIGlzIE5vbmU6CiAgICAgICAgICAgIGlmIHNlbGYuRnVuY3Rpb25FdmFsdWF0b3IgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBzZWxmLl9fcmV0dXJuVHlwZSA9IHNlbGYuRnVuY3Rpb25FdmFsdWF0b3IuZ2V0UmV0dXJuVHlwZShzZWxmKQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgcGFzcwogICAgICAgICAgICAgICAgI3ByaW50KGYiIyMjIE1pc3NpbmcgSGFuZGxlcjoge3NlbGYuZnVuY3Rpb25OYW1lfSIpCiAgICAgICAgcmV0dXJuIHNlbGYuX19yZXR1cm5UeXBlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGhlbHBVcmwoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBOb25lCgogICAgZGVmIEdldFJlc3VsdChzZWxmLCBmb3JjZTogYm9vbCA9IEZhbHNlKToKICAgICAgICBpZiBzZWxmLl9fcmVzdWx0IGlzIE5vbmUgb3IgZm9yY2UgPT0gVHJ1ZToKICAgICAgICAgICAgaWYgc2VsZi5GdW5jdGlvbkV2YWx1YXRvciBpcyBub3QgTm9uZSBhbmQgc2VsZi5Nb2RlbC5zdGF0ZSA9PSAiUmVhZHkiOgogICAgICAgICAgICAgICAgc2VsZi5fX3Jlc3VsdCA9IHNlbGYuRnVuY3Rpb25FdmFsdWF0b3IuRXZhbHVhdGUoc2VsZikKICAgICAgICAgICAgICAgIAogICAgICAgIHJldHVybiBzZWxmLl9fcmVzdWx0CiAgICAKICAgIGRlZiBDcmVhdGVQYXJhbWV0ZXJzRnJvbVN0cmluZyhzZWxmLCBwYXJhbWV0ZXJTdHJpbmc6IHN0cik6CiAgICAgICAgcGFyYW1ldGVyczogTGlzdFtFeHByZXNzaW9uXSA9IFtdCiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lIGFuZCBzZWxmLmV4cHJlc3Npb24ucGFyc2VyIGlzIG5vdCBOb25lOgogICAgICAgICAgICBwYXJhbWV0ZXJTdHJpbmdzOiBMaXN0W3N0cl0gPSBzZWxmLmV4cHJlc3Npb24ucGFyc2VyLlBhcnNlRXhwcmVzc2lvblN0cmluZyhwYXJhbWV0ZXJTdHJpbmcpCiAgICAKICAgICAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlIGltcG9ydCBJc3N1ZVR5cGUKICAgICAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlQ29udGFpbmVyIGltcG9ydCBJc3N1ZUNvbnRhaW5lcgoKICAgICAgICAgICAgZm9yIHBzIGluIHBhcmFtZXRlclN0cmluZ3M6CgogICAgICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgICAgIHBhcmFtZXRlciA9IHNlbGYuZXhwcmVzc2lvbi5BZGRFeHByZXNzaW9uKHBzLCBwcm9wZXJ0aWVzPXt9KQogICAgICAgICAgICAgICAgZXhjZXB0IEV4Y2VwdGlvbiBhcyBlOgogICAgICAgICAgICAgICAgICAgIHByaW50KGYiRXJyb3IgY3JlYXRpbmcgcGFyYW1ldGVyIHtwc30gaW4gZnVuY3Rpb24ge3NlbGYuZnVuY3Rpb25OYW1lfToge2V9IikKCiAgICAgICAgICAgICAgICBpZiBwYXJhbWV0ZXIgaXMgTm9uZToKICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYsIElzc3VlQ29udGFpbmVyKToKICAgICAgICAgICAgICAgICAgICAgICAgc2VsZi5BZGRJc3N1ZSh0eXBlPUlzc3VlVHlwZS5QYXJzaW5nRXJyb3IsIGRlc2NyaXB0aW9uPWYiUGFyYW1ldGVyIGV4cHJlc3Npb24gZmFpbGVkIHRvIHBhcnNlIiwgZGV0YWlscz1wcykKICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICBwYXJhbWV0ZXJzLmFwcGVuZChwYXJhbWV0ZXIpCgogICAgICAgIHJldHVybiBwYXJhbWV0ZXJzCiAgICAKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3N0cl9fKCkKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcGFyYW1ldGVycyA9ICIiCiAgICAgICAgZm9yIHAgaW4gc2VsZi5wYXJhbWV0ZXJzOgogICAgICAgICAgICBwYXJhbWV0ZXJzICs9IHN0cihwKSArICIsICIKCiAgICAgICAgcGFyYW1ldGVycyA9IHBhcmFtZXRlcnMucnN0cmlwKCcsICcpCgogICAgICAgIHJldHVybiBmIntzZWxmLmZ1bmN0aW9uTmFtZX0oe3BhcmFtZXRlcnN9KSAtPiB7c2VsZi5leHByZXNzaW9uLnZhbHVlfSIK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/core/column.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSAuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIC5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCmZyb20gLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCmZyb20gLnRhYmxlIGltcG9ydCBUYWJsZQoKY2xhc3MgQ29sdW1uKFRhYmxlSXRlbSk6CiAgICBkZWYgX19pbml0X18oc2VsZiwgCiAgICAgICAgICAgICAgICAgbmFtZTogc3RyLCAKICAgICAgICAgICAgICAgICB0eXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLkFueSwgCiAgICAgICAgICAgICAgICAgZXhwcmVzc2lvbjogT3B0aW9uYWxbVW5pb25bc3RyfEV4cHJlc3Npb25dXSA9IE5vbmUsIAogICAgICAgICAgICAgICAgIGV4cHJlc3Npb25Db2RlVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvbkNvZGVUeXBlXSA9IE5vbmUsCiAgICAgICAgICAgICAgICAgYWxpYXM6IE9wdGlvbmFsW3N0cl0gPSBOb25lLCAKICAgICAgICAgICAgICAgICBpc1ByaW1hcnlLZXk6IGJvb2wgPSBOb25lLAogICAgICAgICAgICAgICAgIGlzVW5pcXVlS2V5OiBib29sID0gRmFsc2UsCiAgICAgICAgICAgICAgICAgaXNOdWxsYWJsZTogYm9vbCA9IFRydWUsCiAgICAgICAgICAgICAgICAgZm9ybWF0U3RyaW5nOiBPcHRpb25hbFtzdHJdID0gTm9uZSwKICAgICAgICAgICAgICAgICB0YWJsZUFsaWFzOiBPcHRpb25hbFtzdHJdID0gTm9uZSwKICAgICAgICAgICAgICAgICB0YWJsZTogT3B0aW9uYWxbVGFibGVdID0gTm9uZSwKICAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzOiBkaWN0ID0ge30KICAgICAgICAgICAgICAgICApOgogICAgICAgIAogICAgICAgIHN1cGVyKCkuX19pbml0X18obmFtZSwgdHlwZT10eXBlLCBleHByZXNzaW9uPWV4cHJlc3Npb24sIGV4cHJlc3Npb25Db2RlVHlwZT1leHByZXNzaW9uQ29kZVR5cGUsIHRhYmxlQWxpYXM9dGFibGVBbGlhcywgcHJvcGVydGllcz1wcm9wZXJ0aWVzLCB0YWJsZT10YWJsZSkKCiAgICAgICAgaWYgYWxpYXMgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGFsaWFzID0gYWxpYXMuc3RyaXAoIlwiIikKICAgICAgICBzZWxmLl9fYWxpYXM6IE9wdGlvbmFsW3N0cl0gPSBhbGlhcwogICAgICAgIHNlbGYuX19mb3JtYXRTdHJpbmc6IE9wdGlvbmFsW3N0cl0gPSBmb3JtYXRTdHJpbmcKICAgICAgICBzZWxmLl9faXNQcmltYXJ5S2V5OiBib29sID0gaXNQcmltYXJ5S2V5CiAgICAgICAgc2VsZi5pc1VuaXF1ZUtleTogYm9vbCA9IGlzVW5pcXVlS2V5CiAgICAgICAgc2VsZi5pc051bGxhYmxlOiBib29sID0gaXNOdWxsYWJsZQogICAgICAgIHNlbGYuX19pc0ZvcmVpZ25LZXk6IGJvb2wgPSBOb25lCgogICAgQHByb3BlcnR5CiAgICBkZWYgaXNTY2FsYXJWYWx1ZShzZWxmKSAtPiBib29sOgogICAgICAgIHJldHVybiBzZWxmLnR5cGUuaXNTY2FsYXIKCiAgICBAcHJvcGVydHkKICAgIGRlZiBhbGlhcyhzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IGdldFF1b3RlZElkZW50aWZpZXIKICAgICAgICBpZiBzZWxmLl9fYWxpYXMgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBnZXRRdW90ZWRJZGVudGlmaWVyKHNlbGYuX19hbGlhcykKICAgICAgICByZXR1cm4gTm9uZSAgIAoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzQ29sdW1uKHNlbGYpOgogICAgICAgIHJldHVybiBUcnVlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIFJ1bnRpbWUoc2VsZik6CiAgICAgICAgaWYgc2VsZi50YWJsZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYudGFibGUuUnVudGltZQogICAgICAgIHJldHVybiBOb25lCiAgICAKICAgIEBhbGlhcy5zZXR0ZXIKICAgIGRlZiBhbGlhcyhzZWxmLCBhbGlhczogc3RyKToKICAgICAgICBpZiBhbGlhcyA9PSAiIjoKICAgICAgICAgICAgYWxpYXMgPSBOb25lCiAgICAgICAgc2VsZi5fX2FsaWFzID0gYWxpYXMKCiAgICBAcHJvcGVydHkKICAgIGRlZiBmb3JtYXRTdHJpbmcoc2VsZikgLT4gT3B0aW9uYWxbc3RyXToKICAgICAgICByZXR1cm4gc2VsZi5fX2Zvcm1hdFN0cmluZwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0NhbGN1bGF0ZWQoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gKHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0ZvcmVpZ25LZXkoc2VsZikgLT4gYm9vbDoKICAgICAgICBwYXNzCgogICAgQGlzRm9yZWlnbktleS5zZXR0ZXIKICAgIGRlZiBpc0ZvcmVpZ25LZXkoc2VsZiwgdmFsdWU6IGJvb2wpOgogICAgICAgIHBhc3MKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1ByaW1hcnlLZXkoc2VsZikgLT4gYm9vbDoKICAgICAgICBwYXNzCgogICAgQGlzUHJpbWFyeUtleS5zZXR0ZXIKICAgIGRlZiBpc1ByaW1hcnlLZXkoc2VsZiwgdmFsdWU6IGJvb2wpOgogICAgICAgIHBhc3MKICAgIAogICAgZGVmIGdldElkZW50aWZpZXIoc2VsZiwgcXVvdGVNaXhlZENhc2U6IGJvb2wgPSBGYWxzZSwgaW5jbHVkZVRhYmxlQWxpYXM6IGJvb2wgPSBGYWxzZSwgaW5jbHVkZVRhYmxlTmFtZTogYm9vbCA9IEZhbHNlLCBzaG93RXhwcmVzc2lvbjogYm9vbCA9IEZhbHNlKSAtPiBzdHI6CiAgICAgICAgZnJvbSBiaS5zcWwuU1FMU2VsZWN0RXhwcmVzc2lvbiBpbXBvcnQgU1FMU2VsZWN0RXhwcmVzc2lvbgogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IGdldFF1b3RlZElkZW50aWZpZXIKCiAgICAgICAgY29sU3RyaW5nID0gc3VwZXIoKS5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPXF1b3RlTWl4ZWRDYXNlLCBpbmNsdWRlVGFibGVBbGlhcz1pbmNsdWRlVGFibGVBbGlhcywgaW5jbHVkZVRhYmxlTmFtZT1pbmNsdWRlVGFibGVOYW1lKQogICAgICAgIGFsaWFzID0gc2VsZi5fX2FsaWFzCiAgICAgICAgaWYgc2hvd0V4cHJlc3Npb24gPT0gVHJ1ZToKICAgICAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgaWYgYWxpYXMgaXMgTm9uZToKICAgICAgICAgICAgICAgICAgICBhbGlhcyA9IHN1cGVyKCkuZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1GYWxzZSwgaW5jbHVkZVRhYmxlQWxpYXM9RmFsc2UsIGluY2x1ZGVUYWJsZU5hbWU9RmFsc2UpCiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYuZXhwcmVzc2lvbiwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgY29sU3RyaW5nID0gc2VsZi5leHByZXNzaW9uLmdldFNxbFZhbHVlKHdyYXBTdWJRdWVyeT1UcnVlKQogICAgICAgICAgICAgICAgCiAgICAgICAgICAgIGVsaWYgc2VsZi5leHByZXNzaW9uU3RyaW5nIGlzIG5vdCBOb25lIGFuZCBzZWxmLmV4cHJlc3Npb25TdHJpbmcgIT0gIiI6CiAgICAgICAgICAgICAgICBpZiBhbGlhcyBpcyBOb25lOgogICAgICAgICAgICAgICAgICAgIGlmIGNvbFN0cmluZyBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgYWxpYXMgPSBzdXBlcigpLmdldElkZW50aWZpZXIocXVvdGVNaXhlZENhc2U9RmFsc2UsIGluY2x1ZGVUYWJsZUFsaWFzPUZhbHNlLCBpbmNsdWRlVGFibGVOYW1lPUZhbHNlKQogICAgICAgICAgICAgICAgY29sU3RyaW5nID0gc2VsZi5leHByZXNzaW9uU3RyaW5nCiAgICAgICAgICAgIAogICAgICAgICAgICAKICAgICAgICBpZiBhbGlhcyBpcyBub3QgTm9uZSBhbmQgcXVvdGVNaXhlZENhc2U6CiAgICAgICAgICAgIGFsaWFzID0gZ2V0UXVvdGVkSWRlbnRpZmllcihhbGlhcykKCiAgICAgICAgaWYgYWxpYXMgaXMgbm90IE5vbmUgYW5kIGNvbFN0cmluZyAhPSBhbGlhczoKICAgICAgICAgICAgY29sU3RyaW5nID0gZiJ7Y29sU3RyaW5nfSBBUyB7YWxpYXN9IgogICAgICAgIHJldHVybiBjb2xTdHJpbmcKCiAgICBkZWYgdG9TdHIoc2VsZik6CiAgICAgICAgaWYgc2VsZi5hbGlhcyBpcyBOb25lOgogICAgICAgICAgICBhbGlhcyA9IHNlbGYubmFtZQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIGFsaWFzID0gc2VsZi5hbGlhcwogICAgCiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICBuYW1lID0gc2VsZi5leHByZXNzaW9uLmNvbnZlcnRUb1NRTCgpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgbmFtZSA9IHNlbGYubmFtZQogICAgICAgIAogICAgICAgIGlmIG5hbWUgIT0gYWxpYXM6CiAgICAgICAgICAgIHJldHVybiBmIntuYW1lfSBBUyB7YWxpYXN9IgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBuYW1lCiAgICAgICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5mdWxsTmFtZQogICAgICAgIAogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzZWxmLm5hbWV9ICh7c2VsZi50eXBlfSkiCiAgICAKICAgIGRlZiBfX2VxX18oc2VsZiwgdmFsdWUpOgogICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIENvbHVtbik6CiAgICAgICAgICAgIHJldHVybiBzZWxmLm5hbWUgPT0gdmFsdWUubmFtZSBhbmQgc2VsZi50eXBlID09IHZhbHVlLnR5cGUgYW5kIHNlbGYuYWxpYXMgPT0gdmFsdWUuYWxpYXMgCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHZhbHVlLCBzdHIpIGFuZCBzZWxmLl9fYWxpYXMgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLl9fYWxpYXMgaXMgbm90IE5vbmUgYW5kIHNlbGYuX19hbGlhcy51cHBlcigpID09IHZhbHVlLnVwcGVyKCkKICAgICAgICAKICAgICAgICByZXR1cm4gc3VwZXIoKS5fX2VxX18odmFsdWUpCiAgICAgICAgCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGhhc2goKHNlbGYubmFtZSwgc2VsZi50eXBlLCBzZWxmLmV4cHJlc3Npb24sIHNlbGYuYWxpYXMpKQoKVGFibGVJdGVtLnJlZ2lzdGVyKENvbHVtbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/__init__.py FROM (SELECT BASE64_DECODE_STRING('IyBFeHBvcnQgbWFpbiBwYXJzZXIgZnVuY3Rpb25zIGZvciBQb3dlciBCSSBmaWxlcwpmcm9tIC5yZWFkZXJzLnBiaS5wYml0cGFyc2VyIGltcG9ydCAoCiAgICBDcmVhdGVQcm9qZWN0RnJvbUZpbGUsIAogICAgcGFyc2VQcm9qZWN0RGF0YU1vZGVsLCAKICAgIHBhcnNlUHJvamVjdFBhcmFtZXRlcnMsIAogICAgcGFyc2VQcm9qZWN0UmVwb3J0UGFnZXMKKQoKIyBOb3RlOiBBbGwgY29yZSBjbGFzc2VzIGFyZSBhdmFpbGFibGUgZnJvbSBiaS5jb3JlLiogYW5kIGJpLnJlYWRlcnMucGJpLioKIyBMZWdhY3kgdG9wLWxldmVsIGV4cG9ydHMgaGF2ZSBiZWVuIHJlbW92ZWQgLSB1c2Ugc3BlY2lmaWMgaW1wb3J0cyBpbnN0ZWFk'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/pbitparser.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/measure.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkucmVhZGVycy5wYmkubW9kZWxJdGVtIGltcG9ydCBNb2RlbEl0ZW0KCmNsYXNzIE1lYXN1cmUoTW9kZWxJdGVtLCBUYWJsZUl0ZW0pOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIG5hbWU6IHN0ciwgZGF4RXhwcmVzc2lvbjogc3RyLCB0eXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLkFueSwgdGFibGU6IE9wdGlvbmFsW1RhYmxlXSA9IE5vbmUsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgTW9kZWxJdGVtLl9faW5pdF9fKHNlbGYpCiAgICAgICAgVGFibGVJdGVtLl9faW5pdF9fKHNlbGYsIG5hbWUsIHR5cGU9dHlwZSwgZXhwcmVzc2lvbj1kYXhFeHByZXNzaW9uLCBleHByZXNzaW9uQ29kZVR5cGU9RXhwcmVzc2lvbkNvZGVUeXBlLkRBWCwgdGFibGU9dGFibGUsIHByb3BlcnRpZXM9cHJvcGVydGllcykKCiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIE5vbmU6CiAgICAgICAgICAgIHByaW50KGYiTWVhc3VyZSBleHByZXNzaW9uIGlzIG5vdCB2YWxpZDoge2RheEV4cHJlc3Npb259IikKICAgIEBwcm9wZXJ0eQogICAgZGVmIE1vZGVsKHNlbGYpOgogICAgICAgIGlmIHNlbGYudGFibGUgaXMgbm90IE5vbmUgYW5kIGlzaW5zdGFuY2Uoc2VsZi50YWJsZSwgTW9kZWxJdGVtKToKICAgICAgICAgICAgcmV0dXJuIHNlbGYudGFibGUuTW9kZWwKICAgICAgICByZXR1cm4gTm9uZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzQ29sdW1uKHNlbGYpOgogICAgICAgIHJldHVybiBGYWxzZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHZhbHVlKHNlbGYpIC0+IEFueToKICAgICAgICByZXR1cm4gc2VsZi5leHByZXNzaW9uLnZhbHVlCgogICAgQHByb3BlcnR5CiAgICBkZWYgdHlwZShzZWxmKSAtPiBBbnk6CiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5leHByZXNzaW9uLnJldHVyblR5cGUKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gc3VwZXIoKS50eXBlCgogICAgQHByb3BlcnR5CiAgICBkZWYgaXNWYWxpZFNlbWFudGljVmlld1NRTChzZWxmKSAtPiBib29sOgogICAgICAgIGZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KCiAgICAgICAgcmV0dXJuICgKICAgICAgICAgICAgc2VsZi5IYXNJc3N1ZXMgPT0gRmFsc2UKICAgICAgICAgICAgYW5kIHNlbGYudmFsdWUgaXMgbm90IE5vbmUKICAgICAgICAgICAgYW5kIHN0cihzZWxmLnZhbHVlKSAhPSAiIgogICAgICAgICAgICBhbmQgc3RyKHNlbGYudmFsdWUpICE9ICIoKSIKICAgICAgICAgICAgIyBUaGVyZSBhcmUgc29tZSBidWdzIGluIERBWCBPcGVyYXRvciArIERheCBGdW5jdGlvbnMgdGhhdCBhcmUgcmV0dXJuaW5nIE5VTEwuCiAgICAgICAgICAgICMgRm9yIG5vdyB3ZSdsbCBqdXN0IGdvaW5nIHRvIHNraXAgdGhvc2UgbWVhc3VyZXMgZm9yIHNlbWFudGljIHZpZXdzLgogICAgICAgICAgICAjIFRPRE86IEZpeCB0aGUgYnVncyBpbiBEQVggT3BlcmF0b3IgKyBEYXggRnVuY3Rpb25zIHRoYXQgYXJlIHJldHVybmluZyBOVUxMLgogICAgICAgICAgICBhbmQgIk5VTEwiIG5vdCBpbiBzdHIoc2VsZi52YWx1ZSkKICAgICAgICAgICAgYW5kIG5vdCBpc2luc3RhbmNlKHNlbGYudmFsdWUsIFNRTFNlbGVjdEV4cHJlc3Npb24pCiAgICAgICAgKQoKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c2VsZi5uYW1lfSB7c2VsZi5leHByZXNzaW9ufSIKCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3JlcHJfXygpCgogICAgZGVmIF9faGFzaF9fKHNlbGYpOgogICAgICAgIHJldHVybiBoYXNoKChzZWxmLm5hbWUsIHNlbGYudHlwZSwgc2VsZi5leHByZXNzaW9uKSkKCk1vZGVsSXRlbS5yZWdpc3RlcihNZWFzdXJlKQpUYWJsZUl0ZW0ucmVnaXN0ZXIoTWVhc3VyZSk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/partitionMode.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBlbnVtIGltcG9ydCBTdHJFbnVtCgoKY2xhc3MgUGFydGl0aW9uTW9kZShTdHJFbnVtKToKICAgICIiIgogICAgRW51bSBmb3IgUG93ZXIgQkkgdGFibGUgcGFydGl0aW9uIG1vZGVzLgogICAgCiAgICAtIElNUE9SVDogRGF0YSBpcyBpbXBvcnRlZCBpbnRvIFBvd2VyIEJJJ3MgaW4tbWVtb3J5IG1vZGVsCiAgICAtIERJUkVDVF9RVUVSWTogRGF0YSBpcyBxdWVyaWVkIGRpcmVjdGx5IGZyb20gdGhlIHNvdXJjZSAoRGlyZWN0UXVlcnkgbW9kZSkKICAgICIiIgogICAgSU1QT1JUID0gImltcG9ydCIKICAgIERJUkVDVF9RVUVSWSA9ICJkaXJlY3RRdWVyeSIKICAgIE5PTkUgPSAiTm9uZSIKCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/modelItem.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSB0eXBpbmcgaW1wb3J0IE9wdGlvbmFsLCBMaXN0CmZyb20gYmkuY29yZS5pc3N1ZUNvbnRhaW5lciBpbXBvcnQgSXNzdWVDb250YWluZXIKZnJvbSBiaS5yZWFkZXJzLnBiaS50ZXJtIGltcG9ydCBUZXJtCgoKY2xhc3MgTW9kZWxJdGVtKElzc3VlQ29udGFpbmVyLCBBQkMpOgogICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KICAgIGltcG9ydCBiaS5yZWFkZXJzLnBiaS5tb2RlbCBhcyBtCiAgICBmcm9tIGJpLmNvcmUuaXNzdWUgaW1wb3J0IElzc3VlCgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkudGVybSBpbXBvcnQgVGVybQogICAgICAgIElzc3VlQ29udGFpbmVyLl9faW5pdF9fKHNlbGYpCiAgICAgICAgc2VsZi5fX3Rlcm1zOiBMaXN0W1Rlcm1dID0gW10KCiAgICBAcHJvcGVydHkKICAgIGRlZiBSdW50aW1lKHNlbGYpOgogICAgICAgIGlmIHNlbGYuTW9kZWwgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLk1vZGVsLlJ1bnRpbWUKICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBAcHJvcGVydHkKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIE1vZGVsKHNlbGYpIC0+IG0uTW9kZWw6CiAgICAgICAgcGFzcwogICAgCiAgICBkZWYgZ2V0VGFibGUoc2VsZiwgbmFtZTogc3RyKToKICAgICAgICByZXR1cm4gc2VsZi5Nb2RlbC5nZXRUYWJsZShuYW1lKQogICAgCiAgICBkZWYgYWRkVGVybShzZWxmLCBuYW1lOiBzdHIsIHN0YXRlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgd2VpZ2h0OiBPcHRpb25hbFtmbG9hdF0gPSBOb25lLCBjdWx0dXJlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgbGFuZ3VhZ2U6IE9wdGlvbmFsW3N0cl0gPSBOb25lKToKICAgICAgICAiIiJBZGQgYSBsaW5ndWlzdGljIHRlcm0gdG8gdGhpcyBtb2RlbCBpdGVtLiIiIgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkudGVybSBpbXBvcnQgVGVybQogICAgICAgIHRlcm0gPSBUZXJtKHNlbGYsIG5hbWUsIHN0YXRlLCB3ZWlnaHQsIGN1bHR1cmUsIGxhbmd1YWdlKQogICAgICAgIHNlbGYuX190ZXJtcy5hcHBlbmQodGVybSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiB0ZXJtcyhzZWxmKSAtPiBMaXN0W1Rlcm1dOgogICAgICAgICIiIkdldCB0aGUgbGlzdCBvZiBsaW5ndWlzdGljIHRlcm1zIGFzc29jaWF0ZWQgd2l0aCB0aGlzIG1vZGVsIGl0ZW0uIiIiCiAgICAgICAgcmV0dXJuIHNlbGYuX190ZXJtcwoK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/relationship.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSBiaS5jb3JlLnJlbGF0aW9uc2hpcCBpbXBvcnQgUmVsYXRpb25zaGlwIGFzIENvcmVSZWxhdGlvbnNoaXAKZnJvbSBiaS5yZWFkZXJzLnBiaS5tb2RlbEl0ZW0gaW1wb3J0IE1vZGVsSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLm1vZGVsIGltcG9ydCBNb2RlbAoKY2xhc3MgUmVsYXRpb25zaGlwKENvcmVSZWxhdGlvbnNoaXAsIE1vZGVsSXRlbSk6ICAgIAogICAgZGVmIF9faW5pdF9fKHNlbGYsIAogICAgICAgICAgICAgICAgIG1vZGVsOiBNb2RlbCwgCiAgICAgICAgICAgICAgICAgbmFtZTogc3RyLCAKICAgICAgICAgICAgICAgICBmcm9tVGFibGVOYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIGZyb21Db2x1bW5OYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIHRvVGFibGVOYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIHRvQ29sdW1uTmFtZTogc3RyLCAKICAgICAgICAgICAgICAgICBqb2luT25EYXRlQmVoYXZpb3I6IE9wdGlvbmFsW3N0cl0gPSBOb25lLCAKICAgICAgICAgICAgICAgICBjcm9zc0ZpbHRlcmluZ0JlaGF2aW9yOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgCiAgICAgICAgICAgICAgICAgaXNBY3RpdmU6IGJvb2wgPSBUcnVlLAogICAgICAgICAgICAgICAgIHByb3BlcnRpZXM6IE9wdGlvbmFsW2RpY3RdID0ge30sCiAgICAgICAgICAgICAgICAgdXNlVGFibGVJZHM6IGJvb2wgPSBGYWxzZSk6CiAgICAgICAgZnJvbVRhYmxlID0gbW9kZWwuZ2V0VGFibGUoZnJvbVRhYmxlTmFtZSwgdXNlSWQ9dXNlVGFibGVJZHMpCiAgICAgICAgZnJvbUNvbHVtbiA9IGZyb21UYWJsZS5nZXRJdGVtKGZyb21Db2x1bW5OYW1lKQoKICAgICAgICB0b1RhYmxlID0gbW9kZWwuZ2V0VGFibGUodG9UYWJsZU5hbWUsIHVzZUlkPXVzZVRhYmxlSWRzKQogICAgICAgIHRvQ29sdW1uID0gdG9UYWJsZS5nZXRJdGVtKHRvQ29sdW1uTmFtZSkKICAgICAgICAKICAgICAgICBDb3JlUmVsYXRpb25zaGlwLl9faW5pdF9fKHNlbGYsIG5hbWU9bmFtZSwgZnJvbVRhYmxlSXRlbT1mcm9tQ29sdW1uLCB0b1RhYmxlSXRlbT10b0NvbHVtbiwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIE1vZGVsSXRlbS5fX2luaXRfXyhzZWxmKQoKICAgICAgICBzZWxmLl9fbW9kZWwgPSBtb2RlbAogICAgICAgIHNlbGYuX19mcm9tVGFibGVOYW1lID0gZnJvbVRhYmxlTmFtZQogICAgICAgIHNlbGYuX19mcm9tQ29sdW1uTmFtZSA9IGZyb21Db2x1bW5OYW1lCiAgICAgICAgc2VsZi5fX3RvVGFibGVOYW1lID0gdG9UYWJsZU5hbWUKICAgICAgICBzZWxmLl9fdG9Db2x1bW5OYW1lID0gdG9Db2x1bW5OYW1lIAogICAgICAgIHNlbGYuX19qb2luT25EYXRlQmVoYXZpb3IgPSBqb2luT25EYXRlQmVoYXZpb3IKICAgICAgICBzZWxmLl9fY3Jvc3NGaWx0ZXJpbmdCZWhhdmlvciA9IGNyb3NzRmlsdGVyaW5nQmVoYXZpb3IKICAgICAgICBzZWxmLl9faXNBY3RpdmUgPSBpc0FjdGl2ZQogICAgICAgIHNlbGYuX19pc0FjdGl2ZQogICAgICAgIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBNb2RlbChzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX21vZGVsCgogICAgQHByb3BlcnR5CiAgICBkZWYgaXNBY3RpdmUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19pc0FjdGl2ZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBmcm9tVGFibGVOYW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fZnJvbVRhYmxlTmFtZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHRvVGFibGVOYW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fdG9UYWJsZU5hbWUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBmcm9tQ29sdW1uTmFtZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2Zyb21Db2x1bW5OYW1lCgogICAgQHByb3BlcnR5CiAgICBkZWYgdG9Db2x1bW5OYW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fdG9Db2x1bW5OYW1lCgogICAgQHByb3BlcnR5CiAgICBkZWYgY3Jvc3NGaWx0ZXJpbmdCZWhhdmlvcihzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2Nyb3NzRmlsdGVyaW5nQmVoYXZpb3IKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgam9pbk9uRGF0ZUJlaGF2aW9yKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fam9pbk9uRGF0ZUJlaGF2aW9yCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzU25vd2ZsYWtlKHNlbGYpOgogICAgICAgIGZyb20gLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUKICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYuZnJvbVRhYmxlSXRlbS50YWJsZSwgU2VtYW50aWNUYWJsZSkgYW5kIGlzaW5zdGFuY2Uoc2VsZi50b1RhYmxlSXRlbS50YWJsZSwgU2VtYW50aWNUYWJsZSk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLmZyb21UYWJsZUl0ZW0udGFibGUuaXNTbm93Zmxha2UgYW5kIHNlbGYudG9UYWJsZUl0ZW0udGFibGUuaXNTbm93Zmxha2UKICAgICAgICByZXR1cm4gRmFsc2UKCiAgICBkZWYgZ2V0U3FsVmFsdWUoc2VsZikgLT4gc3RyOgogICAgICAgICMgVG8gYmUgdXNlZCBpbiBTZW1hbnRpYyBWaWV3IFNRTCBnZW5lcmF0aW9uCiAgICAgICAgZnJvbSB1dGlsLnNub3dmbGFrZU9iamVjdCBpbXBvcnQgZ2V0UXVvdGVkSWRlbnRpZmllcgogICAgICAgIAogICAgICAgIHF1b3RlZEZyb21UYWJsZU5hbWUgPSBnZXRRdW90ZWRJZGVudGlmaWVyKHNlbGYuZnJvbVRhYmxlTmFtZSkKICAgICAgICBxdW90ZWRUb1RhYmxlTmFtZSA9IGdldFF1b3RlZElkZW50aWZpZXIoc2VsZi50b1RhYmxlTmFtZSkKICAgICAgICBxdW90ZWRSZWxhdGlvbnNoaXBOYW1lID0gZ2V0UXVvdGVkSWRlbnRpZmllcihmIntzZWxmLmZyb21UYWJsZU5hbWV9X1RPX3tzZWxmLnRvVGFibGVOYW1lfSIpCiAgICAgICAgcXVvdGVkRnJvbUNvbHVtbk5hbWUgPSBnZXRRdW90ZWRJZGVudGlmaWVyKHNlbGYuZnJvbUNvbHVtbk5hbWUpCiAgICAgICAgcXVvdGVkVG9Db2x1bW5OYW1lID0gZ2V0UXVvdGVkSWRlbnRpZmllcihzZWxmLnRvQ29sdW1uTmFtZSkKICAgICAgICAKICAgICAgICByZWxhdGlvbnNoaXBfc3FsID0gKAogICAgICAgICAgICBmIntxdW90ZWRSZWxhdGlvbnNoaXBOYW1lfSBBU1xuIgogICAgICAgICAgICBmIiAgICAgICAgICAge3F1b3RlZEZyb21UYWJsZU5hbWV9ICh7cXVvdGVkRnJvbUNvbHVtbk5hbWV9KSAiCiAgICAgICAgICAgIGYiUkVGRVJFTkNFUyB7cXVvdGVkVG9UYWJsZU5hbWV9ICh7cXVvdGVkVG9Db2x1bW5OYW1lfSkiCiAgICAgICAgKQoKICAgICAgICByZXR1cm4gcmVsYXRpb25zaGlwX3NxbAoKTW9kZWxJdGVtLnJlZ2lzdGVyKFJlbGF0aW9uc2hpcCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/term.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IE9wdGlvbmFsLCBEaWN0LCBBbnkKCmNsYXNzIFRlcm06CiAgICAiIiIKICAgIFJlcHJlc2VudHMgYSBsaW5ndWlzdGljIHRlcm0gZnJvbSBQb3dlciBCSSdzIGxpbmd1aXN0aWNNZXRhZGF0YS4KICAgIFRlcm1zIGNhbiBiZSBhc3NvY2lhdGVkIHdpdGggdGFibGVzIG9yIGNvbHVtbnMgYW5kIGFyZSB1c2VkIGZvciBuYXR1cmFsIGxhbmd1YWdlIHF1ZXJpZXMuCiAgICAKICAgIEV4YW1wbGUgZnJvbSBQQklUOgogICAgewogICAgICAgICJUZXJyaXRvcnkiOiB7CiAgICAgICAgICAgICJTdGF0ZSI6ICJTdWdnZXN0ZWQiLAogICAgICAgICAgICAiV2VpZ2h0IjogMC45CiAgICAgICAgfQogICAgfQogICAgIiIiCiAgICAKICAgIGRlZiBfX2luaXRfXyhzZWxmLCAKICAgICAgICAgICAgICAgICBwYXJlbnQ6IEFueSwKICAgICAgICAgICAgICAgICBuYW1lOiBzdHIsIAogICAgICAgICAgICAgICAgIHN0YXRlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgCiAgICAgICAgICAgICAgICAgd2VpZ2h0OiBPcHRpb25hbFtmbG9hdF0gPSBOb25lLAogICAgICAgICAgICAgICAgIGN1bHR1cmU6IE9wdGlvbmFsW3N0cl0gPSBOb25lLAogICAgICAgICAgICAgICAgIGxhbmd1YWdlOiBPcHRpb25hbFtzdHJdID0gTm9uZSk6CiAgICAgICAgIiIiCiAgICAgICAgSW5pdGlhbGl6ZSBhIFRlcm0uCiAgICAgICAgCiAgICAgICAgQXJnczoKICAgICAgICAgICAgcGFyZW50OiBUaGUgcGFyZW50IE1vZGVsSXRlbSAoQ29sdW1uIG9yIFRhYmxlKSB0aGlzIHRlcm0gYmVsb25ncyB0bwogICAgICAgICAgICBuYW1lOiBUaGUgdGVybSB0ZXh0IChlLmcuLCAiVGVycml0b3J5IiwgInRlcnIgbm0iKQogICAgICAgICAgICBzdGF0ZTogVGhlIHN0YXRlIG9mIHRoZSB0ZXJtIChlLmcuLCAiR2VuZXJhdGVkIiwgIlN1Z2dlc3RlZCIpCiAgICAgICAgICAgIHdlaWdodDogVGhlIHdlaWdodC9jb25maWRlbmNlIG9mIHRoZSB0ZXJtIChlLmcuLCAwLjksIDAuOTkpCiAgICAgICAgICAgIGN1bHR1cmU6IFRoZSBjdWx0dXJlIGNvZGUgKGUuZy4sICJlbi1VUyIpCiAgICAgICAgICAgIGxhbmd1YWdlOiBUaGUgbGFuZ3VhZ2UgY29kZSAoZS5nLiwgImVuLVVTIikKICAgICAgICAiIiIKICAgICAgICBzZWxmLl9fcGFyZW50ID0gcGFyZW50CiAgICAgICAgc2VsZi5fX25hbWUgPSBuYW1lCiAgICAgICAgc2VsZi5fX3N0YXRlID0gc3RhdGUKICAgICAgICBzZWxmLl9fd2VpZ2h0ID0gd2VpZ2h0CiAgICAgICAgc2VsZi5fX2N1bHR1cmUgPSBjdWx0dXJlCiAgICAgICAgc2VsZi5fX2xhbmd1YWdlID0gbGFuZ3VhZ2UKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgTW9kZWwoc2VsZik6CiAgICAgICAgIiIiR2V0IHRoZSBNb2RlbCBmcm9tIHRoZSBwYXJlbnQgb2JqZWN0LiIiIgogICAgICAgIGlmIHNlbGYuX19wYXJlbnQgaXMgbm90IE5vbmUgYW5kIGhhc2F0dHIoc2VsZi5fX3BhcmVudCwgJ01vZGVsJyk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLl9fcGFyZW50Lk1vZGVsCiAgICAgICAgcmV0dXJuIE5vbmUKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcGFyZW50KHNlbGYpIC0+IE1vZGVsSXRlbToKICAgICAgICAiIiJUaGUgcGFyZW50IE1vZGVsSXRlbSAoQ29sdW1uIG9yIFRhYmxlKSB0aGlzIHRlcm0gYmVsb25ncyB0by4iIiIKICAgICAgICByZXR1cm4gc2VsZi5fX3BhcmVudAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lKHNlbGYpIC0+IHN0cjoKICAgICAgICAiIiJUaGUgdGVybSB0ZXh0LiIiIgogICAgICAgIHJldHVybiBzZWxmLl9fbmFtZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBzdGF0ZShzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgICIiIlRoZSBzdGF0ZSBvZiB0aGUgdGVybSAoZS5nLiwgJ0dlbmVyYXRlZCcsICdTdWdnZXN0ZWQnKS4iIiIKICAgICAgICByZXR1cm4gc2VsZi5fX3N0YXRlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHdlaWdodChzZWxmKSAtPiBPcHRpb25hbFtmbG9hdF06CiAgICAgICAgIiIiVGhlIHdlaWdodC9jb25maWRlbmNlIG9mIHRoZSB0ZXJtLiIiIgogICAgICAgIHJldHVybiBzZWxmLl9fd2VpZ2h0CiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGN1bHR1cmUoc2VsZikgLT4gT3B0aW9uYWxbc3RyXToKICAgICAgICAiIiJUaGUgY3VsdHVyZSBjb2RlIChlLmcuLCAnZW4tVVMnKS4iIiIKICAgICAgICByZXR1cm4gc2VsZi5fX2N1bHR1cmUKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgbGFuZ3VhZ2Uoc2VsZikgLT4gT3B0aW9uYWxbc3RyXToKICAgICAgICAiIiJUaGUgbGFuZ3VhZ2UgY29kZSAoZS5nLiwgJ2VuLVVTJykuIiIiCiAgICAgICAgcmV0dXJuIHNlbGYuX19sYW5ndWFnZQogICAgCiAgICBkZWYgdG9EaWN0KHNlbGYpIC0+IERpY3Rbc3RyLCBBbnldOgogICAgICAgICIiIgogICAgICAgIENvbnZlcnQgdGhlIHRlcm0gdG8gYSBkaWN0aW9uYXJ5IGZvcm1hdCBtYXRjaGluZyBQQklUIHN0cnVjdHVyZS4KICAgICAgICAKICAgICAgICBSZXR1cm5zOgogICAgICAgICAgICBEaWN0IHdpdGggdGhlIHRlcm0gbmFtZSBhcyBrZXkgYW5kIHByb3BlcnRpZXMgYXMgdmFsdWUKICAgICAgICAiIiIKICAgICAgICByZXN1bHQgPSB7fQogICAgICAgIAogICAgICAgIGlmIHNlbGYuX19zdGF0ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmVzdWx0WyJTdGF0ZSJdID0gc2VsZi5fX3N0YXRlCiAgICAgICAgCiAgICAgICAgaWYgc2VsZi5fX3dlaWdodCBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmVzdWx0WyJXZWlnaHQiXSA9IHNlbGYuX193ZWlnaHQKICAgICAgICAKICAgICAgICByZXR1cm4ge3NlbGYuX19uYW1lOiByZXN1bHR9IGlmIHJlc3VsdCBlbHNlIHtzZWxmLl9fbmFtZToge319CiAgICAKICAgIGRlZiBfX3JlcHJfXyhzZWxmKSAtPiBzdHI6CiAgICAgICAgcGFydHMgPSBbZiJuYW1lPSd7c2VsZi5fX25hbWV9JyJdCiAgICAgICAgaWYgc2VsZi5fX3N0YXRlOgogICAgICAgICAgICBwYXJ0cy5hcHBlbmQoZiJzdGF0ZT0ne3NlbGYuX19zdGF0ZX0nIikKICAgICAgICBpZiBzZWxmLl9fd2VpZ2h0IGlzIG5vdCBOb25lOgogICAgICAgICAgICBwYXJ0cy5hcHBlbmQoZiJ3ZWlnaHQ9e3NlbGYuX193ZWlnaHR9IikKICAgICAgICByZXR1cm4gZiJUZXJtKHsnLCAnLmpvaW4ocGFydHMpfSkiCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gc2VsZi5fX25hbWUKICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCBvdGhlcikgLT4gYm9vbDoKICAgICAgICBpZiBub3QgaXNpbnN0YW5jZShvdGhlciwgVGVybSk6CiAgICAgICAgICAgIHJldHVybiBGYWxzZQogICAgICAgIHJldHVybiAoc2VsZi5fX25hbWUgPT0gb3RoZXIuX19uYW1lIGFuZCAKICAgICAgICAgICAgICAgIHNlbGYuX19zdGF0ZSA9PSBvdGhlci5fX3N0YXRlIGFuZAogICAgICAgICAgICAgICAgc2VsZi5fX3dlaWdodCA9PSBvdGhlci5fX3dlaWdodCkKICAgIAogICAgZGVmIF9faGFzaF9fKHNlbGYpIC0+IGludDoKICAgICAgICByZXR1cm4gaGFzaCgoc2VsZi5fX25hbWUsIHNlbGYuX19zdGF0ZSwgc2VsZi5fX3dlaWdodCkpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tablePartition.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsLCBEaWN0LCBTZXQKZnJvbSAubW9kZWxJdGVtIGltcG9ydCBNb2RlbEl0ZW0KZnJvbSAucGFydGl0aW9uTW9kZSBpbXBvcnQgUGFydGl0aW9uTW9kZQoKY2xhc3MgVGFibGVQYXJ0aXRpb24oTW9kZWxJdGVtKToKICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uQ29kZVR5cGUgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQogICAgZnJvbSAuc2VtYW50aWNUYWJsZSBpbXBvcnQgU2VtYW50aWNUYWJsZQogICAgZnJvbSBiaS5jb3JlLmlzc3VlIGltcG9ydCBJc3N1ZQogICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KCiAgICBkZWYgX19pbml0X18oc2VsZiwgdGFibGU6IFNlbWFudGljVGFibGUsIG5hbWU6IHN0ciwgbW9kZTogc3RyLCBleHByZXNzaW9uOiBFeHByZXNzaW9uLCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIGZyb20gYmkuY29yZS5pc3N1ZVR5cGUgaW1wb3J0IElzc3VlVHlwZQogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IFNub3dmbGFrZU9iamVjdAoKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCkKICAgICAgICBzZWxmLl9fdGFibGUgPSB0YWJsZQogICAgICAgIHNlbGYuX19uYW1lID0gbmFtZQogICAgICAgIHNlbGYuX19tb2RlOiBQYXJ0aXRpb25Nb2RlID0gUGFydGl0aW9uTW9kZS5OT05FCgogICAgICAgIHNlbGYuX19leHByZXNzaW9uID0gZXhwcmVzc2lvbgogICAgICAgIAoKICAgICAgICBzZWxmLl9fcHJvcGVydGllcyA9IHByb3BlcnRpZXMKICAgICAgICBzZWxmLk1vdmVMb2dpY0ludG9Tbm93Zmxha2UgPSBGYWxzZQogICAgICAgIHNlbGYuQ3JlYXRlU25vd2ZsYWtlT2JqZWN0ID0gRmFsc2UKICAgICAgICBzZWxmLl9fc25vd2ZsYWtlT2JqZWN0OiBTbm93Zmxha2VPYmplY3QgPSBOb25lCiAgICAgICAgCiAgICAgICAgdHJ5OgogICAgICAgICAgICBzZWxmLl9fbW9kZSA9IFBhcnRpdGlvbk1vZGUobW9kZSkKICAgICAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6CiAgICAgICAgICAgIHByaW50IChmIkVycm9yIHBhcnNpbmcgdGFibGUgcGFydGl0aW9uIG1vZGU6IHtlfSIpCiAgICAgICAgICAgICMgRGVmYXVsdCB0byBJbXBvcnQgTW9kZSBpZiBtb2RlIGlzIG5vdCBzcGVjaWZpZWQuCiAgICAgICAgICAgIHNlbGYuX19tb2RlID0gUGFydGl0aW9uTW9kZS5JTVBPUlQKICAgICAgICAgICAgCiAgICAgICAgIyBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBUYWJsZUV4cHJlc3Npb24KICAgICAgICAjIGlmIGlzaW5zdGFuY2Uoc2VsZi5fX2V4cHJlc3Npb24sIFRhYmxlRXhwcmVzc2lvbik6CiAgICAgICAgIyAgICAgZXhpc3RpbmdDb2x1bW5zID0gc2VsZi5fX2V4cHJlc3Npb24uZ2V0Q29sdW1uTmFtZXMocXVvdGVNaXhlZENhc2U9RmFsc2UpCiAgICAgICAgIyAgICAgZm9yIGNvbCBpbiB0YWJsZS5jb2x1bW5zOgogICAgICAgICMgICAgICAgICBpZiBjb2wuc291cmNlQ29sdW1uTmFtZSBpcyBub3QgTm9uZToKICAgICAgICAjICAgICAgICAgICAgIGlmIGNvbC5zb3VyY2VDb2x1bW4gaXMgTm9uZSBhbmQgY29sLnNvdXJjZUNvbHVtbk5hbWUgbm90IGluIGV4aXN0aW5nQ29sdW1uczoKICAgICAgICAjICAgICAgICAgICAgICAgICBzZWxmLl9fZXhwcmVzc2lvbi5hZGRDb2x1bW4obmFtZT1jb2wuc291cmNlQ29sdW1uTmFtZSwgdHlwZT1jb2wudHlwZSwgaXNQcmltYXJ5S2V5PWNvbC5pc1ByaW1hcnlLZXksIGlzVW5pcXVlS2V5PWNvbC5pc1VuaXF1ZUtleSwgaXNOdWxsYWJsZT1jb2wuaXNOdWxsYWJsZSwgZm9ybWF0U3RyaW5nPWNvbC5mb3JtYXRTdHJpbmcpCiAgICAgICAgIyAgICAgICAgICAgICBlbHNlOgogICAgICAgICMgICAgICAgICAgICAgICAgIHBhc3MKCiAgICBAcHJvcGVydHkKICAgIGRlZiB0YWJsZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3RhYmxlCgogICAgQHByb3BlcnR5CiAgICBkZWYgTW9kZWwoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX190YWJsZS5Nb2RlbAoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHByb3BlcnRpZXMoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19wcm9wZXJ0aWVzCgogICAgQHByb3BlcnR5CiAgICBkZWYgbmFtZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX25hbWUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBtb2RlKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fbW9kZQogICAgCiAgICBAbW9kZS5zZXR0ZXIKICAgIGRlZiBtb2RlKHNlbGYsIHZhbHVlOiBQYXJ0aXRpb25Nb2RlKToKICAgICAgICBzZWxmLl9fbW9kZSA9IHZhbHVlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIG9yaWdpbmFsTW9kZShzZWxmKSAtPiBQYXJ0aXRpb25Nb2RlOgogICAgICAgICIiIlRoZSBvcmlnaW5hbCBtb2RlIGZyb20gUG93ZXIgQkkgYmVmb3JlIGFueSB1c2VyIGNoYW5nZXMiIiIKICAgICAgICBvcmlnaW5hbF9kYXRhID0gc2VsZi5fX3Byb3BlcnRpZXMuZ2V0KCJvcmlnaW5hbCIsIHt9KQogICAgICAgIG1vZGVfc3RyID0gb3JpZ2luYWxfZGF0YS5nZXQoIm1vZGUiLCAiTm9uZSIpCiAgICAgICAgdHJ5OgogICAgICAgICAgICByZXR1cm4gUGFydGl0aW9uTW9kZShtb2RlX3N0cikKICAgICAgICBleGNlcHQ6CiAgICAgICAgICAgIHJldHVybiBQYXJ0aXRpb25Nb2RlLk5PTkUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBleHByZXNzaW9uKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fZXhwcmVzc2lvbgoKICAgIEBwcm9wZXJ0eSAKICAgIGRlZiByb290RXhwcmVzc2lvbihzZWxmKToKICAgICAgICBpZiBzZWxmLmV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGhpc3RvcnkgPSBzZWxmLmV4cHJlc3Npb24uZ2V0SGlzdG9yeSgpCiAgICAgICAgICAgIGlmIGxlbihoaXN0b3J5KToKICAgICAgICAgICAgICAgIHJldHVybiBoaXN0b3J5WzBdCiAgICAgICAgcmV0dXJuIE5vbmUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0NhY2hlZChzZWxmKToKICAgICAgICBpZiBzZWxmLmV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLmV4cHJlc3Npb24uaXNDYWNoZWQKICAgICAgICByZXR1cm4gRmFsc2UKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNGaWxlKHNlbGYpOgogICAgICAgIGlmIHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuZXhwcmVzc2lvbi5pc0ZpbGUKICAgICAgICByZXR1cm4gRmFsc2UKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0RhdGFiYXNlKHNlbGYpOgogICAgICAgIGlmIHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuZXhwcmVzc2lvbi5pc0RhdGFiYXNlCiAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzU25vd2ZsYWtlKHNlbGYpOgogICAgICAgIGlmIHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuZXhwcmVzc2lvbi5pc1Nub3dmbGFrZQogICAgICAgIHJldHVybiBGYWxzZQoKICAgIGRlZiBHZXRBbGxEZXBlbmRlbmNpZXMoc2VsZikgLT4gU2V0W01vZGVsSXRlbV06CiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5leHByZXNzaW9uLkdldEFsbERlcGVuZGVuY2llcygpCiAgICAgICAgcmV0dXJuIHNldCgpCgogICAgQHByb3BlcnR5CiAgICBkZWYgSGFzSXNzdWVzKHNlbGYpOgogICAgICAgIHJldHVybiBzdXBlcigpLkhhc0lzc3VlcyBvciBzZWxmLmV4cHJlc3Npb24uSGFzSXNzdWVzCgogICAgQHByb3BlcnR5CiAgICBkZWYgU25vd2ZsYWtlT2JqZWN0KHNlbGYpOgogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3QgaW1wb3J0IFNub3dmbGFrZU9iamVjdAogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3RUeXBlIGltcG9ydCBTbm93Zmxha2VPYmplY3RUeXBlCiAgICAgICAgaWYgc2VsZi5fX3Nub3dmbGFrZU9iamVjdCBpcyBOb25lOgogICAgICAgICAgICBzZl9vYmpfbmFtZSA9IGYie3NlbGYudGFibGUubmFtZX1fUEFSVElUSU9OIgogICAgICAgICAgICBpZiBzZWxmLnRhYmxlLm5hbWUgIT0gc2VsZi5uYW1lIGFuZCBsZW4oc2VsZi50YWJsZS5wYXJ0aXRpb25zKSA+IDE6CiAgICAgICAgICAgICAgICBzZl9vYmpfbmFtZSA9IGYie3NmX29ial9uYW1lfV97c2VsZi5uYW1lfSIKCiAgICAgICAgICAgIGRibmFtZSA9IHNlbGYuTW9kZWwuQ3VycmVudERhdGFiYXNlCiAgICAgICAgICAgIHNjaGVtYSA9IHNlbGYuTW9kZWwuT2JqZWN0Q3JlYXRlU2NoZW1hCgogICAgICAgICAgICAjIERlZmF1bHQgdG8gYSB2aWV3CiAgICAgICAgICAgIHNlbGYuX19zbm93Zmxha2VPYmplY3QgPSBTbm93Zmxha2VPYmplY3QodHlwZT1Tbm93Zmxha2VPYmplY3RUeXBlLlZJRVcsIG5hbWU9c2Zfb2JqX25hbWUsIGRhdGFiYXNlPWRibmFtZSwgc2NoZW1hPXNjaGVtYSwgY2FzZV9pbnNlbnNpdGl2ZT1GYWxzZSkKCiAgICAgICAgcmV0dXJuIHNlbGYuX19zbm93Zmxha2VPYmplY3QKCiAgICBkZWYgZ2V0U3FsVmFsdWUoc2VsZiwgd3JhcFN1YlF1ZXJ5OiBib29sID0gRmFsc2UpOgogICAgICAgIGlmIHNlbGYuQ3JlYXRlU25vd2ZsYWtlT2JqZWN0ID09IFRydWU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLlNub3dmbGFrZU9iamVjdC5nZXRfZnFuKCkKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWwoKQoKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgICAgICByZXR1cm4gRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShzZWxmLmV4cHJlc3Npb24pCiAgICAgICAgcmV0dXJuIGYiRVJST1I6IHBhcnRpdGlvbiBleHByZXNzaW9uIGlzIE5vbmUiCiAgICAKICAgIGRlZiBHZXRSZWZlcmVuY2VUYWJsZXMoc2VsZik6CiAgICAgICAgIiIiCiAgICAgICAgVE9ETzogdGhpcyBpcyBhIGhhY2sgdG8gZ2V0IGRlcGVuZGVuY3kgaW5mb3JtYXRpb24gZm9yIHRoZSBwYXJ0aXRpb24uCiAgICAgICAgR2V0IGFsbCB0aGUgdGFibGVzIHRoYXQgdGhpcyBwYXJ0aXRpb24gaXMgcmVmZXJlbmNpbmcuCiAgICAgICAgVGhpcyBwcm92aWRlcyBkZXBlbmRlbmN5IGluZm9ybWF0aW9uIGZvciB0aGUgcGFydGl0aW9uLgogICAgICAgICIiIgogICAgICAgIGZyb20gLnNlbWFudGljVGFibGUgaW1wb3J0IFNlbWFudGljVGFibGUKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwuZnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBGdW5jdGlvbkV4cHJlc3Npb24KICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IExldEV4cHJlc3Npb24KICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBEYXRhYmFzZVRhYmxlRXhwcmVzc2lvbgoKICAgICAgICByZWZUYWJsZXMgPSBzZXQoKQogICAgICAgIGlmIHNlbGYuZXhwcmVzc2lvbiBpcyBub3QgTm9uZSBhbmQgaXNpbnN0YW5jZShzZWxmLmV4cHJlc3Npb24sIExldEV4cHJlc3Npb24pOgogICAgICAgICAgICBmb3Igayx2IGluIHNlbGYuZXhwcmVzc2lvbi5leHByZXNzaW9ucy5pdGVtcygpOgogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZSh2LCBGdW5jdGlvbkV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgIHJlZlRhYmxlcy51cGRhdGUoW3QgZm9yIHQgaW4gdi5wYXJhbWV0ZXJzIGlmIGlzaW5zdGFuY2UodCwgU2VtYW50aWNUYWJsZSldKQogICAgICAgICAgICAgICAgZWxpZiB2LnZhbHVlIGFuZCBpc2luc3RhbmNlKHYudmFsdWUsIERhdGFiYXNlVGFibGVFeHByZXNzaW9uKSBhbmQgaXNpbnN0YW5jZSh2LnZhbHVlLk93bmVyLCBTZW1hbnRpY1RhYmxlKToKICAgICAgICAgICAgICAgICAgICByZWZUYWJsZXMuYWRkKHYudmFsdWUuT3duZXIpCgogICAgICAgICMgUmVtb3ZlIHRoZSB0YWJsZSB0aGlzIHBhcml0aW9uIGJlbG9uZ3MgdG8uCiAgICAgICAgaWYgc2VsZi50YWJsZToKICAgICAgICAgICAgcmVmVGFibGVzLmRpc2NhcmQoc2VsZi50YWJsZSkKCiAgICAgICAgcmV0dXJuIHJlZlRhYmxlcwoKICAgIGRlZiBHZXRBbGxJc3N1ZXMoc2VsZik6CiAgICAgICAgaXNzdWVzID0gc2V0KCkKICAgICAgICBpZiBzZWxmLmV4cHJlc3Npb24gaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGlzc3VlcyA9IGlzc3Vlcy51bmlvbihzZWxmLmV4cHJlc3Npb24uR2V0QWxsSXNzdWVzKCkpCiAgICAgICAgcmV0dXJuIGlzc3VlcwoKICAgIGRlZiBHZXRJc3N1ZURldGFpbHMoc2VsZiwgaXNzdWU6IElzc3VlLCBpbmNsdWRlQ2hpbGRyZW46IGJvb2wgPSBGYWxzZSk6CiAgICAgICAgZnJvbSBiaS5jb3JlLmlzc3VlQ29udGFpbmVyIGltcG9ydCBJc3N1ZUNvbnRhaW5lcgogICAgICAgIGZyb20gYmkuY29yZS5pc3N1ZURldGFpbHMgaW1wb3J0IElzc3VlRGV0YWlsCiAgICAgICAgcmVzdWx0czogRGljdFtJc3N1ZUNvbnRhaW5lciwgU2V0W0lzc3VlRGV0YWlsXV0gPSB7fQoKICAgICAgICBkZXRhaWxzID0gc3VwZXIoKS5HZXRJc3N1ZURldGFpbHMoaXNzdWUpIAogICAgICAgIGlmIGxlbihkZXRhaWxzKSA+IDA6CiAgICAgICAgICAgIHJlc3VsdHNbc2VsZl0gPSBkZXRhaWxzCgogICAgICAgIGlmIGluY2x1ZGVDaGlsZHJlbiA9PSBUcnVlIGFuZCBzZWxmLnJvb3RFeHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICBmb3Igayx2IGluIHNlbGYucm9vdEV4cHJlc3Npb24uR2V0SXNzdWVEZXRhaWxzKGlzc3VlLCBpbmNsdWRlQ2hpbGRyZW49aW5jbHVkZUNoaWxkcmVuKS5pdGVtcygpOgogICAgICAgICAgICAgICAgcmVzdWx0c1trXSA9IHYKCiAgICAgICAgcmV0dXJuIHJlc3VsdHMKCiAgICBkZWYgR2V0Q29udmVydGVkRXhwcmVzc2lvbihzZWxmLCBzcWw6IE9wdGlvbmFsW3N0cl0gPSBOb25lKToKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwuY29ubmVjdG9yRXhwcmVzc2lvbiBpbXBvcnQgRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uCgogICAgICAgIHNxbCA9IHNlbGYuZ2V0U3FsVmFsdWUoKSBpZiBzcWwgaXMgTm9uZSBlbHNlIHNxbAogICAgICAgIGlmIHNlbGYuQ3JlYXRlU25vd2ZsYWtlT2JqZWN0ID09IFRydWU6CiAgICAgICAgICAgIHNxbCA9IGYiU0VMRUNUICogRlJPTSB7c2VsZi5Tbm93Zmxha2VPYmplY3QuZ2V0X2ZxbigpfSIKICAgICAgICAgICAgCiAgICAgICAgaWYgc3FsIGlzIG5vdCBOb25lOgogICAgICAgICAgICAjIEVzY2FwZSBkb3VibGUgcXVvdGVzIHdpdGggIiIgaW4gUG93ZXIgUXVlcnkuCiAgICAgICAgICAgIHNxbCA9IHNxbC5yZXBsYWNlKCciJywnIiInKS5yZXBsYWNlKCJcbiIsIiAiKS5yZXBsYWNlKCJcdCIsIiAiKQogICAgICAgICAgICBpZiBzZWxmLmlzRGF0YWJhc2U6CiAgICAgICAgICAgICAgICBjb25uZWN0b3JTdHJpbmcgPSAiIgogICAgICAgICAgICAgICAgZm9yIGQgaW4gc2VsZi5HZXRBbGxEZXBlbmRlbmNpZXMoKToKICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGQsIERhdGFiYXNlQ29ubmVjdG9yRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgICAgIGNvbm5lY3RvclN0cmluZyA9IGQudG9NU3RyaW5nKCkKICAgICAgICAgICAgICAgIHJldHVybiBmImxldFxuICAgIFNvdXJjZSA9IFZhbHVlLk5hdGl2ZVF1ZXJ5KHtjb25uZWN0b3JTdHJpbmd9KXt7W05hbWU9XCJ7c2VsZi5Nb2RlbC5DdXJyZW50RGF0YWJhc2V9XCJdfX1bRGF0YV0sIFwie3NxbH1cIiwgbnVsbCwgW0VuYWJsZUZvbGRpbmc9dHJ1ZV0pXG5pblxuICAgIFNvdXJjZSIKCiAgICBkZWYgR2V0T2JqZWN0Q3JlYXRlRERMKHNlbGYsIHNxbDogT3B0aW9uYWxbc3RyXSA9IE5vbmUsIHNmX2RibmFtZTogT3B0aW9uYWxbc3RyXSA9IE5vbmUsIHNmX3NjaGVtYTogT3B0aW9uYWxbc3RyXSA9IE5vbmUpOgogICAgICAgIGZyb20gdXRpbC5zbm93Zmxha2VPYmplY3RUeXBlIGltcG9ydCBTbm93Zmxha2VPYmplY3RUeXBlCiAgICAgICAgCiAgICAgICAgaWYgc3FsIGlzIE5vbmU6ICMgTmVlZCB0byBnZXQgdGhlIGFjdHVhbCBTUUwgdXNlZCB0byBjcmVhdGUgcGFydGl0aW9uIHZpZXcuCiAgICAgICAgICAgIHNxbCA9IHNlbGYuZ2V0U3FsKCkKCiAgICAgICAgIyBPdmVyd3JpdGUgdGhlIGRhdGFiYXNlIGFuZCBzY2hlbWEgaWYgcHJvdmlkZWQuCiAgICAgICAgaWYgc2ZfZGJuYW1lIGlzIG5vdCBOb25lOgogICAgICAgICAgICBzZWxmLlNub3dmbGFrZU9iamVjdC5kYXRhYmFzZSA9IHNmX2RibmFtZQogICAgICAgIAogICAgICAgIGlmIHNmX3NjaGVtYSBpcyBub3QgTm9uZToKICAgICAgICAgICAgc2VsZi5Tbm93Zmxha2VPYmplY3Quc2NoZW1hID0gc2Zfc2NoZW1hCgogICAgICAgIGlmIHNxbCBpcyBub3QgTm9uZTogICAgICAgICAgICAKICAgICAgICAgICAgc2ZPYmplY3RGUU4gPSBzZWxmLlNub3dmbGFrZU9iamVjdC5nZXRfZnFuKCkKCiAgICAgICAgaWYgc2VsZi5leHByZXNzaW9uIGlzIG5vdCBOb25lOgogICAgICAgICAgICBvcmlnaW5hbEV4cHJlc3Npb24gPSBzZWxmLmV4cHJlc3Npb24uc291cmNlU3RyaW5nLnJlcGxhY2UoIlxuIiwgIiAiKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIG9yaWdpbmFsRXhwcmVzc2lvbiA9ICIiCgogICAgICAgIG1hdGNoIHNlbGYuU25vd2ZsYWtlT2JqZWN0LnR5cGU6CiAgICAgICAgICAgIGNhc2UgU25vd2ZsYWtlT2JqZWN0VHlwZS5WSUVXOgogICAgICAgICAgICAgICAgcmV0dXJuIGYiQ1JFQVRFIE9SIFJFUExBQ0UgVklFVyB7c2ZPYmplY3RGUU59XG5DT01NRU5UID0gJ3tvcmlnaW5hbEV4cHJlc3Npb259J1xuQVNcbntzcWx9IgogICAgICAgICAgICBjYXNlIFNub3dmbGFrZU9iamVjdFR5cGUuTUFURVJJQUxJWkVEX1ZJRVc6CiAgICAgICAgICAgICAgICByZXR1cm4gZiJDUkVBVEUgT1IgUkVQTEFDRSBNQVRFUklBTElaRUQgVklFVyB7c2ZPYmplY3RGUU59XG5DT01NRU5UID0gJ3tvcmlnaW5hbEV4cHJlc3Npb259J1xuQVNcbntzcWx9IgogICAgICAgICAgICBjYXNlIF86CiAgICAgICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYie3NlbGYuU25vd2ZsYWtlT2JqZWN0LnR5cGV9IGlzIG5vdCBhIHN1cHBvcnRlZCB0eXBlIGZvciBEREwgY3JlYXRpb24iKQoKICAgIAoKTW9kZWxJdGVtLnJlZ2lzdGVyKFRhYmxlUGFydGl0aW9uKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/page.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbCwgTGlzdCwgVW5pb24sIFNldApmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZAppbXBvcnQgYmkucmVhZGVycy5wYmkucmVwb3J0CmltcG9ydCBiaS5yZWFkZXJzLnBiaS5wYWdlSXRlbQoKY2xhc3MgUGFnZToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCByZXBvcnQ6IGJpLnJlYWRlcnMucGJpLlJlcG9ydCwgbmFtZTogc3RyLCBkaXNwbGF5TmFtZTogc3RyLCBpZDogc3RyLCB3aWR0aDogaW50LCBoZWlnaHQ6IGludCwgcHJvcGVydGllczogZGljdCA9IHt9LCBmaWx0ZXJzOiBkaWN0ID0ge30pOgogICAgICAgIHNlbGYucmVwb3J0ID0gcmVwb3J0CiAgICAgICAgc2VsZi5uYW1lID0gbmFtZQogICAgICAgIHNlbGYuZGlzcGxheU5hbWUgPSBkaXNwbGF5TmFtZQogICAgICAgIHNlbGYuaWQgPSBpZAogICAgICAgIHNlbGYud2lkdGggPSB3aWR0aAogICAgICAgIHNlbGYuaGVpZ2h0ID0gaGVpZ2h0CiAgICAgICAgc2VsZi5wcm9wZXJ0aWVzID0gcHJvcGVydGllcwogICAgICAgIHNlbGYuZmlsdGVycyA9IGZpbHRlcnMKICAgICAgICBzZWxmLml0ZW1zOiBMaXN0W2JpLnJlYWRlcnMucGJpLnBhZ2VJdGVtLlBhZ2VJdGVtXSA9IFtdCgogICAgZGVmIGFkZEl0ZW0oc2VsZiwgbmFtZTogc3RyLCB4OiBpbnQsIHk6IGludCwgejppbnQsIGhlaWdodDogaW50LCB3aWR0aDogaW50LCB0eXBlOiBzdHIsIGRpc3BsYXlNb2RlOiBzdHIgPSAidmlzaWJsZSIsIHF1ZXJ5OiBkaWN0ID0ge30sIGZpbHRlcnM6IExpc3RbZGljdF09W10scHJvcGVydGllczogZGljdCA9IHt9KSAtPiBiaS5wYWdlSXRlbS5QYWdlSXRlbToKICAgICAgICBpdGVtID0gYmkucGFnZUl0ZW0uUGFnZUl0ZW0oc2VsZiwgbmFtZSwgeCwgeSwgeiwgaGVpZ2h0LCB3aWR0aCwgdHlwZSwgZGlzcGxheU1vZGUsIHF1ZXJ5LCBmaWx0ZXJzLCBwcm9wZXJ0aWVzKQogICAgICAgIHNlbGYuaXRlbXMuYXBwZW5kKGl0ZW0pCiAgICAgICAgcmV0dXJuIGl0ZW0KICAgIAogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIGlmIHNlbGYuZGlzcGxheU5hbWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLmRpc3BsYXlOYW1lCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIGYie3NlbGYubmFtZX0iCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/model.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/MOperatorExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KCmZyb20gYmkuY29yZS5leHByZXNzaW9uUGFyZW50IGltcG9ydCBFeHByZXNzaW9uUGFyZW50CmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLm9wZXJhdG9yIGltcG9ydCBPcGVyYXRvcgpmcm9tIC5tRXhwcmVzc2lvbiBpbXBvcnQgTUV4cHJlc3Npb24KCmNsYXNzIE1PcGVyYXRvckV4cHJlc3Npb24oTUV4cHJlc3Npb24sIE9wZXJhdG9yKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBvcGVyYW5kczogbGlzdCwgb3BlcmF0b3JzOiBsaXN0LCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc2VsZi5fX3ZhbHVlID0gTm9uZQogICAgICAgIE1FeHByZXNzaW9uLl9faW5pdF9fKHNlbGYsIHR5cGU9RXhwcmVzc2lvblR5cGUuT3BlcmF0b3IsIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICBPcGVyYXRvci5fX2luaXRfXyhzZWxmKQogICAgICAgIHNlbGYuX19vcGVyYW5kczogTGlzdFtNRXhwcmVzc2lvbl0gPSBbXQogICAgICAgIHNlbGYuX19vcGVyYXRvcnM6IExpc3Rbc3RyXSA9IFtdCiAgICAgICAgZm9yIG9wZXJhbmQgaW4gb3BlcmFuZHM6CiAgICAgICAgICAgIG8gPSBzZWxmLkFkZEV4cHJlc3Npb24ob3BlcmFuZCwgcHJvcGVydGllcz17fSkKICAgICAgICAgICAgaWYgbyBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIHNlbGYuX19vcGVyYW5kcy5hcHBlbmQobykKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHBhc3MKICAgICAgICBzZWxmLl9fb3BlcmF0b3JzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgIGZvciBvIGluIG9wZXJhdG9yczoKICAgICAgICAgICAgc2VsZi5fX29wZXJhdG9ycy5hcHBlbmQobykKCiAgICBAcHJvcGVydHkKICAgIGRlZiBvcGVyYW5kcyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX29wZXJhbmRzCgogICAgQHByb3BlcnR5CiAgICBkZWYgb3BlcmF0b3JzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fb3BlcmF0b3JzCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIE9wZXJhdG9yLnJldHVyblR5cGUuZmdldChzZWxmKQogICAgCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIGlmIHNlbGYuX192YWx1ZSBpcyBOb25lOgogICAgICAgICAgICBzZWxmLl9fdmFsdWUgPSBPcGVyYXRvci5nZXRTcWwoc2VsZikKICAgICAgICByZXR1cm4gc2VsZi5fX3ZhbHVlCiAgICAKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0U3FsVmFsdWUoKQoKICAgIGRlZiBHZXRBbGxJc3N1ZXMoc2VsZik6CiAgICAgICAgYWxsSXNzdWVzID0gc2V0KHNlbGYuSXNzdWVzKQoKICAgICAgICBmb3IgYyBpbiBzZWxmLm9wZXJhbmRzOgogICAgICAgICAgICBpZiBjIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgYWxsSXNzdWVzID0gYWxsSXNzdWVzLnVuaW9uKGMuR2V0QWxsSXNzdWVzKCkpCgogICAgICAgIHJldHVybiBhbGxJc3N1ZXMKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgIyBSZXR1cm4gdGhlIHRyYW5zbGF0ZWQgU1FMIGV4cHJlc3Npb24uCiAgICAgICAgIyBEaXJlY3RDb252ZXJzaW9uIGV4cGVjdGVzIHRoZSB2YWx1ZSB0byBiZSB0aGUgdHJhbnNsYXRlZCBTUUwgZXhwcmVzc2lvbi4KICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSgpCgogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYiT3BlcmF0b3I6IHtzZWxmLnNvdXJjZVN0cmluZ30iCiAgICAKTUV4cHJlc3Npb24ucmVnaXN0ZXIoTU9wZXJhdG9yRXhwcmVzc2lvbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functionExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgT3B0aW9uYWwsIFVuaW9uLCBBbnkKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC5tRXhwcmVzc2lvbiBpbXBvcnQgTUV4cHJlc3Npb24KCmNsYXNzIEZ1bmN0aW9uRXhwcmVzc2lvbihNRXhwcmVzc2lvbiwgRnVuY3Rpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIG5hbWU6IHN0ciwgcGFyYW1ldGVyU3RyaW5nOiBzdHIsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCiAgICAgICAgTUV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1FeHByZXNzaW9uVHlwZS5GdW5jdGlvbiwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIEZ1bmN0aW9uLl9faW5pdF9fKHNlbGYsIGZ1bmN0aW9uTmFtZT1uYW1lLCBwYXJhbWV0ZXJTdHJpbmc9cGFyYW1ldGVyU3RyaW5nKQogICAgICAgIAogICAgICAgIHNlbGYuX19zb3VyY2UgPSBOb25lCiAgICAgICAgc291cmNlcyA9IFtdCgogICAgICAgIGZvciBwIGluIHNlbGYucGFyYW1ldGVyczoKICAgICAgICAgICAgaWYgcCBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UocCwgVGFibGUpIG9yIG5vdCBwLmlzVmFsdWU6CiAgICAgICAgICAgICAgICAgICAgc291cmNlcy5hcHBlbmQocCkKCiAgICAgICAgaWYgbGVuKHNvdXJjZXMpID09IDA6CiAgICAgICAgICAgIHNlbGYuX19zb3VyY2UgPSBzZWxmCiAgICAgICAgZWxpZiBsZW4oc291cmNlcykgPT0gMToKICAgICAgICAgICAgc2VsZi5fX3NvdXJjZSA9IHNlbGYucGFyYW1ldGVyc1swXQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHNlbGYuX19zb3VyY2UgPSBzb3VyY2VzCgogICAgZGVmIHVwZGF0ZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5HZXRSZXN1bHQoKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIEZ1bmN0aW9uLnJldHVyblR5cGUuZmdldChzZWxmKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBzb3VyY2Uoc2VsZikgLT4gVW5pb25bRXhwcmVzc2lvbnxMaXN0W0V4cHJlc3Npb25dXToKICAgICAgICByZXR1cm4gc2VsZi5fX3NvdXJjZQogICAgCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIHJldHVybiBFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYudmFsdWUpCiAgICAKICAgICMgc2V0U291cmNlIGlzIHRvIGFsbG93IGEgZnVuY3Rpb24gaGFuZGxlciB0byBzZXQgdGhlIHNvdXJjZSBvbiB0aGUgRnVuY3Rpb25FeHByZXNzaW9uLCBvdGhlcndpc2Ugd2UnbGwgdXNlIHRoZSBwYXJhbWV0ZXJzIHRvIGRlZmluZSB0aGUgc291cmNlCiAgICBkZWYgc2V0U291cmNlKHNlbGYsIHNvdXJjZTogTUV4cHJlc3Npb24pOgogICAgICAgIHNlbGYuX19zb3VyY2UgPSBzb3VyY2UKCgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fc3RyX18oKQogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICBwYXJhbWV0ZXJzID0gW10KICAgICAgICBmb3IgcCBpbiBzZWxmLnBhcmFtZXRlcnM6CiAgICAgICAgICAgIGlmIHAgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBwYXJhbWV0ZXJzLmFwcGVuZChwLmdldFNxbFZhbHVlKCkpCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICBwYXJhbWV0ZXJzLmFwcGVuZCgiTlVMTCIpCiAgICAgICAgICAgICAgICAKICAgICAgICBwYXJhbWV0ZXJTdHJpbmcgPSAiLCAiLmpvaW4ocGFyYW1ldGVycykKCiAgICAgICAgcmV0dXJuIGYie3NlbGYuZnVuY3Rpb25OYW1lfSh7cGFyYW1ldGVyU3RyaW5nfSkgLT4ge3NlbGYudmFsdWV9IgoKRnVuY3Rpb24ucmVnaXN0ZXIoRnVuY3Rpb25FeHByZXNzaW9uKQpNRXhwcmVzc2lvbi5yZWdpc3RlcihGdW5jdGlvbkV4cHJlc3Npb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/tableExpression.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/mExpression.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/RecordExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IFZhbHVlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gdHlwaW5nIGltcG9ydCBBbnksIERpY3QsIExpc3QsIE9wdGlvbmFsCgoKY2xhc3MgUmVjb3JkRXhwcmVzc2lvbihWYWx1ZUV4cHJlc3Npb24pOgogICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB2YWx1ZTogZGljdCwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIHNlbGYuX19pbm5lckRpY3Q6IERpY3Rbc3RyLEFueV0gPSB7fQoKICAgICAgICBzdXBlcigpLl9faW5pdF9fKHR5cGU9RXhwcmVzc2lvblR5cGUuUmVjb3JkLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgCiAgICAgICAgZm9yIGssdiBpbiB2YWx1ZS5pdGVtcygpOgogICAgICAgICAgICBrID0gay5zdHJpcCgpCiAgICAgICAgICAgICMgcmVjb3JkIG5hbWVzIG1pZ2h0IGJlIGluIHRoZSBmb3JtICMiUmVjb3JkIE5hbWUiCiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2Uoaywgc3RyKSBhbmQgay5zdGFydHN3aXRoKCIjXCIiKSBhbmQgay5lbmRzd2l0aCgiXCIiKToKICAgICAgICAgICAgICAgIGsgPSBrWzI6bGVuKGspLTFdCiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgc2VsZi5fX2lubmVyRGljdFtrXSA9IHNlbGYuQWRkRXhwcmVzc2lvbihleHByZXNzaW9uPXYsIHByb3BlcnRpZXM9e30pCiAgICAgICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpIC0+IGRpY3Q6CiAgICAgICAgcmV0dXJuIHNlbGYuX19pbm5lckRpY3QKCiAgICBAcHJvcGVydHkKICAgIGRlZiBrZXlzKHNlbGYpIC0+IExpc3Rbc3RyXToKICAgICAgICByZXR1cm4gbGlzdChzZWxmLnZhbHVlLmtleXMoKSkKCiAgICBkZWYgZmlsdGVyKHNlbGYsIGZpbHRlcjogTGlzdFtzdHJdKToKICAgICAgICByZXN1bHQgPSB7fQogICAgICAgIGZvciBrIGluIGZpbHRlcjoKICAgICAgICAgICAgaWYgayBpbiBzZWxmLnZhbHVlOgogICAgICAgICAgICAgICAgcmVzdWx0W2tdID0gc2VsZi52YWx1ZVtrXQogICAgICAgIGlmIGxlbihyZXN1bHQpID4gMDoKICAgICAgICAgICAgcmV0dXJuIFJlY29yZEV4cHJlc3Npb24ocmVzdWx0LCBzZWxmLCBwcm9wZXJ0aWVzPXsiZmlsdGVyIjogZmlsdGVyfSkKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCiAgICAgICAgZnJvbSBkYXRldGltZSBpbXBvcnQgZGF0ZXRpbWUsIGRhdGUsIHRpbWUKICAgICAgICBwYXJhbWV0ZXJzID0gW10KICAgICAgICBmb3Igayx2IGluIHNlbGYudmFsdWUuaXRlbXMoKToKICAgICAgICAgICAgcGFyYW1ldGVycy5hcHBlbmQoZiIne2t9JyIpCiAgICAgICAgICAgIHBhcmFtZXRlcnMuYXBwZW5kKEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUodikpCgogICAgICAgIHNxbCA9IGYiT0JKRUNUX0NPTlNUUlVDVCh7JywnLmpvaW4ocGFyYW1ldGVycyl9KSIKICAgICAgICByZXR1cm4gc3FsCgogICAgZGVmIGdldFZhbHVlKHNlbGYsIG5hbWU6IE9wdGlvbmFsW3N0cl0gPSBOb25lKToKICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIAogICAgICAgIGlmIG5hbWUgaXMgTm9uZToKICAgICAgICAgICAgdmFsdWUgPSB7fQogICAgICAgICAgICBpZiBzZWxmLl9faW5uZXJEaWN0IGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgZm9yIGssIHYgaW4gc2VsZi52YWx1ZS5pdGVtcygpOgogICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UodiwgVmFsdWVFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgdiA9IHYuZ2V0VmFsdWUoKQogICAgICAgICAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2LCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgdiA9IHYudmFsdWUKICAgICAgICAgICAgICAgICAgICB2YWx1ZVtrXSA9IHYKICAgICAgICBlbHNlOgogICAgICAgICAgICB2YWx1ZSA9IHNlbGYuZ2V0SXRlbShuYW1lLCBUcnVlKQogICAgICAgIHJldHVybiB2YWx1ZQogICAgCiAgICBkZWYgZ2V0SXRlbShzZWxmLCBuYW1lOiBzdHIsIHJldHVyblZhbHVlOiBib29sID0gRmFsc2UpOgogICAgICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCiAgICAgICAgdmFsdWUgPSBzZWxmLmdldEV4cHJlc3Npb24obmFtZSkKICAgICAgICBpZiByZXR1cm5WYWx1ZSA9PSBUcnVlOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHZhbHVlLCBWYWx1ZUV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgdmFsdWUgPSB2YWx1ZS5nZXRWYWx1ZSgpCiAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2YWx1ZSwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICB2YWx1ZSA9IHZhbHVlLnZhbHVlCiAgICAgICAgcmV0dXJuIHZhbHVlCiAgICAKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIG5hbWUpOgogICAgICAgIGlmIHNlbGYuX19pbm5lckRpY3QgaXMgbm90IE5vbmUgYW5kIG5hbWUgaW4gc2VsZi5fX2lubmVyRGljdDoKICAgICAgICAgICAgcmV0dXJuIHNlbGYuX19pbm5lckRpY3QuZ2V0KG5hbWUpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIHN1cGVyKCkuZ2V0RXhwcmVzc2lvbihuYW1lKQoKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICBpdGVtcyA9IFtdCiAgICAgICAgZm9yIGssdiBpbiBzZWxmLl9faW5uZXJEaWN0Lml0ZW1zKCk6CiAgICAgICAgICAgIGl0ZW1zLmFwcGVuZChmIntrfSA9ICB7dn0iKQogICAgICAgIGl0ZW1zU3RyID0gIiwiLmpvaW4oaXRlbXMpCgogICAgICAgIHJldHVybiBmIntzZWxmLnR5cGV9OiB7aXRlbXNTdHJ9IgoKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIGl0ZW1zID0gW10KICAgICAgICBmb3Igayx2IGluIHNlbGYuX19pbm5lckRpY3QuaXRlbXMoKToKICAgICAgICAgICAgaXRlbXMuYXBwZW5kKGYie2t9ID0gIHtzdHIodil9IikKICAgICAgICBpdGVtc1N0ciA9ICIsIi5qb2luKGl0ZW1zKQoKICAgICAgICByZXR1cm4gZiJbIHtpdGVtc1N0cn0gXSIKCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHN1cGVyKCkuX19oYXNoX18oKQogICAgClZhbHVlRXhwcmVzc2lvbi5yZWdpc3RlcihSZWNvcmRFeHByZXNzaW9uKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/connectorExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgT3B0aW9uYWwsIEFueSwgRGljdCwgU2V0CmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmNvbm5lY3RvciBpbXBvcnQgQ29ubmVjdG9yLCBDb25uZWN0b3JUeXBlCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC5tRXhwcmVzc2lvbiBpbXBvcnQgTUV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLm5hbWVkSXRlbSBpbXBvcnQgTmFtZWRJdGVtQ29udGFpbmVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uUGFyZW50IGltcG9ydCBFeHByZXNzaW9uUGFyZW50CgoKCmNsYXNzIENvbm5lY3RvckV4cHJlc3Npb24oTUV4cHJlc3Npb24sIENvbm5lY3RvciwgQUJDKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB0eXBlOiBDb25uZWN0b3JUeXBlLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgTUV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1FeHByZXNzaW9uVHlwZS5Db25uZWN0b3IsIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICBDb25uZWN0b3IuX19pbml0X18oc2VsZiwgdHlwZT10eXBlLCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIHJldHVybiAiIiAjVE9ETyAKICAgIAogICAgZGVmIHVwZGF0ZShzZWxmKToKICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYuc291cmNlVHlwZS5uYW1lfSIKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19yZXByX18oKQogICAgCk1FeHByZXNzaW9uLnJlZ2lzdGVyKENvbm5lY3RvckV4cHJlc3Npb24pCkNvbm5lY3Rvci5yZWdpc3RlcihDb25uZWN0b3JFeHByZXNzaW9uKQoKY2xhc3MgTXVsdGlwbGVDb25uZWN0b3JFeHByZXNzaW9uKENvbm5lY3RvckV4cHJlc3Npb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIGNvbm5lY3RvcnM6IFNldFtDb25uZWN0b3JFeHByZXNzaW9uXSwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIHN1cGVyKCkuX19pbml0X18odHlwZT1Db25uZWN0b3JUeXBlLk11bHRpcGxlLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgZm9yIGMgaW4gY29ubmVjdG9yczoKICAgICAgICAgICAgc2VsZi5BZGREZXBlbmRlbmN5KGMpCgogICAgICAgIHNlbGYuX19jb25uZWN0b3JzID0gY29ubmVjdG9ycwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBjb25uZWN0b3JzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fY29ubmVjdG9ycwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0ZpbGUoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gYW55KGMuaXNGaWxlIGZvciBjIGluIHNlbGYuY29ubmVjdG9ycykKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNDYWNoZWQoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gYW55KGMuaXNDYWNoZWQgZm9yIGMgaW4gc2VsZi5jb25uZWN0b3JzKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0RhdGFiYXNlKHNlbGYpIC0+IGJvb2w6CiAgICAgICAgcmV0dXJuIGFueShjLmlzRGF0YWJhc2UgZm9yIGMgaW4gc2VsZi5jb25uZWN0b3JzKQogICAgCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGhhc2goKHNlbGYuc291cmNlVHlwZSwgc3RyKHNlbGYuY29ubmVjdG9ycykpKQoKQ29ubmVjdG9yRXhwcmVzc2lvbi5yZWdpc3RlcihNdWx0aXBsZUNvbm5lY3RvckV4cHJlc3Npb24pCgpjbGFzcyBCYXNlRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uKENvbm5lY3RvckV4cHJlc3Npb24sIE5hbWVkSXRlbUNvbnRhaW5lcik6CiAgICBmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCiAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgZGVmIF9faW5pdF9fKHNlbGYsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgZGF0YWJhc2VUeXBlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgY2hpbGRUeXBlczogRGljdFtzdHIsdHlwZV0gPSB7fSwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBpZiAiZGF0YWJhc2VUeXBlIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgaWYgZGF0YWJhc2VUeXBlIGlzIE5vbmU6CiAgICAgICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJNdXN0IHByb3ZpZGUgYSBkYXRhYmFzZSB0eXBlIikKICAgICAgICAgICAgCiAgICAgICAgICAgIHByb3BlcnRpZXNbImRhdGFiYXNlVHlwZSJdID0gZGF0YWJhc2VUeXBlCgogICAgICAgIENvbm5lY3RvckV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1Db25uZWN0b3JUeXBlLkRhdGFiYXNlLCBwYXJlbnQ9IHBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIE5hbWVkSXRlbUNvbnRhaW5lci5fX2luaXRfXyhzZWxmLCBjaGlsZFR5cGVzKQogICAgICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgZGF0YWJhc2VUeXBlKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gc2VsZi5nZXRQcm9wZXJ0eSgiZGF0YWJhc2VUeXBlIikKICAgIAogICAgZGVmIGNvbm5lY3Rpb25TdHJpbmcoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBzZWxmLmdldFByb3BlcnR5KCJjb25uZWN0aW9uU3RyaW5nIikKICAgIAogICAgZGVmIGNvbm5lY3Rpb25EZXRhaWxzKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gc2VsZi5nZXRQcm9wZXJ0eSgiY29ubmVjdGlvbkRldGFpbHMiKQogICAgCiAgICBkZWYgZXhlY3V0ZVF1ZXJ5KHNlbGYsIHF1ZXJ5OiBzdHIpIC0+IEV4cHJlc3Npb246CiAgICAgICAgcmV0dXJuIE5vbmUKCiAgICBkZWYgdG9NU3RyaW5nKHNlbGYpOgogICAgICAgIHJldHVybiAiIgogICAgCiAgICBkZWYgZ2V0VGFibGVDb2x1bW5zKHNlbGYsIHRhYmxlOiBFeHByZXNzaW9uKSAtPiBMaXN0W0NvbHVtbl06CiAgICAgICAgcmV0dXJuIFtdCgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzdXBlcigpLl9fcmVwcl9fKCl9IHtzZWxmLmRhdGFiYXNlVHlwZX0iCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzdXBlcigpLl9fc3RyX18oKX0ge3NlbGYuZGF0YWJhc2VUeXBlfSIKICAgIAogICAgZGVmIF9fZXFfXyhzZWxmLCB2YWx1ZSk6CiAgICAgICAgaWYgc3VwZXIoKS5fX2VxX18odmFsdWUpOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHZhbHVlLCBCYXNlRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIHJldHVybiB2YWx1ZS5kYXRhYmFzZVR5cGUgPT0gc2VsZi5kYXRhYmFzZVR5cGUKICAgICAgICByZXR1cm4gRmFsc2UKICAgIAogICAgZGVmIF9faGFzaF9fKHNlbGYpOgogICAgICAgIHJldHVybiBoYXNoKChzZWxmLnR5cGUsIHNlbGYuc291cmNlVHlwZSwgc2VsZi5kYXRhYmFzZVR5cGUpKQoKQ29ubmVjdG9yRXhwcmVzc2lvbi5yZWdpc3RlcihCYXNlRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uKQoKY2xhc3MgRmlsZUNvbm5lY3RvckV4cHJlc3Npb24oQ29ubmVjdG9yRXhwcmVzc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZiwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwYXRoOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgZW5jb2Rpbmc6IE9wdGlvbmFsW3N0cl0gPSBOb25lLCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIGltcG9ydCBpbwogICAgICAgIGlmICJwYXRoIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgcHJvcGVydGllc1sicGF0aCJdID0gcGF0aAogICAgICAgIGlmICJlbmNvZGluZyIgbm90IGluIHByb3BlcnRpZXM6CiAgICAgICAgICAgIHByb3BlcnRpZXNbImVuY29kaW5nIl0gPSBlbmNvZGluZwoKICAgICAgICBzdXBlcigpLl9faW5pdF9fKHR5cGU9Q29ubmVjdG9yVHlwZS5GaWxlLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgc2VsZi5fX2NvbnRlbnRzOiBieXRlcyA9IE5vbmUKICAgICAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBhdGgoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBzZWxmLmdldFByb3BlcnR5KCJwYXRoIikKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgZW5jb2Rpbmcoc2VsZikgLT4gT3B0aW9uYWxbc3RyXToKICAgICAgICByZXR1cm4gc2VsZi5nZXRQcm9wZXJ0eSgiZW5jb2RpbmciKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc0F2YWlsYWJsZShzZWxmKToKICAgICAgICBpbXBvcnQgb3MKICAgICAgICByZXR1cm4gb3MucGF0aC5leGlzdHMoc2VsZi5wYXRoKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBieXRlcyhzZWxmKSAtPiBPcHRpb25hbFtieXRlc106CiAgICAgICAgaW1wb3J0IGlvCiAgICAgICAgaWYgc2VsZi5fX2NvbnRlbnRzIGlzIE5vbmU6CiAgICAgICAgICAgIGlmIHNlbGYuaXNBdmFpbGFibGU6CiAgICAgICAgICAgICAgICB0cnk6CiAgICAgICAgICAgICAgICAgICAgd2l0aCAoaW8ub3BlbihzZWxmLnBhdGgsICJyYiIpKSBhcyBmOgogICAgICAgICAgICAgICAgICAgICAgICBzZWxmLl9fY29udGVudHMgPSBmLnJlYWQoKQogICAgICAgICAgICAgICAgZXhjZXB0OgogICAgICAgICAgICAgICAgICAgIHBhc3MKICAgICAgICByZXR1cm4gc2VsZi5fX2NvbnRlbnRzCgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzdXBlcigpLl9fcmVwcl9fKCl9IHtzZWxmLnBhdGh9IgogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c3VwZXIoKS5fX3N0cl9fKCl9IHtzZWxmLnBhdGh9IgogICAgCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlKToKICAgICAgICBpZiBzdXBlcigpLl9fZXFfXyh2YWx1ZSk6CiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIEZpbGVDb25uZWN0b3JFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIHJldHVybiB2YWx1ZS5wYXRoID09IHNlbGYucGF0aAogICAgICAgIHJldHVybiBGYWxzZQoKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaCgoc2VsZi50eXBlLCBzZWxmLnNvdXJjZVR5cGUsIHNlbGYucGF0aCkpCgpDb25uZWN0b3JFeHByZXNzaW9uLnJlZ2lzdGVyKEZpbGVDb25uZWN0b3JFeHByZXNzaW9uKQoKY2xhc3MgQ2FjaGVkQ29ubmVjdG9yRXhwcmVzc2lvbihDb25uZWN0b3JFeHByZXNzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBkYXRhOiBBbnksIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgZW5jb2Rpbmc6IE9wdGlvbmFsW3N0cl09Tm9uZSwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBpZiAiZGF0YSIgbm90IGluIHByb3BlcnRpZXM6CiAgICAgICAgICAgIHByb3BlcnRpZXNbImRhdGEiXSA9IGRhdGEKICAgICAgICBzdXBlcigpLl9faW5pdF9fKHR5cGU9Q29ubmVjdG9yVHlwZS5DYWNoZWQsIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGRhdGEoc2VsZikgLT4gc3RyOgogICAgICAgIHJldHVybiBzZWxmLmdldFByb3BlcnR5KCJkYXRhIikKCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlKToKICAgICAgICBpZiBzdXBlcigpLl9fZXFfXyh2YWx1ZSk6CiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIENhY2hlZENvbm5lY3RvckV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgcmV0dXJuIHZhbHVlLmRhdGEgPT0gc2VsZi5kYXRhCiAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaCgoc2VsZi50eXBlLCBzZWxmLnNvdXJjZVR5cGUpKQoKQ29ubmVjdG9yRXhwcmVzc2lvbi5yZWdpc3RlcihDYWNoZWRDb25uZWN0b3JFeHByZXNzaW9uKQoKCmNsYXNzIERhdGFiYXNlQ29ubmVjdG9yRXhwcmVzc2lvbihCYXNlRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uKToKICAgIGZyb20gYmkucmVhZGVycy5wYmkuY29sdW1uIGltcG9ydCBDb2x1bW4KICAgIGZyb20gYmkucmVhZGVycy5wYmkudG1kbC5tRXhwcmVzc2lvbiBpbXBvcnQgTUV4cHJlc3Npb24KCiAgICBkZWYgX19pbml0X18oc2VsZiwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCByb2xlOiBPcHRpb25hbFtzdHJdID0gTm9uZSwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IERhdGFiYXNlRXhwcmVzc2lvbiwgRGF0YWJhc2VTY2hlbWFFeHByZXNzaW9uCiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhwYXJlbnQ9cGFyZW50LCBjaGlsZFR5cGVzPXsgIkRhdGFiYXNlIjogRGF0YWJhc2VFeHByZXNzaW9uLCAiU2NoZW1hIjogRGF0YWJhc2VTY2hlbWFFeHByZXNzaW9uIH0sIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgIAogICAgZGVmIGdldFRhYmxlQ29sdW1ucyhzZWxmLCB0YWJsZTogTUV4cHJlc3Npb24pIC0+IE9wdGlvbmFsW0xpc3RbQ29sdW1uXV06CiAgICAgICAgY29scyA9IE5vbmUKICAgICAgICBpZiBzZWxmLmlzU25vd2ZsYWtlOgogICAgICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBEYXRhYmFzZVRhYmxlRXhwcmVzc2lvbgogICAgICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBzZWxmLk1vZGVsLlNlc3Npb24gaXMgbm90IE5vbmUgYW5kIGlzaW5zdGFuY2UodGFibGUsIERhdGFiYXNlVGFibGVFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIGNtZCA9IGYiREVTQyBUQUJMRSB7dGFibGUuZnVsbHlRdWFsaWZpZWROYW1lfSIKICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICByZXN1bHRzID0gc2VsZi5Nb2RlbC5TZXNzaW9uLnNxbChjbWQpLmNvbGxlY3QoKQogICAgICAgICAgICAgICAgICAgIGNvbHMgPSBbXQogICAgICAgICAgICAgICAgICAgIGZvciByb3cgaW4gcmVzdWx0czoKICAgICAgICAgICAgICAgICAgICAgICAgY29sTmFtZSA9IHJvd1snbmFtZSddCiAgICAgICAgICAgICAgICAgICAgICAgIHR5cGVTdHIgPSByb3dbJ3R5cGUnXQogICAgICAgICAgICAgICAgICAgICAgICBpZiAiKCIgaW4gdHlwZVN0cjoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGkgPSB0eXBlU3RyLmluZGV4KCIoIikKICAgICAgICAgICAgICAgICAgICAgICAgICAgIGlmIGkgPiAwOgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHR5cGVTdHIgPSB0eXBlU3RyWzA6aV0KICAgICAgICAgICAgICAgICAgICAgICAgY29sVHlwZSA9IEV4cHJlc3Npb25UeXBlLkZyb21TdHJpbmcodHlwZVN0cikKICAgICAgICAgICAgICAgICAgICAgICAgaXNOdWxsaWJsZSA9IChyb3dbJ251bGw/J10gPT0gIlkiKQogICAgICAgICAgICAgICAgICAgICAgICBpc1ByaW1hcnlLZXkgPSAocm93WydwcmltYXJ5IGtleSddID09ICJZIikKICAgICAgICAgICAgICAgICAgICAgICAgY29scy5hcHBlbmQoQ29sdW1uKG5hbWU9Y29sTmFtZSwgdHlwZT1jb2xUeXBlLCBpc051bGxhYmxlPWlzTnVsbGlibGUsIGlzUHJpbWFyeUtleT1pc1ByaW1hcnlLZXkpKQogICAgICAgICAgICAgICAgZXhjZXB0OgogICAgICAgICAgICAgICAgICAgIGNvbHMgPSBOb25lCiAgICAgICAgcmV0dXJuIGNvbHMKCiAgICBAcHJvcGVydHkKICAgIGRlZiBkZWZhdWx0S2luZChzZWxmKSAtPiBzdHI6CiAgICAgICAgaWYgc2VsZi5pc1Nub3dmbGFrZToKICAgICAgICAgICAgcmV0dXJuICJEYXRhYmFzZSIKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gIlNjaGVtYSIKICAgIAogICAgZGVmIHRvTVN0cmluZyhzZWxmKToKICAgICAgICBpZiBzZWxmLmlzU25vd2ZsYWtlOgogICAgICAgICAgICByZXR1cm4gZiJTbm93Zmxha2UuRGF0YWJhc2VzKFwie3NlbGYuZ2V0UHJvcGVydHkoJ2FjY291bnQnLCcnKX1cIixcIntzZWxmLmdldFByb3BlcnR5KCd3YXJlaG91c2UnLCcnKX1cIixbSW1wbGVtZW50YXRpb249XCIyLjBcIl0iCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuICIiCiAgICAKICAgIGRlZiBfX3JlcHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3N0cl9fKCkKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYuZGF0YWJhc2VUeXBlfSIKICAgICAgICAKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gc3VwZXIoKS5fX2hhc2hfXygpCgpCYXNlRGF0YWJhc2VDb25uZWN0b3JFeHByZXNzaW9uLnJlZ2lzdGVyKERhdGFiYXNlQ29ubmVjdG9yRXhwcmVzc2lvbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/anonFunctionExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIHR5cGluZyBpbXBvcnQgTGlzdCwgT3B0aW9uYWwsIFVuaW9uLCBBbnkKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLm1FeHByZXNzaW9uIGltcG9ydCBNRXhwcmVzc2lvbgppbXBvcnQgcmVnZXgKCmNsYXNzIEFub255bW91c0Z1bmN0aW9uRXhwcmVzc2lvbihNRXhwcmVzc2lvbiwgRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIGV4cHJlc3Npb25TdHJpbmc6IHN0ciwgcGFyYW1ldGVyU3RyaW5nOiBzdHIgPSAiIiwgcmV0dXJuVHlwZTogRXhwcmVzc2lvblR5cGUgPSBFeHByZXNzaW9uVHlwZS5BbnksIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc2VsZi5fX2V4cHJlc3Npb25TdHJpbmcgPSBleHByZXNzaW9uU3RyaW5nCiAgICAgICAgc2VsZi5fX2V4cHJlc3Npb246IEV4cHJlc3Npb24gPSBOb25lCiAgICAgICAgc2VsZi5fX2Z1bmN0aW9uOiBGdW5jdGlvbiA9IE5vbmUKCiAgICAgICAgTUV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1FeHByZXNzaW9uVHlwZS5GdW5jdGlvbiwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIEZ1bmN0aW9uSGFuZGxlci5fX2luaXRfXyhzZWxmLCBuYW1lPSIiLCByZXR1cm5UeXBlPXJldHVyblR5cGUpCgogICAgICAgIHNlbGYuX19wb3NpdGlvbmFsUGFyYW1ldGVyczogTGlzdFtGdW5jdGlvblBhcmFtZXRlcl0gPSBbXQoKICAgICAgICBwYXJhbXRlclBhdHRlcm4gPSByIig/PG5hbWU+XHcrKShccythc1xzKyg/PHR5cGU+XHcrKSl7MCwxfSIKICAgICAgICBmb3IgcCBpbiBwYXJhbWV0ZXJTdHJpbmcuc3BsaXQoIiwiKToKICAgICAgICAgICAgcGFyYW1ldGVyTWF0Y2ggPSByZWdleC5tYXRjaChwYXJhbXRlclBhdHRlcm4sIHAuc3RyaXAoKSkKICAgICAgICAgICAgaWYgcGFyYW1ldGVyTWF0Y2ggaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBwTmFtZSA9IHBhcmFtZXRlck1hdGNoLmdyb3VwKCJuYW1lIikKICAgICAgICAgICAgICAgIHBUeXBlID0gcGFyYW1ldGVyTWF0Y2guZ3JvdXAoInR5cGUiKQogICAgICAgICAgICAgICAgaWYgcFR5cGUgaXMgbm90IE5vbmUgYW5kIGxlbihwVHlwZSkgPiAwOgogICAgICAgICAgICAgICAgICAgIHBUeXBlID0gRXhwcmVzc2lvblR5cGUuRnJvbVN0cmluZyhwVHlwZSkKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgcFR5cGUgPSBFeHByZXNzaW9uVHlwZS5BbnkKICAgICAgICAgICAgICAgIHNlbGYuX19wb3NpdGlvbmFsUGFyYW1ldGVycy5hcHBlbmQoRnVuY3Rpb25QYXJhbWV0ZXIobmFtZT1wTmFtZSwgdHlwZT1wVHlwZSkpCgogICAgICAgIAogICAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBzZWxmLl9fcG9zaXRpb25hbFBhcmFtZXRlcnMKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgc291cmNlKHNlbGYpOgogICAgICAgIGlmIHNlbGYuX19mdW5jdGlvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuX19mdW5jdGlvbi5wYXJhbWV0ZXJzCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKToKICAgICAgICBpZiBzZWxmLl9fZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuX19leHByZXNzaW9uLnZhbHVlIAogICAgCiAgICBkZWYgZ2V0RXhwcmVzc2lvbihzZWxmLCBuYW1lKToKICAgICAgICBleHByZXNzaW9uID0gTm9uZQogICAgICAgIGlmIHNlbGYuX19mdW5jdGlvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgZXhwcmVzc2lvbiA9IHNlbGYuZ2V0UGFyYW1ldGVyKG5hbWUpCiAgICAgICAgaWYgZXhwcmVzc2lvbiBpcyBOb25lOgogICAgICAgICAgICBleHByZXNzaW9uID0gTUV4cHJlc3Npb24uZ2V0RXhwcmVzc2lvbihzZWxmLCBuYW1lKQogICAgICAgIHJldHVybiBleHByZXNzaW9uCiAgICAKICAgIGRlZiBTZXRGdW5jdGlvbihzZWxmLCBmdW5jdGlvbjogRnVuY3Rpb24pOgogICAgICAgICNzZWxmLlNldFBhcmFtZXRlclZhbHVlcyhmdW5jdGlvbi5wYXJhbWV0ZXJzKQogICAgICAgIHNlbGYuX19mdW5jdGlvbiA9IGZ1bmN0aW9uCiAgICAgICAgRnVuY3Rpb25IYW5kbGVyLlNldEZ1bmN0aW9uKHNlbGYsIGZ1bmN0aW9uKQogICAgICAgIHNlbGYuX19leHByZXNzaW9uID0gc2VsZi5BZGRFeHByZXNzaW9uKHNlbGYuX19leHByZXNzaW9uU3RyaW5nLCBwcm9wZXJ0aWVzPXt9KQoKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZXZhbHVhdGUoKQogICAgICAgIAogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICBpZiBzZWxmLl9fZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuX19leHByZXNzaW9uLmdldFNxbCgpCgogICAgIyBkZWYgU2V0UGFyYW1ldGVyVmFsdWVzKHNlbGYsIHBhcmFtZXRlcnM6IExpc3RbRXhwcmVzc2lvbl0pOgogICAgIyAgICAgZm9yIGksIHYgaW4gZW51bWVyYXRlKHBhcmFtZXRlcnMpOgogICAgIyAgICAgICAgIHBhcmFtZXRlciA9IHNlbGYuX19wYXJhbWV0ZXJzW2ldCiAgICAjICAgICAgICAgc2VsZi5fX3BhcmFtZXRlclZhbHVlc1twYXJhbWV0ZXIubmFtZV0gPSB2CiAgICAjICAgICBzZWxmLl9fZXhwcmVzc2lvbiA9IHNlbGYuQWRkRXhwcmVzc2lvbihzZWxmLl9fZXhwcmVzc2lvblN0cmluZykKCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19zdHJfXygpCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHBhcmFtZXRlcnMgPSAiLCAiLmpvaW4oW3N0cihwKSBmb3IgcCBpbiBzZWxmLnBvc2l0aW9uYWxQYXJhbWV0ZXJzXSkKCiAgICAgICAgcmV0dXJuIGYiKHtwYXJhbWV0ZXJzfSkgPT4ge3NlbGYuX19leHByZXNzaW9uU3RyaW5nfSIKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihBbm9ueW1vdXNGdW5jdGlvbkV4cHJlc3Npb24pCk1FeHByZXNzaW9uLnJlZ2lzdGVyKEFub255bW91c0Z1bmN0aW9uRXhwcmVzc2lvbikKCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/ValueNativeQuery.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwuY29ubmVjdG9yRXhwcmVzc2lvbiBpbXBvcnQgQ29ubmVjdG9yRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBRdWVyeVRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RleHQtZnJvbQpjbGFzcyBWYWx1ZU5hdGl2ZVF1ZXJ5KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVmFsdWUuTmF0aXZlUXVlcnkiLCBFeHByZXNzaW9uVHlwZS5BbnkpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YXJnZXQiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInF1ZXJ5IiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJwYXJhbWV0ZXJzIiwgRXhwcmVzc2lvblR5cGUuQW55LCBUcnVlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigib3B0aW9ucyIsIEV4cHJlc3Npb25UeXBlLlJlY29yZCwgVHJ1ZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFyZ2V0ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFyZ2V0IikKICAgICAgICBxdWVyeSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInF1ZXJ5IikKICAgICAgICBwYXJhbWV0ZXJzID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgicGFyYW1ldGVycyIpCiAgICAgICAgb3B0aW9ucyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm9wdGlvbnMiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhcmdldCwgQ29ubmVjdG9yRXhwcmVzc2lvbik6CiAgICAgICAgICAgIHJldHVybiBRdWVyeVRhYmxlRXhwcmVzc2lvbihxdWVyeSwgdGFyZ2V0LCBzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcGFyYW1ldGVycywgb3B0aW9ucykKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihWYWx1ZU5hdGl2ZVF1ZXJ5KQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/DateFrom.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL2RhdGUtZnJvbQpjbGFzcyBEYXRlRnJvbShGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkRhdGUuRnJvbSIsIEV4cHJlc3Npb25UeXBlLkRhdGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbICBGdW5jdGlvblBhcmFtZXRlcigidmFsdWUiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwgRnVuY3Rpb25QYXJhbWV0ZXIoImN1bHR1cmUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKV0KICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB2YWx1ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInZhbHVlIikKICAgICAgICBjdWx0dXJlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY3VsdHVyZSIpCiAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iVE9fREFURSIsIHBhcmFtZXRlcnM9W3ZhbHVlXSwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5EYXRlKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKERhdGVGcm9tKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/SnowflakeDatabases.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gdHlwaW5nIGltcG9ydCBMaXN0LCBBbnkKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLm1FeHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlci1xdWVyeS9jb25uZWN0b3JzL3Nub3dmbGFrZQojIHNwZWNpZmljcyBoZXJlOiBodHRwczovL2dpdGh1Yi5jb20vbWljcm9zb2Z0L0RhdGFDb25uZWN0b3JzL2Jsb2IvbWFzdGVyL3NhbXBsZXMvT0RCQy9Tbm93Zmxha2VPREJDL1Nub3dmbGFrZU9EQkMucHEKY2xhc3MgU25vd2ZsYWtlRGF0YWJhc2VzKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiU25vd2ZsYWtlLkRhdGFiYXNlcyIsIEV4cHJlc3Npb25UeXBlLkNvbm5lY3RvciwgMikKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImFjY291bnQiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ3YXJlaG91c2UiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImltcGxlbWVudGF0aW9uIiwgRXhwcmVzc2lvblR5cGUuTnVtYmVyLCBUcnVlKQogICAgICAgICAgICAgICBdCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIG5hbWVkUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiUm9sZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiSW1wbGVtZW50YXRpb24iLCBFeHByZXNzaW9uVHlwZS5OdW1iZXIsIFRydWUpCiAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkudG1kbC5jb25uZWN0b3JFeHByZXNzaW9uIGltcG9ydCBEYXRhYmFzZUNvbm5lY3RvckV4cHJlc3Npb24KCiAgICAgICAgYWNjb3VudCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImFjY291bnQiKQogICAgICAgIHdhcmVob3VzZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIndhcmVob3VzZSIpCiAgICAgICAgaW1wbGVtZW50YXRpb24gPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJpbXBsZW1lbnRhdGlvbiIpCiAgICAgICAgcm9sZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInJvbGUiKQogICAgICAgIHByb3BlcnRpZXMgPSB7ImltcGxlbWVudGFpb24iOiBpbXBsZW1lbnRhdGlvbiwgInJvbGUiOiByb2xlICwgIndhcmVob3VzZSI6IHdhcmVob3VzZSwgImFjY291bnQiOiBhY2NvdW50LCAiZGF0YWJhc2VUeXBlIjogInNub3dmbGFrZSJ9CiAgICAgICAgCiAgICAgICAgcmV0dXJuIERhdGFiYXNlQ29ubmVjdG9yRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIHByb3BlcnRpZXM9cHJvcGVydGllcykKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihTbm93Zmxha2VEYXRhYmFzZXMp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/FileContents.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwuY29ubmVjdG9yRXhwcmVzc2lvbiBpbXBvcnQgRmlsZUNvbm5lY3RvckV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS9maWxlLWNvbnRlbnRzCmNsYXNzIEZpbGVDb250ZW50cyhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkZpbGUuQ29udGVudHMiLCBFeHByZXNzaW9uVHlwZS5CaW5hcnkpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJwYXRoIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigib3B0aW9ucyIsIEV4cHJlc3Npb25UeXBlLlJlY29yZCwgVHJ1ZSkKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgcGF0aCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInBhdGgiKQogICAgICAgIGlmIHBhdGguc3RhcnRzd2l0aCgiQzpcXE1hY1xcSG9tZSIpOgogICAgICAgICAgICBwYXRoID0gcGF0aC5yZXBsYWNlKCJDOlxcTWFjXFxIb21lIiwiL3VzZXJzL2Fnb2xkYmVyZyIpLnJlcGxhY2UoIlxcIiwiLyIpCgogICAgICAgIHByb3BlcnRpZXM9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24ucHJvcGVydGllcwogICAgICAgIHByb3BlcnRpZXNbInBhdGgiXSA9IHBhdGgKICAgICAgICBwcm9wZXJ0aWVzWyJvcHRpb25zIl0gPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJvcHRpb25zIikKICAgICAgICAKICAgICAgICByZXR1cm4gRmlsZUNvbm5lY3RvckV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoRmlsZUNvbnRlbnRzKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/DateIsInCurrentYear.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxTdGF0ZW1lbnRFeHByZXNzaW9uIGltcG9ydCBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vZGF0ZS1Jc0luQ3VycmVudFllYXIKCmNsYXNzIERhdGVJc0luQ3VycmVudFllYXIoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJEYXRlLklzSW5DdXJyZW50WWVhciIsIEV4cHJlc3Npb25UeXBlLkxvZ2ljYWwpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbICBGdW5jdGlvblBhcmFtZXRlcigiZGF0ZVRpbWUiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKV0KICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBkYXRlVGltZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImRhdGVUaW1lIikKICAgICAgICAKICAgICAgICByZXR1cm4gU1FMU3RhdGVtZW50RXhwcmVzc2lvbihzdGF0ZW1lbnQ9ZiIoWUVBUigne2RhdGVUaW1lfScpID0gWUVBUihDVVJSRU5UX0RBVEUoKSkpIiwgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLkxvZ2ljYWwpCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoRGF0ZUlzSW5DdXJyZW50WWVhcik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TextFrom.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCmZyb20gYmkuc3FsLlNRTEZ1bmN0aW9uRXhwcmVzc2lvbiBpbXBvcnQgU1FMRnVuY3Rpb25FeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGV4dC1mcm9tCmNsYXNzIFRleHRGcm9tKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGV4dC5Gcm9tIiwgRXhwcmVzc2lvblR5cGUuVGV4dCkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidmFsdWUiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImN1bHR1cmUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKQogICAgICAgICAgICAgICAgIF0KCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGltcG9ydCBsb2NhbGUKICAgICAgICB0ZXh0OiBzdHIgPSBOb25lCiAgICAgICAgdmFsdWUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ2YWx1ZSIpCiAgICAgICAgY3VsdHVyZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImN1bHR1cmUiKQoKICAgICAgICBpZiBjdWx0dXJlIGlzIG5vdCBOb25lOgogICAgICAgICAgICBsb2NhbGUuc2V0bG9jYWxlKGxvY2FsZS5MQ19BTEwsIGN1bHR1cmUpCgogICAgICAgIHRleHQgPSBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IlRPX1ZBUkNIQVIiLCBwYXJhbWV0ZXJzPVt2YWx1ZV0sIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGV4dCkgICAgICAgIAogICAgICAgIAogICAgICAgIGlmIGN1bHR1cmUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGxvY2FsZS5zZXRsb2NhbGUobG9jYWxlLkxDX0FMTCwgJycpIAogICAgICAgIAogICAgICAgIHJldHVybiB0ZXh0CgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGV4dEZyb20pCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TextProper.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RleHQtcHJvcGVyCiMgTWFwcyB0byBTbm93Zmxha2UgSU5JVENBUChzdHJpbmcpCmNsYXNzIFRleHRQcm9wZXIoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJUZXh0LlByb3BlciIsIEV4cHJlc3Npb25UeXBlLlRleHQpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0ZXh0IiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY3VsdHVyZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpLAogICAgICAgIF0KICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB0ZXh0ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGV4dCIpCiAgICAgICAgIyBjdWx0dXJlIGlzIGN1cnJlbnRseSBpZ25vcmVkIGZvciBTbm93Zmxha2UgSU5JVENBUAogICAgICAgIF9jdWx0dXJlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY3VsdHVyZSIpCgogICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24oCiAgICAgICAgICAgIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwKICAgICAgICAgICAgZnVuY3Rpb25OYW1lPSJJTklUQ0FQIiwKICAgICAgICAgICAgcGFyYW1ldGVycz1bdGV4dF0sCiAgICAgICAgICAgIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGV4dCwKICAgICAgICApCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGV4dFByb3BlcikKCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableSelectRows.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFdyYXBwZXJUYWJsZUV4cHJlc3Npb24sIFRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLXNlbGVjdHJvd3MKY2xhc3MgVGFibGVTZWxlY3RSb3dzKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuU2VsZWN0Um93cyIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29uZGl0aW9uIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSkKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB0YWJsZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRhYmxlIikKICAgICAgICBjb25kaXRpb24gPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJjb25kaXRpb24iKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZUV4cHJlc3Npb24pOgogICAgICAgICAgICByZXR1cm4gV3JhcHBlclRhYmxlRXhwcmVzc2lvbih0YWJsZSwgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBjb25kaXRpb25zPWNvbmRpdGlvbikKICAgICAgICBlbHNlOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYidGFibGUgaXMgbm90IGEgVGFibGVFeHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQogICAgCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZVNlbGVjdFJvd3Mp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/BinaryFromText.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL2JpbmFyeS1mcm9tdGV4dApjbGFzcyBCaW5hcnlGcm9tVGV4dChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkJpbmFyeS5Gcm9tVGV4dCIsIEV4cHJlc3Npb25UeXBlLkJpbmFyeSwgMSkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRleHQiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImVuY29kaW5nIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgVHJ1ZSwgIkJpbmFyeUVuY29kaW5nLkJhc2U2NCIpCiAgICAgICAgICAgICAgICAgXQogICAgICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBpbXBvcnQgYmFzZTY0CiAgICAgICAgdGV4dEJ5dGVzID0gTm9uZQogICAgICAgIHRleHQgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0ZXh0IikKICAgICAgICBlbmNvZGluZyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImVuY29kaW5nIikKICAgICAgICBpZiB0ZXh0IGlzIG5vdCBOb25lOiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgaWYgZW5jb2RpbmcgPT0gIkJpbmFyeUVuY29kaW5nLkJhc2U2NCI6CiAgICAgICAgICAgICAgICB0ZXh0Qnl0ZXMgPSBiYXNlNjQuYjY0ZGVjb2RlKHRleHQpICAgICAgICAKICAgICAgICAgICAgZWxpZiBlbmNvZGluZyA9PSAiQmluYXJ5RW5jb2RpbmcuSGV4IjoKICAgICAgICAgICAgICAgIHRleHRCeXRlcyA9IGJ5dGVzLmZyb21oZXgodGV4dCkKICAgICAgICByZXR1cm4gdGV4dEJ5dGVzCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoQmluYXJ5RnJvbVRleHQp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/ExcelWorkbook.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwuY29ubmVjdG9yRXhwcmVzc2lvbiBpbXBvcnQgQ29ubmVjdG9yRXhwcmVzc2lvbiwgQ2FjaGVkQ29ubmVjdG9yRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBXb3JrYm9va1RhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuaXNzdWUgaW1wb3J0IElzc3VlLCBJc3N1ZVR5cGUKCmNsYXNzIEV4Y2VsV29ya2Jvb2soRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJFeGNlbC5Xb3JrYm9vayIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCAxKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigid29ya2Jvb2siLCBFeHByZXNzaW9uVHlwZS5CaW5hcnksIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInVzZUhlYWRlcnMiLCBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsLCBUcnVlLCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImRlbGF5VHlwZXMiLCBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsLCBUcnVlLCBGYWxzZSkKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIG5hbWVkUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiVXNlSGVhZGVycyIsIEV4cHJlc3Npb25UeXBlLkxvZ2ljYWwsIFRydWUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIkRlbGF5VHlwZXMiLCBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsLCBUcnVlLCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIkluZmVyU2hlZXREaW1lbnNpb25zIiwgRXhwcmVzc2lvblR5cGUuTG9naWNhbCwgVHJ1ZSwgRmFsc2UpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGltcG9ydCBwYW5kYXMgYXMgcGQKICAgICAgICBpbXBvcnQgaW8KICAgICAgICB3b3JrYm9vayA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIndvcmtib29rIikKICAgICAgICBjb25uZWN0b3IgPSBOb25lCiAgICAgICAgZGF0YSA9IE5vbmUKCiAgICAgICAgaWYgaXNpbnN0YW5jZSh3b3JrYm9vaywgQ29ubmVjdG9yRXhwcmVzc2lvbik6CiAgICAgICAgICAgIGNvbm5lY3RvciA9IHdvcmtib29rCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHdvcmtib29rLCBieXRlcyk6CiAgICAgICAgICAgIGNvbm5lY3RvciA9IENhY2hlZENvbm5lY3RvckV4cHJlc3Npb24oZGF0YT13b3JrYm9vaywgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoInVua25vd24gd29ya2Jvb2sgdHlwZSIpCiAgICAgICAgCiAgICAgICAgdXNlSGVhZGVycyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInVzZUhlYWRlcnMiKQogICAgICAgIERlbGF5VHlwZXMgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJEZWxheVR5cGVzIikKICAgICAgICBJbmZlclNoZWV0RGltZW5zaW9ucyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIkluZmVyU2hlZXREaW1lbnNpb25zIikKICAgICAgICBwcm9wZXJ0aWVzID0geyJvcmlnaW5hbCI6IHNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnNvdXJjZVN0cmluZywgInVzZUhlYWRlcnMiOiB1c2VIZWFkZXJzLCAiRGVsYXlUeXBlcyI6IERlbGF5VHlwZXMsICJJbmZlclNoZWV0RGltZW5pb25zIjogSW5mZXJTaGVldERpbWVuc2lvbnMgfQogICAgICAgIHJldHVybiBXb3JrYm9va1RhYmxlRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGNvbm5lY3Rvcj1jb25uZWN0b3IsIHByb3BlcnRpZXM9cHJvcGVydGllcykgICAgCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoRXhjZWxXb3JrYm9vayk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/JsonDocument.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS9qc29uLWRvY3VtZW50CmNsYXNzIEpzb25Eb2N1bWVudChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkpzb24uRG9jdW1lbnQiLCBFeHByZXNzaW9uVHlwZS5BbnksIDEpICMgcmV0dXJucyBBIHJlY29yZCwgbGlzdCwgb3IgcHJpbWl0aXZlIHZhbHVlIHJlcHJlc2VudGluZyB0aGUgSlNPTiBkYXRhIGNvbnRhaW5lZCBpbiB0aGUganNvblRleHQKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImpzb25UZXh0IiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJlbmNvZGluZyIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUsICJUZXh0RW5jb2RpbmcuVXRmOCIpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgaW1wb3J0IGpzb24KICAgICAgICBqc29uQ29udGVudCA9IE5vbmUKICAgICAgICBqc29uVGV4dCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImpzb25UZXh0IikKICAgICAgICBpZiBpc2luc3RhbmNlKGpzb25UZXh0LCBFeHByZXNzaW9uKToKICAgICAgICAgICAganNvblRleHQgPSBqc29uVGV4dC52YWx1ZQogICAgICAgICAgICAKICAgICAgICBlbmNvZGluZyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImVuY29kaW5nIikKICAgICAgICBqc29uQ29udGVudCA9IGpzb24ubG9hZHMoanNvblRleHQpCiAgICAgICAgcmV0dXJuIGpzb25Db250ZW50CgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoSnNvbkRvY3VtZW50KQogICAgICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TablePromoteHeaders.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IENhY2hlZFRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1wcm9tb3RlaGVhZGVycwojIFByb21vdGVzIHRoZSBmaXJzdCByb3cgb2YgdmFsdWVzIGFzIHRoZSBuZXcgY29sdW1uIGhlYWRlcnMgKGkuZS4gY29sdW1uIG5hbWVzKS4gQnkgZGVmYXVsdCwgb25seSB0ZXh0IG9yIG51bWJlciB2YWx1ZXMgYXJlIHByb21vdGVkIHRvIGhlYWRlcnMuIFZhbGlkIG9wdGlvbnM6CiMgUHJvbW90ZUFsbFNjYWxhcnM6IElmIHNldCB0byB0cnVlLCBhbGwgdGhlIHNjYWxhciB2YWx1ZXMgaW4gdGhlIGZpcnN0IHJvdyBhcmUgcHJvbW90ZWQgdG8gaGVhZGVycyB1c2luZyB0aGUgQ3VsdHVyZSwgaWYgc3BlY2lmaWVkIChvciBjdXJyZW50IGRvY3VtZW50IGxvY2FsZSkuIEZvciB2YWx1ZXMgdGhhdCBjYW5ub3QgYmUgY29udmVydGVkIHRvIHRleHQsIGEgZGVmYXVsdCBjb2x1bW4gbmFtZSB3aWxsIGJlIHVzZWQuCiMgQ3VsdHVyZTogQSBjdWx0dXJlIG5hbWUgc3BlY2lmeWluZyB0aGUgY3VsdHVyZSBmb3IgdGhlIGRhdGEuCgpjbGFzcyBUYWJsZVByb21vdGVIZWFkZXJzKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuUHJvbW90ZUhlYWRlcnMiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgMSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm9wdGlvbnMiLCBFeHByZXNzaW9uVHlwZS5SZWNvcmQsIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lZFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoIlByb21vdGVBbGxTY2FsYXJzIiwgRXhwcmVzc2lvblR5cGUuTG9naWNhbCwgRmFsc2UsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIkN1bHR1cmUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGltcG9ydCBkYXRldGltZQogICAgICAgIGltcG9ydCBsb2NhbGUKCiAgICAgICAgdGFibGU6IFRhYmxlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUiLCBnZXRJbm5lclZhbHVlPUZhbHNlKQogICAgICAgIHByb21vdGVBbGxTY2FsYXJzOiBib29sID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiUHJvbW90ZUFsbFNjYWxhcnMiKQogICAgICAgIGN1bHR1cmU6IHN0ciA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImN1bHR1cmUiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZSk6CiAgICAgICAgICAgIGlmIHRhYmxlLmRhdGFJc0F2YWlsYWJsZSBhbmQgbGVuKHRhYmxlLnJvd3MpID4gMDoKICAgICAgICAgICAgICAgIGlmIGN1bHR1cmUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgbG9jYWxlLnNldGxvY2FsZShsb2NhbGUuTENfQUxMLCBjdWx0dXJlKSAKICAgICAgICAgICAgCiAgICAgICAgICAgICAgICBkYXRhID0gdGFibGUucm93cy5jb3B5KCkKICAgICAgICAgICAgICAgIGZpcnN0Um93ID0gZGF0YS5wb3AoMCkKICAgICAgICAgICAgICAgIG5ld0NvbHMgPSBbXQoKICAgICAgICAgICAgICAgICMgZW51bWVyYXRlIHRoZSBmaXJzdCByb3cgdG8gY3JlYXRlIHRoZSBuZXcgY29sdW1uIG5hbWVzCiAgICAgICAgICAgICAgICBmb3IgaSwgdiBpbiBlbnVtZXJhdGUoZmlyc3RSb3cpOgogICAgICAgICAgICAgICAgICAgIGNvbCA9IHRhYmxlLmNvbHVtbnNbaV0KICAgICAgICAgICAgICAgICAgICBpZiB2IGlzIG5vdCBOb25lIGFuZCAoKHByb21vdGVBbGxTY2FsYXJzIGFuZCBjb2wuaXNTY2FsYXJWYWx1ZSkgb3IgKGNvbC50eXBlIGluIFtFeHByZXNzaW9uVHlwZS5OdW1iZXIsIEV4cHJlc3Npb25UeXBlLlRleHRdKSk6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGNvbC50eXBlID09IEV4cHJlc3Npb25UeXBlLkRhdGU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBuZXdOYW1lID0gZGF0ZXRpbWUuZGF0ZXRpbWUuc3RyZnRpbWUodiwgIiV4IikKICAgICAgICAgICAgICAgICAgICAgICAgZWxpZiBjb2wudHlwZSA9PSBFeHByZXNzaW9uVHlwZS5EYXRlVGltZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIG5ld05hbWUgPSBkYXRldGltZS5kYXRldGltZS5zdHJmdGltZSh2LCAiJWMiKQogICAgICAgICAgICAgICAgICAgICAgICBlbGlmIHYgaXMgbm90IE5vbmUgYW5kIGxlbihzdHIodikpID4gMDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgIG5ld05hbWUgPSBzdHIodikKICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIG5ld05hbWUgPSBjb2wuZ2V0SWRlbnRpZmllcihGYWxzZSkKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICBuZXdOYW1lID0gY29sLmdldElkZW50aWZpZXIoRmFsc2UpCiAgICAgICAgICAgICAgICAgICAgZXhwcmVzc2lvblN0ciA9IE5vbmUKICAgICAgICAgICAgICAgICAgICBpZiBjb2wuZXhwcmVzc2lvbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgZXhwcmVzc2lvblN0ciA9IGNvbC5leHByZXNzaW9uLnNvdXJjZVN0cmluZwogICAgICAgICAgICAgICAgICAgIG5ld0NvbHMuYXBwZW5kKENvbHVtbihuYW1lPW5ld05hbWUsIHR5cGU9Y29sLnR5cGUsIGV4cHJlc3Npb25Db2RlVHlwZT1jb2wuY2hpbGRFeHByZXNzaW9uQ29kZVR5cGUsIGV4cHJlc3Npb249ZXhwcmVzc2lvblN0ciwgaXNQcmltYXJ5S2V5PWNvbC5pc1ByaW1hcnlLZXksIGlzVW5pcXVlS2V5PWNvbC5pc1VuaXF1ZUtleSwgaXNOdWxsYWJsZT1jb2wuaXNOdWxsYWJsZSwgZm9ybWF0U3RyaW5nPWNvbC5mb3JtYXRTdHJpbmcsIHByb3BlcnRpZXM9Y29sLnByb3BlcnRpZXMpKQogICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgIGlmIGN1bHR1cmUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICAgICAgbG9jYWxlLnNldGxvY2FsZShsb2NhbGUuTENfQUxMLCAnJykgCiAgICAgICAgICAgICAgICByZXR1cm4gQ2FjaGVkVGFibGVFeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgY29sdW1ucz1uZXdDb2xzLCBkYXRhPWRhdGEpCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAjVE9ETyAtIHNob3VsZCBwcm9iYWJseSBjcmVhdGUgYSBxdWVyeSB3aGljaCBkb2VzIHRoZSBwcm9tb3RlIGhlYWRlciBsb2dpYwogICAgICAgICAgICAgICAgIyBzb21ldGhpbmcgbGlrZTogU0VMRUNUICogRlJPTSAoU0VMRUNUICosIFJvd051bWJlcigpIE9WRVIgKE9SREVSIEJZIE5VTEwpIEZST00gKHRhYmxlKSBXSEVSRSBSb3dOdW1iZXIgPiAxKSAtLSBidXQgbmVlZHRvIGZpZ3VyZSBvdXQgaG93IHRvIGFsaWFzIHRoZSBjb2x1bW5zCiAgICAgICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJDYW4ndCBwcm9tb3RlIGhlYWRlcnMgZm9yIHRhYmxlIHdpdGggbm8gcm93cyBvZiBkYXRhIikKICAgICAgICBlbHNlOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJFeHBlY3RpbmcgYSB0YWJsZSBleHByZXNzaW9uIikKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRhYmxlUHJvbW90ZUhlYWRlcnMpCgoKICAgICAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableFirstN.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uLCBMaXN0RXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFdyYXBwZXJUYWJsZUV4cHJlc3Npb24sIFRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLWZpcnN0bgpjbGFzcyBUYWJsZUZpcnN0TihGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLkZpcnN0TiIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY291bnRPckNvbmRpdGlvbiIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgY291bnRPckNvbmRpdGlvbiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvdW50T3JDb25kaXRpb24iKQogICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIFRhYmxlRXhwcmVzc2lvbik6CiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoY291bnRPckNvbmRpdGlvbiwgaW50KToKICAgICAgICAgICAgICAgIHJldHVybiBXcmFwcGVyVGFibGVFeHByZXNzaW9uKHRhYmxlLCBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIHRvcE49Y291bnRPckNvbmRpdGlvbikKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIlRhYmxlLkZpcnN0TiB3aXRoIGNvbmRpdGlvbiBpcyBub3QgY3VycmVudGx5IHN1cHBvcnRlZCIpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmInRhYmxlIGlzIG5vdCBhIFRhYmxlRXhwcmVzc2lvbjoge3N0cih0YWJsZSl9IikKICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGFibGVGaXJzdE4p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableNestedJoin.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBXcmFwcGVyVGFibGVFeHByZXNzaW9uLCBKb2luS2luZCwgSm9pblRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxTZWxlY3RFeHByZXNzaW9uIGltcG9ydCBTUUxTZWxlY3RFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTFN0YXRlbWVudEV4cHJlc3Npb24gaW1wb3J0IFNRTFN0YXRlbWVudEV4cHJlc3Npb24KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1uZXN0ZWRqb2luCmNsYXNzIFRhYmxlTmVzdGVkSm9pbihGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLk5lc3RlZEpvaW4iLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlMSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJrZXkxIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlMiIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJrZXkyIiwgRXhwcmVzc2lvblR5cGUuRnVuY3Rpb24sIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigibmV3Q29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiam9pbktpbmQiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlLCAiSm9pbktpbmQuTGVmdE91dGVyIiksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImtleUVxdWFsaXR5Q29tcGFyZXJzIiwgRXhwcmVzc2lvblR5cGUuTGlzdCwgRmFsc2UpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuY29sdW1uIGltcG9ydCBDb2x1bW4KICAgICAgICBmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCiAgICAgICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25Db2RlVHlwZSBpbXBvcnQgRXhwcmVzc2lvbkNvZGVUeXBlCiAgICAgICAgCiAgICAgICAgdGFibGUxID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUxIikKICAgICAgICBrZXkxID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgia2V5MSIpCiAgICAgICAgdGFibGUyID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUyIikKICAgICAgICBrZXkyID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgia2V5MiIpCiAgICAgICAgbmV3Q29sdW1uTmFtZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm5ld0NvbHVtbk5hbWUiKQogICAgICAgIGpvaW5LaW5kID0gSm9pbktpbmQuRnJvbVN0cmluZyhzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJqb2luS2luZCIpKQogICAgICAgIGtleUVxdWFsaXR5Q29tcGFyZXJzID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgia2V5RXF1YWxpdHlDb21wYXJlcnMiKQogICAgICAgIAogICAgICAgIGlmIG5vdCBpc2luc3RhbmNlKHRhYmxlMSwgVGFibGUpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJleHBlY3RpbmcgdGFibGUgZXhwcmVzc2lvbiIpCiAgICAgICAgCiAgICAgICAgaWYgbm90IGlzaW5zdGFuY2UodGFibGUyLCBUYWJsZSk6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoImV4cGVjdGluZyB0YWJsZSBleHByZXNzaW9uIikKICAgICAgICAKICAgICAgICBrZXkxY29sczogTGlzdFtDb2x1bW5dID0gW10KICAgICAgICBmb3IgYyBpbiBKb2luVGFibGVFeHByZXNzaW9uLmdldEtleUNvbHVtbnMoa2V5MSk6CiAgICAgICAgICAgIGNvbCA9IHRhYmxlMS5nZXRDb2x1bW4oYykKICAgICAgICAgICAga2V5MWNvbHMuYXBwZW5kKGNvbCkKCiAgICAgICAga2V5MmNvbHM6IExpc3RbQ29sdW1uXSA9IFtdCiAgICAgICAgZm9yIGMgaW4gSm9pblRhYmxlRXhwcmVzc2lvbi5nZXRLZXlDb2x1bW5zKGtleTIpOgogICAgICAgICAgICBjb2wgPSB0YWJsZTIuZ2V0Q29sdW1uKGMpCiAgICAgICAgICAgIGtleTJjb2xzLmFwcGVuZChjb2wpCgogICAgICAgIHByaW1hcnlUYWJsZSA9IHRhYmxlMQogICAgICAgIHNlY29uZGFyeVRhYmxlID0gdGFibGUyCiAgICAgICAgcHJpbWFyeUtleXMgPSBrZXkxY29scwogICAgICAgIHNlY29uZGFyeUtleXMgPSBrZXkyY29scwogICAgICAgIHByaW1hcnlUYWJsZUFsaWFzID0gIlQxIgogICAgICAgIHNlY29uZGFyeVRhYmxlQWxpYXMgPSAiVDIiCgogICAgICAgIGlmIGpvaW5LaW5kLmlzUmlnaHQ6CiAgICAgICAgICAgIHByaW1hcnlUYWJsZSA9IHRhYmxlMgogICAgICAgICAgICBzZWNvbmRhcnlUYWJsZSA9IHRhYmxlMQogICAgICAgICAgICBwcmltYXJ5S2V5cyA9IGtleTJjb2xzCiAgICAgICAgICAgIHNlY29uZGFyeUtleXMgPSBrZXkxY29scwogICAgICAgICAgICBwcmltYXJ5VGFibGVBbGlhcyA9ICJUMiIKICAgICAgICAgICAgc2Vjb25kYXJ5VGFibGVBbGlhcyA9ICJUMSIKCiAgICAgICAgaWYgbGVuKGtleTFjb2xzKSAhPSBsZW4oa2V5MmNvbHMpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYia2V5MSBhbmQga2V5MiBtdXN0IGNvbnRhaW4gdGhlIHNhbWUgbnVtYmVyIG9mIGVsZW1lbnRzIikKCiAgICAgICAga2V5RXF1aXR5Q29tcGFyZXJzID0gSm9pblRhYmxlRXhwcmVzc2lvbi5nZXRLZXlDb2x1bW5zKGtleUVxdWFsaXR5Q29tcGFyZXJzKQoKICAgICAgICBpZiBsZW4oa2V5RXF1aXR5Q29tcGFyZXJzKSA9PSAwOgogICAgICAgICAgICBrZXlFcXVpdHlDb21wYXJlcnMgPSBbIj0iXSAqIGxlbihrZXkxY29scykKCiAgICAgICAgZWxpZiBsZW4oa2V5RXF1aXR5Q29tcGFyZXJzKSAhPSBsZW4oa2V5MWNvbHMpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYia2V5RXF1aXR5Q29tcGFyZXJzIG11c3QgY29udGFpbiB0aGUgc2FtZSBudW1iZXIgb2YgZWxlbWVudHMgYXMga2V5MSBhbmQga2V5MiIpCiAgICAgICAgCiAgICAgICAgY29uc3RyYWludHMgPSBbXQogICAgICAgIGZvciBpLCB2IGluIGVudW1lcmF0ZShrZXkxY29scyk6CiAgICAgICAgICAgIGNvbXBhcmVyID0ga2V5RXF1aXR5Q29tcGFyZXJzW2ldCiAgICAgICAgICAgIHByaW1hcnlDb2wgPSBwcmltYXJ5S2V5c1tpXQogICAgICAgICAgICBzZWNvbmRhcnlDb2wgPSBzZWNvbmRhcnlLZXlzW2ldCiAgICAgICAgICAgIGNvbnN0cmFpbnRzLmFwcGVuZChTUUxTdGF0ZW1lbnRFeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgc3RhdGVtZW50PWYie3ByaW1hcnlUYWJsZUFsaWFzfS57cHJpbWFyeUNvbC5uYW1lfSB7Y29tcGFyZXJ9IHtzZWNvbmRhcnlUYWJsZUFsaWFzfS57c2Vjb25kYXJ5Q29sLm5hbWV9IikpCiAgICAgICAgICAgIAogICAgICAgIGNvbEV4cHJlc3Npb24gPSBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9Ik9CSkVDVF9DT05TVFJVQ1QiLCBwYXJhbWV0ZXJzPVsiKiJdKQogICAgICAgIGNvbCA9IENvbHVtbihuYW1lPW5ld0NvbHVtbk5hbWUsIGV4cHJlc3Npb249Y29sRXhwcmVzc2lvbikKICAgICAgICBuZXN0ZWRKb2luQ29sdW1uQ29tbWFuZCA9IFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmcm09c2Vjb25kYXJ5VGFibGUsIGNvbHVtbnM9W2NvbF0sIGNvbnN0cmFpbnRzPWNvbnN0cmFpbnRzKQoKICAgICAgICBqb2luVGFibGUgPSBXcmFwcGVyVGFibGVFeHByZXNzaW9uKHByaW1hcnlUYWJsZSwgc2VsZi5mdW5jdGlvbkV4cHJlc3Npb24pCiAgICAgICAgam9pblRhYmxlLmFkZENvbHVtbihleHByZXNzaW9uPW5lc3RlZEpvaW5Db2x1bW5Db21tYW5kLCBuYW1lPW5ld0NvbHVtbk5hbWUsIHR5cGU9RXhwcmVzc2lvblR5cGUuVGFibGUpCiAgICAgICAgCiAgICAgICAgcmV0dXJuIGpvaW5UYWJsZQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRhYmxlTmVzdGVkSm9pbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableRemoveColumns.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gdHlwaW5nIGltcG9ydCBMaXN0LCBBbnkKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLnRhYmxlRXhwcmVzc2lvbiBpbXBvcnQgV3JhcHBlclRhYmxlRXhwcmVzc2lvbiwgVGFibGVFeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGFibGUtcmVtb3ZlY29sdW1ucwpjbGFzcyBUYWJsZVJlbW92ZUNvbHVtbnMoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJUYWJsZS5SZW1vdmVDb2x1bW5zIiwgRXhwcmVzc2lvblR5cGUuVGFibGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJjb2x1bW5zIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm1pc3NpbmdGaWVsZCIsIEV4cHJlc3Npb25UeXBlLk51bWJlciwgVHJ1ZSkKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB0YWJsZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRhYmxlIikKICAgICAgICBjb2xzX3RvX3JlbW92ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbnMiKQogICAgICAgIG1pc3NpbmdGaWVsZCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm1pc3NpbmdGaWVsZCIpCiAgICAgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGVFeHByZXNzaW9uKToKICAgICAgICAgICAgY29sdW1ucyA9IFtdCiAgICAgICAgICAgIGZvciBjIGluIHRhYmxlLmNvbHVtbnM6CiAgICAgICAgICAgICAgICBpZiBjIG5vdCBpbiBjb2xzX3RvX3JlbW92ZToKICAgICAgICAgICAgICAgICAgICBjb2x1bW5zLmFwcGVuZChjKQoKICAgICAgICAgICAgd3JhcHBlZFRhYmxlID0gIFdyYXBwZXJUYWJsZUV4cHJlc3Npb24od3JhcHBlZFRhYmxlPXRhYmxlLCBjb2x1bW5zPWNvbHVtbnMsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcHJvcGVydGllcz1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbi5wcm9wZXJ0aWVzKQogICAgICAgICAgICByZXR1cm4gd3JhcHBlZFRhYmxlCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIkV4cGVjdGluZyB0YWJsZSBleHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQogICAgICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGFibGVSZW1vdmVDb2x1bW5zKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableSelectColumns.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uLCBMaXN0RXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFdyYXBwZXJUYWJsZUV4cHJlc3Npb24sIFRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLXNlbGVjdGNvbHVtbnMKY2xhc3MgVGFibGVTZWxlY3RDb2x1bW5zKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuU2VsZWN0Q29sdW1ucyIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1ucyIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJtaXNzaW5nRmllbGQiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKQogICAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHRhYmxlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUiKQogICAgICAgIGNvbHVtbnMgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJjb2x1bW5zIikKICAgICAgICBtaXNzaW5nRmllbGQgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJtaXNzaW5nRmllbGQiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZUV4cHJlc3Npb24pOgogICAgICAgICAgICBjb2xzID0gW10KICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb2x1bW5zLCBMaXN0RXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICBjb2x1bW5zID0gY29sdW1ucy5nZXRWYWx1ZSgpCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBpc2luc3RhbmNlKGNvbHVtbnMsIGxpc3QpOgogICAgICAgICAgICAgICAgZm9yIGMgaW4gY29sdW1uczoKICAgICAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKGMsIE1FeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgYyA9IGMudmFsdWUKICAgICAgICAgICAgICAgICAgICBjb2xzLmFwcGVuZChDb2x1bW4obmFtZT1jKSkKICAgICAgICAgICAgICAgIHJldHVybiBXcmFwcGVyVGFibGVFeHByZXNzaW9uKHRhYmxlLCBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGNvbHVtbnM9Y29scykKICAgICAgICBlbHNlOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYidGFibGUgaXMgbm90IGEgVGFibGVFeHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQogICAgCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZVNlbGVjdENvbHVtbnMp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableRenameColumns.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBRdWVyeVRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IExpc3RFeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCmZyb20gYmkuY29yZS50YWJsZSBpbXBvcnQgVGFibGUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1yZW5hbWVjb2x1bW5zCmNsYXNzIFRhYmxlUmVuYW1lQ29sdW1ucyhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLlJlbmFtZUNvbHVtbnMiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInJlbmFtZXMiLCBFeHByZXNzaW9uVHlwZS5MaXN0LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm1pc3NpbmdGaWVsZCIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgcmVuYW1lcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInJlbmFtZXMiKQogICAgICAgIG1pc3NpbmdGaWVsZCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm1pc3NpbmdGaWVsZCIpCiAgICAgICAgZXhjbHVkZSA9IFtdCiAgICAgICAgaW5jbHVkZSA9IFtdCgogICAgICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIFRhYmxlKToKICAgICAgICAgICAgZm9yIHJlbmFtZSBpbiByZW5hbWVzOgogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShyZW5hbWUsIExpc3RFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICBmcm9tQ29sID0gcmVuYW1lLnZhbHVlWzBdLnZhbHVlCiAgICAgICAgICAgICAgICAgICAgaWYgZnJvbUNvbCAhPSBmcm9tQ29sLnVwcGVyKCk6CiAgICAgICAgICAgICAgICAgICAgICAgIGZyb21Db2wgPSBmIlwie2Zyb21Db2x9XCIiCiAgICAgICAgICAgICAgICAgICAgdG9Db2wgPSByZW5hbWUudmFsdWVbMV0udmFsdWUKICAgICAgICAgICAgICAgICAgICBpZiB0b0NvbCAhPSB0b0NvbC51cHBlcigpOgogICAgICAgICAgICAgICAgICAgICAgICB0b0NvbCA9IGYiXCJ7dG9Db2x9XCIiCiAgICAgICAgICAgICAgICAgICAgZXhjbHVkZS5hcHBlbmQoZnJvbUNvbCkKICAgICAgICAgICAgICAgICAgICBpbmNsdWRlLmFwcGVuZChmIntmcm9tQ29sfSBBUyB7dG9Db2x9IikKICAgICAgICAgICAgZXhjbHVkZVN0ciA9ICIsXG5cdCIuam9pbihleGNsdWRlKQogICAgICAgICAgICBleGNsdWRlU3RyID0gZiJFWENMVURFICh7ZXhjbHVkZVN0cn0pIgogICAgICAgICAgICBpbmNsdWRlU3RyID0gIlxuXHQsIi5qb2luKGluY2x1ZGUpCiAgICAgICAgICAgIHNxbCA9IGYiU0VMRUNUIFxuXHQqXG5cdHtleGNsdWRlU3RyfSxcblx0e2luY2x1ZGVTdHJ9XG5GUk9NXG5cdCh7dGFibGUuZ2V0U3FsVmFsdWUoKX0pIgogICAgICAgICAgICB3cmFwcGVkVGFibGUgPSBRdWVyeVRhYmxlRXhwcmVzc2lvbigKICAgICAgICAgICAgICAgIHF1ZXJ5PXNxbCwKICAgICAgICAgICAgICAgIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwKICAgICAgICAgICAgICAgIHByb3BlcnRpZXM9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24ucHJvcGVydGllcywKICAgICAgICAgICAgICAgIGJhc2VUYWJsZT10YWJsZQogICAgICAgICAgICApCiAgICAgICAgICAgIHJldHVybiB3cmFwcGVkVGFibGUKICAgICAgICBlbHNlOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYidGFibGUgaXMgbm90IGEgVGFibGVFeHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRhYmxlUmVuYW1lQ29sdW1ucykKICAgICAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TextContains.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTEZ1bmN0aW9uRXhwcmVzc2lvbiBpbXBvcnQgU1FMRnVuY3Rpb25FeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGV4dC1jb250YWlucwpjbGFzcyBUZXh0Q29udGFpbnMoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJUZXh0LkNvbnRhaW5zIiwgRXhwcmVzc2lvblR5cGUuTG9naWNhbCkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRleHQiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInN1YnN0cmluZyIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29tcGFyZXIiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKSwKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB0ZXh0ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGV4dCIpCiAgICAgICAgc3Vic3RyaW5nID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgic3Vic3RyaW5nIiwgZ2V0SW5uZXJWYWx1ZT1GYWxzZSkKICAgICAgICBjb21wYXJlciA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbXBhcmVyIikKICAgICAgICAKICAgICAgICByZXR1cm4gU1FMRnVuY3Rpb25FeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgZnVuY3Rpb25OYW1lPSJDT05UQUlOUyIsIHBhcmFtZXRlcnM9W3RleHQsIHN1YnN0cmluZ10pCiAgICAgICAgCiAgICAgICAgCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUZXh0Q29udGFpbnMp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableTransformColumnTypes.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24sIFR5cGVFeHByZXNzaW9uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFRhYmxlRXhwcmVzc2lvbiwgV3JhcHBlclRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkuY29yZS50YWJsZSBpbXBvcnQgVGFibGUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1hZGRpbmRleGNvbHVtbgpjbGFzcyBUYWJsZVRyYW5zZm9ybUNvbHVtblR5cGVzKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuVHJhbnNmb3JtQ29sdW1uVHlwZXMiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInR5cGVUcmFuc2Zvcm1hdGlvbnMiLCBFeHByZXNzaW9uVHlwZS5MaXN0LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImN1bHR1cmUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlKQogICAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHRhYmxlOiBUYWJsZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRhYmxlIikKICAgICAgICB0eXBlVHJhbnNmb3JtYXRpb25zOiBMaXN0W2xpc3RdID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidHlwZVRyYW5zZm9ybWF0aW9ucyIpCiAgICAgICAgY3VsdHVyZTogc3RyID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY3VsdHVyZSIpCgogICAgICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIFRhYmxlKToKICAgICAgICAgICAgd3JhcHBlZFRhYmxlID0gV3JhcHBlclRhYmxlRXhwcmVzc2lvbih3cmFwcGVkVGFibGU9dGFibGUsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbikKCiAgICAgICAgICAgIGZvciB0eXBlVHJhbnNmb3JtIGluIHR5cGVUcmFuc2Zvcm1hdGlvbnM6CiAgICAgICAgICAgICAgICBjb2xOYW1lID0gdHlwZVRyYW5zZm9ybVswXQogICAgICAgICAgICAgICAgY29sVHlwZSA9IHR5cGVUcmFuc2Zvcm1bMV0KICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoY29sVHlwZSwgVHlwZUV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgIGNvbFR5cGUgPSBjb2xUeXBlLnJldHVyblR5cGUKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoY29sTmFtZSwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgY29sTmFtZSA9IGNvbE5hbWUudmFsdWUKICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb2xOYW1lLCBDb2x1bW4pOgogICAgICAgICAgICAgICAgICAgIGNvbE5hbWUgPSBjb2xOYW1lLmdldElkZW50aWZpZXIoKQoKICAgICAgICAgICAgICAgIHdyYXBwZWRUYWJsZS5jaGFuZ2VDb2x1bW5UeXBlKGNvbE5hbWUsIGNvbFR5cGUpCiAgICAgICAgICAgICAgICAKICAgICAgICAgICAgcmV0dXJuIHdyYXBwZWRUYWJsZQogICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoInRhYmxlIHBhcmFtZXRlciBpcyBub3QgYSBUYWJsZSBFeHByZXNzaW9uIikKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZVRyYW5zZm9ybUNvbHVtblR5cGVzKQogICAgICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableAddIndexColumn.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFdyYXBwZXJUYWJsZUV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIGJpLnNxbC5TUUxTdGF0ZW1lbnRFeHByZXNzaW9uIGltcG9ydCBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGFibGUtYWRkaW5kZXhjb2x1bW4KY2xhc3MgVGFibGVBZGRJbmRleENvbHVtbihGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLkFkZEluZGV4Q29sdW1uIiwgRXhwcmVzc2lvblR5cGUuVGFibGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJuZXdDb2x1bW5OYW1lIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJpbml0aWFsVmFsdWUiLCBFeHByZXNzaW9uVHlwZS5OdW1iZXIsIFRydWUsIDApLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJpbmNyZW1lbnQiLCBFeHByZXNzaW9uVHlwZS5OdW1iZXIsIFRydWUsIDEpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgbmV3Q29sdW1uTmFtZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm5ld0NvbHVtbk5hbWUiKQogICAgICAgIGluaXRpYWxWYWx1ZTogaW50ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiaW5pdGlhbFZhbHVlIikKICAgICAgICBpbmNyZW1lbnQ6IGludCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImluY3JlbWVudCIpCgogICAgICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIFRhYmxlKToKICAgICAgICAgICAgd3JhcHBlclRhYmxlID0gV3JhcHBlclRhYmxlRXhwcmVzc2lvbih3cmFwcGVkVGFibGU9dGFibGUsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbikKICAgICAgICAgICAgZXhwID0gU1FMU3RhdGVtZW50RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIHN0YXRlbWVudD1mIihST1dfTlVNQkVSKCkgT1ZFUiAoT1JERVIgQlkgTlVMTCkgKyB7aW5pdGlhbFZhbHVlfSkgKiB7aW5jcmVtZW50fSIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuTnVtYmVyKQogICAgICAgICAgICB3cmFwcGVyVGFibGUuYWRkQ29sdW1uKG5hbWU9bmV3Q29sdW1uTmFtZSwgdHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIsIGV4cHJlc3Npb249ZXhwKQogICAgICAgIAogICAgICAgICAgICByZXR1cm4gd3JhcHBlclRhYmxlCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiRXhwZWN0ZWQgdGFibGUgdmFsdWUiKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRhYmxlQWRkSW5kZXhDb2x1bW4p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/CsvDocument.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC5jb25uZWN0b3JFeHByZXNzaW9uIGltcG9ydCBGaWxlQ29ubmVjdG9yRXhwcmVzc2lvbiwgQ29ubmVjdG9yRXhwcmVzc2lvbiwgQ2FjaGVkQ29ubmVjdG9yRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuaXNzdWUgaW1wb3J0IElzc3VlLCBJc3N1ZVR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLnRhYmxlRXhwcmVzc2lvbiBpbXBvcnQgQ2FjaGVkVGFibGVFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vY3N2LWRvY3VtZW50CmNsYXNzIENzdkRvY3VtZW50KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiQ3N2LkRvY3VtZW50IiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIDEpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJzb3VyY2UiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImNvbHVtbnMiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJkZWxpbWl0ZXIiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUsICIsIiksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImV4dHJhVmFsdWVzIiwgRXhwcmVzc2lvblR5cGUuTnVtYmVyLCBUcnVlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZW5jb2RpbmciLCBFeHByZXNzaW9uVHlwZS5OdW1iZXIsIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lZFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoIkRlbGltaXRlciIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUsICIsIiksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJDb2x1bW5zIiwgRXhwcmVzc2lvblR5cGUuQW55LCBUcnVlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiRW5jb2RpbmciLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlLCAiVGV4dEVuY29kaW5nLlV0ZjgiKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiQ3N2U3R5bGUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlLCAiQ3N2U3R5bGUuUXVvdGVBZnRlckRlbGltaXRlciIpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJRdW90ZVN0eWxlIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgVHJ1ZSwgIlF1b3RlU3R5bGUuQ3N2IiksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIkluY2x1ZGVCeXRlT3JkZXJNYXJrIiwgRXhwcmVzc2lvblR5cGUuTG9naWNhbCwgVHJ1ZSwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJFeHRyYVZhbHVlcyIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGltcG9ydCBwYW5kYXMgYXMgcGQKICAgICAgICBpbXBvcnQgaW8KICAgICAgICBjc3ZUYWJsZTogcGQuRGF0YUZyYW1lID0gTm9uZQogICAgICAgIGNvbE5hbWVzID0gTm9uZQogICAgICAgIGNvbFR5cGVzID0gTm9uZQogICAgICAgIGR0eXBlcyA9IHt9CiAgICAgICAgc291cmNlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgic291cmNlIikKICAgICAgICBjb2x1bW5zID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY29sdW1ucyIpCiAgICAgICAgY29ubmVjdG9yID0gTm9uZQoKICAgICAgICBpZiBpc2luc3RhbmNlKHNvdXJjZSwgQ29ubmVjdG9yRXhwcmVzc2lvbik6CiAgICAgICAgICAgIGNvbm5lY3RvciA9IHNvdXJjZQoKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb25uZWN0b3IsIEZpbGVDb25uZWN0b3JFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIHNvdXJjZSA9IGNvbm5lY3Rvci5ieXRlcwogICAgICAgICAgICAgICAgaWYgbm90IGNvbm5lY3Rvci5pc0F2YWlsYWJsZToKICAgICAgICAgICAgICAgICAgICBzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbi5BZGRJc3N1ZSh0eXBlPUlzc3VlVHlwZS5FeGNlcHRpb25SdW5uaW5nRnVuY3Rpb24sIGRlc2NyaXB0aW9uPWYiRmlsZSBub3QgZm91bmQiLCBkZXRhaWxzPWNvbm5lY3Rvci5wYXRoKQoKICAgICAgICBpZiBpc2luc3RhbmNlKGNvbHVtbnMsIGludCk6CiAgICAgICAgICAgIGNvbE5hbWVzID0gW10KICAgICAgICAgICAgZm9yIGkgaW4gcmFuZ2UoMCwgY29sdW1ucyk6CiAgICAgICAgICAgICAgICBjb2xOYW1lID0gZiJDb2x1bW57aSsxfSIKICAgICAgICAgICAgICAgIGNvbE5hbWVzLmFwcGVuZChjb2xOYW1lKQogICAgICAgICAgICAgICAgZHR5cGVzW2NvbE5hbWVdID0gcGQuU3RyaW5nRHR5cGUoKQoKICAgICAgICBlbGlmIGlzaW5zdGFuY2UoY29sdW1ucywgbGlzdCk6CiAgICAgICAgICAgIGNvbE5hbWVzID0gW10KICAgICAgICAgICAgZm9yIGNvbCBpbiBjb2x1bW5zOgogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb2wsIHN0cik6CiAgICAgICAgICAgICAgICAgICAgY29sTmFtZSA9IGNvbAogICAgICAgICAgICAgICAgICAgIGR0eXBlc1tjb2xOYW1lXSA9IHBkLlN0cmluZ0R0eXBlKCkKICAgICAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZShjb2wsIENvbHVtbik6CiAgICAgICAgICAgICAgICAgICAgY29sTmFtZSA9IGNvbC5uYW1lCiAgICAgICAgICAgICAgICAgICAgY29sVHlwZSA9IGNvbC50eXBlCiAgICAgICAgICAgICAgICBjb2xOYW1lcy5hcHBlbmQoY29sTmFtZSkKICAgICAgICAgICAgICAgIGR0eXBlc1tjb2xOYW1lXSA9IGNvbFR5cGUKICAgICAgICBkZWxpbWl0ZXIgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJkZWxpbWl0ZXIiKQogICAgICAgIGlmIGRlbGltaXRlciBpcyBub3QgTm9uZSBhbmQgbGVuKGRlbGltaXRlcikgPiAwOgogICAgICAgICAgICBkZWxpbWl0ZXIgPSBkZWxpbWl0ZXJbMF0KCiAgICAgICAgZW5jb2RpbmcgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJlbmNvZGluZyIpCiAgICAgICAgaWYgZW5jb2RpbmcgPT0gMTI1MjoKICAgICAgICAgICAgZW5jb2RpbmcgPSAid2luZG93cy0xMjUyIgogICAgICAgIAogICAgICAgIENzdlN0eWxlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiQ3N2U3R5bGUiKQogICAgICAgIFF1b3RlU3R5bGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJRdW90ZVN0eWxlIikKICAgICAgICBJbmNsdWRlQnl0ZU9yZGVyTWFyayA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIkluY2x1ZGVCeXRlT3JkZXJNYXJrIikKICAgICAgICBleHRyYVZhbHVlcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImV4dHJhVmFsdWVzIikKICAgICAgICBkYXRhID0gTm9uZQogICAgICAgIGlmIHNvdXJjZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgZGF0YSA9IHBkLnJlYWRfY3N2KGlvLkJ5dGVzSU8oc291cmNlKSwgZGVsaW1pdGVyPWRlbGltaXRlciwgbmFtZXM9Y29sTmFtZXMsIGR0eXBlPWR0eXBlcywgZW5jb2Rpbmc9ZW5jb2RpbmcsIGhlYWRlcj1Ob25lKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIGRhdGEgPSBwZC5EYXRhRnJhbWUoY29sdW1ucz1jb2xOYW1lcykKCiAgICAgICAgcmVzdWx0ID0gQ2FjaGVkVGFibGVFeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgY29ubmVjdG9yPWNvbm5lY3RvciwgZGF0YT1kYXRhLCBwcm9wZXJ0aWVzPXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnByb3BlcnRpZXMpCiAgICAgICAgCiAgICAgICAgcmV0dXJuIHJlc3VsdAogICAgICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoQ3N2RG9jdW1lbnQp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableJoin.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBKb2luVGFibGVFeHByZXNzaW9uLCBKb2luS2luZApmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLW5lc3RlZGpvaW4KY2xhc3MgVGFibGVKb2luKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuSm9pbiIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUxIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImtleTEiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigidGFibGUyIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImtleTIiLCBFeHByZXNzaW9uVHlwZS5GdW5jdGlvbiwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJqb2luS2luZCIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUsICJKb2luS2luZC5Jbm5lciIpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJqb2luQWxnb3JpdGhtIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgVHJ1ZSwgIkpvaW5BbGdvcml0aG0uRHluYW1pYyIpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJrZXlFcXVhbGl0eUNvbXBhcmVycyIsIEV4cHJlc3Npb25UeXBlLkxpc3QsIEZhbHNlKQogICAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHRhYmxlMSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRhYmxlMSIpCiAgICAgICAga2V5MSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImtleTEiKQogICAgICAgIHRhYmxlMiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRhYmxlMiIpCiAgICAgICAga2V5MiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImtleTIiKQogICAgICAgIGpvaW5LaW5kID0gSm9pbktpbmQuZnJvbVN0cmluZyhzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJqb2luS2luZCIpKQogICAgICAgIGtleUVxdWFsaXR5Q29tcGFyZXJzID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgia2V5RXF1YWxpdHlDb21wYXJlcnMiKQogICAgICAgIGpvaW5BbGdvcml0aG0gPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJqb2luQWxnb3JpdGhtIikKCiAgICAgICAgcmV0dXJuIEpvaW5UYWJsZUV4cHJlc3Npb24odGFibGUxPXRhYmxlMSwgdGFibGUyPXRhYmxlMiwga2V5MT1rZXkxLCBrZXkyPWtleTIsIGpvaW5LaW5kPWpvaW5LaW5kLCBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGtleUVxdWl0eUNvbXBhcmVycz1rZXlFcXVhbGl0eUNvbXBhcmVycykKICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGFibGVKb2luKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TextCombine.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGV4dC1jb21iaW5lCmNsYXNzIFRleHRDb21iaW5lKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGV4dC5Db21iaW5lIiwgRXhwcmVzc2lvblR5cGUuVGV4dCkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRleHRzIiwgRXhwcmVzc2lvblR5cGUuTGlzdCwgVHJ1ZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJzZXBhcmF0b3IiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBUcnVlLCAiIikKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICB0ZXh0cyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRleHRzIikKICAgICAgICBzZXBhcmF0b3IgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJzZXBhcmF0b3IiKQogICAgICAgIAoKICAgICAgICBpZiBpc2luc3RhbmNlKHRleHRzLCBsaXN0KToKICAgICAgICAgICAgCiAgICAgICAgICAgIHR4dHMgPSBbXQogICAgICAgICAgICBpID0gMAogICAgICAgICAgICBmb3IgdCBpbiB0ZXh0czoKICAgICAgICAgICAgICAgIHR4dCA9ICIiCiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHQsIE1FeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICB0eHQgPSB0LnZhbHVlCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHR4dCA9IHQKCiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHR4dCwgTUV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgIHR4dCA9IHR4dC52YWx1ZQogICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICBpZiBpIDwgbGVuKHRleHRzKSAtIDE6CiAgICAgICAgICAgICAgICAgICAgdHh0ICs9IHNlcGFyYXRvcgogICAgICAgICAgICAgICAgdHh0cy5hcHBlbmQodHh0KQogICAgICAgICAgICAgICAgaSA9IGkrMQoKICAgICAgICAgICAgcmV0dXJuIGYiQ09OQ0FUKHsnLCcuam9pbih0eHRzKX0pIgogICAgICAgIAogICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkV4cGVjdGVkIGxpc3Qgb2YgdGV4dHMiKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRleHRDb21iaW5lKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/ListContains.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IExpc3RFeHByZXNzaW9uLCBNRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxTdGF0ZW1lbnRFeHByZXNzaW9uIGltcG9ydCBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vbGlzdC1jb250YWlucwoKY2xhc3MgTGlzdENvbnRhaW5zKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiTGlzdC5Db250YWlucyIsIEV4cHJlc3Npb25UeXBlLkxvZ2ljYWwpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbICBGdW5jdGlvblBhcmFtZXRlcigibGlzdCIsIEV4cHJlc3Npb25UeXBlLkxpc3QsIEZhbHNlKSwKICAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJlcXVhdGlvbkNyaXRlcmlhIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgVHJ1ZSwgIkNvbXBhcmVyLkVxdWFscyIpXQogICAgICAgICAgICAgICAgICAgICMgY29tcGFyZXIgZnVuY3Rpb25zIGRlZmluZWQgaGVyZTogaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS9jb21wYXJlci1mdW5jdGlvbnMKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBsaXN0ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgibGlzdCIpCiAgICAgICAgdmFsdWUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ2YWx1ZSIpCiAgICAgICAgZXF1YXRpb25Dcml0ZXJpYSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImVxdWF0aW9uQ3JpdGVyaWEiKQogICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIE1FeHByZXNzaW9uKToKICAgICAgICAgICAgdmFsdWVGdW5jdGlvbiA9IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iVE9fVkFSSUFOVCIsIHBhcmFtZXRlcnM9W3ZhbHVlXSwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5BbnkpCgogICAgICAgICAgICBjbWQgPSAiIgoKICAgICAgICAgICAgbWF0Y2ggKGVxdWF0aW9uQ3JpdGVyaWEpOgogICAgICAgICAgICAgICAgY2FzZSAiQ29tcGFyZXIuT3JkaW5hbElnbm9yZUNhc2UiOgogICAgICAgICAgICAgICAgICAgIGNtZCA9ICJHRVRfSUdOT1JFX0NBU0UiCiAgICAgICAgICAgICAgICBjYXNlICJDb21wYXJlci5FcXVhbHMiOgogICAgICAgICAgICAgICAgICAgIGNtZCA9ICJHRVQiCiAgICAgICAgICAgICAgICBjYXNlIF86CiAgICAgICAgICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIlVuc3VwcG9ydGVkIGVxdWF0aW9uIGNyaXRlcmlhOiB7ZXF1YXRpb25Dcml0ZXJpYX0iKQogICAgICAgICAgICAgICAgCiAgICAgICAgICAgIGdldEZ1bmN0aW9uID0gU1FMRnVuY3Rpb25FeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgZnVuY3Rpb25OYW1lPWNtZCwgcGFyYW1ldGVycz1bbGlzdCwgdmFsdWVdKQoKICAgICAgICAgICAgcmV0dXJuIFNRTFN0YXRlbWVudEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBzdGF0ZW1lbnQ9ZiIoe2dldEZ1bmN0aW9uLmdldFNxbFZhbHVlKCl9ID0ge3ZhbHVlRnVuY3Rpb24uZ2V0U3FsVmFsdWUoKX0pIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIklzc3VlIHZhbHVlIHBhcmFtZXRlciBpcyBub3QgTUV4cHJlc3Npb24iKQogICAgICAgICAgICAgICAgICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoTGlzdENvbnRhaW5zKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableReplaceValue.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24sIExpc3RFeHByZXNzaW9uCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFRhYmxlRXhwcmVzc2lvbiwgUXVlcnlUYWJsZUV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1yZXBsYWNldmFsdWUKY2xhc3MgVGFibGVSZXBsYWNlVmFsdWUoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJUYWJsZS5SZXBsYWNlVmFsdWUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm9sZFZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm5ld1ZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInJlcGxhY2VyIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImNvbHVtbnNUb1NlYXJjaCIsIEV4cHJlc3Npb25UeXBlLkxpc3QsIEZhbHNlKQogICAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHRhYmxlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUiKQogICAgICAgIG9sZFZhbHVlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgib2xkVmFsdWUiKQogICAgICAgIG5ld1ZhbHVlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgibmV3VmFsdWUiKQogICAgICAgIHJlcGxhY2VyID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgicmVwbGFjZXIiKQogICAgICAgIGNvbHVtbnNUb1NlYXJjaCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbnNUb1NlYXJjaCIpCgogICAgICAgICMgVmFsaWRhdGlvbnMKICAgICAgICBpZiBub3QgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGVFeHByZXNzaW9uKToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIkV4cGVjdGluZyB0YWJsZSBleHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQoKICAgICAgICAjIFN1cHBvcnQgb25lIGNvbHVtbiBmb3Igbm93CiAgICAgICAgY29scyA9IGNvbHVtbnNUb1NlYXJjaAogICAgICAgIGlmIGlzaW5zdGFuY2UoY29sdW1uc1RvU2VhcmNoLCBMaXN0RXhwcmVzc2lvbik6CiAgICAgICAgICAgIGNvbHMgPSBjb2x1bW5zVG9TZWFyY2guZ2V0VmFsdWUoKQogICAgICAgIGlmIG5vdCBpc2luc3RhbmNlKGNvbHMsIGxpc3QpIG9yIGxlbihjb2xzKSAhPSAxOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJUYWJsZS5SZXBsYWNlVmFsdWUgb25seSBzdXBwb3J0cyBvbmUgY29sdW1uIHRvIHNlYXJjaCBmb3Igbm93LiIpCgogICAgICAgIHRhcmdldF9jb2xfbmFtZSA9IGNvbHNbMF0KICAgICAgICBpZiBpc2luc3RhbmNlKHRhcmdldF9jb2xfbmFtZSwgRXhwcmVzc2lvbik6CiAgICAgICAgICAgIHRhcmdldF9jb2xfbmFtZSA9IHRhcmdldF9jb2xfbmFtZS52YWx1ZQoKICAgICAgICBpZiByZXBsYWNlciA9PSAiUmVwbGFjZXIuUmVwbGFjZVRleHQiOgogICAgICAgICAgICAjIEJ1aWxkIFNFTEVDVCBSRVBMQUNFKGNvbCwgb2xkLCBuZXcpIEZST00gKHRhYmxlKQogICAgICAgICAgICBiYXNlX3NxbCA9IHRhYmxlLmdldFNxbFZhbHVlKCkKCiAgICAgICAgICAgICMgUmVzb2x2ZSB0aGUgY29sdW1uIGlkZW50aWZpZXIgd2l0aCBxdW90aW5nIGlmIHBvc3NpYmxlCiAgICAgICAgICAgIGNvbF9pZGVudGlmaWVyID0gdGFyZ2V0X2NvbF9uYW1lCiAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgIGNvbF9vYmogPSB0YWJsZS5nZXRDb2x1bW4odGFyZ2V0X2NvbF9uYW1lKQogICAgICAgICAgICAgICAgaWYgY29sX29iaiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBjb2xfaWRlbnRpZmllciA9IGNvbF9vYmouZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1UcnVlLCBpbmNsdWRlVGFibGVBbGlhcz1GYWxzZSwgaW5jbHVkZVRhYmxlTmFtZT1GYWxzZSkKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZSh0YXJnZXRfY29sX25hbWUsIHN0cikgYW5kIHRhcmdldF9jb2xfbmFtZSAhPSB0YXJnZXRfY29sX25hbWUudXBwZXIoKToKICAgICAgICAgICAgICAgICAgICAgICAgY29sX2lkZW50aWZpZXIgPSBmJyJ7dGFyZ2V0X2NvbF9uYW1lfSInCiAgICAgICAgICAgIGV4Y2VwdCBFeGNlcHRpb246CiAgICAgICAgICAgICAgICBwYXNzCgogICAgICAgICAgICBvbGRfc3FsID0gRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShvbGRWYWx1ZSkKICAgICAgICAgICAgbmV3X3NxbCA9IEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUobmV3VmFsdWUpCiAgICAgICAgICAgICMgVE9ETzogdGhpcyBuZWVkcyB0byBiZSBhIHByb3BlciBTUUwgZXhwcmVzc2lvbiBpbnN0ZWFkLgogICAgICAgICAgICBzcWwgPSAoCiAgICAgICAgICAgICAgICBmIlNFTEVDVCBSRVBMQUNFKHtjb2xfaWRlbnRpZmllcn0sIHtvbGRfc3FsfSwge25ld19zcWx9KVxuIgogICAgICAgICAgICAgICAgZiJGUk9NXG5cdCh7YmFzZV9zcWx9KSIKICAgICAgICAgICAgKQogICAgICAgICAgICByZXR1cm4gUXVlcnlUYWJsZUV4cHJlc3Npb24oCiAgICAgICAgICAgICAgICBxdWVyeT1zcWwsCiAgICAgICAgICAgICAgICBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sCiAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzPXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnByb3BlcnRpZXMsCiAgICAgICAgICAgICAgICBiYXNlVGFibGU9dGFibGUsCiAgICAgICAgICAgICkKICAgICAgICAjIFRPRE86IHN1cHBvcnQgUmVwbGFjZXIuUmVwbGFjZVZhbHVlLgogICAgICAgIGVsaWYgcmVwbGFjZXIgPT0gIlJlcGxhY2VyLlJlcGxhY2VWYWx1ZSI6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIlRhYmxlLlJlcGxhY2VWYWx1ZSB3aXRoIFJlcGxhY2VyLlJlcGxhY2VWYWx1ZSBpcyBub3Qgc3VwcG9ydGVkIHlldCIpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIlVuc3VwcG9ydGVkIHJlcGxhY2VyOiB7cmVwbGFjZXJ9IikKCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGFibGVSZXBsYWNlVmFsdWUp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableAddColumn.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFRhYmxlRXhwcmVzc2lvbiwgV3JhcHBlclRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IE1FeHByZXNzaW9uLCBFYWNoRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLWFkZGNvbHVtbgpjbGFzcyBUYWJsZUFkZENvbHVtbihGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLkFkZENvbHVtbiIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigibmV3Q29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1uR2VuZXJhdG9yIiwgRXhwcmVzc2lvblR5cGUuRnVuY3Rpb24sIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1uVHlwZSIsIEV4cHJlc3Npb25UeXBlLlR5cGUsIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgbmV3Q29sdW1uTmFtZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm5ld0NvbHVtbk5hbWUiKQogICAgICAgIGNvbHVtbkdlbmVyYXRvciA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbkdlbmVyYXRvciIpCiAgICAgICAgY29sdW1uVHlwZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtblR5cGUiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKGNvbHVtbkdlbmVyYXRvciwgRWFjaEV4cHJlc3Npb24pOgogICAgICAgICAgICBjb2x1bW5HZW5lcmF0b3IgPSBjb2x1bW5HZW5lcmF0b3IuZXhwcmVzc2lvbgogICAgICAgICAgICAKICAgICAgICBpZiBpc2luc3RhbmNlKGNvbHVtblR5cGUsIE1FeHByZXNzaW9uKToKICAgICAgICAgICAgY29sdW1uVHlwZSA9IGNvbHVtblR5cGUudmFsdWUKCiAgICAgICAgaWYgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGUpOgogICAgICAgICAgICB3cmFwcGVkVGFibGUgPSBXcmFwcGVyVGFibGVFeHByZXNzaW9uKHRhYmxlLCBzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbikKICAgICAgICAgICAgd3JhcHBlZFRhYmxlLmFkZENvbHVtbihuYW1lPW5ld0NvbHVtbk5hbWUsIHR5cGU9Y29sdW1uVHlwZSwgZXhwcmVzc2lvbj1jb2x1bW5HZW5lcmF0b3IpCiAgICAgICAgCiAgICAgICAgICAgIHJldHVybiB3cmFwcGVkVGFibGUKICAgIAogICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoZiJ0YWJsZSBpcyBub3QgYSBUYWJsZSBFeHByZXNzaW9uOiB7c3RyKHRhYmxlKX0iKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRhYmxlQWRkQ29sdW1uKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TextStart.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RleHQtc3RhcnQKY2xhc3MgVGV4dFN0YXJ0KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGV4dC5TdGFydCIsIEV4cHJlc3Npb25UeXBlLlRleHQpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0ZXh0IiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY291bnQiLCBFeHByZXNzaW9uVHlwZS5OdW1iZXIsIEZhbHNlKSwKICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGV4dCA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInRleHQiKQogICAgICAgIGNvdW50ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY291bnQiKQogICAgICAgICMgTWFwIHRvIFNub3dmbGFrZSBMRUZUKHRleHQsIGNvdW50KQogICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24oCiAgICAgICAgICAgIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwKICAgICAgICAgICAgZnVuY3Rpb25OYW1lPSJMRUZUIiwKICAgICAgICAgICAgcGFyYW1ldGVycz1bdGV4dCwgY291bnRdLAogICAgICAgICAgICByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLlRleHQsCiAgICAgICAgKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFRleHRTdGFydCkKCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableFromRecords.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gdHlwaW5nIGltcG9ydCBMaXN0LCBBbnkKZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLlJlY29yZEV4cHJlc3Npb24gaW1wb3J0IFJlY29yZEV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLm1FeHByZXNzaW9uIGltcG9ydCBMaXN0RXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBDYWNoZWRUYWJsZUV4cHJlc3Npb24sIFRhYmxlRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGFibGUtZnJvbXJlY29yZHMKY2xhc3MgVGFibGVGcm9tUmVjb3JkcyhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLkZyb21SZWNvcmRzIiwgRXhwcmVzc2lvblR5cGUuVGFibGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJyZWNvcmRzIiwgRXhwcmVzc2lvblR5cGUuTGlzdCwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1ucyIsIEV4cHJlc3Npb25UeXBlLkFueSwgVHJ1ZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm1pc3NpbmdGaWVsZCIsIEV4cHJlc3Npb25UeXBlLk51bWJlciwgVHJ1ZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgcmVjb3JkcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInJlY29yZHMiKQogICAgICAgIGNvbHVtbnNQYXJhbSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbnMiKQogICAgICAgIHJvd3MgPSBbXQogICAgICAgIGNvbHVtbnM6IExpc3RbQ29sdW1uXSA9IFtdCiAgICAgICAgY29sTmFtZXMgPSBbXQogICAgICAgICAgICAgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZShyZWNvcmRzLCBsaXN0KToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb2x1bW5zUGFyYW0sIFRhYmxlRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICBjb2x1bW5zID0gY29sdW1uc1BhcmFtLmNvbHVtbnMKICAgICAgICAgICAgICAgIGZvciBjb2wgaW4gY29sdW1uczoKICAgICAgICAgICAgICAgICAgICBjb2xOYW1lcy5hcHBlbmQoY29sLm5hbWUpCgogICAgICAgICAgICBlbGlmIGNvbHVtbnNQYXJhbSBpcyBOb25lOgogICAgICAgICAgICAgICAgZm9yIHJlY29yZCBpbiByZWNvcmRzOgogICAgICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UocmVjb3JkLCBSZWNvcmRFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgZm9yIGNvbE5hbWUgaW4gcmVjb3JkLmtleXM6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiBjb2xOYW1lIG5vdCBpbiBjb2xOYW1lczoKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBjb2xOYW1lcy5hcHBlbmQoY29sTmFtZSkKCiAgICAgICAgICAgIGZvciBjb2xOYW1lIGluIGNvbE5hbWVzOgogICAgICAgICAgICAgICAgY29sdW1ucy5hcHBlbmQoQ29sdW1uKG5hbWU9Y29sTmFtZSkpICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKCiAgICAgICAgICAgIGNvbENvdW50ID0gbGVuKGNvbHVtbnMpCiAgICAgICAgICAgIGZvciByZWNvcmQgaW4gcmVjb3JkczoKICAgICAgICAgICAgICAgIHJvdyA9IFtOb25lXSAqIGNvbENvdW50CiAgICAgICAgICAgICAgICBpZiBpc2luc3RhbmNlKHJlY29yZCwgUmVjb3JkRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgZm9yIGssIHYgaW4gcmVjb3JkLnZhbHVlLml0ZW1zKCk6CiAgICAgICAgICAgICAgICAgICAgICAgIGNvbEluZGV4ID0gY29sTmFtZXMuaW5kZXgoaykKICAgICAgICAgICAgICAgICAgICAgICAgcm93W2NvbEluZGV4XSA9IHYKICAgICAgICAgICAgICAgIHJvd3MuYXBwZW5kKHJvdykKICAgICAgICAKICAgICAgICBtaXNzaW5nRmllbGQgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJtaXNzaW5nRmllbGQiKQoKICAgICAgICByZXR1cm4gQ2FjaGVkVGFibGVFeHByZXNzaW9uKGNvbHVtbnM9Y29sdW1ucywgZGF0YT1yb3dzLCBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24pCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoVGFibGVGcm9tUmVjb3Jkcyk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableDistinct.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBRdWVyeVRhYmxlRXhwcmVzc2lvbiwgVGFibGVFeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbkhhbmRsZXIgaW1wb3J0IEZ1bmN0aW9uSGFuZGxlciwgRnVuY3Rpb25QYXJhbWV0ZXIsIEV4cHJlc3Npb25UeXBlCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9wb3dlcnF1ZXJ5LW0vdGFibGUtZGlzdGluY3QKY2xhc3MgVGFibGVEaXN0aW5jdChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlRhYmxlLkRpc3RpbmN0IiwgRXhwcmVzc2lvblR5cGUuVGFibGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJlcXVhdGlvbkNyaXRlcmlhIiwgRXhwcmVzc2lvblR5cGUuQW55LCBUcnVlKQogICAgICAgICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgZXF1YXRpb25Dcml0ZXJpYSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImVxdWF0aW9uQ3JpdGVyaWEiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZUV4cHJlc3Npb24pOgogICAgICAgICAgICBpZiBlcXVhdGlvbkNyaXRlcmlhIGlzIE5vbmU6CiAgICAgICAgICAgICAgICByZXR1cm4gU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZybT10YWJsZSwgZGlzdGluY3Q9VHJ1ZSkKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIGNvbHMgPSBbXQogICAgICAgICAgICAgICAgZ3JvdXBCeSA9IFtdCiAgICAgICAgICAgICAgICBmb3IgYyBpbiB0YWJsZS5jb2x1bW5zOgogICAgICAgICAgICAgICAgICAgIGlmIGMuZ2V0SWRlbnRpZmllcigpIGluIGVxdWF0aW9uQ3JpdGVyaWE6CiAgICAgICAgICAgICAgICAgICAgICAgIGNvbHMuYXBwZW5kKGMpCiAgICAgICAgICAgICAgICAgICAgICAgIGdyb3VwQnkuYXBwZW5kKGMpCiAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgYXZGdW5jID0gU1FMRnVuY3Rpb25FeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgZnVuY3Rpb25OYW1lPSJBTllfVkFMVUUiLCBwYXJhbWV0ZXJzPVtjXSkKCiAgICAgICAgICAgICAgICAgICAgICAgIGNvbHMuYXBwZW5kKENvbHVtbihuYW1lPWMubmFtZSwgdHlwZT1jLnR5cGUsIGV4cHJlc3Npb249YXZGdW5jKSkKCiAgICAgICAgICAgICAgICByZXR1cm4gU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZybT10YWJsZSwgY29sdW1ucz1jb2xzLCBncm91cEJ5PWdyb3VwQnkpCgogICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkV4cGVjdGluZyBhIHRhYmxlIGV4cHJlc3Npb24iKQogICAgCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZURpc3RpbmN0KQogICAgICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableExpandTableColumn.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC50YWJsZUV4cHJlc3Npb24gaW1wb3J0IFdyYXBwZXJUYWJsZUV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS50bWRsLm1FeHByZXNzaW9uIGltcG9ydCBNRXhwcmVzc2lvbiwgTGlzdEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL3Bvd2VycXVlcnktbS90YWJsZS1leHBhbmR0YWJsZWNvbHVtbgpjbGFzcyBUYWJsZUV4cGFuZFRhYmxlQ29sdW1uKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiVGFibGUuRXhwYW5kVGFibGVDb2x1bW4iLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImNvbHVtbiIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1uTmFtZXMiLCBFeHByZXNzaW9uVHlwZS5MaXN0LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoIm5ld0NvbHVtbk5hbWVzIiwgRXhwcmVzc2lvblR5cGUuTGlzdCwgVHJ1ZSkKICAgICAgICAgICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgY29sdW1uID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY29sdW1uIikKICAgICAgICBjb2x1bW5OYW1lcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbk5hbWVzIikKICAgICAgICBuZXdDb2x1bW5OYW1lcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm5ld0NvbHVtbk5hbWVzIikKICAgICAgICAKICAgICAgICBpZiBub3QgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGUpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJFeHBlY3RpbmcgdGFibGUiKQogICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UoY29sdW1uTmFtZXMsIExpc3RFeHByZXNzaW9uKToKICAgICAgICAgICAgY29sdW1uTmFtZXMgPSBjb2x1bW5OYW1lcy52YWx1ZQoKICAgICAgICBpZiBpc2luc3RhbmNlKG5ld0NvbHVtbk5hbWVzLCBMaXN0RXhwcmVzc2lvbik6CiAgICAgICAgICAgIG5ld0NvbHVtbk5hbWVzID0gbmV3Q29sdW1uTmFtZXMudmFsdWUKICAgICAgICAgICAgCiAgICAgICAgaWYgbm90IGlzaW5zdGFuY2UoY29sdW1uTmFtZXMsIGxpc3QpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJFeHBlY3RpbmcgbGlzdCIpCiAgICAgICAgaWYgbm90IGlzaW5zdGFuY2UobmV3Q29sdW1uTmFtZXMsIGxpc3QpOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJFeHBlY3RpbmcgbGlzdCIpCiAgICAgICAgCiAgICAgICAgbmV3VGFibGUgPSBXcmFwcGVyVGFibGVFeHByZXNzaW9uKHdyYXBwZWRUYWJsZT10YWJsZSwgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uKQogICAgICAgIGZvciBpIGluIHJhbmdlKGxlbihjb2x1bW5OYW1lcykpOgogICAgICAgICAgICBjb2xOYW1lID0gZiJ7Y29sdW1ufS57Y29sdW1uTmFtZXNbaV19IgogICAgICAgICAgICBhbGlhcyA9IG5ld0NvbHVtbk5hbWVzW2ldCiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoYWxpYXMsIE1FeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIGFsaWFzID0gYWxpYXMudmFsdWUKCiAgICAgICAgICAgIG5ld1RhYmxlLmFkZENvbHVtbihuYW1lPWNvbE5hbWUsIGFsaWFzPWFsaWFzKQoKICAgICAgICByZXR1cm4gbmV3VGFibGUKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZUV4cGFuZFRhYmxlQ29sdW1uKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/TableFromRows.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwudGFibGVFeHByZXNzaW9uIGltcG9ydCBDYWNoZWRUYWJsZUV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS5jb2x1bW4gaW1wb3J0IENvbHVtbgpmcm9tIGJpLnJlYWRlcnMucGJpLnRtZGwubUV4cHJlc3Npb24gaW1wb3J0IExpc3RFeHByZXNzaW9uLCBUeXBlRXhwcmVzc2lvbiwgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL3RhYmxlLWZyb21yb3dzCmNsYXNzIFRhYmxlRnJvbVJvd3MoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJUYWJsZS5Gcm9tUm93cyIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigicm93cyIsIEV4cHJlc3Npb25UeXBlLkxpc3QsIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImNvbHVtbnMiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUpCiAgICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgcm93cyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInJvd3MiKQogICAgICAgIGNvbHVtbnNQYXJhbSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbnMiKQogICAgICAgCiAgICAgICAgY29sdW1uczogTGlzdFtDb2x1bW5dID0gW10KICAgICAgICAgICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2Uocm93cywgTGlzdEV4cHJlc3Npb24pOgogICAgICAgICAgICByb3dzID0gcm93cy52YWx1ZQoKICAgICAgICAKICAgICAgICBpZiBjb2x1bW5zUGFyYW0gaXMgTm9uZToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShyb3dzLCBsaXN0KSBhbmQgbGVuKHJvd3MpID4gMDoKICAgICAgICAgICAgICAgIGZpcnN0Um93ID0gcm93c1swXQogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShmaXJzdFJvdywgTGlzdEV4cHJlc3Npb24pOgogICAgICAgICAgICAgICAgICAgIGZpcnN0Um93ID0gZmlyc3RSb3cudmFsdWUKICAgICAgICAgICAgICAgIGNvbENvdW50ID0gbGVuKGZpcnN0Um93KQogICAgICAgICAgICAgICAgZm9yIGkgaW4gcmFuZ2UoMCwgY29sQ291bnQpOgogICAgICAgICAgICAgICAgICAgIGNvbHVtbnMuYXBwZW5kKENvbHVtbihuYW1lPWYiQ29sdW1ue2krMX0iKSkKICAgICAgICBlbGlmIGlzaW5zdGFuY2UoY29sdW1uc1BhcmFtLCBMaXN0RXhwcmVzc2lvbik6CiAgICAgICAgICAgIGNvbHVtbnNQYXJhbSA9IGNvbHVtbnNQYXJhbS52YWx1ZQogICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UoY29sdW1uc1BhcmFtLCBsaXN0KToKICAgICAgICAgICAgZm9yIGMgaW4gY29sdW1uc1BhcmFtOgogICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICBjID0gYy52YWx1ZQoKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoYywgc3RyKToKICAgICAgICAgICAgICAgICAgICBjb2x1bW5zLmFwcGVuZChDb2x1bW4obmFtZT1jKSkKICAgICAgICBlbGlmIGlzaW5zdGFuY2UoY29sdW1uc1BhcmFtLCBUeXBlRXhwcmVzc2lvbikgYW5kIGNvbHVtbnNQYXJhbS52YWx1ZSA9PSBFeHByZXNzaW9uVHlwZS5UYWJsZToKICAgICAgICAgICAgY29sdW1ucyA9IGNvbHVtbnNQYXJhbS5wcm9wZXJ0aWVzLmdldCgiY29sdW1ucyIpCiAgICAgICAgICAgIAogICAgICAgIHJldHVybiBDYWNoZWRUYWJsZUV4cHJlc3Npb24oY29sdW1ucz1jb2x1bW5zLCBkYXRhPXJvd3MsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbikKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihUYWJsZUZyb21Sb3dzKQogICAgICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/DateEndOfMonth.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL2RhdGUtZW5kb2Ztb250aApjbGFzcyBEYXRlRW5kT2ZNb250aChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkRhdGUuRW5kT2ZNb250aCIsIEV4cHJlc3Npb25UeXBlLkRhdGUpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJkYXRlVGltZSIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpIF0KICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBkYXRlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiZGF0ZVRpbWUiKQogICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IkxBU1RfREFZIiwgcGFyYW1ldGVycz1bZGF0ZV0sIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuRGF0ZSkKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihEYXRlRW5kT2ZNb250aCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/functions/BinaryDecompress.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgppbXBvcnQgemxpYgppbXBvcnQgZ3ppcAoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvcG93ZXJxdWVyeS1tL2JpbmFyeS1kZWNvbXByZXNzCmNsYXNzIEJpbmFyeURlY29tcHJlc3MoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJCaW5hcnkuRGVjb21wcmVzcyIsIEV4cHJlc3Npb25UeXBlLkJpbmFyeSwgMSkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImJpbmFyeSIsIEV4cHJlc3Npb25UeXBlLkJpbmFyeSwgVHJ1ZSksIAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJjb21wcmVzc2lvblR5cGUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGJpbmFyeSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImJpbmFyeSIpCiAgICAgICAgaWYgaXNpbnN0YW5jZShiaW5hcnksIEV4cHJlc3Npb24pOgogICAgICAgICAgICBiaW5hcnkgPSBiaW5hcnkudmFsdWUKICAgICAgICBjb21wcmVzc2lvblR5cGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJjb21wcmVzc2lvblR5cGUiKQogICAgICAgIGRlY29tcHJlc3NlZEJ5dGVzID0gTm9uZQoKICAgICAgICBpZiBiaW5hcnkgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGlmIGNvbXByZXNzaW9uVHlwZSA9PSAiQ29tcHJlc3Npb24uRGVmbGF0ZSI6CiAgICAgICAgICAgICAgICBkZWNvbXByZXNzZWRCeXRlcyA9IHpsaWIuZGVjb21wcmVzcyhiaW5hcnksIHdiaXRzPS0xNSkKICAgICAgICAgICAgZWxpZiBjb21wcmVzc2lvblR5cGUgPT0gIkNvbXByZXNzaW9uLkdaaXAiOgogICAgICAgICAgICAgICAgZGVjb21wcmVzc2VkQnl0ZXMgPSBnemlwLmRlY29tcHJlc3MoYmluYXJ5KQoKICAgICAgICByZXR1cm4gZGVjb21wcmVzc2VkQnl0ZXMKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKEJpbmFyeURlY29tcHJlc3Mp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/namedItem.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbCwgRGljdCwgTGlzdCwgU2V0CmZyb20gYWJjIGltcG9ydCBBQkMsIGFic3RyYWN0bWV0aG9kCgpjbGFzcyBOYW1lZEl0ZW1Db250YWluZXIoKTogIAogICAgZGVmIF9faW5pdF9fKHNlbGYsIGNoaWxkVHlwZXM6IERpY3Rbc3RyLCB0eXBlXSA9IHt9KToKICAgICAgICBzZWxmLl9fY2hpbGRJdGVtVHlwZXMgPSBjaGlsZFR5cGVzCiAgICAgICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBjaGlsZEl0ZW1UeXBlcyhzZWxmKSAtPiBEaWN0W3N0ciwgdHlwZV06CiAgICAgICAgcmV0dXJuIHNlbGYuX19jaGlsZEl0ZW1UeXBlcwogICAgCiAgICBAcHJvcGVydHkKICAgIEBhYnN0cmFjdG1ldGhvZAogICAgZGVmIGRlZmF1bHRLaW5kKHNlbGYpIC0+IHN0cjoKICAgICAgICBwYXNzCiAgICAKICAgIGRlZiBnZXRJdGVtKHNlbGYsIG5hbWU6IHN0ciwgdHlwZU5hbWU6IHN0ciA9IE5vbmUpIC0+IE9wdGlvbmFsW05hbWVkSXRlbV06CiAgICAgICAgaXRlbSA9IE5vbmUKICAgICAgICBpZiB0eXBlTmFtZSBpcyBOb25lOgogICAgICAgICAgICB0eXBlTmFtZSA9IHNlbGYuZGVmYXVsdEtpbmQKICAgICAgICAgICAgCiAgICAgICAgaWYgdHlwZU5hbWUgaW4gc2VsZi5jaGlsZEl0ZW1UeXBlczoKICAgICAgICAgICAgaXRlbVR5cGUgPSBzZWxmLmNoaWxkSXRlbVR5cGVzLmdldCh0eXBlTmFtZSkKICAgICAgICAgICAgaWYgaXNzdWJjbGFzcyhpdGVtVHlwZSwgTmFtZWRJdGVtKToKICAgICAgICAgICAgICAgIGl0ZW0gPSBpdGVtVHlwZShuYW1lPW5hbWUsIHBhcmVudD1zZWxmKQogICAgICAgIHJldHVybiBpdGVtCiAgICAKY2xhc3MgTmFtZWRJdGVtKE5hbWVkSXRlbUNvbnRhaW5lciwgQUJDKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBuYW1lOiBzdHIsIHBhcmVudDogTmFtZWRJdGVtQ29udGFpbmVyLCBjaGlsZFR5cGVzOiBEaWN0W3N0cix0eXBlXSA9IHt9KToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKGNoaWxkVHlwZXM9Y2hpbGRUeXBlcykKICAgICAgICBzZWxmLl9fbmFtZSA9IG5hbWUKICAgICAgICBpZiBpc2luc3RhbmNlKHBhcmVudCwgTmFtZWRJdGVtQ29udGFpbmVyKToKICAgICAgICAgICAgc2VsZi5fX3BhcmVudCA9IHBhcmVudAogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHNlbGYuX19wYXJlbnQgPSBOb25lCgogICAgQHByb3BlcnR5CiAgICBkZWYgbmFtZShzZWxmKSAtPiBzdHI6CiAgICAgICAgcmV0dXJuIHNlbGYuX19uYW1lCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBhcmVudChzZWxmKSAtPiBOYW1lZEl0ZW1Db250YWluZXI6CiAgICAgICAgcmV0dXJuIHNlbGYuX19wYXJlbnQKICAgIAogICAgQHByb3BlcnR5CiAgICBAYWJzdHJhY3RtZXRob2QKICAgIGRlZiBpdGVtVHlwZShzZWxmKSAtPiBzdHI6CiAgICAgICAgcGFzcwogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBmdWxseVF1YWxpZmllZE5hbWUoc2VsZikgLT4gc3RyOgogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZi5wYXJlbnQsIE5hbWVkSXRlbSk6CiAgICAgICAgICAgIHJldHVybiBmIntzZWxmLnBhcmVudC5mdWxseVF1YWxpZmllZE5hbWV9LntzZWxmLm5hbWV9IgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBzZWxmLm5hbWUKCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYubmFtZX0ge3NlbGYuaXRlbVR5cGV9IgogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3JlcHJfXygp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/mExpressionParser.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/expressionInfo.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IE9wdGlvbmFsLCBBbnkKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKY2xhc3MgRXhwcmVzc2lvbkluZm8oKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCAKICAgICAgICAgICAgICAgICB0eXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLlVua25vd24sIAogICAgICAgICAgICAgICAgIHZhbHVlOiBPcHRpb25hbFtBbnldID0gTm9uZSwKICAgICAgICAgICAgICAgICBkZXRhaWxzOiBkaWN0ID0ge30sCiAgICAgICAgICAgICAgICAgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBzZWxmLnR5cGUgPSB0eXBlCiAgICAgICAgc2VsZi52YWx1ZSA9IHZhbHVlCiAgICAgICAgc2VsZi5kZXRhaWxzID0gZGV0YWlscwogICAgICAgIHNlbGYucHJvcGVydGllcz0gcHJvcGVydGllcwoKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaCgoc2VsZi50eXBlLCBzZWxmLnZhbHVlLCBzZWxmLnByb3BlcnRpZXMuZ2V0KCJvcmlnaW5hbCIpKSk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/tmdl/ConditionalExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIGJpLmNvcmUuY29uZGl0aW9uYWwgaW1wb3J0IENvbmRpdGlvbmFsCmZyb20gYmkucmVhZGVycy5wYmkudG1kbC5tRXhwcmVzc2lvbiBpbXBvcnQgTUV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBEaWN0LCBMaXN0CgoKY2xhc3MgQ29uZGl0aW9uYWxFeHByZXNzaW9uKE1FeHByZXNzaW9uLCBDb25kaXRpb25hbCk6CiAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudAogICAgZGVmIF9faW5pdF9fKHNlbGYsIGNvbmRpdGlvbjogc3RyLCB0cnVlRXhwcmVzc2lvbjogc3RyLCBmYWxzZUV4cHJlc3Npb246IHN0ciwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIE1FeHByZXNzaW9uLl9faW5pdF9fKHNlbGYsIHR5cGU9RXhwcmVzc2lvblR5cGUuQ29uZGl0aW9uYWwsIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICBjb25kaXRpb24gPSBzZWxmLkFkZEV4cHJlc3Npb24oY29uZGl0aW9uLCBwcm9wZXJ0aWVzPXt9KQogICAgICAgIHRydWVFeHByZXNzaW9uID0gc2VsZi5BZGRFeHByZXNzaW9uKHRydWVFeHByZXNzaW9uLCBwcm9wZXJ0aWVzPXt9KQogICAgICAgIGZhbHNlRXhwcmVzc2lvbiA9IHNlbGYuQWRkRXhwcmVzc2lvbihmYWxzZUV4cHJlc3Npb24sIHByb3BlcnRpZXM9e30pCiAgICAgICAgQ29uZGl0aW9uYWwuX19pbml0X18oc2VsZj1zZWxmLCBjb25kaXRpb249Y29uZGl0aW9uLCB0cnVlVmFsdWU9dHJ1ZUV4cHJlc3Npb24sIGZhbHNlVmFsdWU9ZmFsc2VFeHByZXNzaW9uKQoKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcmV0dXJuVHlwZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi50cnVlVmFsdWUucmV0dXJuVHlwZQogICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpOgogICAgICAgIGZyb20gYmkuY29yZS5vcGVyYXRvciBpbXBvcnQgT3BlcmF0b3IKICAgICAgICBpZiBpc2luc3RhbmNlKHNlbGYuY29uZGl0aW9uLCBPcGVyYXRvcik6CiAgICAgICAgICAgIGlmIGxlbihzZWxmLmNvbmRpdGlvbi5vcGVyYW5kcykgPT0gMiBhbmQgc2VsZi5jb25kaXRpb24ub3BlcmF0b3JzWzBdID09ICI9IjoKICAgICAgICAgICAgICAgIGlmIHNlbGYuY29uZGl0aW9uLm9wZXJhbmRzWzBdLnZhbHVlID09IHNlbGYuY29uZGl0aW9uLm9wZXJhbmRzWzFdLnZhbHVlOgogICAgICAgICAgICAgICAgICAgIHJldHVybiBzZWxmLnRydWVWYWx1ZQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgc2VsZi5mYWxzZVZhbHVlCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuZmFsc2VWYWx1ZQoKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KICAgICAgICBzcWwgPSBmIkNBU0UgV0hFTiB7RXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShzZWxmLmNvbmRpdGlvbil9IFRIRU4ge0V4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUoc2VsZi50cnVlVmFsdWUpfSBFTFNFIHtFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYuZmFsc2VWYWx1ZSl9IEVORCBDQVNFIgogICAgICAgIHJldHVybiBzcWwKCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0U3FsVmFsdWUoKQoKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIGZyb20gYmkuY29yZS5zY3JpcHQgaW1wb3J0IFNjcmlwdAogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZi5wYXJlbnQsIE1FeHByZXNzaW9uKSBhbmQgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgU2NyaXB0KToKICAgICAgICAgICAgcmV0dXJuIHNlbGYudmFsdWUKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gc2VsZi5fX3JlcHJfXygpCgpNRXhwcmVzc2lvbi5yZWdpc3RlcihDb25kaXRpb25hbEV4cHJlc3Npb24pCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/semanticTable.py FROM (SELECT BASE64_DECODE_STRING('...'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/pbir/transformTable.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIE9wdGlvbmFsLCBVbmlvbgpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCgpjbGFzcyBUcmFuc2Zvcm1UYWJsZToKICAgICIiIgogICAgUmVwcmVzZW50cyBhIHRhYmxlIHRyYW5zZm9ybWF0aW9uIGluIFBvd2VyIEJJIHJlcG9ydCBxdWVyaWVzLgogICAgVXNlZCBmb3IgcGFyc2luZyByZXBvcnQtbGV2ZWwgdmlzdWFsaXphdGlvbnMgKG5vdCB0aGUgc2VtYW50aWMgbW9kZWwpLgogICAgCiAgICBOb3RlOiBDdXJyZW50bHkgdW51c2VkIGFzIHJlcG9ydCBwYWdlIHBhcnNpbmcgaXMgbm90IGltcGxlbWVudGVkLgogICAgS2VwdCBmb3IgcG90ZW50aWFsIGZ1dHVyZSByZXBvcnQgdmlzdWFsaXphdGlvbiBzdXBwb3J0LgogICAgIiIiCiAgICBkZWYgX19pbml0X18oc2VsZiwgbmFtZTogc3RyLCBwcm9wZXJ0aWVzOiBPcHRpb25hbFtkaWN0XSA9IHt9KToKICAgICAgICBzZWxmLm5hbWUgPSBuYW1lCiAgICAgICAgc2VsZi5jb2x1bW5zOiBMaXN0W0NvbHVtbl0gPSBbXQogICAgICAgIHNlbGYuaXRlbXM6IExpc3RbVGFibGVJdGVtXSA9IFtdCiAgICAgICAgc2VsZi5wcm9wZXJ0aWVzID0gcHJvcGVydGllcwogICAgCiAgICBkZWYgYWRkQ29sdW1uKHNlbGYsIGNvbHVtbjogQ29sdW1uKToKICAgICAgICBzZWxmLmNvbHVtbnMuYXBwZW5kKGNvbHVtbikKICAgIAogICAgZGVmIGFkZEl0ZW0oc2VsZiwgaXRlbTogVGFibGVJdGVtKToKICAgICAgICBzZWxmLml0ZW1zLmFwcGVuZChpdGVtKQogICAgCiAgICBkZWYgZ2V0Q29sdW1uKHNlbGYsIGNvbHVtbk5hbWU6IHN0cikgLT4gT3B0aW9uYWxbQ29sdW1uXToKICAgICAgICBmb3IgYyBpbiBzZWxmLmNvbHVtbnM6CiAgICAgICAgICAgICMgR2V0IHNpbXBsZSB1bnF1b3RlZCBuYW1lIChubyB0YWJsZSBwcmVmaXgsIG5vIGFsaWFzLCBubyBxdW90ZXMpCiAgICAgICAgICAgIHNpbXBsZV9uYW1lID0gYy5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPUZhbHNlLCBpbmNsdWRlVGFibGVBbGlhcz1GYWxzZSwgaW5jbHVkZVRhYmxlTmFtZT1GYWxzZSkKICAgICAgICAgICAgIyBHZXQgZnVsbHkgcXVhbGlmaWVkIG5hbWUgZm9yIGNhc2VzIHdoZXJlIHNlYXJjaCBpbmNsdWRlcyB0YWJsZSBwcmVmaXgKICAgICAgICAgICAgcXVhbGlmaWVkX25hbWUgPSBjLmdldElkZW50aWZpZXIocXVvdGVNaXhlZENhc2U9RmFsc2UsIGluY2x1ZGVUYWJsZUFsaWFzPUZhbHNlLCBpbmNsdWRlVGFibGVOYW1lPVRydWUpCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBzaW1wbGVfbmFtZSA9PSBjb2x1bW5OYW1lIG9yIHF1YWxpZmllZF9uYW1lID09IGNvbHVtbk5hbWU6CiAgICAgICAgICAgICAgICByZXR1cm4gYwogICAgICAgIHJldHVybiBOb25lCiAgICAKICAgIGRlZiBnZXRJdGVtKHNlbGYsIG5hbWU6IHN0cikgLT4gT3B0aW9uYWxbVGFibGVJdGVtXToKICAgICAgICBpdGVtID0gTm9uZQogICAgICAgIGZvciBtIGluIHNlbGYuaXRlbXM6CiAgICAgICAgICAgICMgR2V0IHNpbXBsZSB1bnF1b3RlZCBuYW1lIChubyB0YWJsZSBwcmVmaXgsIG5vIGFsaWFzLCBubyBxdW90ZXMpCiAgICAgICAgICAgIHNpbXBsZV9uYW1lID0gbS5nZXRJZGVudGlmaWVyKHF1b3RlTWl4ZWRDYXNlPUZhbHNlLCBpbmNsdWRlVGFibGVBbGlhcz1GYWxzZSwgaW5jbHVkZVRhYmxlTmFtZT1GYWxzZSkKICAgICAgICAgICAgIyBHZXQgZnVsbHkgcXVhbGlmaWVkIG5hbWUgZm9yIGNhc2VzIHdoZXJlIHNlYXJjaCBpbmNsdWRlcyB0YWJsZSBwcmVmaXgKICAgICAgICAgICAgcXVhbGlmaWVkX25hbWUgPSBtLmdldElkZW50aWZpZXIocXVvdGVNaXhlZENhc2U9RmFsc2UsIGluY2x1ZGVUYWJsZUFsaWFzPUZhbHNlLCBpbmNsdWRlVGFibGVOYW1lPVRydWUpCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBzaW1wbGVfbmFtZSA9PSBuYW1lIG9yIHF1YWxpZmllZF9uYW1lID09IG5hbWU6CiAgICAgICAgICAgICAgICBpdGVtID0gbQogICAgICAgICAgICAgICAgYnJlYWsKICAgICAgICBpZiBpdGVtIGlzIE5vbmUgYW5kIG5hbWUuZW5kc3dpdGgoIl9vIik6CiAgICAgICAgICAgIGl0ZW0gPSBzZWxmLmdldEl0ZW0obmFtZVswOmxlbihuYW1lKS0yXSkKICAgICAgICByZXR1cm4gaXRlbQogICAgCiAgICBkZWYgZ2V0Q29sdW1uT3JJdGVtKHNlbGYsIG5hbWU6IHN0cikgLT4gT3B0aW9uYWxbVW5pb25bQ29sdW1uLCBUYWJsZUl0ZW1dXToKICAgICAgICBpdGVtID0gc2VsZi5nZXRDb2x1bW4obmFtZSkKICAgICAgICBpZiBpdGVtIGlzIE5vbmU6CiAgICAgICAgICAgIGl0ZW0gPSBzZWxmLmdldEl0ZW0obmFtZSkKICAgICAgICByZXR1cm4gaXRlbQogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYiVHJhbnNmb3JtVGFibGUoe3NlbGYubmFtZX0pIgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/pageItem.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbCwgTGlzdCwgVW5pb24sIFNldApmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZAppbXBvcnQgYmkucmVhZGVycy5wYmkucGFnZQoKY2xhc3MgUGFnZUl0ZW0oKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBwYWdlOiBiaS5yZWFkZXJzLnBiaS5wYWdlLlBhZ2UsIG5hbWU6IHN0ciwgeDogaW50LCB5OiBpbnQsIHo6aW50LCBoZWlnaHQ6IGludCwgd2lkdGg6IGludCwgdHlwZTogc3RyLCBkaXNwbGF5TW9kZTogc3RyLCBxdWVyeTogZGljdCA9IHt9LCBmaWx0ZXJzOiBMaXN0W2RpY3RdID0gW10sIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc2VsZi5wYWdlID0gcGFnZQogICAgICAgIHNlbGYubmFtZSA9IG5hbWUKICAgICAgICBzZWxmLnggPSB4CiAgICAgICAgc2VsZi55ID0geQogICAgICAgIHNlbGYueiA9IHoKICAgICAgICBzZWxmLmhlaWdodCA9IGhlaWdodAogICAgICAgIHNlbGYud2lkdGggPSB3aWR0aAogICAgICAgIHNlbGYudHlwZSA9IHR5cGUKICAgICAgICBzZWxmLmRpc3BsYXlNb2RlID0gZGlzcGxheU1vZGUKICAgICAgICBzZWxmLnByb3BlcnRpZXMgPSBwcm9wZXJ0aWVzCiAgICAgICAgc2VsZi5maWx0ZXJzID0gZmlsdGVycwogICAgICAgIHNlbGYucXVlcnkgPSBxdWVyeQoKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgZnVsbHlRdWFsaWZpZWROYW1lKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gZiJ7c2VsZi5wYWdlLmRpc3BsYXlOYW1lfS57c2VsZi5uYW1lfSIKICAgIAogICAgI0BhYnN0cmFjdG1ldGhvZAogICAgZGVmIHRvU3RyZWFtbGl0KHNlbGYpIC0+IHN0cjogCiAgICAgICAgcmV0dXJuICIiCgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzZWxmLnR5cGV9ICh7c2VsZi55fSwge3NlbGYueH0sIHtzZWxmLndpZHRofSwge3NlbGYuaGVpZ2h0fSkgKHtzZWxmLmRpc3BsYXlNb2RlfSkiCgpjbGFzcyBQYWdlSXRlbVRleHQoUGFnZUl0ZW0pOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIHBhZ2U6IGJpLnJlYWRlcnMucGJpLnBhZ2UuUGFnZSwgbmFtZTogc3RyLCB4OiBpbnQsIHk6IGludCwgejppbnQsIGhlaWdodDogaW50LCB3aWR0aDogaW50LCB0eXBlOiBzdHIsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhwYWdlLCBuYW1lLCB4LCB5LCB6LCBoZWlnaHQsIHdpZHRoLCAidGV4dCIsIHByb3BlcnRpZXMpCgogICAgZGVmIHRvU3RyZWFtbGl0KHNlbGYpOgogICAgICAgICNjbnRsID0gc3QuY29uCiAgICAgICAgcmV0dXJuICIiCiAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/semanticTableType.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBlbnVtIGltcG9ydCBTdHJFbnVtCgpjbGFzcyBTZW1hbnRpY1RhYmxlVHlwZShTdHJFbnVtKToKICAgIE5PTkUgPSAiTm9uZSIKICAgIERJTUVOU0lPTiA9ICJEaW1lbnNpb24iCiAgICBGQUNUID0gIkZhY3QiCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXOperatorExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUub3BlcmF0b3IgaW1wb3J0IE9wZXJhdG9yCmZyb20gLkRBWEV4cHJlc3Npb24gaW1wb3J0IERBWEV4cHJlc3Npb24sIERBWEV4cHJlc3Npb25UeXBlLCBFeHByZXNzaW9uCgpmcm9tIGVudW0gaW1wb3J0IEVudW0KaW1wb3J0IHJlZ2V4CgoKY2xhc3MgREFYT3BlcmF0b3JFeHByZXNzaW9uKERBWEV4cHJlc3Npb24sIE9wZXJhdG9yKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBvcGVyYW5kczogbGlzdCwgb3BlcmF0b3JzOiBsaXN0LCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc2VsZi5fX3ZhbHVlID0gTm9uZQogICAgICAgIERBWEV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1EQVhFeHByZXNzaW9uVHlwZS5PcGVyYXRvciwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIE9wZXJhdG9yLl9faW5pdF9fKHNlbGYpCiAgICAgICAgc2VsZi5fX29wZXJhbmRzOiBMaXN0W0RBWEV4cHJlc3Npb25dID0gW10KICAgICAgICBzZWxmLl9fb3BlcmF0b3JzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgIGZvciBvcGVyYW5kIGluIG9wZXJhbmRzOgogICAgICAgICAgICBvID0gc2VsZi5BZGRFeHByZXNzaW9uKGV4cHJlc3Npb25TdHJpbmc9b3BlcmFuZCwgcHJvcGVydGllcz17fSkKICAgICAgICAgICAgaWYgbyBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIHNlbGYuX19vcGVyYW5kcy5hcHBlbmQobykKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHBhc3MKICAgICAgICBzZWxmLl9fb3BlcmF0b3JzOiBMaXN0W3N0cl0gPSBbXQogICAgICAgIGZvciBvIGluIG9wZXJhdG9yczoKICAgICAgICAgICAgc2VsZi5fX29wZXJhdG9ycy5hcHBlbmQobykKICAgICAgICAKCiAgICBAcHJvcGVydHkKICAgIGRlZiBvcGVyYW5kcyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX29wZXJhbmRzCgogICAgQHByb3BlcnR5CiAgICBkZWYgb3BlcmF0b3JzKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fb3BlcmF0b3JzCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIE9wZXJhdG9yLnJldHVyblR5cGUuZmdldChzZWxmKQogICAgICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLmdldFZhbHVlKCkKICAgIAogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICBpZiBzZWxmLl9fdmFsdWUgaXMgTm9uZToKICAgICAgICAgICAgc2VsZi5fX3ZhbHVlID0gT3BlcmF0b3IuZ2V0U3FsKHNlbGYpCiAgICAgICAgcmV0dXJuIHNlbGYuX192YWx1ZQoKICAgIGRlZiBnZXRWYWx1ZShzZWxmLCBleHBhbmRNZWFzdXJlczogYm9vbCA9IEZhbHNlKToKICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSgpICAgCgogICAgZGVmIEdldEFsbElzc3VlcyhzZWxmKToKICAgICAgICBhbGxJc3N1ZXMgPSBzZXQoc2VsZi5Jc3N1ZXMpCgogICAgICAgIGZvciBjIGluIHNlbGYub3BlcmFuZHM6CiAgICAgICAgICAgIGlmIGMgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBhbGxJc3N1ZXMgPSBhbGxJc3N1ZXMudW5pb24oYy5HZXRBbGxJc3N1ZXMoKSkKCiAgICAgICAgcmV0dXJuIGFsbElzc3VlcwogICAgCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICAjIFJldHVybiB0aGUgdHJhbnNsYXRlZCBTUUwgZXhwcmVzc2lvbi4KICAgICAgICAjIERpcmVjdENvbnZlcnNpb24gZXhwZWN0ZXMgdGhlIHZhbHVlIHRvIGJlIHRoZSB0cmFuc2xhdGVkIFNRTCBleHByZXNzaW9uLgogICAgICAgIHJldHVybiBzZWxmLmdldFNxbFZhbHVlKCkKICAgIAogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIk9wZXJhdG9yOiB7c2VsZi5zb3VyY2VTdHJpbmd9IgogICAgCkRBWEV4cHJlc3Npb24ucmVnaXN0ZXIoREFYT3BlcmF0b3JFeHByZXNzaW9uKQpPcGVyYXRvci5yZWdpc3RlcihEQVhPcGVyYXRvckV4cHJlc3Npb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXValueExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uLCBEQVhFeHByZXNzaW9uVHlwZSwgRXhwcmVzc2lvbgppbXBvcnQgcGFuZGFzIGFzIHBkCgogICAgCmNsYXNzIERBWFZhbHVlRXhwcmVzc2lvbihEQVhFeHByZXNzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB2YWx1ZTogQW55LCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyh0eXBlPURBWEV4cHJlc3Npb25UeXBlLlZhbHVlLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgIyBJbml0aWFsaXplIGVhcmx5IHNvIF9fc3RyX18vZ2V0U3FsVmFsdWUgZHVyaW5nIGVycm9yIHJlcG9ydGluZyBkb24ndCBhY2Nlc3MKICAgICAgICAjIGFuIGF0dHJpYnV0ZSB0aGF0IGhhc24ndCBiZWVuIHNldCB5ZXQKICAgICAgICBzZWxmLl9fdmFsdWUgPSBOb25lCiAgICAgICAgaWYgaXNpbnN0YW5jZSh2YWx1ZSwgc3RyKToKICAgICAgICAgICAgaWYgdmFsdWUuc3RhcnRzd2l0aCgnIicpOgogICAgICAgICAgICAgICAgdmFsdWUgPSB2YWx1ZS5sc3RyaXAoJyInKS5yc3RyaXAoJyInKQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgdHJ5OgogICAgICAgICAgICAgICAgICAgIHZhbHVlID0gaW50KHZhbHVlKQogICAgICAgICAgICAgICAgZXhjZXB0OgogICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgdmFsdWUgPSBmbG9hdCh2YWx1ZSkKICAgICAgICAgICAgICAgICAgICBleGNlcHQ6CiAgICAgICAgICAgICAgICAgICAgICAgIHBhc3MKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIGxpc3QpOgogICAgICAgICAgICBpZiBsZW4odmFsdWUpID4gMDoKICAgICAgICAgICAgICAgIHJvd3MgPSBbXQogICAgICAgICAgICAgICAgIyBjaGVjayB0byBzZWUgaWYgaXQncyBhIGxpc3Qgb3IgYSB0YWJsZQogICAgICAgICAgICAgICAgaWYgYWxsKGlzaW5zdGFuY2UodixsaXN0KSBmb3IgdiBpbiB2YWx1ZSk6ICMgaXQncyBhIHRhYmxlCiAgICAgICAgICAgICAgICAgICAgY29sdW1uTmFtZXMgPSBbXQogICAgICAgICAgICAgICAgICAgIGZvciBpIGluIHJhbmdlKDAsbGVuKHZhbHVlWzBdKSk6CiAgICAgICAgICAgICAgICAgICAgICAgIGNvbHVtbk5hbWVzLmFwcGVuZChmIlZhbHVle2krMX0iKQogICAgICAgICAgICAgICAgICAgIGZvciByIGluIHZhbHVlOgogICAgICAgICAgICAgICAgICAgICAgICByb3cgPSB7fQogICAgICAgICAgICAgICAgICAgICAgICBmb3IgaSwgY2VsbCBpbiBlbnVtZXJhdGUocik6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBjb2xOYW1lID0gY29sdW1uTmFtZXNbaV0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIHJvd1tjb2xOYW1lXSA9IHNlbGYuQWRkRXhwcmVzc2lvbihjZWxsLCBwcm9wZXJ0aWVzPXt9KQogICAgICAgICAgICAgICAgICAgICAgICByb3dzLmFwcGVuZChyb3cpCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIGZvciB2IGluIHZhbHVlOgogICAgICAgICAgICAgICAgICAgICAgICByb3cgPSB7fQogICAgICAgICAgICAgICAgICAgICAgICByb3dbIlZhbHVlIl0gPSBzZWxmLkFkZEV4cHJlc3Npb24odiwgcHJvcGVydGllcz17fSkKICAgICAgICAgICAgICAgICAgICAgICAgcm93cy5hcHBlbmQocm93KQogICAgICAgICAgICAgICAgdmFsdWUgPSBwZC5qc29uX25vcm1hbGl6ZShyb3dzKQogICAgICAgICAgICAgICAgCiAgICAgICAgc2VsZi5fX3ZhbHVlID0gdmFsdWUKICAgIAoKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcmV0dXJuVHlwZShzZWxmKToKICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuRnJvbVZhbHVlKHNlbGYuX192YWx1ZSkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcmF3VmFsdWUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX192YWx1ZQogICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpOgogICAgICAgIHJldHVybiBFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYuX192YWx1ZSkKCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZi5fX3ZhbHVlLCBwZC5EYXRhRnJhbWUpOgogICAgICAgICAgICBjb2x1bW5OYW1lcyA9ICIsICIuam9pbihzZWxmLl9fdmFsdWUuY29sdW1ucykKCiAgICAgICAgICAgIHJvd3MgPSBbXQogICAgICAgICAgICBmb3IgciBpbiBzZWxmLl9fdmFsdWUudmFsdWVzLnRvbGlzdCgpOgogICAgICAgICAgICAgICAgcm93ID0gW10KICAgICAgICAgICAgICAgIGZvciBjZWxsIGluIHI6CiAgICAgICAgICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjZWxsLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgICAgICAgICAgcm93LmFwcGVuZChzdHIoY2VsbC52YWx1ZSkpCiAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgcm93LmFwcGVuZChzdHIoY2VsbCkpCiAgICAgICAgICAgICAgICByb3dTdHIgPSAiLCAiLmpvaW4ocm93KQogICAgICAgICAgICAgICAgcm93U3RyID0gZiIoe3Jvd1N0cn0pIgogICAgICAgICAgICAgICAgcm93cy5hcHBlbmQocm93U3RyKQoKICAgICAgICAgICAgcyA9ICIsICIuam9pbihyb3dzKQogICAgICAgICAgICByZXR1cm4gZiIoVkFMVUVTIHtzfSkgQVMgdjEoe2NvbHVtbk5hbWVzfSkiCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUoc2VsZi5fX3ZhbHVlKQogICAgICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0U3FsVmFsdWUoKQoKREFYRXhwcmVzc2lvbi5yZWdpc3RlcihEQVhWYWx1ZUV4cHJlc3Npb24pCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBMaXN0LCBVbmlvbgpmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudCwgRXhwcmVzc2lvbkNvZGVUeXBlCmZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCmZyb20gYmkucmVhZGVycy5wYmkubW9kZWxJdGVtIGltcG9ydCBNb2RlbEl0ZW0KZnJvbSBlbnVtIGltcG9ydCBFbnVtCmltcG9ydCByZWdleAoKY2xhc3MgREFYRXhwcmVzc2lvblR5cGUoRW51bSk6CiAgICBVbmtub3duID0gMCwKICAgIFJlZmVyZW5jZSA9IDEsCiAgICBGdW5jdGlvbiA9IDIsCiAgICBWYWx1ZSA9IDMsCiAgICBPcGVyYXRvciA9IDQsCiAgICBWYXJpYWJsZSA9IDUsCiAgICBTY3JpcHQgPSA2CgogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLm5hbWUKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYubmFtZQogICAgCiAgICBkZWYgX19lcV9fKHNlbGYsIHZhbHVlKToKICAgICAgICByZXR1cm4gc3VwZXIoKS5fX2VxX18odmFsdWUpCiAgICAKICAgIGRlZiBfX2hhc2hfXyhzZWxmKToKICAgICAgICByZXR1cm4gaGFzaChzZWxmLnZhbHVlKQogICAgCiAgICBkZWYgdG9JY29uKHNlbGYpOgogICAgICAgIHJldHVybiAiIgoKY2xhc3MgREFYRXhwcmVzc2lvbihFeHByZXNzaW9uLCBNb2RlbEl0ZW0sIEFCQyk6CiAgICBkZWYgX19pbml0X18oc2VsZiwgdHlwZTogREFYRXhwcmVzc2lvblR5cGUsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBNb2RlbEl0ZW0uX19pbml0X18oc2VsZikKICAgICAgICBFeHByZXNzaW9uLl9faW5pdF9fKHNlbGYsIGNvZGVUeXBlPUV4cHJlc3Npb25Db2RlVHlwZS5EQVgsIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICBzZWxmLl9fdHlwZSA9IHR5cGUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBjaGlsZEV4cHJlc3Npb25Db2RlVHlwZShzZWxmKToKICAgICAgICByZXR1cm4gRXhwcmVzc2lvbkNvZGVUeXBlLkRBWAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBSdW50aW1lKHNlbGYpOgogICAgICAgIGlmIHNlbGYuTW9kZWwgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLk1vZGVsLlJ1bnRpbWUKICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBNb2RlbChzZWxmKToKICAgICAgICBpZiBoYXNhdHRyKHNlbGYucGFyZW50LCJNb2RlbCIpOgogICAgICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQuTW9kZWwKICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1NjcmlwdChzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi50eXBlID09IERBWEV4cHJlc3Npb25UeXBlLlNjcmlwdAoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRnVuY3Rpb24oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYudHlwZSA9PSBEQVhFeHByZXNzaW9uVHlwZS5GdW5jdGlvbgogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1ZhbHVlKHNlbGYpIC0+IGJvb2w6CiAgICAgICAgcmV0dXJuIHNlbGYudHlwZSA9PSBEQVhFeHByZXNzaW9uVHlwZS5WYWx1ZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiB0YWJsZShzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0KICAgICAgICBmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCiAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgVGFibGUpOgogICAgICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQKICAgICAgICBlbGlmIGlzaW5zdGFuY2Uoc2VsZi5wYXJlbnQsIFRhYmxlSXRlbSk6CiAgICAgICAgICAgIHJldHVybiBzZWxmLnBhcmVudC50YWJsZQogICAgICAgIGVsaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgREFYRXhwcmVzc2lvbik6CiAgICAgICAgICAgIHJldHVybiBzZWxmLnBhcmVudC50YWJsZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX190eXBlCiAgICAKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIHZhcmlhYmxlTmFtZTogc3RyKToKICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQuZ2V0RXhwcmVzc2lvbih2YXJpYWJsZU5hbWUpCiAgICAKICAgIGRlZiBnZXRUYWJsZShzZWxmLCB0YWJsZU5hbWU6IHN0cik6CiAgICAgICAgaWYgdGFibGVOYW1lIGlzIE5vbmU6CiAgICAgICAgICAgIGlmIHNlbGYudGFibGUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICByZXR1cm4gc2VsZi50YWJsZQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgcmV0dXJuIE5vbmUKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQuTW9kZWwuZ2V0VGFibGUodGFibGVOYW1lKQogICAgICAgICAgIAoKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiAiIgogICAgCiAgICBkZWYgX19oYXNoX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGhhc2goKHNlbGYuY29kZVR5cGUsIHNlbGYudHlwZSwgc3RyKHNlbGYuc291cmNlU3RyaW5nKSkpCiAgICAKRXhwcmVzc2lvbi5yZWdpc3RlcihEQVhFeHByZXNzaW9uKQpNb2RlbEl0ZW0ucmVnaXN0ZXIoREFYRXhwcmVzc2lvbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXReferenceExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uLCBEQVhFeHByZXNzaW9uVHlwZSwgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuaXNzdWVUeXBlIGltcG9ydCBJc3N1ZVR5cGUKaW1wb3J0IHJlZ2V4CmZyb20gYmkucmVhZGVycy5wYmkuc2VtYW50aWNUYWJsZSBpbXBvcnQgU2VtYW50aWNUYWJsZQoKY2xhc3MgREFYUmVmZXJlbmNlRXhwcmVzc2lvbihEQVhFeHByZXNzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB2YWx1ZTogc3RyLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgCiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyh0eXBlPURBWEV4cHJlc3Npb25UeXBlLlJlZmVyZW5jZSwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIAogICAgICAgIHRhYmxlQ29sdW1uTWF0Y2ggPSByZWdleC5tYXRjaChyIid7MCwxfShbXlxbJ10rKXswLDF9J3swLDF9KFxbW15cXV0rXF0pezAsMX0iLHZhbHVlKQogICAgICAgIHNlbGYuX190YWJsZU5hbWU6IHN0ciA9IE5vbmUKICAgICAgICBzZWxmLl9fY29sdW1uTmFtZTogc3RyID0gTm9uZQogICAgICAgIHNlbGYuX192YWx1ZSA9IE5vbmUKICAgICAgICAKICAgICAgICBzZWxmLl9fdGFibGU6IFNlbWFudGljVGFibGUgPSBOb25lCiAgICAgICAgaWYgc2VsZi50YWJsZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgc2VsZi5fX3RhYmxlID0gc2VsZi50YWJsZQogICAgICAgIHNlbGYuX19jb2x1bW46IFRhYmxlSXRlbSA9IE5vbmUKICAgICAgICAKICAgICAgICBpZiB0YWJsZUNvbHVtbk1hdGNoIGlzIG5vdCBOb25lOgogICAgICAgICAgICBpZiB0YWJsZUNvbHVtbk1hdGNoLmdyb3VwKDEpIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgc2VsZi5fX3RhYmxlTmFtZSA9IHRhYmxlQ29sdW1uTWF0Y2guZ3JvdXAoMSkKICAgICAgICAgICAgICAgIGlmIChpc2luc3RhbmNlKHNlbGYuX190YWJsZSwgU2VtYW50aWNUYWJsZSkgYW5kIHNlbGYuX190YWJsZS5uYW1lICE9IHNlbGYuX190YWJsZU5hbWUpOgogICAgICAgICAgICAgICAgICAgIHNlbGYuX190YWJsZSA9IHNlbGYuTW9kZWwuZ2V0VGFibGUoc2VsZi5fX3RhYmxlTmFtZSkKCiAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgREFYRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICBzZWxmLl9fdGFibGUgPSBzZWxmLnBhcmVudC50YWJsZQoKICAgICAgICAgICAgaWYgdGFibGVDb2x1bW5NYXRjaC5ncm91cCgyKSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIHNlbGYuX19jb2x1bW5OYW1lID0gdGFibGVDb2x1bW5NYXRjaC5ncm91cCgyKS5sc3RyaXAoIlsiKS5yc3RyaXAoIl0iKQogICAgICAgICAgICAKICAgICAgICBlbHNlOgogICAgICAgICAgICBwYXJlbnQuQWRkSXNzdWUodHlwZT1Jc3N1ZVR5cGUuUGFyc2luZ0Vycm9yLCBkZXNjcmlwdGlvbj0iUmVmZXJlbmNlIGV4cHJlc3Npb24gZGlkIG5vdCBwYXJzZSBjb3JyZWN0bHkiLCBkZXRhaWxzPXZhbHVlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBSZWZUYWJsZShzZWxmKToKICAgICAgICBpZiBzZWxmLl9fdGFibGUgaXMgTm9uZToKICAgICAgICAgICAgc2VsZi5fX3RhYmxlID0gc2VsZi5Nb2RlbC5nZXRUYWJsZShzZWxmLl9fdGFibGVOYW1lKQogICAgICAgICAgICBpZiBzZWxmLl9fdGFibGUgaXMgTm9uZSBhbmQgc2VsZi5Nb2RlbC5zdGF0ZSA9PSAiUmVhZHkiOgogICAgICAgICAgICAgICAgIyBjaGVjayB0byBzZWUgaWYgdGhlIHJlZmVyZW5jZSBpcyB0byBhbiBleHByZXNzaW9uIChjb3VsZCBiZSBwYXJ0IG9mIGEgc2NyaXB0KQogICAgICAgICAgICAgICAgZXhwVmFyaWFibGU6IERBWEV4cHJlc3Npb24gPSBzZWxmLmdldEV4cHJlc3Npb24oc2VsZi5fX3RhYmxlTmFtZSkKICAgICAgICAgICAgICAgIGlmIGV4cFZhcmlhYmxlIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgICAgIHBhc3MgI0JVR0JVRzogTmVlZCB0byBmaXggYW55IHJlY3Vyc2lvbiBpc3N1ZXMgaGVyZQogICAgICAgICAgICAgICAgICAgICNwcmludChmImV4cFZhcmlhYmxlOiB7ZXhwVmFyaWFibGUuc291cmNlU3RyaW5nfSIpCiAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICNpZiBleHBWYXJpYWJsZS52YWx1ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAjICAgIHNlbGYuX192YWx1ZSA9IGV4cFZhcmlhYmxlLnZhbHVlCiAgICAgICAgICAgICAgICAgICAgI2Vsc2U6CiAgICAgICAgICAgICAgICAgICAgIyAgICBzZWxmLkFkZElzc3VlKHR5cGU9SXNzdWVUeXBlLkludmFsaWRSZWZlcmVuY2UsIGRlc2NyaXB0aW9uPSJSZWZlcmVuY2UgdG8gdmFyaWFibGUgd2hpY2ggZG9lcyBub3QgcmV0dXJuIGEgdmFsdWUiLCBkZXRhaWxzPWYiVmFyaWFibGUge3NlbGYuX190YWJsZU5hbWV9IGRvZXMgbm90IHJldHVybiBhIHZhbHVlIikKICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgc2VsZi5BZGRJc3N1ZSh0eXBlPUlzc3VlVHlwZS5JbnZhbGlkUmVmZXJlbmNlLCBkZXNjcmlwdGlvbj0iUmVmZXJlbmNlIHRhYmxlIGRvZXMgbm90IGV4aXN0IiwgZGV0YWlscz1mIkNhbiBub3QgZmluZCB0YWJsZToge3NlbGYuX190YWJsZU5hbWV9IikKICAgICAgICByZXR1cm4gc2VsZi5fX3RhYmxlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIFJlZkNvbHVtbihzZWxmKToKICAgICAgICBpZiBzZWxmLl9fY29sdW1uIGlzIE5vbmU6CiAgICAgICAgICAgIGlmIHNlbGYuX19jb2x1bW5OYW1lIGlzIG5vdCBOb25lIGFuZCBzZWxmLlJlZlRhYmxlIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgc2VsZi5fX2NvbHVtbiA9IHNlbGYuUmVmVGFibGUuZ2V0SXRlbShzZWxmLl9fY29sdW1uTmFtZSkKICAgICAgICAgICAgCiAgICAgICAgICAgIGlmIHNlbGYuX19jb2x1bW4gaXMgTm9uZSBhbmQgc2VsZi5fX2NvbHVtbk5hbWUgaXMgbm90IE5vbmUgYW5kIHNlbGYuTW9kZWwuc3RhdGUgPT0gIlJlYWR5IjoKICAgICAgICAgICAgICAgIG0gPSBzZWxmLk1vZGVsLkZpbmRNZWFzdXJlKHNlbGYuX19jb2x1bW5OYW1lKQogICAgICAgICAgICAgICAgaWYgbSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBzZWxmLl9fY29sdW1uID0gbQogICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICBzZWxmLkFkZElzc3VlKHR5cGU9SXNzdWVUeXBlLkludmFsaWRSZWZlcmVuY2UsIGRlc2NyaXB0aW9uPSJSZWZlcmVuY2UgY29sdW1uIGRvZXMgbm90IGV4aXN0IiwgZGV0YWlscz1mIkNhbiBub3QgZmluZCBjb2x1bW46IHtzZWxmLl9fY29sdW1uTmFtZX0iKQogICAgICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIHNlbGYuX19jb2x1bW4KICAgIAogICAgZGVmIEdldEFsbERlcGVuZGVuY2llcyhzZWxmKToKICAgICAgICBkZXBlbmRlbmNpZXMgPSBzdXBlcigpLkdldEFsbERlcGVuZGVuY2llcygpCiAgICAgICAgaWYgc2VsZi52YWx1ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgZGVwZW5kZW5jaWVzLmFkZChzZWxmLnZhbHVlKQogICAgICAgIHJldHVybiBkZXBlbmRlbmNpZXMKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1JlZmVyZW5jZShzZWxmKToKICAgICAgICByZXR1cm4gVHJ1ZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiByZXR1cm5UeXBlKHNlbGYpOgogICAgICAgIGZyb20gYmkuY29yZS50YWJsZSBpbXBvcnQgVGFibGUKICAgICAgICBpZiBzZWxmLlJlZkNvbHVtbiBpcyBub3QgTm9uZToKICAgICAgICAgICAgcmV0dXJuIHNlbGYuUmVmQ29sdW1uLnR5cGUKICAgICAgICBlbGlmIHNlbGYuUmVmVGFibGUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2Uoc2VsZi5SZWZUYWJsZSwgVGFibGUpOgogICAgICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLlRhYmxlCiAgICAgICAgICAgIGVsaWYgaXNpbnN0YW5jZShzZWxmLlJlZlRhYmxlLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgICAgIHJldHVybiBzZWxmLlJlZlRhYmxlLnR5cGUKICAgICAgICByZXR1cm4gRXhwcmVzc2lvblR5cGUuVW5rbm93bgogICAgICAgICAgICAKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgaWYgc2VsZi5fX3ZhbHVlIGlzIE5vbmU6CiAgICAgICAgICAgIGlmIHNlbGYuUmVmQ29sdW1uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgc2VsZi5fX3ZhbHVlID0gc2VsZi5SZWZDb2x1bW4KICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHNlbGYuX192YWx1ZSA9IHNlbGYuUmVmVGFibGUKICAgICAgICByZXR1cm4gc2VsZi5fX3ZhbHVlCiAgICAKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgaWYgc2VsZi52YWx1ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnZhbHVlLCBTZW1hbnRpY1RhYmxlKToKICAgICAgICAgICAgICAgIHJldHVybiBzZWxmLnZhbHVlLlNub3dmbGFrZU9iamVjdC5nZXRfZnFuKCkKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYudmFsdWUpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmV0dXJuICIiCiAgICAgICAgICAgICAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLmdldFNxbFZhbHVlKCkKCkRBWEV4cHJlc3Npb24ucmVnaXN0ZXIoREFYUmVmZXJlbmNlRXhwcmVzc2lvbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DirectConversion.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueSwgT3B0aW9uYWwKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvbG9va3VwdmFsdWUtZnVuY3Rpb24tZGF4CmNsYXNzIERpcmVjdENvbnZlcnNpb24oRnVuY3Rpb25IYW5kbGVyLCBBQkMpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIERBWEZ1bmN0aW9uTmFtZTogc3RyLCBTUUxGdW5jdGlvbk5hbWU6IHN0ciwgcmV0dXJuVHlwZTogRXhwcmVzc2lvblR5cGUsIHBhcmFtZXRlckNvdW50OiBpbnQgPSAwLCBzcWxQYXJhbWV0ZXJPcmRlcjogT3B0aW9uYWxbTGlzdFtpbnRdXSA9IE5vbmUpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18obmFtZT1EQVhGdW5jdGlvbk5hbWUsIHJldHVyblR5cGU9cmV0dXJuVHlwZSkgCgogICAgICAgIGlmIHNxbFBhcmFtZXRlck9yZGVyIGlzIG5vdCBOb25lOgogICAgICAgICAgICBpZiBsZW4oc3FsUGFyYW1ldGVyT3JkZXIpID4gcGFyYW1ldGVyQ291bnQ6CiAgICAgICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJzcWxQYXJhbWV0ZXJPcmRlciBoYXMgdG9vIG1hbnkgZW50cmllcyIpCiAgICAgICAgICAgIGVsaWYgbWF4KHNxbFBhcmFtZXRlck9yZGVyKSA+PSBwYXJhbWV0ZXJDb3VudDoKICAgICAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoInNxbCBwYXJhbWV0ZXIgb3JkZXIgY29udGFpbnMgYW4gaW5kZXggaGlnaGVyIHRoYW4gRGF4IFBhcmFtZXRlciBDb3VudCIpCiAgICAgICAgc2VsZi5fX3NxbEZ1bmN0aW9uTmFtZSA9IFNRTEZ1bmN0aW9uTmFtZQogICAgICAgIHNlbGYuX19wYXJhbWV0ZXJTaWduYXR1cmU6IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdID0gW10gCiAgICAgICAgc2VsZi5fX3NxbFBhcmFtZXRlck9yZGVyID0gc3FsUGFyYW1ldGVyT3JkZXIKICAgICAgICAKICAgICAgICBmb3IgaSBpbiByYW5nZShwYXJhbWV0ZXJDb3VudCk6CiAgICAgICAgICAgIHNlbGYuX19wYXJhbWV0ZXJTaWduYXR1cmUuYXBwZW5kKEZ1bmN0aW9uUGFyYW1ldGVyKGYicHtpKzF9IiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSkpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHNxbEZ1bmN0aW9uTmFtZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX3NxbEZ1bmN0aW9uTmFtZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gc2VsZi5fX3BhcmFtZXRlclNpZ25hdHVyZQogICAgICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0KICAgICAgICBwYXJhbXMgPSBbXQoKICAgICAgICBmb3IgcGFyYW1TaWcgaW4gc2VsZi5wb3NpdGlvbmFsUGFyYW1ldGVyczoKICAgICAgICAgICAgcGFyYW1zLmFwcGVuZChzZWxmLmdldFBhcmFtZXRlcihwYXJhbVNpZy5uYW1lKSkKICAgICAgICAgICAgCiAgICAgICAgaWYgc2VsZi5fX3NxbFBhcmFtZXRlck9yZGVyIGlzIG5vdCBOb25lOgogICAgICAgICAgICBvcmRlcmVkUGFyYW1zID0gW10KICAgICAgICAgICAgZm9yIGkgaW4gc2VsZi5fX3NxbFBhcmFtZXRlck9yZGVyOgogICAgICAgICAgICAgICAgb3JkZXJlZFBhcmFtcy5hcHBlbmQocGFyYW1zW2ldKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIG9yZGVyZWRQYXJhbXMgPSBwYXJhbXMKCiAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT1zZWxmLnNxbEZ1bmN0aW9uTmFtZSwgcGFyYW1ldGVycz1vcmRlcmVkUGFyYW1zKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKERpcmVjdENvbnZlcnNpb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXScriptExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgRGljdCwgVW5pb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuc2NyaXB0IGltcG9ydCBTY3JpcHQKZnJvbSAuREFYRXhwcmVzc2lvbiBpbXBvcnQgREFYRXhwcmVzc2lvbiwgREFYRXhwcmVzc2lvblR5cGUsIEV4cHJlc3Npb24KaW1wb3J0IHJlZ2V4CiAgICAKY2xhc3MgREFYU2NyaXB0RXhwcmVzc2lvbihEQVhFeHByZXNzaW9uLCBTY3JpcHQpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIGJvZHk6IHN0ciwgcmV0dXJuTmFtZTogc3RyLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgREFYRXhwcmVzc2lvbi5fX2luaXRfXyhzZWxmPXNlbGYsIHR5cGU9REFYRXhwcmVzc2lvblR5cGUuU2NyaXB0LCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgU2NyaXB0Ll9faW5pdF9fKHNlbGY9c2VsZikKICAgICAgICAKICAgICAgICB2YXJpYWJsZXNQYXR0ZXJuID0gciJccypWQVJccysoPzx2YXI+W2EtekEtWl9dW2EtekEtWjAtOV9dKylccys9XHMrKD88ZXhwPi4rPyg/PShcblZBUil8JCkpIgogICAgICAgIGZvciB2YXJNYXRjaCBpbiByZWdleC5maW5kaXRlcih2YXJpYWJsZXNQYXR0ZXJuLCBib2R5LCBmbGFncz1yZWdleC5ET1RBTEx8cmVnZXguSUdOT1JFQ0FTRXxyZWdleC5VKToKICAgICAgICAgICAgdmFyaWFibGVOYW1lID0gdmFyTWF0Y2guZ3JvdXAoInZhciIpCiAgICAgICAgICAgIHZhckV4cHJlc3Npb25TdHJpbmcgPSB2YXJNYXRjaC5ncm91cCgiZXhwIikKICAgICAgICAgICAgdmFyRXhwcmVzc2lvbiA9IHNlbGYuQWRkRXhwcmVzc2lvbih2YXJFeHByZXNzaW9uU3RyaW5nLCBwcm9wZXJ0aWVzPXt9KQogICAgICAgICAgICBzZWxmLkFkZFZhcmlhYmxlRXhwcmVzc2lvbih2YXJpYWJsZU5hbWU9dmFyaWFibGVOYW1lLCBleHByZXNzaW9uPXZhckV4cHJlc3Npb24pCgogICAgICAgIHZhbHVlRXhwcmVzc2lvbiA9IHNlbGYuZXhwcmVzc2lvbnMuZ2V0KHJldHVybk5hbWUpCiAgICAgICAgc2VsZi5TZXRWYWx1ZSh2YWx1ZUV4cHJlc3Npb24pCiAgICAgICAgICAgICAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGV4cHJlc3Npb25zKHNlbGYpOgogICAgICAgIHJldHVybiBTY3JpcHQuZXhwcmVzc2lvbnMuZmdldChzZWxmKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiByZXR1cm5UeXBlKHNlbGYpOgogICAgICAgIHJldHVybiBTY3JpcHQucmV0dXJuVHlwZS5mZ2V0KHNlbGYpCiAgICAKICAgIGRlZiBHZXRBbGxJc3N1ZXMoc2VsZik6CiAgICAgICAgYWxsSXNzdWVzID0gc2V0KHNlbGYuSXNzdWVzKQoKICAgICAgICBmb3Igayx2IGluIHNlbGYuZXhwcmVzc2lvbnMuaXRlbXMoKToKICAgICAgICAgICAgaWYgdiBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIGFsbElzc3VlcyA9IGFsbElzc3Vlcy51bmlvbih2LkdldEFsbElzc3VlcygpKQoKICAgICAgICByZXR1cm4gYWxsSXNzdWVzCiAgICAKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIHZhcmlhYmxlTmFtZTogc3RyKToKICAgICAgICBleHByZXNzaW9uID0gc2VsZi5nZXRWYXJpYWJsZSh2YXJpYWJsZU5hbWUpCiAgICAgICAgaWYgZXhwcmVzc2lvbiBpcyBOb25lOgogICAgICAgICAgICBleHByZXNzaW9uID0gIERBWEV4cHJlc3Npb24uZ2V0RXhwcmVzc2lvbihzZWxmLCB2YXJpYWJsZU5hbWUpCiAgICAgICAgcmV0dXJuIGV4cHJlc3Npb24KICAgICAgICAgICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpOgogICAgICAgIHJldHVybiBTY3JpcHQudmFsdWVFeHByZXNzaW9uLmZnZXQoc2VsZikKCiAgICBkZWYgX19zdHJfXyhzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSgpCiAgICAKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgcmV0dXJuIEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUoc2VsZi52YWx1ZSkKICAgIApEQVhFeHByZXNzaW9uLnJlZ2lzdGVyKERBWFNjcmlwdEV4cHJlc3Npb24pClNjcmlwdC5yZWdpc3RlcihEQVhTY3JpcHRFeHByZXNzaW9uKQoK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Unichar.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3VuaWNoYXItZnVuY3Rpb24tZGF4CiMgU25vd2ZsYWtlOiBDSFIoPGNvZGVfcG9pbnQ+KQpjbGFzcyBVbmlDaGFyKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIERBWEZ1bmN0aW9uTmFtZT0iVU5JQ0hBUiIsCiAgICAgICAgICAgIFNRTEZ1bmN0aW9uTmFtZT0iQ0hSIiwKICAgICAgICAgICAgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5UZXh0LAogICAgICAgICAgICBwYXJhbWV0ZXJDb3VudD0xCiAgICAgICAgKQoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihVbmlDaGFyKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Max.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L21heC1mdW5jdGlvbi1kYXgKY2xhc3MgTWF4KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiTUFYIiwgRXhwcmVzc2lvblR5cGUuQW55KSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbjFfb3JfY29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbjIiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUpCiAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWFJlZmVyZW5jZUV4cHJlc3Npb24gaW1wb3J0IERBWFJlZmVyZW5jZUV4cHJlc3Npb24KCiAgICAgICAgZTEgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJleHByZXNzaW9uMV9vcl9jb2x1bW5OYW1lIikKICAgICAgICBlMiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImV4cHJlc3Npb24yIikKICAgICAgICAKICAgICAgICBpZiBpc2luc3RhbmNlKGUxLCBUYWJsZUl0ZW0pOgogICAgICAgICAgICAjIGRpZmZlcmVudCBiZWhhdmlvciBiYXNlZCBvbiBvbmUgb3IgdHdvIHBhcmFtZXRlcnMKICAgICAgICAgICAgaWYgZTIgaXMgTm9uZToKICAgICAgICAgICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9Ik1BWCIsIHBhcmFtZXRlcnM9W2UxXSwgcmV0dXJuVHlwZT1lMS50eXBlKQogICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iR1JFQVRFU1RfSUdOT1JFX05VTExTIiwgcGFyYW1ldGVycz1bZTEsIGUyXSwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Gcm9tVmFsdWUoZTEpKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoImV4cGVjdGluZyBhIHJlZmVyZW5jZSB0byBhIFRhYmxlIEl0ZW0gKGNvbHVtbikiKQoKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKE1heCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Distinct.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxTZWxlY3RFeHByZXNzaW9uIGltcG9ydCBTUUxTZWxlY3RFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvZGlzdGluY3QtZnVuY3Rpb24tZGF4CmNsYXNzIERpc3RpbmN0KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiRElTVElOQ1QiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImNvbHVtbiIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIAogICAgICAgIGNvbHVtbiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImNvbHVtbiIpICAKICAgICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UoY29sdW1uLCBUYWJsZUl0ZW0pOgogICAgICAgICAgICByZXR1cm4gU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGNvbHVtbnM9W2NvbHVtbl0sIGZybT1jb2x1bW4udGFibGUsIGRpc3RpbmN0PVRydWUpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiRXhwZWN0ZWQgY29sdW1uIHR5cGUiKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKERpc3RpbmN0KQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/IsFiltered.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTFZhbHVlRXhwcmVzc2lvbiBpbXBvcnQgU1FMVmFsdWVFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvaXNmaWx0ZXJlZC1mdW5jdGlvbi1kYXgKY2xhc3MgSXNGaWx0ZXJlZChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIklTRklMVEVSRUQiLCBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsKSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiVGFibGVOYW1lT3JDb2x1bW5OYW1lIiwgRXhwcmVzc2lvblR5cGUuQ29sdW1uLCBGYWxzZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtIAoKICAgICAgICBUYWJsZU5hbWVPckNvbHVtbk5hbWUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJUYWJsZU5hbWVPckNvbHVtbk5hbWUiKSAgICAKICAgICAgICByZXR1cm4gU1FMVmFsdWVFeHByZXNzaW9uKHZhbHVlPUZhbHNlLCBwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24pCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoSXNGaWx0ZXJlZCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Left.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2xlZnQtZnVuY3Rpb24tZGF4CiMgaHR0cHM6Ly9kb2NzLnNub3dmbGFrZS5jb20vZW4vc3FsLXJlZmVyZW5jZS9mdW5jdGlvbnMvbGVmdApjbGFzcyBMZWZ0KERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIERBWEZ1bmN0aW9uTmFtZT0iTEVGVCIsCiAgICAgICAgICAgIFNRTEZ1bmN0aW9uTmFtZT0iTEVGVCIsCiAgICAgICAgICAgIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGV4dCwKICAgICAgICAgICAgcGFyYW1ldGVyQ291bnQ9MiwKICAgICAgICApCgpEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKExlZnQpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Values.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhSZWZlcmVuY2VFeHByZXNzaW9uIGltcG9ydCBEQVhSZWZlcmVuY2VFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC92YWx1ZXMtZnVuY3Rpb24tZGF4CmNsYXNzIFZhbHVlcyhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIlZBTFVFUyIsIEV4cHJlc3Npb25UeXBlLlRhYmxlKSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiVGFibGVOYW1lT3JDb2x1bW5OYW1lIiwgRXhwcmVzc2lvblR5cGUuQ29sdW1uLCBGYWxzZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZSAgICAgICAgCiAgICAgICAgc3FsID0gIiIKICAgICAgICBUYWJsZU5hbWVPckNvbHVtbk5hbWUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJUYWJsZU5hbWVPckNvbHVtbk5hbWUiKSAgICAKICAgICAgICBpZiBpc2luc3RhbmNlKFRhYmxlTmFtZU9yQ29sdW1uTmFtZSwgVGFibGVJdGVtKToKICAgICAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBjb2x1bW5zPVtUYWJsZU5hbWVPckNvbHVtbk5hbWVdLCBmcm09VGFibGVOYW1lT3JDb2x1bW5OYW1lLnRhYmxlLCBkaXN0aW5jdD1UcnVlKQogICAgICAgIGVsaWYgaXNpbnN0YW5jZShUYWJsZU5hbWVPckNvbHVtbk5hbWUsIFRhYmxlKToKICAgICAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmcm09VGFibGVOYW1lT3JDb2x1bW5OYW1lKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkV4cGVjdGVkIGVpdGhlciB0YWJsZSBpdGVtIG9yIHRhYmxlIikKICAgICAgICAKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFZhbHVlcykKICAgICAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Year.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3llYXItZnVuY3Rpb24tZGF4CmNsYXNzIFllYXIoRGlyZWN0Q29udmVyc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhEQVhGdW5jdGlvbk5hbWU9IllFQVIiLCBTUUxGdW5jdGlvbk5hbWU9IllFQVIiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwgcGFyYW1ldGVyQ291bnQ9MSkgIAoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihZZWFyKQogICAgCiAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Calculate.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24sIFNRTEV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYT3BlcmF0b3JFeHByZXNzaW9uIGltcG9ydCBEQVhPcGVyYXRvckV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBEQVhGdW5jdGlvbkV4cHJlc3Npb24sIERBWEZ1bmN0aW9uRmlsdGVyVHlwZQoKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC9jYWxjdWxhdGUtZnVuY3Rpb24tZGF4CmNsYXNzIENhbGN1bGF0ZShGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkNBTENVTEFURSIsIEV4cHJlc3Npb25UeXBlLkFueSkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImV4cHJlc3Npb24iLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwgCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImZpbHRlcjEiLCBFeHByZXNzaW9uVHlwZS5BbnksIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZmlsdGVyMiIsIEV4cHJlc3Npb25UeXBlLkFueSwgVHJ1ZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZ2V0UmV0dXJuVHlwZShzZWxmKToKICAgICAgICBleHByZXNzaW9uID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiZXhwcmVzc2lvbiIpCiAgICAgICAgaWYgaXNpbnN0YW5jZShleHByZXNzaW9uLCBFeHByZXNzaW9uKToKICAgICAgICAgICAgcmV0dXJuIGV4cHJlc3Npb24ucmV0dXJuVHlwZQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBFeHByZXNzaW9uVHlwZS5Gcm9tVmFsdWUoZXhwcmVzc2lvbikKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICByZXR1cm4gc2VsZi5fX2RvQ2FsY3VsYXRlKCkKICAgIAogICAgZGVmIF9fZG9DYWxjdWxhdGUoc2VsZikgLT4gU1FMU2VsZWN0RXhwcmVzc2lvbjoKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0gICAgICAgIAogICAgICAgIAogICAgICAgIGV4cHJlc3Npb24gPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJleHByZXNzaW9uIikKICAgICAgICBmaWx0ZXJzID0gW10KICAgICAgICBmb3IgaSBpbiByYW5nZSgxLGxlbihzZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbi5wYXJhbWV0ZXJzKSk6CiAgICAgICAgICAgIHZhbHVlID0gc2VsZi5nZXRQYXJhbWV0ZXJJbmRleFZhbHVlKGksIHJldHVyblZhbHVlPUZhbHNlKQogICAgICAgICAgICBpZiB2YWx1ZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIERBWEZ1bmN0aW9uRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgbWF0Y2ggdmFsdWUuZmlsdGVyVHlwZToKICAgICAgICAgICAgICAgICAgICAgICAgY2FzZSBEQVhGdW5jdGlvbkZpbHRlclR5cGUuTW9kaWZpZXI6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBpZiB2YWx1ZS5mdW5jdGlvbk5hbWUgPT0gIkFMTCI6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgcGFzcwogICAgICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXNzICMgY2FuJ3Qgc3VwcG9ydCB0aGlzCiAgICAgICAgICAgICAgICAgICAgICAgIGNhc2UgREFYRnVuY3Rpb25GaWx0ZXJUeXBlLkJvb2xlYW46CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBmaWx0ZXJzLmFwcGVuZCh2YWx1ZSkKICAgICAgICAgICAgICAgICAgICAgICAgY2FzZSBEQVhGdW5jdGlvbkZpbHRlclR5cGUuVGFibGU6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBwYXNzCiAgICAgICAgICAgICAgICAjIGlmIHRoZXkgYXJlIG9wZXJhdG9ycywgdGhlbiB0aGV5IGFyZSBmaWx0ZXJzCiAgICAgICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2UodmFsdWUsIERBWE9wZXJhdG9yRXhwcmVzc2lvbik6CiAgICAgICAgICAgICAgICAgICAgZmlsdGVycy5hcHBlbmQodmFsdWUpCiAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgIHBhc3MKCgogICAgICAgIAogICAgICAgIGNvbHVtbnMgPSBbXSAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbiwgVGFibGUpOgogICAgICAgICAgICBjb2x1bW5zID0gZXhwcmVzc2lvbi5jb2x1bW5zCiAgICAgICAgICAgIGZybSA9IGV4cHJlc3Npb24KICAgICAgICAgICAgcmV0dXJuVHlwZSA9IEV4cHJlc3Npb25UeXBlLlRhYmxlCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKGV4cHJlc3Npb24sIFRhYmxlSXRlbSk6CiAgICAgICAgICAgIGNvbHVtbnMuYXBwZW5kKGV4cHJlc3Npb24pCiAgICAgICAgICAgIHJldHVyblR5cGUgPSBleHByZXNzaW9uLnR5cGUKICAgICAgICAgICAgZnJtID0gZXhwcmVzc2lvbi50YWJsZQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoZiJQYXJhbWV0ZXIgdHlwZXMgd2VyZSBub3QgYXMgZXhwZWN0ZWQgKHRhYmxlIG9yIHRhYmxlSXRlbSk6IHtleHByZXNzaW9ufSIpCiAgICAgICAgCiAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCByZXR1cm5UeXBlPXJldHVyblR5cGUsIGNvbHVtbnM9Y29sdW1ucywgZnJtPWZybSwgY29uc3RyYWludHM9ZmlsdGVycykKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKENhbGN1bGF0ZSkgICAKICAgICAgICAKICAgICAgICA='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Rept.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvcmVwdC1mdW5jdGlvbi1kYXgKIyBodHRwczovL2RvY3Muc25vd2ZsYWtlLmNvbS9lbi9zcWwtcmVmZXJlbmNlL2Z1bmN0aW9ucy9yZXBlYXQKY2xhc3MgUmVwdChEaXJlY3RDb252ZXJzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKAogICAgICAgICAgICBEQVhGdW5jdGlvbk5hbWU9IlJFUFQiLAogICAgICAgICAgICBTUUxGdW5jdGlvbk5hbWU9IlJFUEVBVCIsCiAgICAgICAgICAgIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGV4dCwKICAgICAgICAgICAgcGFyYW1ldGVyQ291bnQ9MiwKICAgICAgICApCgpEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKFJlcHQpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/HasOneValue.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxTdGF0ZW1lbnRFeHByZXNzaW9uIGltcG9ydCBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvaGFzb25ldmFsdWUtZnVuY3Rpb24tZGF4CmNsYXNzIEhhc09uZVZhbHVlKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiSEFTT05FVkFMVUUiLCBFeHByZXNzaW9uVHlwZS5Mb2dpY2FsKSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKQogICAgICAgICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYUmVmZXJlbmNlRXhwcmVzc2lvbiBpbXBvcnQgREFYUmVmZXJlbmNlRXhwcmVzc2lvbgoKICAgICAgICBjb2x1bW5OYW1lID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY29sdW1uTmFtZSIpCgogICAgICAgIGlmIGlzaW5zdGFuY2UoY29sdW1uTmFtZSwgVGFibGVJdGVtKToKICAgICAgICAgICAgcmV0dXJuIFNRTFN0YXRlbWVudEV4cHJlc3Npb24oc3RhdGVtZW50PWYiQ09VTlQoe2NvbHVtbk5hbWUubmFtZX0pID0gMSIsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkV4cGVjdGluZyBjb2x1bW4iKQogICAgICAgIAogICAgICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoSGFzT25lVmFsdWUp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Second.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3NlY29uZC1mdW5jdGlvbi1kYXgKY2xhc3MgU2Vjb25kKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJTRUNPTkQiLCBTUUxGdW5jdGlvbk5hbWU9IlNFQ09ORCIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuTnVtYmVyLCBwYXJhbWV0ZXJDb3VudD0xKSAgCgpEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKFNlY29uZCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Abs.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EaXJlY3RDb252ZXJzaW9uIGltcG9ydCBEaXJlY3RDb252ZXJzaW9uCgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2Ficy1mdW5jdGlvbi1kYXgKIyBodHRwczovL2RvY3Muc25vd2ZsYWtlLmNvbS9lbi9zcWwtcmVmZXJlbmNlL2Z1bmN0aW9ucy9hYnMKY2xhc3MgQWJzKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIERBWEZ1bmN0aW9uTmFtZT0iQUJTIiwKICAgICAgICAgICAgU1FMRnVuY3Rpb25OYW1lPSJBQlMiLAogICAgICAgICAgICByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwKICAgICAgICAgICAgcGFyYW1ldGVyQ291bnQ9MSwKICAgICAgICApCgoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihBYnMpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/DateDiff.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmlzc3VlVHlwZSBpbXBvcnQgSXNzdWVUeXBlCmZyb20gYmkuc3FsLlNRTEZ1bmN0aW9uRXhwcmVzc2lvbiBpbXBvcnQgU1FMRnVuY3Rpb25FeHByZXNzaW9uCmltcG9ydCByZQoKCmNsYXNzIERhdGVEaWZmKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygKICAgICAgICAgICAgbmFtZT0iREFURURJRkYiLAogICAgICAgICAgICByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlcgogICAgICAgICkKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsKICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInN0YXJ0X2RhdGUiLCBFeHByZXNzaW9uVHlwZS5EYXRlLCBGYWxzZSksCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJlbmRfZGF0ZSIsIEV4cHJlc3Npb25UeXBlLkRhdGUsIEZhbHNlKSwKICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImludGVydmFsIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpCiAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gc3RyOgogICAgICAgICIiIgogICAgICAgIENvbnZlcnQgREFURURJRkYgREFYIHRvIFNRTCBEQVRFRElGRiBmdW5jdGlvbgogICAgICAgIAogICAgICAgIERBVEVESUZGIHN5bnRheDoKICAgICAgICBEQVRFRElGRihzdGFydF9kYXRlLCBlbmRfZGF0ZSwgaW50ZXJ2YWwpCiAgICAgICAgCiAgICAgICAgRXhhbXBsZXM6CiAgICAgICAgREFURURJRkYoRU9NT05USChUb2RheSgpLC0yKSwgVG9kYXkoKSwgREFZKQogICAgICAgIC0+IERBVEVESUZGKGRheSwgTEFTVF9EQVkoQ1VSUkVOVF9EQVRFKCkgLSBJTlRFUlZBTCAnMiBtb250aHMnKSwgQ1VSUkVOVF9EQVRFKCkpCiAgICAgICAgIiIiCiAgICAgICAgc3RhcnRfZGF0ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyKCJzdGFydF9kYXRlIikKICAgICAgICBlbmRfZGF0ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyKCJlbmRfZGF0ZSIpCiAgICAgICAgaW50ZXJ2YWwgPSBzZWxmLmdldFBhcmFtZXRlcigiaW50ZXJ2YWwiKQogICAgICAgIAogICAgICAgICMgR2VuZXJhdGUgU25vd2ZsYWtlIERBVEVESUZGCiAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iREFURURJRkYiLCBwYXJhbWV0ZXJzPVtpbnRlcnZhbCwgc3RhcnRfZGF0ZSwgZW5kX2RhdGVdKQogICAgICAgIAojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihEYXRlRGlmZikK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Divide.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2RpdmlkZS1mdW5jdGlvbi1kYXgKIyBOb3RlOiBEQVggRElWSURFKG51bWVyYXRvciwgZGVub21pbmF0b3IgWywgYWx0ZXJuYXRlcmVzdWx0XSkgaGFzIGFuIG9wdGlvbmFsIHRoaXJkIHBhcmFtZXRlcgojIGZvciBhbHRlcm5hdGUgcmVzdWx0IHdoZW4gZGl2aXNpb24gYnkgemVybyBvY2N1cnMuIEZvciBub3csIHdlIGlnbm9yZSB0aGUgdGhpcmQgcGFyYW1ldGVyCiMgYW5kIHVzZSBTbm93Zmxha2UncyBESVYwTlVMTCB3aGljaCByZXR1cm5zIDAgb24gZGl2aXNpb24gYnkgemVybyBvciBOVUxMLgpjbGFzcyBEaXZpZGUoRGlyZWN0Q29udmVyc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygKICAgICAgICAgICAgREFYRnVuY3Rpb25OYW1lPSJESVZJREUiLAogICAgICAgICAgICBTUUxGdW5jdGlvbk5hbWU9IkRJVjBOVUxMIiwKICAgICAgICAgICAgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIsCiAgICAgICAgICAgIHBhcmFtZXRlckNvdW50PTIKICAgICAgICApCgoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihEaXZpZGUpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Format.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2Zvcm1hdC1mdW5jdGlvbi1kYXgKY2xhc3MgRm9ybWF0KEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiRk9STUFUIiwgRXhwcmVzc2lvblR5cGUuVGV4dCkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBUcnVlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZm9ybWF0X3N0cmluZyIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJsb2NhbGVfbmFtZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpCiAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHZhbHVlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidmFsdWUiKQogICAgICAgIGZzID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiZm9ybWF0X3N0cmluZyIpCgogICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IlRPX1ZBUkNIQVIiLCBwYXJhbWV0ZXJzPVt2YWx1ZSwgZnNdKQogICAgCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihGb3JtYXQp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Right.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3JpZ2h0LWZ1bmN0aW9uLWRheAojIGh0dHBzOi8vZG9jcy5zbm93Zmxha2UuY29tL2VuL3NxbC1yZWZlcmVuY2UvZnVuY3Rpb25zL3JpZ2h0CmNsYXNzIFJpZ2h0KERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIERBWEZ1bmN0aW9uTmFtZT0iUklHSFQiLAogICAgICAgICAgICBTUUxGdW5jdGlvbk5hbWU9IlJJR0hUIiwKICAgICAgICAgICAgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5UZXh0LAogICAgICAgICAgICBwYXJhbWV0ZXJDb3VudD0yLAogICAgICAgICkKCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoUmlnaHQpCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/All.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC9hbGwtZnVuY3Rpb24tZGF4CmNsYXNzIEFsbChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkFMTCIsIEV4cHJlc3Npb25UeXBlLkxvZ2ljYWwpICMgCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSkgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbSAgICAgICAgCiAgICAgICAgCiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpIAogICAgICAgIGNvbHVtbnMgPSBbXQogICAgICAgIGZvciBpIGluIHJhbmdlKDEsbGVuKHNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnBhcmFtZXRlcnMpKToKICAgICAgICAgICAgdmFsdWUgPSBzZWxmLmdldFBhcmFtZXRlckluZGV4VmFsdWUoaSkKICAgICAgICAgICAgaWYgdmFsdWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBjb2x1bW5zLmFwcGVuZCh2YWx1ZSkKICAgICAgICAgICAgICAgIAogICAgICAgIGlmIGlzaW5zdGFuY2UodGFibGUsIERBWEV4cHJlc3Npb24pOgogICAgICAgICAgICB0YWJsZSA9IHRhYmxlLnZhbHVlCiAgICAgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGUpOgogICAgICAgICAgICByZXR1cm4gU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGNvbHVtbnM9dGFibGUuY29sdW1ucywgZnJtPXRhYmxlKQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh0YWJsZSwgVGFibGVJdGVtKToKICAgICAgICAgICAgY29sdW1ucyA9IFt0YWJsZV0KICAgICAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBjb2x1bW5zPWNvbHVtbnMsIGZybT10YWJsZSkKICAgICAgICAKICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJFeHBlY3RpbmcgdGFibGUgb3IgY29sdW1uIikKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihBbGwp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/DistinctCount.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2Rpc3RpbmN0Y291bnQtZnVuY3Rpb24tZGF4CmNsYXNzIERpc3RpbmN0Q291bnQoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJESVNUSU5DVENPVU5UIiwgRXhwcmVzc2lvblR5cGUuTnVtYmVyKSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiY29sdW1uIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgVHJ1ZSkgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCiAgICAgICAgCiAgICAgICAgY29sdW1uID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgiY29sdW1uIikgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZShjb2x1bW4sIFRhYmxlSXRlbSk6CiAgICAgICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IkNPVU5UIiwgcGFyYW1ldGVycz1bY29sdW1uXSwgYXR0cmlidXRlPSJESVNUSU5DVCIpCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiRXhwZWN0ZWQgY29sdW1uIHR5cGUiKQoKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKERpc3RpbmN0Q291bnQp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/CountRows.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRnVuY3Rpb25IYW5kbGVyLCBGdW5jdGlvblBhcmFtZXRlciwgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMU2VsZWN0RXhwcmVzc2lvbiBpbXBvcnQgU1FMU2VsZWN0RXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuY29sdW1uIGltcG9ydCBDb2x1bW4KCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC9jb3VudHJvd3MtZnVuY3Rpb24tZGF4CmNsYXNzIENvdW50Um93cyhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18obmFtZT0iQ09VTlRST1dTIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIpCgogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgIyBPbmUgcGFyYW1ldGVyOiBhIHRhYmxlIGV4cHJlc3Npb24KICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSkgXQoKICAgIGRlZiBldmFsdWF0ZShzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGUgaW1wb3J0IFRhYmxlCgogICAgICAgICMgV2Ugd2FudCB0aGUgZXhwcmVzc2lvbiBvYmplY3QsIG5vdCBhdXRvLXVud3JhcHBlZCB2YWx1ZQogICAgICAgIHRhYmxlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidGFibGUiKQoKICAgICAgICBpZiBpc2luc3RhbmNlKHRhYmxlLCBUYWJsZSk6CiAgICAgICAgICAgIGNvdW50RnVuY3Rpb24gPSBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IkNPVU5UIiwgcGFyYW1ldGVycz1bIioiXSkKICAgICAgICAgICAgY29sID0gQ29sdW1uKG5hbWU9IlJPV0NPVU5UIiwgZXhwcmVzc2lvbj1jb3VudEZ1bmN0aW9uLCB0eXBlPUV4cHJlc3Npb25UeXBlLk51bWJlcikKICAgICAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmcm09dGFibGUsIGNvbHM9W2NvbF0pCiAgICAgICAgZWxzZToKICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiQ09VTlRST1dTIHJlcXVpcmVzIGEgdGFibGUgcGFyYW1ldGVyIikKCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoQ291bnRSb3dzKQoK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/LookupValue.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvbG9va3VwdmFsdWUtZnVuY3Rpb24tZGF4CmNsYXNzIExvb2t1cFZhbHVlKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiTE9PS1VQVkFMVUUiLCBFeHByZXNzaW9uVHlwZS5BbnkpICMgCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbIEZ1bmN0aW9uUGFyYW1ldGVyKCJyZXN1bHRfY29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigic2VhcmNoX2NvbHVtbk5hbWUiLCBFeHByZXNzaW9uVHlwZS5UZXh0LCBGYWxzZSksCiAgICAgICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInNlYXJjaF92YWx1ZSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiYWx0ZXJuYXRlUmVzdWx0IiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpCiAgICAgICAgICAgICAgIF0KICAgIAogICAgZGVmIGdldFJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmVzdWx0ID0gc2VsZi5fX2RvTG9va3VwKCkKICAgICAgICByZXR1cm4gcmVzdWx0LnJldHVyblR5cGUKICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICByZXR1cm4gc2VsZi5fX2RvTG9va3VwKCkKICAgIAogICAgZGVmIF9fZG9Mb29rdXAoc2VsZik6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtICAgICAgICAKICAgICAgIAogICAgICAgIHJlc3VsdENvbHVtbiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInJlc3VsdF9jb2x1bW5OYW1lIikgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZShyZXN1bHRDb2x1bW4sIFRhYmxlSXRlbSk6CiAgICAgICAgICAgIHJldHVyblR5cGUgPSByZXN1bHRDb2x1bW4udHlwZQogICAgICAgICAgICBsb29rdXBUYWJsZSA9IHJlc3VsdENvbHVtbi50YWJsZQogICAgICAgICAgICAgICAKICAgICAgICAgICAgc2VhcmNoQ29sdW1uID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgic2VhcmNoX2NvbHVtbk5hbWUiKQogICAgICAgICAgICBzZWFyY2hWYWx1ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInNlYXJjaF92YWx1ZSIpCiAgICAgICAgICAgIAogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNlYXJjaENvbHVtbiwgVGFibGVJdGVtKSBhbmQgaXNpbnN0YW5jZShzZWFyY2hWYWx1ZSwgVGFibGVJdGVtKToKICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgY29sdW1uczogTGlzdFtDb2x1bW5dID0gW0NvbHVtbihuYW1lPXJlc3VsdENvbHVtbi5uYW1lLCB0eXBlPXJlc3VsdENvbHVtbi50eXBlLCBleHByZXNzaW9uPVNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT1mIkFOWV9WQUxVRSIsIHBhcmFtZXRlcnM9W3Jlc3VsdENvbHVtbl0sIHJldHVyblR5cGU9cmV0dXJuVHlwZSkpXQogICAgICAgICAgICAgICAgY29uc3RyYWludHM6IExpc3Rbc3RyXSA9IFtmIntzZWFyY2hDb2x1bW4uZnVsbE5hbWV9ID0ge3NlYXJjaFZhbHVlLmZ1bGxOYW1lfSJdCgogICAgICAgICAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmcm09bG9va3VwVGFibGUsIGNvbHVtbnM9Y29sdW1ucywgY29uc3RyYWludHM9Y29uc3RyYWludHMsIHJldHVyblR5cGU9cmV0dXJuVHlwZSkKICAgICAgICAgICAgCiAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiUGFyYW1ldGVyIHR5cGVzIHdlcmUgbm90IGFzIGV4cGVjdGVkIikKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihMb29rdXBWYWx1ZSk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Min.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L21pbi1mdW5jdGlvbi1kYXgKY2xhc3MgTWluKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygiTUlOIiwgRXhwcmVzc2lvblR5cGUuQW55KSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbjFfb3JfY29sdW1uTmFtZSIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbjIiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUpCiAgICAgICAgICAgICAgIF0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWFJlZmVyZW5jZUV4cHJlc3Npb24gaW1wb3J0IERBWFJlZmVyZW5jZUV4cHJlc3Npb24KCiAgICAgICAgZTEgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJleHByZXNzaW9uMV9vcl9jb2x1bW5OYW1lIikKICAgICAgICBlMiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImV4cHJlc3Npb24yIikKICAgICAgICAKICAgICAgICAjIGRpZmZlcmVudCBiZWhhdmlvciBiYXNlZCBvbiBvbmUgb3IgdHdvIHBhcmFtZXRlcnMKICAgICAgICBpZiBlMiBpcyBOb25lOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKGUxLCBEQVhSZWZlcmVuY2VFeHByZXNzaW9uKTogCiAgICAgICAgICAgICAgICBlMSA9IGUxLnZhbHVlCgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKGUxLCBUYWJsZUl0ZW0pOgogICAgICAgICAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iTUlOIiwgcGFyYW1ldGVycz1bZTFdLCByZXR1cm5UeXBlPWUxLnR5cGUpCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKCJleHBlY3RpbmcgYSByZWZlcmVuY2UgdG8gYSBUYWJsZSBJdGVtIChjb2x1bW4pIikKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gU1FMRnVuY3Rpb25FeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgZnVuY3Rpb25OYW1lPSJMRUFTVF9JR05PUkVfTlVMTFMiLCBwYXJhbWV0ZXJzPVtlMSwgZTJdLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLkZyb21WYWx1ZShlMSkpCgpGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoTWluKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Day.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2RheS1mdW5jdGlvbi1kYXgKY2xhc3MgRGF5KERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJEQVkiLCBTUUxGdW5jdGlvbk5hbWU9IkRBWSIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuTnVtYmVyLCBwYXJhbWV0ZXJDb3VudD0xKSAgCgpEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKERheSk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Today.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvdG9kYXktZnVuY3Rpb24tZGF4CmNsYXNzIFRvZGF5KERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJUT0RBWSIsIFNRTEZ1bmN0aW9uTmFtZT0iQ1VSUkVOVF9EQVRFIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5EYXRlLCBwYXJhbWV0ZXJDb3VudD0xKSAgCgojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoVG9kYXkpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Round.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EaXJlY3RDb252ZXJzaW9uIGltcG9ydCBEaXJlY3RDb252ZXJzaW9uCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvcm91bmQtZnVuY3Rpb24tZGF4CmNsYXNzIFJvdW5kKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJST1VORCIsIFNRTEZ1bmN0aW9uTmFtZT0iUk9VTkQiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwgcGFyYW1ldGVyQ291bnQ9MikgIAoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihSb3VuZCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/DatesBetween.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMU3RhdGVtZW50RXhwcmVzc2lvbiBpbXBvcnQgU1FMU3RhdGVtZW50RXhwcmVzc2lvbgppbXBvcnQgcmUKCgpjbGFzcyBEYXRlc0JldHdlZW4oRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKAogICAgICAgICAgICBuYW1lPSJEQVRFU0JFVFdFRU4iLAogICAgICAgICAgICByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLkxvZ2ljYWwKICAgICAgICApCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJkYXRlX2NvbHVtbiIsIEV4cHJlc3Npb25UeXBlLkNvbHVtbiwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigic3RhcnRfZGF0ZSIsIEV4cHJlc3Npb25UeXBlLkRhdGUsIEZhbHNlKSwKICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImVuZF9kYXRlIiwgRXhwcmVzc2lvblR5cGUuRGF0ZSwgRmFsc2UpCiAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gc3RyOgogICAgICAgICIiIgogICAgICAgIENvbnZlcnQgREFURVNCRVRXRUVOIERBWCB0byBTUUwgV0hFUkUgY2xhdXNlIHdpdGggZGF0ZSByYW5nZQogICAgICAgIAogICAgICAgIERBVEVTQkVUV0VFTiBzeW50YXg6CiAgICAgICAgREFURVNCRVRXRUVOKGRhdGVfY29sdW1uLCBzdGFydF9kYXRlLCBlbmRfZGF0ZSkKICAgICAgICAKICAgICAgICBFeGFtcGxlczoKICAgICAgICBEQVRFU0JFVFdFRU4odndEaW1EYXRlVXNhZ2VbRGF0ZV0sIEVPTU9OVEgoVE9EQVkoKSwtMiksIFRPREFZKCkpCiAgICAgICAgLT4gV0hFUkUgdndEaW1EYXRlVXNhZ2UuRGF0ZSBCRVRXRUVOIExBU1RfREFZKENVUlJFTlRfREFURSgpIC0gSU5URVJWQUwgJzInIG1vbnRoKSBBTkQgQ1VSUkVOVF9EQVRFKCkKICAgICAgICAKICAgICAgICBOb3RlOiBUaGlzIGZ1bmN0aW9uIGlzIHR5cGljYWxseSB1c2VkIHdpdGhpbiBDQUxDVUxBVEUoKSBmb3IgZmlsdGVyaW5nLAogICAgICAgIHNvIHdlIHJldHVybiBhIGZpbHRlciBjb25kaXRpb24gcmF0aGVyIHRoYW4gYSBjb21wbGV0ZSBTRUxFQ1Qgc3RhdGVtZW50LgogICAgICAgICIiIgogICAgICAgIAogICAgICAgIGRhdGVfY29sdW1uID0gc2VsZi5nZXRQYXJhbWV0ZXIoImRhdGVfY29sdW1uIikKICAgICAgICBzdGFydF9kYXRlID0gc2VsZi5nZXRQYXJhbWV0ZXIoInN0YXJ0X2RhdGUiKQogICAgICAgIGVuZF9kYXRlID0gc2VsZi5nZXRQYXJhbWV0ZXIoImVuZF9kYXRlIikKICAgICAgICAKICAgICAgICAjIEdlbmVyYXRlIFNRTCBmaWx0ZXIgY29uZGl0aW9uCiAgICAgICAgcmV0dXJuIFNRTFN0YXRlbWVudEV4cHJlc3Npb24oc3RhdGVtZW50PWYie2RhdGVfY29sdW1ufSBCRVRXRUVOIHtzdGFydF9kYXRlfSBBTkQge2VuZF9kYXRlfSIsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsKQogICAgICAgICAgICAKICAgICAgICAKCgojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihEYXRlc0JldHdlZW4pCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Hour.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2hvdXItZnVuY3Rpb24tZGF4CmNsYXNzIEhvdXIoRGlyZWN0Q29udmVyc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhEQVhGdW5jdGlvbk5hbWU9IkhPVVIiLCBTUUxGdW5jdGlvbk5hbWU9IkhPVVIiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwgcGFyYW1ldGVyQ291bnQ9MSkgIAoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihIb3VyKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Minute.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L21pbnV0ZS1mdW5jdGlvbi1kYXgKY2xhc3MgTWludXRlKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJNSU5VVEUiLCBTUUxGdW5jdGlvbk5hbWU9Ik1JTlVURSIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuTnVtYmVyLCBwYXJhbWV0ZXJDb3VudD0xKSAgCgpEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKE1pbnV0ZSk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/CountA.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2NvdW50YS1mdW5jdGlvbi1kYXgKY2xhc3MgQ291bnRBKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgICMgTWFwIERBWCBDT1VOVEEgdG8gU1FMIENPVU5ULCBzaW5nbGUgcGFyYW1ldGVyCiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhEQVhGdW5jdGlvbk5hbWU9IkNPVU5UQSIsIFNRTEZ1bmN0aW9uTmFtZT0iQ09VTlQiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwgcGFyYW1ldGVyQ291bnQ9MSkKCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoQ291bnRBKQoK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/IsBlank.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0KZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KZnJvbSBiaS5zcWwuU1FMU3RhdGVtZW50RXhwcmVzc2lvbiBpbXBvcnQgU1FMU3RhdGVtZW50RXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2lzYmxhbmstZnVuY3Rpb24tZGF4CiMgTm90ZTogSVNCTEFOSyBjaGVja3MgaWYgYSB2YWx1ZSBpcyBibGFuayAoTlVMTCBvciBlbXB0eSBzdHJpbmcgZm9yIHRleHQgY29sdW1ucyBpbiBEQVggY29udGV4dCkKIyBXZSBjb252ZXJ0IHRoaXMgdG8gU25vd2ZsYWtlJ3MgSUZGIHdpdGggYXBwcm9wcmlhdGUgY2hlY2tzIGJhc2VkIG9uIGRhdGEgdHlwZQpjbGFzcyBJc0JsYW5rKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhuYW1lPSJJU0JMQU5LIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbRnVuY3Rpb25QYXJhbWV0ZXIoInZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSldCgogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgoKICAgICAgICB2YWx1ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInZhbHVlIikKCiAgICAgICAgaWZudWxsID0gU1FMRnVuY3Rpb25FeHByZXNzaW9uKHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgZnVuY3Rpb25OYW1lPSJJRk5VTEwiLCBwYXJhbWV0ZXJzPVt2YWx1ZSwiJyciXSwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5UZXh0KQogICAgICAgIHJldHVybiBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uKHN0YXRlbWVudD1mIntpZm51bGwuZ2V0U3FsVmFsdWUoKX0gPSAnJyIsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsKQoKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihJc0JsYW5rKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/CombineValues.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBiaS5jb3JlLmZ1bmN0aW9uSGFuZGxlciBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2NvbWJpbmV2YWx1ZXMtZnVuY3Rpb24tZGF4CiMgU25vd2ZsYWtlIGVxdWl2YWxlbnQ6IENPTkNBVF9XUyhkZWxpbWl0ZXIsIGV4cHIxLCBleHByMiwgLi4uKQpjbGFzcyBDb21iaW5lVmFsdWVzKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIERBWEZ1bmN0aW9uTmFtZT0iQ09NQklORVZBTFVFUyIsCiAgICAgICAgICAgIFNRTEZ1bmN0aW9uTmFtZT0iQ09OQ0FUX1dTIiwKICAgICAgICAgICAgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5UZXh0LAogICAgICAgICAgICBwYXJhbWV0ZXJDb3VudD0zLAogICAgICAgICkKCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoQ29tYmluZVZhbHVlcykKCgo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/SelectColumns.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkuc3FsLlNRTFNlbGVjdEV4cHJlc3Npb24gaW1wb3J0IFNRTFNlbGVjdEV4cHJlc3Npb24KZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYUmVmZXJlbmNlRXhwcmVzc2lvbiBpbXBvcnQgREFYUmVmZXJlbmNlRXhwcmVzc2lvbgppbXBvcnQgcmUKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC9zZWxlY3Rjb2x1bW5zLWZ1bmN0aW9uLWRheApjbGFzcyBTZWxlY3RDb2x1bW5zKEZ1bmN0aW9uSGFuZGxlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXygKICAgICAgICAgICAgbmFtZT0iU0VMRUNUQ09MVU1OUyIsCiAgICAgICAgICAgIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGFibGUKICAgICAgICApCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZSIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBGYWxzZSksCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJuYW1lMSIsIEV4cHJlc3Npb25UeXBlLlRleHQsIEZhbHNlKSwKICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoImV4cHJlc3Npb24xIiwgRXhwcmVzc2lvblR5cGUuQW55LCBGYWxzZSksCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJuYW1lMiIsIEV4cHJlc3Npb25UeXBlLlRleHQsIFRydWUpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbjIiLCBFeHByZXNzaW9uVHlwZS5BbnksIFRydWUpCiAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gc3RyOgogICAgICAgICIiIgogICAgICAgIENvbnZlcnQgU0VMRUNUQ09MVU1OUyBEQVggdG8gU1FMIFNFTEVDVCBzdGF0ZW1lbnQKICAgICAgICAKICAgICAgICBTRUxFQ1RDT0xVTU5TIHN5bnRheDoKICAgICAgICBTRUxFQ1RDT0xVTU5TKHRhYmxlLCAibmFtZTEiLCBleHByZXNzaW9uMSwgIm5hbWUyIiwgZXhwcmVzc2lvbjIsIC4uLikKICAgICAgICAKICAgICAgICBFeGFtcGxlczoKICAgICAgICBTRUxFQ1RDT0xVTU5TKEdyb3VwcywgIkNhdGVnb3J5IiwgR3JvdXBzW0NhdGVnb3J5XSwgIkdyb3VwIiwgR3JvdXBzW0dyb3VwXSkKICAgICAgICAtPiBTRUxFQ1QgR3JvdXBzLkNhdGVnb3J5IEFTIENhdGVnb3J5LCBHcm91cHMuR3JvdXAgQVMgR3JvdXAgRlJPTSBHcm91cHMKICAgICAgICAiIiIKCiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlcigidGFibGUiKQogICAgICAgIGNvbHVtbnM6IExpc3RbQ29sdW1uXQogICAgICAgIGZvciBpIGluIHJhbmdlKDEsbGVuKHNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnBhcmFtZXRlcnMpKToKICAgICAgICAgICAgbmFtZSA9IHNlbGYuZ2V0UGFyYW1ldGVySW5kZXhWYWx1ZShpLCByZXR1cm5WYWx1ZT1UcnVlKQogICAgICAgICAgICBleHByZXNzaW9uID0gc2VsZi5nZXRQYXJhbWV0ZXJJbmRleFZhbHVlKGkrMSwgcmV0dXJuVmFsdWU9RmFsc2UpCiAgICAgICAgICAgIGlmIGlzaW5zdGFuY2UoZXhwcmVzc2lvbiwgREFYUmVmZXJlbmNlRXhwcmVzc2lvbikgYW5kIGV4cHJlc3Npb24uUmVmQ29sdW1uIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgY29sdW1ucy5hcHBlbmQoQ29sdW1uKG5hbWU9ZXhwcmVzc2lvbi5SZWZDb2x1bW4ubmFtZSwgYWxpYXM9bmFtZSwgdHlwZT1leHByZXNzaW9uLlJlZkNvbHVtbi50eXBlKSkKICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgIGNvbHVtbnMuYXBwZW5kKENvbHVtbihuYW1lPW5hbWUsIHR5cGU9RXhwcmVzc2lvblR5cGUuRnJvbVZhbHVlKGV4cHJlc3Npb24pLCBleHByZXNzaW9uPWV4cHJlc3Npb24pKQogICAgICAgICAgICBpICs9IDEKICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBjb2x1bW5zPWNvbHVtbnMsIGZybT10YWJsZSkKCgojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihTZWxlY3RDb2x1bW5zKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/EoMonth.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMRnVuY3Rpb25FeHByZXNzaW9uIGltcG9ydCBTUUxGdW5jdGlvbkV4cHJlc3Npb24KaW1wb3J0IHJlCgoKY2xhc3MgRW9Nb250aChGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oCiAgICAgICAgICAgIG5hbWU9IkVPTU9OVEgiLAogICAgICAgICAgICByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLkRhdGUKICAgICAgICApCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJzdGFydF9kYXRlIiwgRXhwcmVzc2lvblR5cGUuRGF0ZSwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigibW9udGhzIiwgRXhwcmVzc2lvblR5cGUuTnVtYmVyLCBGYWxzZSkKICAgICAgICBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBzdHI6CiAgICAgICAgIiIiCiAgICAgICAgQ29udmVydCBFT01PTlRIIERBWCB0byBTUUwgTEFTVF9EQVkgd2l0aCBkYXRlIGFyaXRobWV0aWMKICAgICAgICAKICAgICAgICBFT01PTlRIIHN5bnRheDoKICAgICAgICBFT01PTlRIKHN0YXJ0X2RhdGUsIG1vbnRocykKICAgICAgICAKICAgICAgICBFeGFtcGxlczoKICAgICAgICBFT01PTlRIKFRvZGF5KCksIC0yKQogICAgICAgIC0+IExBU1RfREFZKENVUlJFTlRfREFURSgpIC0gSU5URVJWQUwgJzInIG1vbnRoKQogICAgICAgIAogICAgICAgIEVPTU9OVEgoVGFibGVbRGF0ZUNvbHVtbl0sIDApCiAgICAgICAgLT4gTEFTVF9EQVkoVGFibGUuRGF0ZUNvbHVtbikKICAgICAgICAiIiIKCiAgICAgICAgc3RhcnRfZGF0ZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInN0YXJ0X2RhdGUiKSAgIAogICAgICAgIG1vbnRocyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoIm1vbnRocyIpIAoKICAgICAgICBpZiBtb250aHMgPT0gMDoKICAgICAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iTEFTVF9EQVkiLCBwYXJhbWV0ZXJzPVtzdGFydF9kYXRlXSkKICAgICAgICBlbHNlOgogICAgICAgICAgICBvZmZzZXRfZGF0ZSA9IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iREFURUFERCIsIHBhcmFtZXRlcnM9WyJNT05USCIsIG1vbnRocywgc3RhcnRfZGF0ZV0pCiAgICAgICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IkxBU1RfREFZIiwgcGFyYW1ldGVycz1bb2Zmc2V0X2RhdGVdKQogICAgICAgIAoKCiMgUmVnaXN0ZXIgdGhlIGZ1bmN0aW9uIGhhbmRsZXIKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKEVvTW9udGgpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Sum.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3N1bS1mdW5jdGlvbi1kYXgKY2xhc3MgU3VtKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJTVU0iLCBTUUxGdW5jdGlvbk5hbWU9IlNVTSIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuTnVtYmVyLCBwYXJhbWV0ZXJDb3VudD0xKSAgCgogICAgCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoU3VtKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/SelectedValue.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkuc3FsLlNRTFZhbHVlRXhwcmVzc2lvbiBpbXBvcnQgU1FMVmFsdWVFeHByZXNzaW9uCmZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCmltcG9ydCByZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3NlbGVjdGVkdmFsdWUtZnVuY3Rpb24tZGF4CmNsYXNzIFNlbGVjdGVkVmFsdWUoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKAogICAgICAgICAgICBuYW1lPSJTRUxFQ1RFRFZBTFVFIiwKICAgICAgICAgICAgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5BbnkKICAgICAgICApCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJjb2x1bW5OYW1lIiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiYWx0ZXJuYXRlUmVzdWx0IiwgRXhwcmVzc2lvblR5cGUuVGV4dCwgRmFsc2UpCiAgICAgICAgXQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gc3RyOgogICAgICAgIGNvbHVtbk5hbWUgPSBzZWxmLmdldFBhcmFtZXRlcigiY29sdW1uTmFtZSIpCiAgICAgICAgYWx0ZXJuYXRlUmVzdWx0ID0gc2VsZi5nZXRQYXJhbWV0ZXIoImFsdGVybmF0ZVJlc3VsdCIpCiAgICAgICAgCiAgICAgICAgaWYgaXNpbnN0YW5jZShhbHRlcm5hdGVSZXN1bHQsIEV4cHJlc3Npb24pOgogICAgICAgICAgICBhbHRlcm5hdGVSZXN1bHQgPSBhbHRlcm5hdGVSZXN1bHQudmFsdWUKICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIFNRTFZhbHVlRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIHZhbHVlPWFsdGVybmF0ZVJlc3VsdCwgcHJvcGVydGllcz17fSkKCgojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihTZWxlY3RlZFZhbHVlKQo='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/PreviousMonth.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxTZWxlY3RFeHByZXNzaW9uIGltcG9ydCBTUUxTZWxlY3RFeHByZXNzaW9uCmZyb20gYmkuY29yZS50YWJsZSBpbXBvcnQgVGFibGUKZnJvbSBiaS5jb3JlLnRhYmxlSXRlbSBpbXBvcnQgVGFibGVJdGVtCmZyb20gYmkuY29yZS5jb2x1bW4gaW1wb3J0IENvbHVtbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3ByZXZpb3VzbW9udGgtZnVuY3Rpb24tZGF4CmNsYXNzIFByZXZpb3VzTW9udGgoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJQUkVWSU9VU01PTlRIIiwgRXhwcmVzc2lvblR5cGUuVGFibGUpICMgCgogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoImRhdGVzIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSBdCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKToKICAgICAgICBkYXRlcyA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImRhdGVzIikKCiAgICAgICAgaWYgaXNpbnN0YW5jZShkYXRlcywgVGFibGVJdGVtKToKICAgICAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iREFURUFERCIsIHBhcmFtZXRlcnM9WyJNT05USCIsIC0xLCBkYXRlc10pCgogICAgICAgIGVsaWYgaXNpbnN0YW5jZShkYXRlcywgVGFibGUpOgogICAgICAgICAgICAjIHNob3VsZCBoYXZlIGEgc2luZ2xlIGNvbHVtbiBvZiB0eXBlIGRhdGUKICAgICAgICAgICAgY29sdW1uRXhwcmVzc2lvbiA9IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iREFURUFERCIsIHBhcmFtZXRlcnM9WyJNT05USCIsIC0xLCBkYXRlcy5jb2x1bW5zWzBdXSkKICAgICAgICAgICAgY29sdW1ucyA9IFtDb2x1bW4oIkRBVEVTIiwgdHlwZT1FeHByZXNzaW9uVHlwZS5EYXRlLCBleHByZXNzaW9uPWNvbHVtbkV4cHJlc3Npb24pXQogICAgICAgICAgICByZXR1cm4gU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGNvbHVtbnM9Y29sdW1ucywgZnJtPWRhdGVzKQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJhaXNlIFZhbHVlRXJyb3IoIkV4cGVjdGVkIGVpdGhlciBkYXRlIHJlZmVyZW5jZSBvciB0YWJsZSBvZiBkYXRlcyIpCiAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFByZXZpb3VzTW9udGgp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Currency.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxWYWx1ZUV4cHJlc3Npb24gaW1wb3J0IFNRTFZhbHVlRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2N1cnJlbmN5LWZ1bmN0aW9uLWRheApjbGFzcyBDdXJyZW5jeShGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIkNVUlJFTkNZIiwgRXhwcmVzc2lvblR5cGUuVGV4dCkgIyAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcG9zaXRpb25hbFBhcmFtZXRlcnMoc2VsZikgLT4gTGlzdFtGdW5jdGlvblBhcmFtZXRlcl06CiAgICAgICAgcmV0dXJuIFsgRnVuY3Rpb25QYXJhbWV0ZXIoInZhbHVlIiwgRXhwcmVzc2lvblR5cGUuQW55LCBUcnVlKV0KICAgIAogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIGZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCgogICAgICAgIHZhbHVlID0gc2VsZi5nZXRQYXJhbWV0ZXIoInZhbHVlIikKICAgICAgICBzcWw6IGZsb2F0ID0gTm9uZQogICAgICAgIGlmIGlzaW5zdGFuY2UodmFsdWUsIEV4cHJlc3Npb24pIGFuZCB2YWx1ZS5pc1ZhbHVlOgogICAgICAgICAgICB2ID0gdmFsdWUudmFsdWUKICAgICAgICBlbHNlOgogICAgICAgICAgICB2ID0gdmFsdWUKICAgICAgICAKICAgICAgICBpZiBpc2luc3RhbmNlKHYsIHN0cik6CiAgICAgICAgICAgIHJldHVybiBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IlRPX05VTUJFUiIsIHBhcmFtZXRlcnM9W3ZdKQogICAgICAgIGVsaWYgaXNpbnN0YW5jZSh2LCBib29sKToKICAgICAgICAgICAgaWYgdiA9PSBUcnVlOgogICAgICAgICAgICAgICAgc3FsID0gMS4wMDAwCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICBzcWwgPSAwLjAwMDAKICAgICAgICBlbGlmIGlzaW5zdGFuY2UodiwgaW50KToKICAgICAgICAgICAgc3FsID0gZmxvYXQoInt2fS4wMDAwMCIpCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHYsIGZsb2F0KToKICAgICAgICAgICAgc3FsID0gZmxvYXQoZiJ7djouNWZ9IikKICAgICAgICBlbHNlOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYiZXhwZWN0ZWQgZmxvYXQvc3RyL2Jvb2wvaW50IGJ1dCBnb3Qge3Z9IikKICAgICAgICAKICAgICAgICByZXR1cm4gU1FMVmFsdWVFeHByZXNzaW9uKHZhbHVlPXNxbCwgcGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uKQogICAgICAgIApGdW5jdGlvbkhhbmRsZXIucmVnaXN0ZXIoQ3VycmVuY3kp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Date.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2RhdGUtZnVuY3Rpb24tZGF4CmNsYXNzIERhdGUoRGlyZWN0Q29udmVyc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhEQVhGdW5jdGlvbk5hbWU9IkRBVEUiLCBTUUxGdW5jdGlvbk5hbWU9IkRBVEVfRlJPTV9QQVJUUyIsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuRGF0ZSwgcGFyYW1ldGVyQ291bnQ9MykgIAoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihEYXRlKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Int.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2ludC1mdW5jdGlvbi1kYXgKY2xhc3MgSW50KERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJJTlQiLCBTUUxGdW5jdGlvbk5hbWU9IkZMT09SIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIsIHBhcmFtZXRlckNvdW50PTEpICAKCkRpcmVjdENvbnZlcnNpb24ucmVnaXN0ZXIoSW50KQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Blank.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxWYWx1ZUV4cHJlc3Npb24gaW1wb3J0IFNRTFZhbHVlRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2JsYW5rLWZ1bmN0aW9uLWRheAojIE5vdGU6IEluIFBvd2VyIEJJLCBCTEFOSygpIGNhbiByZXByZXNlbnQgZWl0aGVyIGFuIGVtcHR5IHN0cmluZyBvciBOVUxMIGRlcGVuZGluZyBvbiBjb250ZXh0LgojIEZvciBTbm93Zmxha2UgU1FMIGNvbnZlcnNpb24sIHdlIHVzZSBOVUxMIGFzIHRoZSBlcXVpdmFsZW50LgpjbGFzcyBCbGFuayhGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18obmFtZT0iQkxBTksiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLkFueSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gW10gICMgQkxBTksoKSB0YWtlcyBubyBwYXJhbWV0ZXJzCgogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IEFueToKICAgICAgICByZXR1cm4gU1FMVmFsdWVFeHByZXNzaW9uKHZhbHVlPU5vbmUsIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbikKCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihCbGFuayk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/SumX.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxTZWxlY3RFeHByZXNzaW9uIGltcG9ydCBTUUxTZWxlY3RFeHByZXNzaW9uCmZyb20gYmkuc3FsLlNRTEZ1bmN0aW9uRXhwcmVzc2lvbiBpbXBvcnQgU1FMRnVuY3Rpb25FeHByZXNzaW9uCmZyb20gYmkuY29yZS5jb2x1bW4gaW1wb3J0IENvbHVtbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3N1bXgtZnVuY3Rpb24tZGF4CmNsYXNzIFN1bVgoRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKCJTVU1YIiwgRXhwcmVzc2lvblR5cGUuTnVtYmVyKSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigidGFibGUiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigiZXhwcmVzc2lvbiIsIEV4cHJlc3Npb25UeXBlLkFueSwgVHJ1ZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICAKICAgIGRlZiBldmFsdWF0ZShzZWxmKSAtPiBBbnk6CiAgICAgICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbQogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWFJlZmVyZW5jZUV4cHJlc3Npb24gaW1wb3J0IERBWFJlZmVyZW5jZUV4cHJlc3Npb24KCiAgICAgICAgdGFibGUgPSBzZWxmLmdldFBhcmFtZXRlclZhbHVlKCJ0YWJsZSIpCiAgICAgICAgZXhwcmVzc2lvbiA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoImV4cHJlc3Npb24iKQogICAgICAgIAogICAgICAgIHN1bUV4cHJlc3Npb24gPSBTUUxGdW5jdGlvbkV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBmdW5jdGlvbk5hbWU9IlNVTSIsIHBhcmFtZXRlcnM9W2V4cHJlc3Npb25dLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlcikKCiAgICAgICAgcmV0dXJuIFNRTFNlbGVjdEV4cHJlc3Npb24ocGFyZW50PXNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLCBjb2x1bW5zPVtDb2x1bW4obmFtZT0iU1VNIiwgZXhwcmVzc2lvbj1zdW1FeHByZXNzaW9uKV0sIGZybT10YWJsZSwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIpCiAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKFN1bVgp'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Month.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIC4uRGlyZWN0Q29udmVyc2lvbiBpbXBvcnQgRGlyZWN0Q29udmVyc2lvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCgojIGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvbW9udGgtZnVuY3Rpb24tZGF4CmNsYXNzIE1vbnRoKERpcmVjdENvbnZlcnNpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oREFYRnVuY3Rpb25OYW1lPSJNT05USCIsIFNRTEZ1bmN0aW9uTmFtZT0iTU9OVEgiLCByZXR1cm5UeXBlPUV4cHJlc3Npb25UeXBlLk51bWJlciwgcGFyYW1ldGVyQ291bnQ9MSkgIAoKRGlyZWN0Q29udmVyc2lvbi5yZWdpc3RlcihNb250aCk='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Count.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSAuLkRpcmVjdENvbnZlcnNpb24gaW1wb3J0IERpcmVjdENvbnZlcnNpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2NvdW50LWZ1bmN0aW9uLWRheAojIFNub3dmbGFrZTogQ09VTlQoPGV4cHJlc3Npb24+KSA8LSBkaWZmZXJlbnQgdGhhbiBEQVggd2hlcmUgaXQgY291bnRzIG9ubHkgbm9uLW51bGwgdmFsdWVzLgpjbGFzcyBDb3VudChEaXJlY3RDb252ZXJzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKERBWEZ1bmN0aW9uTmFtZT0iQ09VTlQiLCBTUUxGdW5jdGlvbk5hbWU9IkNPVU5UIiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5OdW1iZXIsIHBhcmFtZXRlckNvdW50PTEpCgogICAgZGVmIGV2YWx1YXRlKHNlbGYpOgogICAgICAgIHJldHVybiBzdXBlcigpLmV2YWx1YXRlKCkKICAgIApEaXJlY3RDb252ZXJzaW9uLnJlZ2lzdGVyKENvdW50KQoK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/Union.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QKZnJvbSBiaS5jb3JlLmZ1bmN0aW9uIGltcG9ydCBGdW5jdGlvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSBiaS5zcWwuU1FMU2VsZWN0RXhwcmVzc2lvbiBpbXBvcnQgU1FMU2VsZWN0RXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxTdGF0ZW1lbnRFeHByZXNzaW9uIGltcG9ydCBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uCmltcG9ydCByZQoKCiMgaHR0cHM6Ly9sZWFybi5taWNyb3NvZnQuY29tL2VuLXVzL2RheC91bmlvbi1mdW5jdGlvbi1kYXgKY2xhc3MgVW5pb24oRnVuY3Rpb25IYW5kbGVyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKAogICAgICAgICAgICBuYW1lPSJVTklPTiIsCiAgICAgICAgICAgIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuVGFibGUKICAgICAgICApCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHBvc2l0aW9uYWxQYXJhbWV0ZXJzKHNlbGYpIC0+IExpc3RbRnVuY3Rpb25QYXJhbWV0ZXJdOgogICAgICAgIHJldHVybiBbCiAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ0YWJsZTEiLCBFeHByZXNzaW9uVHlwZS5UYWJsZSwgRmFsc2UpLAogICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigidGFibGUyIiwgRXhwcmVzc2lvblR5cGUuVGFibGUsIEZhbHNlKSwKICAgICAgICAgICAgRnVuY3Rpb25QYXJhbWV0ZXIoInRhYmxlMyIsIEV4cHJlc3Npb25UeXBlLlRhYmxlLCBUcnVlKQogICAgICAgIF0KICAgIAogICAgZGVmIGV2YWx1YXRlKHNlbGYpIC0+IHN0cjoKICAgICAgICAiIiIKICAgICAgICBDb252ZXJ0IFVOSU9OIERBWCB0byBTUUwgVU5JT04gQUxMIHN0YXRlbWVudAogICAgICAgIAogICAgICAgIFVOSU9OIHN5bnRheDoKICAgICAgICBVTklPTih0YWJsZTEsIHRhYmxlMiwgLi4uKQogICAgICAgIAogICAgICAgIEV4YW1wbGVzOgogICAgICAgIFVOSU9OKFRhYmxlMSwgVGFibGUyKQogICAgICAgIC0+IFNFTEVDVCAqIEZST00gVGFibGUxIFVOSU9OIEFMTCBTRUxFQ1QgKiBGUk9NIFRhYmxlMgogICAgICAgIAogICAgICAgIFVOSU9OKFNFTEVDVENPTFVNTlMoVGFibGUxLCAiQ29sMSIsIFRhYmxlMVtBXSksIFNFTEVDVENPTFVNTlMoVGFibGUyLCAiQ29sMSIsIFRhYmxlMltCXSkpCiAgICAgICAgLT4gKFNFTEVDVCBUYWJsZTEuQSBBUyBDb2wxIEZST00gVGFibGUxKSBVTklPTiBBTEwgKFNFTEVDVCBUYWJsZTIuQiBBUyBDb2wxIEZST00gVGFibGUyKQogICAgICAgICIiIgogICAgICAgIHNlbGVjdFN0YXRlbWVudHMgPSBbXQogICAgICAgIGZvciBpIGluIHJhbmdlKDAsbGVuKHNlbGYuZnVuY3Rpb25FeHByZXNzaW9uLnBhcmFtZXRlcnMpKToKICAgICAgICAgICAgc2VsZWN0U3RhdGVtZW50cy5hcHBlbmQoU1FMU2VsZWN0RXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZybT1zZWxmLmdldFBhcmFtZXRlckluZGV4VmFsdWUoaSwgcmV0dXJuVmFsdWU9RmFsc2UpKSkKICAgICAgICAKICAgICAgICB1bmlvbiA9ICJVTklPTiBBTEwiLmpvaW4oc2VsZWN0U3RhdGVtZW50cykKICAgICAgICByZXR1cm4gU1FMU3RhdGVtZW50RXhwcmVzc2lvbihzdGF0ZW1lbnQ9dW5pb24sIHBhcmVudD1zZWxmLmZ1bmN0aW9uRXhwcmVzc2lvbiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5UYWJsZSkKICAgICAgICAKCgojIFJlZ2lzdGVyIHRoZSBmdW5jdGlvbiBoYW5kbGVyCkZ1bmN0aW9uSGFuZGxlci5yZWdpc3RlcihVbmlvbikK'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/functions/IF.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIEFueQpmcm9tIGJpLmNvcmUuZnVuY3Rpb25IYW5kbGVyIGltcG9ydCBGdW5jdGlvbkhhbmRsZXIsIEZ1bmN0aW9uUGFyYW1ldGVyLCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLnNxbC5TUUxGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IFNRTEZ1bmN0aW9uRXhwcmVzc2lvbgoKIyBodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L2lmLWZ1bmN0aW9uLWRheApjbGFzcyBJZihGdW5jdGlvbkhhbmRsZXIpOgogICAgZGVmIF9faW5pdF9fKHNlbGYpOgogICAgICAgIHN1cGVyKCkuX19pbml0X18oIklGIiwgRXhwcmVzc2lvblR5cGUuQW55KSAjIAogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBwb3NpdGlvbmFsUGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0Z1bmN0aW9uUGFyYW1ldGVyXToKICAgICAgICByZXR1cm4gWyBGdW5jdGlvblBhcmFtZXRlcigibG9naWNhbF90ZXN0IiwgRXhwcmVzc2lvblR5cGUuTG9naWNhbCwgRmFsc2UpLCAKICAgICAgICAgICAgICAgICBGdW5jdGlvblBhcmFtZXRlcigidmFsdWVfaWZfdHJ1ZSIsIEV4cHJlc3Npb25UeXBlLkFueSwgRmFsc2UpLAogICAgICAgICAgICAgICAgIEZ1bmN0aW9uUGFyYW1ldGVyKCJ2YWx1ZV9pZl9mYWxzZSIsIEV4cHJlc3Npb25UeXBlLkFueSwgVHJ1ZSkKICAgICAgICAgICAgICAgXQogICAgCiAgICBkZWYgZ2V0UmV0dXJuVHlwZShzZWxmKToKICAgICAgICB2YWx1ZV9pZl90cnVlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidmFsdWVfaWZfdHJ1ZSIpICAKICAgICAgICBpZiBpc2luc3RhbmNlKHZhbHVlX2lmX3RydWUsIEV4cHJlc3Npb24pOgogICAgICAgICAgICByZXR1cm4gdmFsdWVfaWZfdHJ1ZS5yZXR1cm5UeXBlCiAgICAgICAgZWxzZTogICAKICAgICAgICAgICAgcmV0dXJuIEV4cHJlc3Npb25UeXBlLkZyb21WYWx1ZSh2YWx1ZV9pZl90cnVlKQogICAgCiAgICBkZWYgZXZhbHVhdGUoc2VsZikgLT4gQW55OgogICAgICAgIHJldHVybiBzZWxmLl9fZG9JRigpCiAgICAKICAgIGRlZiBfX2RvSUYoc2VsZikgLT4gU1FMRnVuY3Rpb25FeHByZXNzaW9uOgogICAgICAgIGZyb20gYmkuY29yZS50YWJsZUl0ZW0gaW1wb3J0IFRhYmxlSXRlbSAgICAgICAgCiAgICAgICAgCiAgICAgICAgbG9naWNhbF90ZXN0ID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgibG9naWNhbF90ZXN0IikKICAgICAgICB2YWx1ZV9pZl90cnVlID0gc2VsZi5nZXRQYXJhbWV0ZXJWYWx1ZSgidmFsdWVfaWZfdHJ1ZSIpICAgICAKICAgICAgICB2YWx1ZV9pZl9mYWxzZSA9IHNlbGYuZ2V0UGFyYW1ldGVyVmFsdWUoInZhbHVlX2lmX2ZhbHNlIikKICAgICAgICByZXR1cm5UeXBlID0gc2VsZi5nZXRSZXR1cm5UeXBlKCkKICAgICAgICAgICAgCiAgICAgICAgcmV0dXJuIFNRTEZ1bmN0aW9uRXhwcmVzc2lvbihwYXJlbnQ9c2VsZi5mdW5jdGlvbkV4cHJlc3Npb24sIGZ1bmN0aW9uTmFtZT0iSUZGIiwgcGFyYW1ldGVycz1bbG9naWNhbF90ZXN0LCB2YWx1ZV9pZl90cnVlLCB2YWx1ZV9pZl9mYWxzZV0sIHJldHVyblR5cGU9cmV0dXJuVHlwZSkKICAgICAgICAKICAgICAgICAgICAgCiAgICAgICAgcmFpc2UgVmFsdWVFcnJvcigiUGFyYW1ldGVyIHR5cGVzIHdlcmUgbm90IGFzIGV4cGVjdGVkIikKICAgICAgICAKRnVuY3Rpb25IYW5kbGVyLnJlZ2lzdGVyKElmKQ=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXExpressionParser.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbApmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudCwgRXhwcmVzc2lvbkNvZGVUeXBlCmZyb20gYmkuY29yZS5leHByZXNzaW9uUGFyc2VyIGltcG9ydCBFeHByZXNzaW9uUGFyc2VyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKCmltcG9ydCByZWdleAoKCmNsYXNzIERBWEV4cHJlc3Npb25QYXJzZXIoRXhwcmVzc2lvblBhcnNlcik6CiAgICBkZWYgX19pbml0X18oc2VsZik6CiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhjb2RlVHlwZT1FeHByZXNzaW9uQ29kZVR5cGUuREFYKQogICAgICAgICMgYWRkIHN1cHBvcnQgZm9yIHRoZSBTUUwgY29tbWVudCBjaGFyYWN0ZXIKICAgICAgICBzZWxmLlNpbmdsZUxpbmVDb21tZW50LmFwcGVuZCgiLS0iKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIEJvdW5kYXJ5KHNlbGYpOgogICAgICAgIHJldHVybiB7IigiOiIpIiwieyI6In0iLCJbIjoiXSJ9CiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIE9wZXJhdG9ycyhzZWxmKToKICAgICAgICByZXR1cm4gWyIrIiwiLSIsIi8iLCIqIiwiPSIsIl4iLCI+IiwiPCIsIiYiLCI9PSIsIj49IiwiPD0iLCI8PiIsIiYmIiwifHwiLCIgSU4gIiwiIGluICIsIiBOT1QiLCJSRVRVUk4iXQogICAgCiAgICBkZWYgQ3JlYXRlRnJvbVN0cmluZyhzZWxmLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIGV4cHJlc3Npb25TdHJpbmc6IHN0ciwgcmV0dXJuVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvblR5cGVdID0gRXhwcmVzc2lvblR5cGUuQW55LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIAogICAgICAgICMgZmlyc3Qgd2UgbmVlZCB0byBzb21lIGNsZWFudXAgb2YgdGhlIHN0cmluZyAKICAgICAgICBpZiBpc2luc3RhbmNlKGV4cHJlc3Npb25TdHJpbmcsIGxpc3QpOgogICAgICAgICAgICBlID0gIiIKICAgICAgICAgICAgZm9yIGwgaW4gZXhwcmVzc2lvblN0cmluZzoKICAgICAgICAgICAgICAgIGUgKz0gZiJ7bH1cbiIKICAgICAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9IGUKICAgICAgICAKICAgICAgICBpZiAib3JpZ2luYWwiIG5vdCBpbiBwcm9wZXJ0aWVzOgogICAgICAgICAgICBwcm9wZXJ0aWVzWyJvcmlnaW5hbCJdID0gZXhwcmVzc2lvblN0cmluZwoKICAgICAgICBleHByZXNzaW9uU3RyaW5nID0gZXhwcmVzc2lvblN0cmluZy5zdHJpcCgpCiAgICAgICAgCiAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9IHNlbGYuUmVtb3ZlQ29tbWVudHMoZXhwcmVzc2lvbj1leHByZXNzaW9uU3RyaW5nKQoKICAgICAgICAjIHJlbW92ZSBhbnkgb3V0ZXIgcGFyZW50aGVzaXMgKHRoZXkgYXJlIG5vdCBuZWVkZWQpCiAgICAgICAgZXhwcmVzc2lvblN0cmluZyA9IHNlbGYuUmVtb3ZlT3V0ZXJQYXJlbnRoZXNpcyhleHByZXNzaW9uU3RyaW5nKQoKICAgICAgICAjIGRldGVybWluZSBpZiB0aGVyZSBhcmUgYW55IG9wZXJhdG9ycyBpbiB0aGUgdG9wIGxldmVsIG9mIHRoZSBleHByZXNzaW9uIHN0cmluZyAoZGVmaW5lcyBhIHNldCBvZiBvcGVyYW5kcyBhbmQgb3BlcmF0b3JzIC0gZWFjaCBvcGVyYW5kIGlzIGEgc2VwZXJhdGUgZXhwcmVzc2lvbikKICAgICAgICBvcGVyYXRvckluZm8gPSBzZWxmLlBhcnNlRXhwcmVzc2lvbk9wZXJhdGlvbnMoZXhwcmVzc2lvblN0cmluZykKICAgICAgICAKICAgICAgICBleHAgPSBzZWxmLkNyZWF0ZShwYXJlbnQ9cGFyZW50LCBpbmZvPW9wZXJhdG9ySW5mbywgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIGlmIGV4cCBpcyBOb25lOgogICAgICAgICAgICByYWlzZSBWYWx1ZUVycm9yKGYiSW52YWxpZCBEQVggRXhwcmVzc2lvbjoge2V4cHJlc3Npb25TdHJpbmd9IikKICAgICAgICByZXR1cm4gZXhwCiAgICAKICAgIGRlZiBDcmVhdGVGcm9tVmFsdWUoc2VsZiwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCB2YWx1ZTogQW55LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWFZhbHVlRXhwcmVzc2lvbiBpbXBvcnQgREFYVmFsdWVFeHByZXNzaW9uCiAgICAgICAgaWYgIm9yaWdpbmFsIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgcHJvcGVydGllc1sib3JpZ2luYWwiXSA9IHN0cih2YWx1ZSkKCiAgICAgICAgcmV0dXJuIERBWFZhbHVlRXhwcmVzc2lvbih2YWx1ZT12YWx1ZSwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgCiAgICBkZWYgQ3JlYXRlKHNlbGYsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgaW5mbzogZGljdCwgcHJvcGVydGllczogZGljdCA9IHt9KToKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhGdW5jdGlvbkV4cHJlc3Npb24gaW1wb3J0IERBWEZ1bmN0aW9uRXhwcmVzc2lvbgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWE9wZXJhdG9yRXhwcmVzc2lvbiBpbXBvcnQgREFYT3BlcmF0b3JFeHByZXNzaW9uCiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYUmVmZXJlbmNlRXhwcmVzc2lvbiBpbXBvcnQgREFYUmVmZXJlbmNlRXhwcmVzc2lvbgogICAgICAgIGZyb20gYmkucmVhZGVycy5wYmkuZGF4LkRBWFZhbHVlRXhwcmVzc2lvbiBpbXBvcnQgREFYVmFsdWVFeHByZXNzaW9uCiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5kYXguREFYVmFyaWFibGVFeHByZXNzaW9uIGltcG9ydCBEQVhWYXJpYWJsZUV4cHJlc3Npb24KICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhTY3JpcHRFeHByZXNzaW9uIGltcG9ydCBEQVhTY3JpcHRFeHByZXNzaW9uCgogICAgICAgIG9wZXJhdG9ycyA9IGluZm8uZ2V0KCJvcGVyYXRvcnMiKSBvciBbXQogICAgICAgIG9wZXJhbmRzID0gaW5mby5nZXQoIm9wZXJhbmRzIikgb3IgW10KICAgICAgICBpZiAib3JpZ2luYWwiIG5vdCBpbiBwcm9wZXJ0aWVzOgogICAgICAgICAgICBwcm9wZXJ0aWVzWyJvcmlnaW5hbCJdID0gaW5mby5nZXQoIm9yaWdpbmFsIikuc3RyaXAoKQoKICAgICAgICBpZiBsZW4ob3BlcmFuZHMpID09IDE6CiAgICAgICAgICAgIGV4cHJlc3Npb25TdHJpbmc6IHN0ciA9IG9wZXJhbmRzWzBdLnN0cmlwKCkKICAgICAgICBlbHNlOgogICAgICAgICAgICBleHByZXNzaW9uU3RyaW5nOiBzdHIgPSBpbmZvLmdldCgib3JpZ2luYWwiKS5zdHJpcCgpCgogICAgICAgIHNjcmlwdFBhdHRlcm4gPSByIl4oPzxib2R5Pi4rKVxzK1JFVFVSTlxzKyg/PHJldHVybj5bYS16QS1aX11bYS16QS1aMC05X10rKVxzKiQiCiAgICAgICAgc2NyaXB0UGF0dGVybk1hdGNoID0gcmVnZXgubWF0Y2goc2NyaXB0UGF0dGVybiwgZXhwcmVzc2lvblN0cmluZywgcmVnZXguSUdOT1JFQ0FTRXxyZWdleC5ET1RBTEwpCiAgICAgICAgaWYgc2NyaXB0UGF0dGVybk1hdGNoIGlzIG5vdCBOb25lOgogICAgICAgICAgICBib2R5ID0gc2NyaXB0UGF0dGVybk1hdGNoLmdyb3VwKCJib2R5IikKICAgICAgICAgICAgcmV0VmFyID0gc2NyaXB0UGF0dGVybk1hdGNoLmdyb3VwKCJyZXR1cm4iKQogICAgICAgICAgICByZXR1cm4gREFYU2NyaXB0RXhwcmVzc2lvbihib2R5PWJvZHksIHJldHVybk5hbWU9cmV0VmFyLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgZWxzZToKICAgICAgICAKICAgICAgICAgICAgdmFyaWFibGVQYXR0ZXJuID0gciJccypWQVJccysoW2EtekEtWl9dW2EtekEtWjAtOV9dKylccyo9XHMqKC4rKVteKFZBUiApKFJFVFVSTiApXSQiCiAgICAgICAgICAgIHZhcmlhYmxlUGF0dGVybk1hdGNoID0gcmVnZXgubWF0Y2godmFyaWFibGVQYXR0ZXJuLCBleHByZXNzaW9uU3RyaW5nLCByZWdleC5JR05PUkVDQVNFfHJlZ2V4LkRPVEFMTCkKCiAgICAgICAgICAgIGlmIHZhcmlhYmxlUGF0dGVybk1hdGNoIGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgbmFtZSA9IHZhcmlhYmxlUGF0dGVybk1hdGNoLmdyb3VwKDEpLnN0cmlwKCkKICAgICAgICAgICAgICAgIGVTdHJpbmcgPSB2YXJpYWJsZVBhdHRlcm5NYXRjaC5ncm91cCgyKS5zdHJpcCgpCiAgICAgICAgICAgICAgICByZXR1cm4gREFYVmFyaWFibGVFeHByZXNzaW9uKG5hbWU9bmFtZSwgZXhwcmVzc2lvblN0cmluZz1lU3RyaW5nLCBwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgICAgIAogICAgICAgICAgICBlbGlmIGxlbihvcGVyYXRvcnMpID4gMDoKICAgICAgICAgICAgICAgIHJldHVybiBEQVhPcGVyYXRvckV4cHJlc3Npb24ob3BlcmFuZHM9aW5mby5nZXQoIm9wZXJhbmRzIiksIG9wZXJhdG9ycz1pbmZvLmdldCgib3BlcmF0b3JzIiksIHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICAgICAgZWxpZiBleHByZXNzaW9uU3RyaW5nLnN0YXJ0c3dpdGgoInsiKSBhbmQgZXhwcmVzc2lvblN0cmluZy5lbmRzd2l0aCgifSIpOgogICAgICAgICAgICAgICAgIyBTZWU6IGh0dHBzOi8vbGVhcm4ubWljcm9zb2Z0LmNvbS9lbi11cy9kYXgvdGFibGUtY29uc3RydWN0b3IgCiAgICAgICAgICAgICAgICB0YWJsZUNvbnRlbnRzID0gZXhwcmVzc2lvblN0cmluZ1sxOmxlbihleHByZXNzaW9uU3RyaW5nKS0xXS5zdHJpcCgpCiAgICAgICAgICAgICAgICByb3dzID0gW10KICAgICAgICAgICAgICAgIGZvciB2IGluIHNlbGYuUGFyc2VFeHByZXNzaW9uU3RyaW5nKHRhYmxlQ29udGVudHMpOgogICAgICAgICAgICAgICAgICAgIHYgPSB2LnN0cmlwKCkKICAgICAgICAgICAgICAgICAgICBpZiB2LnN0YXJ0c3dpdGgoIigiKSBhbmQgdi5lbmRzd2l0aCgiKSIpOgogICAgICAgICAgICAgICAgICAgICAgICByb3cgPSBbXQogICAgICAgICAgICAgICAgICAgICAgICByb3dDb250ZW50cyA9IHZbMTpsZW4odiktMV0uc3RyaXAoKQogICAgICAgICAgICAgICAgICAgICAgICBmb3IgYyBpbiBzZWxmLlBhcnNlRXhwcmVzc2lvblN0cmluZyhyb3dDb250ZW50cyk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICByb3cuYXBwZW5kKGMpCiAgICAgICAgICAgICAgICAgICAgICAgIHJvd3MuYXBwZW5kKHJvdykKICAgICAgICAgICAgICAgICAgICBlbHNlOgogICAgICAgICAgICAgICAgICAgICAgICByb3dzLmFwcGVuZCh2KQoKICAgICAgICAgICAgICAgIHJldHVybiBzZWxmLkNyZWF0ZUZyb21WYWx1ZShwYXJlbnQ9cGFyZW50LCB2YWx1ZT1yb3dzLCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICByZWZlcmVuY2VQYXR0ZXJuID0gciJeJz8oW0EtWmEtel9dW15cW1woLF0rKT8nPyhcW1teXFtcKCxdK1xdKT8kIiAjciIoXHcpezAsMX0oXFsoXHcpXF0pezAsMX0iCiAgICAgICAgICAgICAgICByZWZlcmVuY2VNYXRjaCA9IHJlZ2V4Lm1hdGNoKHJlZmVyZW5jZVBhdHRlcm4sIGV4cHJlc3Npb25TdHJpbmcucmVwbGFjZSgiXG4iLCIgIikpCiAgICAgICAgICAgICAgICBpZiByZWZlcmVuY2VNYXRjaCBpcyBub3QgTm9uZSBvciBleHByZXNzaW9uU3RyaW5nLmVuZHN3aXRoKCJdIik6CiAgICAgICAgICAgICAgICAgICAgcmV0dXJuIERBWFJlZmVyZW5jZUV4cHJlc3Npb24odmFsdWU9ZXhwcmVzc2lvblN0cmluZywgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICBmdW5jdGlvbk1hdGNoID0gcmVnZXgubWF0Y2gociJccyooW0EtWmEtel9dKylccypcKCguKilcKSQiLCBleHByZXNzaW9uU3RyaW5nLnJlcGxhY2UoIlxuIiwiICIpLnN0cmlwKCkpCiAgICAgICAgICAgICAgICAgICAgaWYgZnVuY3Rpb25NYXRjaCBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICAgICAgZnVuY3Rpb25OYW1lID0gZnVuY3Rpb25NYXRjaC5ncm91cCgxKS5zdHJpcCgpCiAgICAgICAgICAgICAgICAgICAgICAgIHBhcmFtZXRlcnMgPSBmdW5jdGlvbk1hdGNoLmdyb3VwKDIpLnN0cmlwKCkKICAgICAgICAgICAgICAgICAgICAgICAgcmV0dXJuIERBWEZ1bmN0aW9uRXhwcmVzc2lvbihmdW5jdGlvbk5hbWU9ZnVuY3Rpb25OYW1lLCBwYXJhbWV0ZXJTdHJpbmc9cGFyYW1ldGVycywgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgICAgICAgICAgICAgIGVsc2U6CiAgICAgICAgICAgICAgICAgICAgICAgIGlmIGV4cHJlc3Npb25TdHJpbmcuc3RhcnRzd2l0aCgnIicpIGFuZCBleHByZXNzaW9uU3RyaW5nLmVuZHN3aXRoKCciJyk6CiAgICAgICAgICAgICAgICAgICAgICAgICAgICB2YWx1ZSA9IGV4cHJlc3Npb25TdHJpbmcKICAgICAgICAgICAgICAgICAgICAgICAgZWxzZToKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB2YWx1ZSA9IGludChleHByZXNzaW9uU3RyaW5nKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgZXhjZXB0OgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHRyeToKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgdmFsdWUgPSBmbG9hdChleHByZXNzaW9uU3RyaW5nKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIGV4Y2VwdDoKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgcmFpc2UgVmFsdWVFcnJvcihmIlVuc3VwcG9ydGVkIERBWCBleHByZXNzaW9uOiB7ZXhwcmVzc2lvblN0cmluZ30iKQogICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgIHJldHVybiBzZWxmLkNyZWF0ZUZyb21WYWx1ZShwYXJlbnQ9cGFyZW50LCB2YWx1ZT12YWx1ZSwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQoKCkV4cHJlc3Npb25QYXJzZXIucmVnaXN0ZXIoREFYRXhwcmVzc2lvblBhcnNlcik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXVariableExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uLCBEQVhFeHByZXNzaW9uVHlwZSwgRXhwcmVzc2lvbgogICAgCmNsYXNzIERBWFZhcmlhYmxlRXhwcmVzc2lvbihEQVhFeHByZXNzaW9uKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBuYW1lOiBzdHIsIGV4cHJlc3Npb25TdHJpbmc6IHN0ciwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIHN1cGVyKCkuX19pbml0X18odHlwZT1EQVhFeHByZXNzaW9uVHlwZS5WYXJpYWJsZSwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIHNlbGYuX19uYW1lID0gbmFtZQogICAgICAgIHNlbGYuX19leHByZXNzaW9uID0gc2VsZi5BZGRFeHByZXNzaW9uKGV4cHJlc3Npb25TdHJpbmc9ZXhwcmVzc2lvblN0cmluZywgcHJvcGVydGllcz17fSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBuYW1lKHNlbGYpIC0+IHN0cjoKICAgICAgICByZXR1cm4gc2VsZi5fX25hbWUKICAgIAogICAgZGVmIGdldEV4cHJlc3Npb24oc2VsZiwgdmFyaWFibGVOYW1lOiBzdHIpOgogICAgICAgIGlmIHZhcmlhYmxlTmFtZSA9PSBzZWxmLm5hbWU6CiAgICAgICAgICAgIHJldHVybiBzZWxmLl9fZXhwcmVzc2lvbgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBzdXBlcigpLmdldEV4cHJlc3Npb24odmFyaWFibGVOYW1lKQogICAgICAgICAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19leHByZXNzaW9uLnJldHVyblR5cGUKICAgIAogICAgZGVmIHVwZGF0ZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5fX2V4cHJlc3Npb24KCiAgICBkZWYgZ2V0U3FsKHNlbGYpOgogICAgICAgIHJldHVybiBFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYudmFsdWUpCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLmdldFNxbFZhbHVlKCkKICAgIApEQVhFeHByZXNzaW9uLnJlZ2lzdGVyKERBWFZhcmlhYmxlRXhwcmVzc2lvbikKCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/dax/DAXFunctionExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgTGlzdCwgVW5pb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnJlYWRlcnMucGJpLmRheC5EQVhFeHByZXNzaW9uIGltcG9ydCBEQVhFeHByZXNzaW9uLCBEQVhFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCmZyb20gYmkuY29yZS5pc3N1ZVR5cGUgaW1wb3J0IElzc3VlVHlwZQpmcm9tIGVudW0gaW1wb3J0IEVudW0KCmNsYXNzIERBWEZ1bmN0aW9uRmlsdGVyVHlwZShFbnVtKToKICAgIFVua25vd24gPSAwLAogICAgQm9vbGVhbiA9IDEsCiAgICBUYWJsZSA9IDIsCiAgICBNb2RpZmllciA9IDMKICAgCiAgICAgICAgICAgIApjbGFzcyBEQVhGdW5jdGlvbkV4cHJlc3Npb24oREFYRXhwcmVzc2lvbiwgRnVuY3Rpb24pOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIGZ1bmN0aW9uTmFtZTogc3RyLCBwYXJhbWV0ZXJTdHJpbmc6IHN0ciwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIERBWEV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgdHlwZT1EQVhFeHByZXNzaW9uVHlwZS5GdW5jdGlvbiwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIEZ1bmN0aW9uLl9faW5pdF9fKHNlbGYsIGZ1bmN0aW9uTmFtZT1mdW5jdGlvbk5hbWUsIHBhcmFtZXRlclN0cmluZz1wYXJhbWV0ZXJTdHJpbmcpCiAgICAgICAgc2VsZi5fX3ZhbHVlID0gTm9uZQogICAgICAgIAogICAgICAgICNpZiBzZWxmLk1vZGVsLkZ1bmN0aW9uRXZhbHVhdG9yIGlzIG5vdCBOb25lIGFuZCBzZWxmLk1vZGVsLnN0YXRlID09ICJSZWFkeSI6CiAgICAgICAgIyAgICBzZWxmLl9fdmFsdWUgPSBzZWxmLk1vZGVsLkZ1bmN0aW9uRXZhbHVhdG9yLkV2YWx1YXRlKHNlbGYpCgogICAgICAgIGlmIHNlbGYuSXNVSUNvbnRleHRTZW5zaXRpdmU6CiAgICAgICAgICAgIHBhcmVudC5BZGRJc3N1ZSh0eXBlPUlzc3VlVHlwZS5VbnN1cHBvcnRlZEZlYXR1cmUsIGRlc2NyaXB0aW9uPWYiQ29udGFpbnMgVUkgQ29udGV4dCBTZW5zaXRpdmUgREFYIEZ1bmN0aW9uIHtmdW5jdGlvbk5hbWV9IiwgZGV0YWlscz1mdW5jdGlvbk5hbWUsIHNvdXJjZT1zZWxmLnNvdXJjZVN0cmluZyApCiAgICAKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgaWYgc2VsZi5fX3ZhbHVlIGlzIE5vbmU6CiAgICAgICAgICAgIHNlbGYuX192YWx1ZSA9IHNlbGYuR2V0UmVzdWx0KCkKICAgICAgICByZXR1cm4gc2VsZi5fX3ZhbHVlCgogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICByZXR1cm4gRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShzZWxmLnZhbHVlKQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBmaWx0ZXJUeXBlKHNlbGYpOgogICAgICAgIGZyb20gLkRBWE9wZXJhdG9yRXhwcmVzc2lvbiBpbXBvcnQgREFYT3BlcmF0b3JFeHByZXNzaW9uCiAgICAgICAgaWYgc2VsZi5mdW5jdGlvbk5hbWUudXBwZXIoKSBpbiBbIlJFTU9WRUZJTFRFUlMiLCJBTEwiLCJBTExOT0JMQU5LUk9XIiwiQUxMRVhDRVBUIiwiS0VFUEZJTFRFUlMiLCJVU0VSRUxBVElPTlNISVAiLCJDUk9TU0ZJTFRFUiJdOgogICAgICAgICAgICByZXR1cm4gREFYRnVuY3Rpb25GaWx0ZXJUeXBlLk1vZGlmaWVyCiAgICAgICAgZWxpZiBpc2luc3RhbmNlKHNlbGYudmFsdWUsIERBWE9wZXJhdG9yRXhwcmVzc2lvbikgYW5kICI9IiBpbiBzZWxmLnZhbHVlLm9wZXJhdG9yczoKICAgICAgICAgICAgcmV0dXJuIERBWEZ1bmN0aW9uRmlsdGVyVHlwZS5Cb29sZWFuCiAgICAgICAgZWxpZiBzZWxmLnJldHVyblR5cGUgPT0gRXhwcmVzc2lvblR5cGUuVGFibGU6CiAgICAgICAgICAgIHJldHVybiBEQVhGdW5jdGlvbkZpbHRlclR5cGUuVGFibGUKICAgICAgICBlbHNlOgogICAgICAgICAgICByZXR1cm4gREFYRnVuY3Rpb25GaWx0ZXJUeXBlLlVua25vd24KICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgSXNVSUNvbnRleHRTZW5zaXRpdmUoc2VsZikgLT4gYm9vbDoKICAgICAgICByZXR1cm4gc2VsZi5mdW5jdGlvbk5hbWUudXBwZXIoKSBpbiBbIkFMTCIsICJBTExDUk9TU0ZJTFRFUkVEIiwgIkFMTEVYQ0VQVCIsICJBTExTRUxFQ1RFRCIsICJJU0NST1NTRklMVEVSRUQiLCAiSVNGSUxURVJFRCIsICJJU1NFTEVDVEVETUVBU1VSRSIsICJLRUVQRklMVEVSUyIsICJORVhUIiwgIlNFTEVDVEVETUVBU1VSRSIsICJTRUxFQ1RFRE1FQVNVUkVGT1JNQVRTVFJJTkciLCJTRUxFQ1RFRE1FQVNVUkVOQU1FIiwiVVNFUkNVTFRVUkUiLCJVU0VSTkFNRSIsIlVTRVJPQkpFQ1RJRCIsIlVTRVJQUklOQ0lQQUxOQU1FIiwgIlJFTU9WRUZJTFRFUlMiXQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiByZXR1cm5UeXBlKHNlbGYpOgogICAgICAgIHJldHVybiBGdW5jdGlvbi5yZXR1cm5UeXBlLmZnZXQoc2VsZikKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaGVscFVybChzZWxmKToKICAgICAgICByZXR1cm4gZiJodHRwczovL2xlYXJuLm1pY3Jvc29mdC5jb20vZW4tdXMvZGF4L3tzZWxmLmZ1bmN0aW9uTmFtZS5sb3dlcigpfS1mdW5jdGlvbi1kYXgiCiAgICAKICAgIGRlZiBHZXRBbGxJc3N1ZXMoc2VsZik6CiAgICAgICAgYWxsSXNzdWVzID0gc2V0KHNlbGYuSXNzdWVzKQoKICAgICAgICBmb3IgcCBpbiBzZWxmLnBhcmFtZXRlcnM6CiAgICAgICAgICAgIGlmIHAgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBhbGxJc3N1ZXMgPSBhbGxJc3N1ZXMudW5pb24ocC5HZXRBbGxJc3N1ZXMoKSkKCiAgICAgICAgcmV0dXJuIGFsbElzc3VlcwogICAgCkZ1bmN0aW9uLnJlZ2lzdGVyKERBWEZ1bmN0aW9uRXhwcmVzc2lvbikKREFYRXhwcmVzc2lvbi5yZWdpc3RlcihEQVhGdW5jdGlvbkV4cHJlc3Npb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/report.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKaW1wb3J0IGJpLnJlYWRlcnMucGJpLnByb2plY3QKaW1wb3J0IGJpLnJlYWRlcnMucGJpLnBhZ2UKaW1wb3J0IGJpLnJlYWRlcnMucGJpLnJlcG9ydFJlc291cmNlCgpjbGFzcyBSZXBvcnQoKToKICAgIGRlZiBfX2luaXRfXyhzZWxmLCBwcm9qZWN0OiBiaS5yZWFkZXJzLnBiaS5wcm9qZWN0LlByb2plY3QsIGlkOiBzdHIsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgc2VsZi5wcm9qZWN0ID0gcHJvamVjdAogICAgICAgIHNlbGYuaWQgPSBpZAogICAgICAgIHNlbGYucHJvcGVydGllcyA9IHByb3BlcnRpZXMKICAgICAgICBzZWxmLnBhZ2VzOiBMaXN0W2JpLnJlYWRlcnMucGJpLnBhZ2UuUGFnZV0gPSBbXQogICAgICAgIHNlbGYucmVzb3VyY2VzOiBMaXN0W2JpLnJlYWRlcnMucGJpLnJlcG9ydFJlc291cmNlLlJlcG9ydFJlc291cmNlXSA9IFtdCiAgICAKICAgIGRlZiBhZGRQYWdlKHNlbGYsIG5hbWU6IHN0ciwgaWQ6IHN0ciwgZGlzcGxheU5hbWU6IHN0ciwgd2lkdGg6IGludCwgaGVpZ2h0OiBpbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSwgZmlsdGVyczogZGljdCA9IHt9KSAtPiBiaS5wYWdlLlBhZ2U6CiAgICAgICAgCiAgICAgICAgcGFnZSA9IGJpLnJlYWRlcnMucGJpLnBhZ2UuUGFnZShzZWxmLCBuYW1lLCBkaXNwbGF5TmFtZSwgaWQsIHdpZHRoLCBoZWlnaHQsIHByb3BlcnRpZXMsIGZpbHRlcnMpCiAgICAgICAgc2VsZi5wYWdlcy5hcHBlbmQocGFnZSkKICAgICAgICByZXR1cm4gcGFnZQogICAgCiAgICBkZWYgYWRkUmVzb3VyY2Uoc2VsZiwgcmVzb3VyY2U6IGJpLnJlYWRlcnMucGJpLnJlcG9ydFJlc291cmNlLlJlcG9ydFJlc291cmNlKToKICAgICAgICBzZWxmLnJlc291cmNlcy5hcHBlbmQocmVzb3VyY2UpCgogICAgZGVmIGdldFJlc291cmNlKHNlbGYsIG5hbWU6IHN0cikgLT4gT3B0aW9uYWxbYmkucmVhZGVycy5wYmkucmVwb3J0UmVzb3VyY2UuUmVwb3J0UmVzb3VyY2VdOgogICAgICAgIGZvciByIGluIHNlbGYucmVzb3VyY2VzOgogICAgICAgICAgICBpZiByLmZpbGVuYW1lID09IG5hbWU6CiAgICAgICAgICAgICAgICByZXR1cm4gcgogICAgICAgICAgICAKICAgICAgICByZXR1cm4gTm9uZQogICAgCiAgICBkZWYgX19yZXByX18oc2VsZik6CiAgICAgICAgcmV0dXJuIGYie3NlbGYuaWR9Ig=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/reportResource.py FROM (SELECT BASE64_DECODE_STRING('CgpjbGFzcyBSZXBvcnRSZXNvdXJjZSgpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIGZpbGVuYW1lOiBzdHIsIGJ5dGVzOiBsaXN0KToKICAgICAgICBzZWxmLmZpbGVuYW1lID0gZmlsZW5hbWUKICAgICAgICBzZWxmLmJ5dGVzID0gYnl0ZXMKICAgIAogICAgZGVmIF9fcmVwcl9fKHNlbGYpOgogICAgICAgIHJldHVybiBmIntzZWxmLmZpbGVuYW1lfSAoe2xlbihzZWxmLmJ5dGVzKX0pIg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/project.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQKZnJvbSBiaS5jb3JlLnJ1bnRpbWUgaW1wb3J0IFJ1bnRpbWUKZnJvbSBzbm93Zmxha2Uuc25vd3Bhcmsuc2Vzc2lvbiBpbXBvcnQgU2Vzc2lvbgoKY2xhc3MgUHJvamVjdCgpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIGZpbGVfcGF0aDogc3RyLCBydW50aW1lOiBSdW50aW1lLCBzZXNzaW9uOiBTZXNzaW9uLCBwcm9wZXJ0aWVzOiBkaWN0KToKICAgICAgICBzZWxmLl9fcnVudGltZSA9IHJ1bnRpbWUKICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnJlcG9ydCBpbXBvcnQgUmVwb3J0CiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5tb2RlbCBpbXBvcnQgTW9kZWwKICAgICAgICBzZWxmLmZpbGVfcGF0aCA9IGZpbGVfcGF0aAogICAgICAgIHNlbGYucmVwb3J0OiBSZXBvcnQgPSBOb25lCiAgICAgICAgc2VsZi5wcm9wZXJ0aWVzID0gcHJvcGVydGllcwogICAgICAgIHNlbGYubW9kZWw6IE1vZGVsID0gTW9kZWwocHJvamVjdD1zZWxmLCBzZXNzaW9uPXNlc3Npb24pCiAgICAgICAgc2VsZi5fX25hbWU6IHN0ciA9IE5vbmUKICAgICAgICAKICAgICAgICAKCiAgICBAcHJvcGVydHkKICAgIGRlZiBSdW50aW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fcnVudGltZQogICAgCiAgICBkZWYgY3JlYXRlUmVwb3J0KHNlbGYsIGlkOiBzdHIsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgZnJvbSBiaS5yZWFkZXJzLnBiaS5yZXBvcnQgaW1wb3J0IFJlcG9ydAoKICAgICAgICBzZWxmLnJlcG9ydCA9IFJlcG9ydChzZWxmLCBpZCwgcHJvcGVydGllcykKICAgICAgICByZXR1cm4gc2VsZi5yZXBvcnQKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgTmFtZShzZWxmKToKICAgICAgICBmcm9tIHBhdGhsaWIgaW1wb3J0IFBhdGgKICAgICAgICBpZiBzZWxmLl9fbmFtZSBpcyBOb25lOgogICAgICAgICAgICByZXR1cm4gUGF0aChzZWxmLmZpbGVfcGF0aCkuc3RlbQogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBzZWxmLl9fbmFtZQogICAgICAgIAogICAgQE5hbWUuc2V0dGVyCiAgICBkZWYgTmFtZShzZWxmLCBuYW1lOiBzdHIpOgogICAgICAgIHNlbGYuX19uYW1lID0gbmFtZQogICAgCgogICAgICAgIAoKICAgIA=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/readers/pbi/column.py FROM (SELECT BASE64_DECODE_STRING('CmZyb20gX19mdXR1cmVfXyBpbXBvcnQgYW5ub3RhdGlvbnMKZnJvbSB0eXBpbmcgaW1wb3J0IEFueSwgT3B0aW9uYWwsIExpc3QsIFVuaW9uLCBTZXQsIERpY3QKZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uIGFzIEJhc2VDb2x1bW4sIEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbkNvZGVUeXBlIGltcG9ydCBFeHByZXNzaW9uQ29kZVR5cGUKZnJvbSBiaS5yZWFkZXJzLnBiaS5tb2RlbEl0ZW0gaW1wb3J0IE1vZGVsSXRlbQpmcm9tIGJpLnJlYWRlcnMucGJpLnRlcm0gaW1wb3J0IFRlcm0KCmNsYXNzIENvbHVtbihNb2RlbEl0ZW0sIEJhc2VDb2x1bW4pOgogICAgZnJvbSBiaS5jb3JlLnRhYmxlIGltcG9ydCBUYWJsZQogICAgCiAgICBkZWYgX19pbml0X18oc2VsZiwgCiAgICAgICAgICAgICAgICAgbmFtZTogc3RyLCAKICAgICAgICAgICAgICAgICB0eXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLkFueSwgCiAgICAgICAgICAgICAgICAgZXhwcmVzc2lvbjogT3B0aW9uYWxbVW5pb25bc3RyfEV4cHJlc3Npb25dXSA9IE5vbmUsICAKICAgICAgICAgICAgICAgICBhbGlhczogT3B0aW9uYWxbc3RyXSA9IE5vbmUsIAogICAgICAgICAgICAgICAgIGV4cHJlc3Npb25Db2RlVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvbkNvZGVUeXBlXSA9IEV4cHJlc3Npb25Db2RlVHlwZS5TUUwsIAogICAgICAgICAgICAgICAgIGlzUHJpbWFyeUtleTogYm9vbCA9IE5vbmUsCiAgICAgICAgICAgICAgICAgaXNVbmlxdWVLZXk6IGJvb2wgPSBGYWxzZSwKICAgICAgICAgICAgICAgICBpc051bGxhYmxlOiBib29sID0gVHJ1ZSwKICAgICAgICAgICAgICAgICBmb3JtYXRTdHJpbmc6IE9wdGlvbmFsW3N0cl0gPSBOb25lLAogICAgICAgICAgICAgICAgIHNvdXJjZUNvbHVtbjogT3B0aW9uYWxbc3RyXSA9IE5vbmUsCiAgICAgICAgICAgICAgICAgdGFibGVBbGlhczogT3B0aW9uYWxbc3RyXSA9IE5vbmUsCiAgICAgICAgICAgICAgICAgdGFibGU6IE9wdGlvbmFsW1RhYmxlXSA9IE5vbmUsCiAgICAgICAgICAgICAgICAgcHJvcGVydGllczogZGljdCA9IHt9CiAgICAgICAgICAgICAgICAgKToKICAgICAgICBmcm9tIGJpLmNvcmUudGFibGVJdGVtIGltcG9ydCBUYWJsZUl0ZW0KICAgICAgICBmcm9tIGJpLnJlYWRlcnMucGJpLnRlcm0gaW1wb3J0IFRlcm0KICAgICAgICBNb2RlbEl0ZW0uX19pbml0X18oc2VsZikKICAgICAgICBCYXNlQ29sdW1uLl9faW5pdF9fKHNlbGYsIG5hbWU9bmFtZSwgCiAgICAgICAgICAgICAgICAgICAgICAgICB0eXBlPXR5cGUsIAogICAgICAgICAgICAgICAgICAgICAgICAgZXhwcmVzc2lvbj1leHByZXNzaW9uLCAKICAgICAgICAgICAgICAgICAgICAgICAgIGV4cHJlc3Npb25Db2RlVHlwZT1leHByZXNzaW9uQ29kZVR5cGUsIAogICAgICAgICAgICAgICAgICAgICAgICAgaXNQcmltYXJ5S2V5PWlzUHJpbWFyeUtleSwKICAgICAgICAgICAgICAgICAgICAgICAgIGlzVW5pcXVlS2V5PWlzVW5pcXVlS2V5LAogICAgICAgICAgICAgICAgICAgICAgICAgaXNOdWxsYWJsZT1pc051bGxhYmxlLAogICAgICAgICAgICAgICAgICAgICAgICAgZm9ybWF0U3RyaW5nPWZvcm1hdFN0cmluZywKICAgICAgICAgICAgICAgICAgICAgICAgIHRhYmxlQWxpYXM9dGFibGVBbGlhcywgCiAgICAgICAgICAgICAgICAgICAgICAgICB0YWJsZT10YWJsZSwKICAgICAgICAgICAgICAgICAgICAgICAgIHByb3BlcnRpZXM9cHJvcGVydGllcykKICAgICAgICBzZWxmLl9fc291cmNlQ29sdW1uTmFtZTogc3RyID0gTm9uZQogICAgICAgIHNlbGYuX19zb3VyY2VDb2x1bW46IFRhYmxlSXRlbSA9IE5vbmUKCiAgICAgICAgaWYgc291cmNlQ29sdW1uIGlzIG5vdCBOb25lOgogICAgICAgICAgICBpZiBpc2luc3RhbmNlKHNvdXJjZUNvbHVtbiwgc3RyKToKICAgICAgICAgICAgICAgIHNvdXJjZUNvbHVtbiA9IHNvdXJjZUNvbHVtbi5zdHJpcCgnIicpCiAgICAgICAgICAgICAgICBzZWxmLl9fc291cmNlQ29sdW1uTmFtZTogT3B0aW9uYWxbc3RyXSA9IHNvdXJjZUNvbHVtbgogICAgICAgICAgICBlbGlmIGlzaW5zdGFuY2Uoc291cmNlQ29sdW1uLCBUYWJsZUl0ZW0pOgogICAgICAgICAgICAgICAgc2VsZi5fX3NvdXJjZUNvbHVtbiA9IHNvdXJjZUNvbHVtbgogICAgICAgICAgICAgICAgc2VsZi5fX3NvdXJjZUNvbHVtbk5hbWUgPSBzb3VyY2VDb2x1bW4uZ2V0SWRlbnRpZmllcihGYWxzZSxGYWxzZSxGYWxzZSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBNb2RlbChzZWxmKToKICAgICAgICBpZiBzZWxmLnRhYmxlIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi50YWJsZS5Nb2RlbAogICAgICAgIHJldHVybiBOb25lCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGZvcm1hdFN0cmluZyhzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgIHJldHVybiBzZWxmLl9fZm9ybWF0U3RyaW5nCgogICAgQHByb3BlcnR5CiAgICBkZWYgc291cmNlQ29sdW1uKHNlbGYpIC0+IE9wdGlvbmFsW0NvbHVtbl06CiAgICAgICAgaWYgc2VsZi5fX3NvdXJjZUNvbHVtbiBpcyBOb25lOgogICAgICAgICAgICBmcm9tIC5zZW1hbnRpY1RhYmxlIGltcG9ydCBTZW1hbnRpY1RhYmxlCiAgICAgICAgICAgIGlmIHNlbGYuX19zb3VyY2VDb2x1bW5OYW1lIGlzIG5vdCBOb25lIGFuZCBpc2luc3RhbmNlKHNlbGYudGFibGUsIFNlbWFudGljVGFibGUpOgogICAgICAgICAgICAgICAgc2VsZi5fX3NvdXJjZUNvbHVtbiA9IHNlbGYudGFibGUuZ2V0UGFydGl0aW9uQ29sdW1uKHNlbGYuX19zb3VyY2VDb2x1bW5OYW1lKQogICAgICAgIHJldHVybiBzZWxmLl9fc291cmNlQ29sdW1uCgogICAgQHByb3BlcnR5CiAgICBkZWYgc291cmNlQ29sdW1uTmFtZShzZWxmKSAtPiBPcHRpb25hbFtzdHJdOgogICAgICAgIHJldHVybiBzZWxmLl9fc291cmNlQ29sdW1uTmFtZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRm9yZWlnbktleShzZWxmKSAtPiBib29sOgogICAgICAgICMgVXNlIHJlbGF0aW9uc2hpcHMgdG8gZGV0ZXJtaW5lIGlmIHRoaXMgY29sdW1uIGlzIGEgZm9yZWlnbiBrZXkuCiAgICAgICAgaWYgc2VsZi5fX2lzRm9yZWlnbktleSBpcyBOb25lOgogICAgICAgICAgICBmb3IgciBpbiBzZWxmLnJlbGF0aW9uc2hpcHM6CiAgICAgICAgICAgICAgICBpZiByLmZyb21UYWJsZU5hbWUgPT0gc2VsZi50YWJsZS5uYW1lIGFuZCByLmZyb21Db2x1bW5OYW1lID09IHNlbGYuZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1GYWxzZSwgaW5jbHVkZVRhYmxlQWxpYXM9RmFsc2UsIGluY2x1ZGVUYWJsZU5hbWU9RmFsc2UpOgogICAgICAgICAgICAgICAgICAgIHNlbGYuX19pc0ZvcmVpZ25LZXkgPSBUcnVlCiAgICAgICAgcmV0dXJuIHNlbGYuX19pc0ZvcmVpZ25LZXkKCiAgICBAaXNGb3JlaWduS2V5LnNldHRlciAgCiAgICBkZWYgaXNGb3JlaWduS2V5KHNlbGYsIHZhbHVlOiBib29sKToKICAgICAgICBzZWxmLl9faXNGb3JlaWduS2V5ID0gdmFsdWUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1ByaW1hcnlLZXkoc2VsZikgLT4gYm9vbDoKICAgICAgICAjIFVzZSByZWxhdGlvbnNoaXBzIHRvIGRldGVybWluZSBpZiB0aGlzIGNvbHVtbiBpcyBhIHByaW1hcnkga2V5LgogICAgICAgIGlmIHNlbGYuX19pc1ByaW1hcnlLZXkgaXMgTm9uZToKICAgICAgICAgICAgZm9yIHIgaW4gc2VsZi5yZWxhdGlvbnNoaXBzOgogICAgICAgICAgICAgICAgaWYgci50b1RhYmxlTmFtZSA9PSBzZWxmLnRhYmxlLm5hbWUgYW5kIHIudG9Db2x1bW5OYW1lID09IHNlbGYuZ2V0SWRlbnRpZmllcihxdW90ZU1peGVkQ2FzZT1GYWxzZSwgaW5jbHVkZVRhYmxlQWxpYXM9RmFsc2UsIGluY2x1ZGVUYWJsZU5hbWU9RmFsc2UpOgogICAgICAgICAgICAgICAgICAgIHNlbGYuX19pc1ByaW1hcnlLZXkgPSBUcnVlCgogICAgICAgIGlmIHNlbGYuX19pc1ByaW1hcnlLZXkgaXMgTm9uZToKICAgICAgICAgICAgIyBObyByZWxhdGlvbnNoaXBzIGZvdW5kLCBjaGVjayBzbm93Zmxha2UgdGFibGUgcHJpbWFyeSBrZXlzIGlmIGF2YWlsYWJsZS4KICAgICAgICAgICAgIyBDaGVjayBzbm93Zmxha2UgdGFibGUgcHJpbWFyeSBrZXlzIGlmIGF2YWlsYWJsZS4KICAgICAgICAgICAgc291cmNlQ29sdW1uID0gc2VsZi5zb3VyY2VDb2x1bW4KICAgICAgICAgICAgaWYgc291cmNlQ29sdW1uIGlzIG5vdCBOb25lIGFuZCBzb3VyY2VDb2x1bW4uaXNQcmltYXJ5S2V5IGlzIG5vdCBOb25lOgogICAgICAgICAgICAgICAgc2VsZi5fX2lzUHJpbWFyeUtleSA9IHNvdXJjZUNvbHVtbi5pc1ByaW1hcnlLZXkKICAgICAgICAgICAgCiAgICAgICAgIyBDb2x1bW4gaXMgbm90IGZvdW5kIGluIGFueSByZWxhdGlvbnNoaXBzIG9yIHNub3dmbGFrZSB0YWJsZS4KICAgICAgICBpZiBzZWxmLl9faXNQcmltYXJ5S2V5IGlzIE5vbmU6CiAgICAgICAgICAgIHNlbGYuX19pc1ByaW1hcnlLZXkgPSBGYWxzZQoKICAgICAgICByZXR1cm4gc2VsZi5fX2lzUHJpbWFyeUtleQoKICAgIEBpc1ByaW1hcnlLZXkuc2V0dGVyCiAgICBkZWYgaXNQcmltYXJ5S2V5KHNlbGYsIHZhbHVlOiBib29sKToKICAgICAgICBzZWxmLl9faXNQcmltYXJ5S2V5ID0gdmFsdWUKICAgICAgICAKQmFzZUNvbHVtbi5yZWdpc3RlcihDb2x1bW4pCk1vZGVsSXRlbS5yZWdpc3RlcihDb2x1bW4p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLValueExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBMaXN0LCBVbmlvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gLlNRTEV4cHJlc3Npb24gaW1wb3J0IFNRTEV4cHJlc3Npb24sIEV4cHJlc3Npb24KCiAgICAKY2xhc3MgU1FMVmFsdWVFeHByZXNzaW9uKFNRTEV4cHJlc3Npb24pOiAgICAKICAgIGRlZiBfX2luaXRfXyhzZWxmLCB2YWx1ZTogQW55LCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgaWYgIm9yaWdpbmFsIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgcHJvcGVydGllc1sib3JpZ2luYWwiXSA9IHZhbHVlCiAgICAgICAgc3VwZXIoKS5fX2luaXRfXyhwYXJlbnQ9cGFyZW50LCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMsIHJldHVyblR5cGU9RXhwcmVzc2lvblR5cGUuRnJvbVZhbHVlKHZhbHVlKSkKICAgICAgICBzZWxmLl9fdmFsdWUgPSB2YWx1ZQogICAgCiAgICBkZWYgdXBkYXRlKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLl9fdmFsdWUKCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1ZhbHVlKHNlbGYpOgogICAgICAgIHJldHVybiBUcnVlCiAgICAKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgcmV0dXJuIEV4cHJlc3Npb24uQ29udmVydFRvU3FsVmFsdWUoc2VsZi5fX3ZhbHVlKSAgICAgICAgCgpTUUxFeHByZXNzaW9uLnJlZ2lzdGVyKFNRTFZhbHVlRXhwcmVzc2lvbikKCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLExpressionParser.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBPcHRpb25hbApmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudCwgRXhwcmVzc2lvbkNvZGVUeXBlCmZyb20gYmkuY29yZS5leHByZXNzaW9uUGFyc2VyIGltcG9ydCBFeHByZXNzaW9uUGFyc2VyCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKZnJvbSAuU1FMRXhwcmVzc2lvbiBpbXBvcnQgU1FMRXhwcmVzc2lvbgoKaW1wb3J0IHJlZ2V4CgoKY2xhc3MgU1FMRXhwcmVzc2lvblBhcnNlcihFeHByZXNzaW9uUGFyc2VyKToKICAgIGRlZiBfX2luaXRfXyhzZWxmKToKICAgICAgICBzdXBlcigpLl9faW5pdF9fKGNvZGVUeXBlPUV4cHJlc3Npb25Db2RlVHlwZS5TUUwpCgogICAgQHByb3BlcnR5CiAgICBkZWYgQm91bmRhcnkoc2VsZik6CiAgICAgICAgcmV0dXJuIHsiKCI6IikifQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBPcGVyYXRvcnMoc2VsZik6CiAgICAgICAgcmV0dXJuIFsiKyIsIi0iLCIvIiwiKiIsIj0iLCJeIiwiPiIsIjwiLCImIiwiPT0iLCI+PSIsIjw9IiwiPD4iLCImJiIsInx8IiwiSU4iLCJOT1QiLCJSRVRVUk4iXQogICAgCiAgICBkZWYgQ3JlYXRlRnJvbVN0cmluZyhzZWxmLCBwYXJlbnQ6IEV4cHJlc3Npb25QYXJlbnQsIGV4cHJlc3Npb25TdHJpbmc6IHN0ciwgcmV0dXJuVHlwZTogT3B0aW9uYWxbRXhwcmVzc2lvblR5cGVdID0gRXhwcmVzc2lvblR5cGUuQW55LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIHJhaXNlIE5vdEltcGxlbWVudGVkRXJyb3IoIm5vdCBpbXBsZW1lbnRlZCIpCiAgICAKICAgIGRlZiBDcmVhdGVGcm9tVmFsdWUoc2VsZiwgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCB2YWx1ZTogQW55LCBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIGZyb20gLlNRTFZhbHVlRXhwcmVzc2lvbiBpbXBvcnQgU1FMVmFsdWVFeHByZXNzaW9uCiAgICAgICAgcmV0dXJuIFNRTFZhbHVlRXhwcmVzc2lvbih2YWx1ZT12YWx1ZSwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIApFeHByZXNzaW9uUGFyc2VyLnJlZ2lzdGVyKFNRTEV4cHJlc3Npb25QYXJzZXIpCg=='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBVbmlvbiwgT3B0aW9uYWwKZnJvbSBhYmMgaW1wb3J0IEFCQywgYWJzdHJhY3RtZXRob2QKZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25QYXJlbnQgaW1wb3J0IEV4cHJlc3Npb25QYXJlbnQsIEV4cHJlc3Npb25Db2RlVHlwZQpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgpmcm9tIGJpLmNvcmUuZnVuY3Rpb24gaW1wb3J0IEZ1bmN0aW9uCmZyb20gYmkuY29yZS5leHByZXNzaW9uVHlwZSBpbXBvcnQgRXhwcmVzc2lvblR5cGUKCgpjbGFzcyBTUUxFeHByZXNzaW9uKEV4cHJlc3Npb24sIEFCQyk6CiAgICBkZWYgX19pbml0X18oc2VsZiwgCiAgICAgICAgICAgICAgICAgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCAKICAgICAgICAgICAgICAgICByZXR1cm5UeXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLkFueSwKICAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIEV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgY29kZVR5cGU9RXhwcmVzc2lvbkNvZGVUeXBlLlNRTCwgcGFyZW50PXBhcmVudCwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIEFCQy5fX2luaXRfXyhzZWxmKQogICAgICAgIHNlbGYuX19yZXR1cm5UeXBlID0gcmV0dXJuVHlwZQogICAgICAgIGlmICJkYXRhU291cmNlIiBpbiBwcm9wZXJ0aWVzOgogICAgICAgICAgICBkYXRhU291cmNlTmFtZSA9IHByb3BlcnRpZXMuZ2V0KCJkYXRhU291cmNlIikKICAgICAgICAgICAgaWYgZGF0YVNvdXJjZU5hbWUgaXMgbm90IE5vbmU6CiAgICAgICAgICAgICAgICBkYXRhU291cmNlID0gc2VsZi5Nb2RlbC5HZXREYXRhU291cmNlKGRhdGFTb3VyY2VOYW1lKQogICAgICAgICAgICAgICAgaWYgZGF0YVNvdXJjZSBpcyBub3QgTm9uZToKICAgICAgICAgICAgICAgICAgICBzZWxmLkFkZERlcGVuZGVuY3koZGF0YVNvdXJjZSkKCiAgICBAcHJvcGVydHkKICAgIGRlZiBSdW50aW1lKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLnBhcmVudC5SdW50aW1lCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGNoaWxkRXhwcmVzc2lvbkNvZGVUeXBlKHNlbGYpOgogICAgICAgIHJldHVybiBFeHByZXNzaW9uQ29kZVR5cGUuU1FMCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJlZmVyZW5jZXMoc2VsZik6CiAgICAgICAgaWYgaXNpbnN0YW5jZShzZWxmLnBhcmVudCwgRnVuY3Rpb24pOgogICAgICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQucGFyYW1ldGVycwogICAgICAgIHJldHVybiBOb25lCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIHR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYucmV0dXJuVHlwZQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIHJldHVyblR5cGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX19yZXR1cm5UeXBlCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIERhdGFTb3VyY2Uoc2VsZik6CiAgICAgICAgZHNOYW1lID0gc2VsZi5wcm9wZXJ0aWVzLmdldCgiZGF0YVNvdXJjZSIpCiAgICAgICAgaWYgZHNOYW1lIGlzIG5vdCBOb25lOgogICAgICAgICAgICByZXR1cm4gc2VsZi5Nb2RlbC5HZXREYXRhU291cmNlKG5hbWU9ZHNOYW1lKQoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRnVuY3Rpb24oc2VsZik6CiAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgaXNWYWx1ZShzZWxmKToKICAgICAgICByZXR1cm4gRmFsc2UKCiAgICBkZWYgaXNTY3JpcHQoc2VsZik6CiAgICAgICAgcmV0dXJuIEZhbHNlCiAgICAKICAgIGRlZiBnZXRFeHByZXNzaW9uKHNlbGYsIHZhcmlhYmxlTmFtZTogc3RyKToKICAgICAgICByZXR1cm4gc2VsZi5wYXJlbnQuZ2V0RXhwcmVzc2lvbih2YXJpYWJsZU5hbWUpCgpFeHByZXNzaW9uLnJlZ2lzdGVyKFNRTEV4cHJlc3Npb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLSelectExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgVW5pb24sIExpc3QsIE9wdGlvbmFsCmZyb20gYmkuc3FsLlNRTEV4cHJlc3Npb24gaW1wb3J0IFNRTEV4cHJlc3Npb24sIEV4cHJlc3Npb25QYXJlbnQsIEV4cHJlc3Npb25UeXBlCmZyb20gYmkuY29yZS50YWJsZSBpbXBvcnQgVGFibGUKZnJvbSBiaS5jb3JlLmNvbHVtbiBpbXBvcnQgQ29sdW1uCmZyb20gYmkuc3FsLlNRTFN0YXRlbWVudEV4cHJlc3Npb24gaW1wb3J0IFNRTFN0YXRlbWVudEV4cHJlc3Npb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb24gaW1wb3J0IEV4cHJlc3Npb24KCmNsYXNzIFNRTFNlbGVjdEV4cHJlc3Npb24oU1FMRXhwcmVzc2lvbiwgVGFibGUpOgogICAgZGVmIF9faW5pdF9fKHNlbGYsIAogICAgICAgICAgICAgICAgIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgCiAgICAgICAgICAgICAgICAgY29sdW1uczogT3B0aW9uYWxbTGlzdFtDb2x1bW5dXSA9IFtdLCAKICAgICAgICAgICAgICAgICBmcm06IE9wdGlvbmFsW1RhYmxlXSA9IE5vbmUsCiAgICAgICAgICAgICAgICAgY29uc3RyYWludHM6IE9wdGlvbmFsW2xpc3RdID0gW10sCiAgICAgICAgICAgICAgICAgb3JkZXJCeTogT3B0aW9uYWxbTGlzdFtFeHByZXNzaW9uXV0gPSBbXSwKICAgICAgICAgICAgICAgICBncm91cEJ5OiBPcHRpb25hbFtMaXN0W0V4cHJlc3Npb25dXSA9IFtdLAogICAgICAgICAgICAgICAgIHJldHVyblR5cGU6IE9wdGlvbmFsW0V4cHJlc3Npb25UeXBlXSA9IEV4cHJlc3Npb25UeXBlLlRhYmxlLAogICAgICAgICAgICAgICAgIGRpc3RpbmN0OiBPcHRpb25hbFtib29sXSA9IEZhbHNlLAogICAgICAgICAgICAgICAgIHRvcE46IE9wdGlvbmFsW2ludF0gPSBOb25lLAogICAgICAgICAgICAgICAgIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgCiAgICAgICAgU1FMRXhwcmVzc2lvbi5fX2luaXRfXyhzZWxmPXNlbGYsIHBhcmVudD1wYXJlbnQsIHJldHVyblR5cGU9cmV0dXJuVHlwZSwgcHJvcGVydGllcz1wcm9wZXJ0aWVzKQogICAgICAgIFRhYmxlLl9faW5pdF9fKHNlbGY9c2VsZikKCiAgICAgICAgaWYgZnJtIGlzIG5vdCBOb25lOgogICAgICAgICAgICBzZWxmLkFkZERlcGVuZGVuY3koZnJtKQogICAgICAgIHNlbGYuX19mcm9tID0gZnJtCiAgICAgICAgCiAgICAgICAgZm9yIGksIGNvbnN0cmFpbnQgaW4gZW51bWVyYXRlKGNvbnN0cmFpbnRzKToKICAgICAgICAgICAgaWYgaXNpbnN0YW5jZShjb25zdHJhaW50LCBzdHIpOgogICAgICAgICAgICAgICAgY29uc3RyYWludHNbaV0gPSBTUUxTdGF0ZW1lbnRFeHByZXNzaW9uKHN0YXRlbWVudD1jb25zdHJhaW50LCBwYXJlbnQ9c2VsZiwgcmV0dXJuVHlwZT1FeHByZXNzaW9uVHlwZS5Mb2dpY2FsLCBwcm9wZXJ0aWVzPXt9KQoKICAgICAgICBzZWxmLl9fY29uc3RyYWludHMgPSBjb25zdHJhaW50cwogICAgICAgIHNlbGYuX19vcmRlckJ5ID0gb3JkZXJCeQogICAgICAgIHNlbGYuX19ncm91cEJ5ID0gZ3JvdXBCeQogICAgICAgIHNlbGYuX19kaXN0aW5jdCA9IGRpc3RpbmN0CiAgICAgICAgc2VsZi5fX3RvcE4gPSB0b3BOCgogICAgICAgIGZvciBjb2wgaW4gY29sdW1uczoKICAgICAgICAgICAgY29sdW1uID0gQ29sdW1uKG5hbWU9Y29sLmdldElkZW50aWZpZXIocXVvdGVNaXhlZENhc2U9RmFsc2UpLHR5cGU9Y29sLnR5cGUsIGV4cHJlc3Npb249Y29sLmV4cHJlc3Npb24gb3IgY29sLmV4cHJlc3Npb25TdHJpbmcsIGV4cHJlc3Npb25Db2RlVHlwZT1jb2wuY2hpbGRFeHByZXNzaW9uQ29kZVR5cGUsIGlzUHJpbWFyeUtleT1jb2wuaXNQcmltYXJ5S2V5LCBpc051bGxhYmxlPWNvbC5pc051bGxhYmxlLCBpc1VuaXF1ZUtleT1jb2wuaXNVbmlxdWVLZXksIGZvcm1hdFN0cmluZz1jb2wuZm9ybWF0U3RyaW5nLCB0YWJsZT1zZWxmLCBwcm9wZXJ0aWVzPWNvbC5wcm9wZXJ0aWVzKQogICAgICAgICAgICBzZWxmLmFkZEl0ZW0oY29sdW1uKQogICAgICAgIAogICAgICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgZGF0YUlzQXZhaWxhYmxlKHNlbGYpOgogICAgICAgIHJldHVybiBGYWxzZQogICAgCiAgICBAcHJvcGVydHkKICAgIGRlZiBpc1NxbFF1ZXJ5KHNlbGYpIC0+IGJvb2w6CiAgICAgICAgcmV0dXJuIFRydWUKICAgIAogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICBmcm9tIGJpLmNvcmUuZXhwcmVzc2lvbiBpbXBvcnQgRXhwcmVzc2lvbgogICAgICAgIGZyb21TdHJpbmc6IHN0ciA9IGYiXG5GUk9NIHtFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHNlbGYuX19mcm9tKX0iCiAgICAgICAgd2hlcmVTdHJpbmc6IHN0ciA9ICIiCiAgICAgICAgZ3JvdXBCeVN0cmluZzogc3RyID0gIiIKICAgICAgICBvcmRlckJ5U3RyaW5nOiBzdHIgPSAiIgogICAgICAgIAogICAgICAgIGNvbnN0cmFpbnRzID0gW10KICAgICAgICBmb3IgYyBpbiBzZWxmLl9fY29uc3RyYWludHM6CiAgICAgICAgICAgIGNvbnN0cmFpbnRzLmFwcGVuZChFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKGMpKQoKICAgICAgICBvcmRlckJ5ID0gW10KICAgICAgICBmb3IgbyBpbiBzZWxmLl9fb3JkZXJCeToKICAgICAgICAgICAgb3JkZXJCeS5hcHBlbmQoRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShvKSkKICAgICAgICAKICAgICAgICBncm91cEJ5ID0gW10KICAgICAgICBmb3IgZyBpbiBzZWxmLl9fZ3JvdXBCeToKICAgICAgICAgICAgZ3JvdXBCeS5hcHBlbmQoRXhwcmVzc2lvbi5Db252ZXJ0VG9TcWxWYWx1ZShnKSkKICAgICAgICAKICAgICAgICB3aGVyZVN0cmluZyA9ICgiIEFORCAiKS5qb2luKGNvbnN0cmFpbnRzKQogICAgICAgIG9yZGVyQnlTdHJpbmcgPSAiLCAiLmpvaW4ob3JkZXJCeSkKICAgICAgICBncm91cEJ5U3RyaW5nID0gIiwgIi5qb2luKGdyb3VwQnkpCgogICAgICAgIGlmIHdoZXJlU3RyaW5nICE9ICIiOgogICAgICAgICAgICB3aGVyZVN0cmluZyA9IGYiXG5XSEVSRSB7d2hlcmVTdHJpbmd9IgogICAgICAgIGlmIGdyb3VwQnlTdHJpbmcgIT0gIiI6CiAgICAgICAgICAgIGdyb3VwQnlTdHJpbmcgPSBmIlxuR1JPVVAgQlkge2dyb3VwQnlTdHJpbmd9IgogICAgICAgIGlmIG9yZGVyQnlTdHJpbmcgIT0gIiI6CiAgICAgICAgICAgIG9yZGVyQnlTdHJpbmcgPSBmIlxuT1JERVIgQlkge29yZGVyQnlTdHJpbmd9IgogICAgICAgIAogICAgICAgIGNvbHVtbk5hbWVzID0gIiwgIi5qb2luKHNlbGYuZ2V0Q29sdW1uTmFtZXMoc2hvd0V4cHJlc3Npb249VHJ1ZSkpCgogICAgICAgIGlmIHNlbGYuX19kaXN0aW5jdCA9PSBUcnVlOgogICAgICAgICAgICBkaXN0aW5jdCA9ICJESVNUSU5DVCAiCiAgICAgICAgZWxzZToKICAgICAgICAgICAgZGlzdGluY3QgPSAiIgogICAgICAgIAogICAgICAgIGlmIHNlbGYuX190b3BOIGlzIG5vdCBOb25lIGFuZCBzZWxmLl9fdG9wTiA+PSAwOgogICAgICAgICAgICB0b3BOID0gZiJUT1Age3NlbGYuX190b3BOfSIKICAgICAgICBlbHNlOgogICAgICAgICAgICB0b3BOID0gIiIKCiAgICAgICAgcmV0dXJuIGYiU0VMRUNUIHt0b3BOfXtkaXN0aW5jdH17Y29sdW1uTmFtZXN9e2Zyb21TdHJpbmd9e3doZXJlU3RyaW5nfXtncm91cEJ5U3RyaW5nfXtvcmRlckJ5U3RyaW5nfSIKICAgIAogICAgZGVmIHVwZGF0ZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSgpCiAgICAKICAgIGRlZiBfX3N0cl9fKHNlbGYpOgogICAgICAgIHJldHVybiBzZWxmLnZhbHVlCgpTUUxFeHByZXNzaW9uLnJlZ2lzdGVyKFNRTFNlbGVjdEV4cHJlc3Npb24p'))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLFunctionExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBMaXN0LCBPcHRpb25hbApmcm9tIGFiYyBpbXBvcnQgQUJDLCBhYnN0cmFjdG1ldGhvZApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudCwgRXhwcmVzc2lvbkNvZGVUeXBlCmZyb20gYmkuY29yZS5leHByZXNzaW9uIGltcG9ydCBFeHByZXNzaW9uCmZyb20gYmkuY29yZS5mdW5jdGlvbiBpbXBvcnQgRnVuY3Rpb24KZnJvbSBiaS5jb3JlLmV4cHJlc3Npb25UeXBlIGltcG9ydCBFeHByZXNzaW9uVHlwZQpmcm9tIGJpLnNxbC5TUUxFeHByZXNzaW9uIGltcG9ydCBTUUxFeHByZXNzaW9uCgpjbGFzcyBTUUxGdW5jdGlvbkV4cHJlc3Npb24oU1FMRXhwcmVzc2lvbik6CiAgICBkZWYgX19pbml0X18oc2VsZiwgCiAgICAgICAgICAgICAgICAgcGFyZW50OiBFeHByZXNzaW9uUGFyZW50LCAKICAgICAgICAgICAgICAgICBmdW5jdGlvbk5hbWU6IHN0ciwKICAgICAgICAgICAgICAgICBwYXJhbWV0ZXJzOiBsaXN0ID0gW10sCiAgICAgICAgICAgICAgICAgYXR0cmlidXRlOiBzdHIgPSAiIiwKICAgICAgICAgICAgICAgICByZXR1cm5UeXBlOiBFeHByZXNzaW9uVHlwZSA9IEV4cHJlc3Npb25UeXBlLkFueSwKICAgICAgICAgICAgICAgICBwcm9wZXJ0aWVzOiBkaWN0ID0ge30pOgogICAgICAgIFNRTEV4cHJlc3Npb24uX19pbml0X18oc2VsZiwgcGFyZW50PXBhcmVudCwgcmV0dXJuVHlwZT1yZXR1cm5UeXBlLCBwcm9wZXJ0aWVzPXByb3BlcnRpZXMpCiAgICAgICAgc2VsZi5fX2Z1bmN0aW9uTmFtZSA9IGZ1bmN0aW9uTmFtZQogICAgICAgIHNlbGYuX19wYXJhbWV0ZXJzID0gcGFyYW1ldGVycwogICAgICAgIHNlbGYuX19hdHRyaWJ1dGUgPSBhdHRyaWJ1dGUKICAgIAogICAgQHByb3BlcnR5CiAgICBkZWYgcGFyYW1ldGVycyhzZWxmKSAtPiBMaXN0W0V4cHJlc3Npb25dOgogICAgICAgIHJldHVybiBzZWxmLl9fcGFyYW1ldGVycwoKICAgIEBwcm9wZXJ0eQogICAgZGVmIGlzRnVuY3Rpb24oc2VsZik6CiAgICAgICAgcmV0dXJuIFRydWUKICAgIAogICAgZGVmIHVwZGF0ZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5nZXRTcWxWYWx1ZSgpCiAgICAKICAgIEBwcm9wZXJ0eQogICAgZGVmIGF0dHJpYnV0ZShzZWxmKToKICAgICAgICBpZiBzZWxmLl9fYXR0cmlidXRlIGlzIE5vbmUgb3IgbGVuKHNlbGYuX19hdHRyaWJ1dGUpID09IDA6CiAgICAgICAgICAgIHJldHVybiAiIgogICAgICAgIGVsc2U6CiAgICAgICAgICAgIHJldHVybiBmIntzZWxmLl9fYXR0cmlidXRlfSAiCiAgICAgICAgCiAgICBkZWYgX19nZXRQYXJhbWV0ZXJTdHJpbmcoc2VsZik6CiAgICAgICAgcGFyYW1ldGVyU3RyaW5ncyA9IFtdCiAgICAgICAgZm9yIHAgaW4gc2VsZi5fX3BhcmFtZXRlcnM6CiAgICAgICAgICAgIHYgPSBFeHByZXNzaW9uLkNvbnZlcnRUb1NxbFZhbHVlKHApCiAgICAgICAgICAgIHBhcmFtZXRlclN0cmluZ3MuYXBwZW5kKHYpCiAgICAgICAgcmV0dXJuICIsIi5qb2luKHBhcmFtZXRlclN0cmluZ3MpCgogICAgZGVmIGdldFNxbChzZWxmKToKICAgICAgICByZXR1cm4gZiJ7c2VsZi5fX2Z1bmN0aW9uTmFtZX0oe3NlbGYuYXR0cmlidXRlfXtzZWxmLl9fZ2V0UGFyYW1ldGVyU3RyaW5nKCl9KSIKICAgIAogICAgZGVmIF9fc3RyX18oc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0U3FsVmFsdWUoKSAgICAKCgpTUUxFeHByZXNzaW9uLnJlZ2lzdGVyKFNRTEZ1bmN0aW9uRXhwcmVzc2lvbik='))  SINGLE=TRUE OVERWRITE=TRUE;

COPY INTO @UI.STREAMLIT/bi/sql/SQLStatementExpression.py FROM (SELECT BASE64_DECODE_STRING('ZnJvbSBfX2Z1dHVyZV9fIGltcG9ydCBhbm5vdGF0aW9ucwpmcm9tIHR5cGluZyBpbXBvcnQgQW55LCBMaXN0LCBVbmlvbgpmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblBhcmVudCBpbXBvcnQgRXhwcmVzc2lvblBhcmVudApmcm9tIGJpLmNvcmUuZXhwcmVzc2lvblR5cGUgaW1wb3J0IEV4cHJlc3Npb25UeXBlCmZyb20gLlNRTEV4cHJlc3Npb24gaW1wb3J0IFNRTEV4cHJlc3Npb24sIEV4cHJlc3Npb24KCiAgICAKY2xhc3MgU1FMU3RhdGVtZW50RXhwcmVzc2lvbihTUUxFeHByZXNzaW9uKTogICAgCiAgICBkZWYgX19pbml0X18oc2VsZiwgc3RhdGVtZW50OiBzdHIsIHBhcmVudDogRXhwcmVzc2lvblBhcmVudCwgcmV0dXJuVHlwZTogRXhwcmVzc2lvblR5cGUgPSBFeHByZXNzaW9uVHlwZS5BbnksIHByb3BlcnRpZXM6IGRpY3QgPSB7fSk6CiAgICAgICAgaWYgIm9yaWdpbmFsIiBub3QgaW4gcHJvcGVydGllczoKICAgICAgICAgICAgcHJvcGVydGllc1sib3JpZ2luYWwiXSA9IHN0YXRlbWVudAoKICAgICAgICBzdXBlcigpLl9faW5pdF9fKHBhcmVudD1wYXJlbnQsIHByb3BlcnRpZXM9cHJvcGVydGllcywgcmV0dXJuVHlwZT1yZXR1cm5UeXBlKQogICAgICAgIHNlbGYuX192YWx1ZSA9IHN0YXRlbWVudAogICAgCiAgICBkZWYgaXNWYWx1ZShzZWxmKToKICAgICAgICByZXR1cm4gc2VsZi5yZXR1cm5UeXBlLmlzU2NhbGFyCiAgICAKICAgIGRlZiB1cGRhdGUoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuZ2V0U3FsVmFsdWUoKQoKICAgIGRlZiBnZXRTcWwoc2VsZik6CiAgICAgICAgcmV0dXJuIHNlbGYuX192YWx1ZSAgICAgICAgCgpTUUxFeHByZXNzaW9uLnJlZ2lzdGVyKFNRTFN0YXRlbWVudEV4cHJlc3Npb24pCgo='))  SINGLE=TRUE OVERWRITE=TRUE;



In [ ]:
-- streamlit object

-- Use WH_TO_USE as the streamlit warehouse if it's defined, otherwise use ADMIN_WH.
SET WH_TO_USE = COALESCE(GETVARIABLE('WH_TO_USE'), $ADMIN_WH);

DROP STREAMLIT IF EXISTS UI.STELLAR_BI;

CREATE OR REPLACE STREAMLIT UI.STELLAR_BI
     FROM @UI.STREAMLIT
     MAIN_FILE = 'app.py'
     QUERY_WAREHOUSE = $WH_TO_USE
     COMMENT = $COMMENT_TAG
     ;

-- role grants to streamlit object
GRANT USAGE ON STREAMLIT UI.STELLAR_BI TO ROLE IDENTIFIER($CREATOR);


In [ ]:
-- Use ROLE_TO_USE if it's defined, otherwise use SYSADMIN.
SET ROLE_TO_USE = COALESCE(GETVARIABLE('ROLE_TO_USE'), 'SYSADMIN');
USE ROLE IDENTIFIER($ROLE_TO_USE);

USE DATABASE IDENTIFIER($DB);
CREATE SCHEMA IF NOT EXISTS PBI;
GRANT ALL ON SCHEMA PBI TO ROLE IDENTIFIER($ADMIN);
